## Librerias


In [1]:
import pandas as pd
import numpy as np      # Para np.nan, np.where, etc.
import re
import unicodedata
from datetime import datetime  # Solo si usas datetime.strptime o similar


In [2]:
df = pd.read_excel(
    "Matriz Viabilidad Juridica DAT-2.xlsx",
    sheet_name="VIABILIDAD JURÍDICA",
    header=1,
    dtype={"ID": "string"},
)

c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## ID

In [3]:
import pandas as pd
import numpy as np
import re
import unicodedata

# 1) Cargar archivo base
df = pd.read_excel(
    "Matriz Viabilidad Juridica DAT-2.xlsx",
    sheet_name="VIABILIDAD JURÍDICA",
    header=1,
    dtype={"ID": "string"},
)

# 2) Columnas base para revisión manual
df["ID_RAW"] = df["ID"].astype("string")
df["Fila_Excel"] = df.index + 2  # fila real en Excel

# 3) Normalizar ID → ID_LIMPIO
df["ID_LIMPIO"] = (
    df["ID_RAW"]
      .astype("string")
      .str.replace("\u00A0", " ", regex=False)   # NBSP → espacio normal
      .str.replace(r"\s+", " ", regex=True)     # espacios múltiples → uno
      .str.strip()
)

# 4) Detectar IDs vacíos (después de limpiar)
mask_vacios = df["ID_LIMPIO"].isna() | (df["ID_LIMPIO"] == "")

# 5) Detectar duplicados (solo considerando IDs no vacíos)
mask_no_vacios = ~mask_vacios
mask_dups = mask_no_vacios & df["ID_LIMPIO"].duplicated(keep=False)

# 6) Columna final para migración: ID_SQL
df["ID_SQL"] = df["ID_LIMPIO"]
df.loc[mask_vacios, "ID_SQL"] = "SIN REGISTRO"

# 7) Reordenar columnas: dejar las nuevas junto a la original
cols = list(df.columns)
grupo = ["ID", "ID_RAW", "ID_LIMPIO", "ID_SQL", "Fila_Excel"]
grupo = [c for c in grupo if c in df.columns]
otras = [c for c in cols if c not in grupo]
df = df[grupo + otras]

# 8) Resumen general
print("IDs vacíos (después de limpiar):", int(mask_vacios.sum()))
print("IDs duplicados (después de limpiar):", int(mask_dups.sum()))

filas_vacias_id = df.loc[mask_vacios, "Fila_Excel"].tolist()
filas_dups_id   = df.loc[mask_dups, "Fila_Excel"].tolist()

print("Filas Excel con IDs vacíos:", filas_vacias_id)
print("Filas Excel con IDs duplicados:", filas_dups_id)

# 9) DataFrames para exportar problemas
cols_export = ["Fila_Excel", "ID", "ID_RAW", "ID_LIMPIO", "ID_SQL"]

df_vacios = df.loc[mask_vacios, cols_export].copy()
df_dups   = df.loc[mask_dups, cols_export].copy()

if not df_vacios.empty:
    df_vacios.to_excel("ID_con_problemas_vacios.xlsx", index=False)

if not df_dups.empty:
    df_dups.to_excel("ID_con_problemas_duplicados.xlsx", index=False)

# 10) Excel con TODOS los registros (original + limpio + para migrar)
df[cols_export].to_excel("ID_original_limpio_y_SQL.xlsx", index=False)

print("\nArchivos creados:")
if not df_vacios.empty:
    print(" - ID_con_problemas_vacios.xlsx")
if not df_dups.empty:
    print(" - ID_con_problemas_duplicados.xlsx")
print(" - ID_original_limpio_y_SQL.xlsx")

# 11) Ejemplo de las primeras filas en consola
print("\n=== Ejemplo de ID original, limpio y para migrar ===")
print(
    df.loc[:, cols_export]
      .head(20)
      .to_string(index=False)
)

# 12) Ejemplos de vacíos
print("\n=== Ejemplos de IDs vacíos (después de limpiar) ===")
if not df_vacios.empty:
    print(df_vacios.head(20).to_string(index=False))
else:
    print("No hay IDs vacíos.")

# 13) Ejemplos de duplicados
print("\n=== Ejemplos de IDs duplicados (después de limpiar) ===")
if not df_dups.empty:
    print(df_dups.head(20).to_string(index=False))
else:
    print("No hay IDs duplicados.")


c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


IDs vacíos (después de limpiar): 22
IDs duplicados (después de limpiar): 0
Filas Excel con IDs vacíos: [13239, 20715, 20718, 21082, 23234, 24032, 24033, 24042, 24043, 24044, 24045, 24046, 24047, 24048, 24049, 24050, 24051, 24052, 24053, 24054, 24055, 24056]
Filas Excel con IDs duplicados: []

Archivos creados:
 - ID_con_problemas_vacios.xlsx
 - ID_original_limpio_y_SQL.xlsx

=== Ejemplo de ID original, limpio y para migrar ===
 Fila_Excel      ID  ID_RAW ID_LIMPIO  ID_SQL
          2 ID8099B ID8099B   ID8099B ID8099B
          3 ID8099C ID8099C   ID8099C ID8099C
          4 ID8099D ID8099D   ID8099D ID8099D
          5 ID8099E ID8099E   ID8099E ID8099E
          6 ID9080A ID9080A   ID9080A ID9080A
          7 ID8099F ID8099F   ID8099F ID8099F
          8 ID8099G ID8099G   ID8099G ID8099G
          9 ID10439 ID10439   ID10439 ID10439
         10  ID6287  ID6287    ID6287  ID6287
         11 ID8099A ID8099A   ID8099A ID8099A
         12  ID4656  ID4656    ID4656  ID4656
         13  ID83

In [4]:
df.head()

,ID,ID_RAW,ID_LIMPIO,ID_SQL,Fila_Excel,FMI,No. Expediente,FUENTE,TIPO DE PROCESO AGRARIO,ETAPA PROCESAL,...,ESTADO REVISION\nAPROBADO / DEVUELTO / EN REVISION,OBSERVACIONES REVISOR,ENTREGÓ CARPETA DE SOPORTES\n(SI / NO),FECHA DE ENVÍO A COORDINACIÓN,ESTADO APROBACIÓN COORDINADORA,FECHA DE REMISIÓN SOPORTES A GESTORA DOCUMENTAL,FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL,FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO,FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO,FECHA GESTION DE ETAPA SIT
0,ID8099B,ID8099B,ID8099B,ID8099B,2,062-37357,NaN,WHATSAPP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID8099C,ID8099C,ID8099C,ID8099C,3,062-37359,NaN,WHATSAPP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID8099D,ID8099D,ID8099D,ID8099D,4,062-37361,NaN,WHATSAPP,NaN,NaN,...,APROBADO,SE REMITE PARA FIRMA,NaN,NaN,FIRMADO Y ENVIADO 21/11/24,NaN,NaN,2024-11-21 00:00:00,2024-12-21 00:00:00,NaN
3,ID8099E,ID8099E,ID8099E,ID8099E,5,062-37354,NaN,WHATSAPP,NaN,NaN,...,APROBADO,SE REMITE PARA FIRMA,SI,2025-02-24 00:00:00,APROBACIÓN COORDINACIÓN,2025-02-24 00:00:00,2025-02-24 00:00:00,2025-02-25 00:00:00,NaN,NaN
4,ID9080A,ID9080A,ID9080A,ID9080A,6,142-23909,NaN,WHATSAPP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## FMI

In [5]:
df["FMI_LIMPIO"] = (
    df["FMI"]
      .astype("string")
      .str.replace("\u00A0", " ", regex=False)
      .str.strip()
)

mask_vacios_fmi = df["FMI_LIMPIO"].isna() | (df["FMI_LIMPIO"] == "")

# Formato esperado: 3 dígitos, guion, 5 dígitos  -> 000-00000
pat_fmi = re.compile(r"^\d{3}-\d{5}$")

mask_formato_malo_fmi = ~mask_vacios_fmi & ~df["FMI_LIMPIO"].str.match(pat_fmi)
mask_dups_fmi = ~mask_vacios_fmi & df["FMI_LIMPIO"].duplicated(keep=False)

filas_fmi_vacios = (df.index[mask_vacios_fmi] + 2).tolist()
filas_fmi_formato_malo = (df.index[mask_formato_malo_fmi] + 2).tolist()
filas_fmi_dups = (df.index[mask_dups_fmi] + 2).tolist()

print("Filas Excel FMI vacíos:", filas_fmi_vacios)
print("Filas Excel FMI formato incorrecto:", filas_fmi_formato_malo)
print("Filas Excel FMI duplicados:", filas_fmi_dups)

Filas Excel FMI vacíos: [895, 2905, 3967, 3969, 4298, 6077, 7385, 20942, 20963, 20964, 21130, 21132, 21133, 21134, 21135, 21136, 21141, 21142, 21143, 21145, 21146, 21147, 21148, 21149, 21150, 21151, 21152, 21153, 21154, 21155]
Filas Excel FMI formato incorrecto: [9, 15, 18, 20, 27, 29, 33, 34, 36, 37, 38, 40, 42, 43, 45, 48, 49, 50, 53, 54, 56, 60, 64, 65, 69, 70, 71, 73, 79, 80, 81, 82, 101, 103, 104, 105, 106, 107, 108, 124, 128, 130, 134, 135, 143, 144, 145, 154, 155, 156, 157, 159, 160, 164, 165, 167, 168, 172, 175, 176, 177, 178, 181, 182, 183, 188, 190, 194, 198, 203, 208, 212, 213, 215, 216, 217, 225, 226, 227, 231, 234, 235, 236, 238, 240, 243, 245, 246, 248, 252, 256, 259, 261, 264, 265, 266, 268, 269, 271, 273, 274, 276, 282, 284, 285, 287, 290, 293, 295, 299, 302, 303, 307, 308, 310, 311, 312, 314, 321, 324, 327, 330, 331, 335, 336, 337, 338, 341, 342, 344, 346, 348, 350, 351, 353, 354, 357, 359, 360, 361, 368, 369, 371, 373, 374, 375, 376, 379, 383, 386, 388, 389, 393, 394,

In [6]:
import pandas as pd
import numpy as np
import re
import unicodedata

# Se asume que df ya está cargado con la matriz completa
# y que ya existe o no la columna Fila_Excel.

# 1) Asegurar columna auxiliar con la fila real de Excel
#    (encabezado de datos empieza en la fila 2)
df["Fila_Excel"] = df.index + 2

# 2) Columnas base para FMI (crudo y limpio)
df["FMI_RAW"] = df["FMI"].astype("string")

df["FMI_LIMPIO"] = (
    df["FMI_RAW"]
      .astype("string")
      .str.replace("\u00A0", " ", regex=False)   # NBSP → espacio normal
      .str.replace(r"\s+", " ", regex=True)     # espacios múltiples → uno
      .str.strip()
)

# 3) Columnas de estado / diagnóstico
mask_vacios_fmi = df["FMI_LIMPIO"].isna() | (df["FMI_LIMPIO"] == "")

# En FMI NO vamos a validar formato (no tiene patrón fijo),
# sólo limpiamos, marcamos vacíos y duplicados.

mask_no_vacios_fmi = ~mask_vacios_fmi
mask_dups_fmi = mask_no_vacios_fmi & df["FMI_LIMPIO"].duplicated(keep=False)

# 4) Valor final para migrar → FMI_SQL
df["FMI_SQL"] = df["FMI_LIMPIO"]
df.loc[mask_vacios_fmi, "FMI_SQL"] = "SIN REGISTRO"

# 5) Listas de filas problemáticas (para que las tengas claras)
filas_fmi_vacios = df.loc[mask_vacios_fmi, "Fila_Excel"].tolist()
filas_fmi_dups   = df.loc[mask_dups_fmi,   "Fila_Excel"].tolist()

print("FMI vacíos (después de limpiar):", int(mask_vacios_fmi.sum()))
print("FMI duplicados (después de limpiar):", int(mask_dups_fmi.sum()))
print("Filas Excel FMI vacíos:", filas_fmi_vacios)
print("Filas Excel FMI duplicados:", filas_fmi_dups)

# 6) DataFrames para exportar a Excel
cols_export = ["Fila_Excel", "FMI", "FMI_RAW", "FMI_LIMPIO", "FMI_SQL"]

df_vacios_fmi = df.loc[mask_vacios_fmi, cols_export].copy()
df_dups_fmi   = df.loc[mask_dups_fmi,   cols_export].copy()

# Excel solo con vacíos
if not df_vacios_fmi.empty:
    df_vacios_fmi.to_excel("FMI_con_problemas_vacios.xlsx", index=False)

# Excel solo con duplicados
if not df_dups_fmi.empty:
    df_dups_fmi.to_excel("FMI_con_problemas_duplicados.xlsx", index=False)

# Excel con TODOS los registros (crudo + limpio + valor final para migrar)
df[cols_export].to_excel("FMI_original_limpio_y_SQL.xlsx", index=False)

print("\nArchivos creados:")
if not df_vacios_fmi.empty:
    print(" - FMI_con_problemas_vacios.xlsx")
if not df_dups_fmi.empty:
    print(" - FMI_con_problemas_duplicados.xlsx")
print(" - FMI_original_limpio_y_SQL.xlsx")

# 7) Muestra en consola para revisión rápida
print("\n=== Ejemplo FMI original, limpio y para migrar ===")
print(
    df.loc[:, cols_export]
      .head(20)
      .to_string(index=False)
)

print("\n=== Ejemplos de FMI vacíos (después de limpiar) ===")
if not df_vacios_fmi.empty:
    print(df_vacios_fmi.head(20).to_string(index=False))
else:
    print("No hay FMI vacíos.")

print("\n=== Ejemplos de FMI duplicados (después de limpiar) ===")
if not df_dups_fmi.empty:
    print(df_dups_fmi.head(20).to_string(index=False))
else:
    print("No hay FMI duplicados.")


FMI vacíos (después de limpiar): 30
FMI duplicados (después de limpiar): 24
Filas Excel FMI vacíos: [895, 2905, 3967, 3969, 4298, 6077, 7385, 20942, 20963, 20964, 21130, 21132, 21133, 21134, 21135, 21136, 21141, 21142, 21143, 21145, 21146, 21147, 21148, 21149, 21150, 21151, 21152, 21153, 21154, 21155]
Filas Excel FMI duplicados: [1041, 7497, 7695, 14804, 16491, 19200, 19781, 19834, 19931, 20966, 21028, 21061, 21131, 21137, 21138, 21139, 21140, 21157, 21158, 21167, 21168, 21169, 21170, 23615]

Archivos creados:
 - FMI_con_problemas_vacios.xlsx
 - FMI_con_problemas_duplicados.xlsx
 - FMI_original_limpio_y_SQL.xlsx

=== Ejemplo FMI original, limpio y para migrar ===
 Fila_Excel        FMI    FMI_RAW FMI_LIMPIO    FMI_SQL
          2  062-37357  062-37357  062-37357  062-37357
          3  062-37359  062-37359  062-37359  062-37359
          4  062-37361  062-37361  062-37361  062-37361
          5  062-37354  062-37354  062-37354  062-37354
          6  142-23909  142-23909  142-23909  14

In [7]:
df.head()

,ID,ID_RAW,ID_LIMPIO,ID_SQL,Fila_Excel,FMI,No. Expediente,FUENTE,TIPO DE PROCESO AGRARIO,ETAPA PROCESAL,...,FECHA DE ENVÍO A COORDINACIÓN,ESTADO APROBACIÓN COORDINADORA,FECHA DE REMISIÓN SOPORTES A GESTORA DOCUMENTAL,FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL,FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO,FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO,FECHA GESTION DE ETAPA SIT,FMI_LIMPIO,FMI_RAW,FMI_SQL
0,ID8099B,ID8099B,ID8099B,ID8099B,2,062-37357,NaN,WHATSAPP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,062-37357,062-37357,062-37357
1,ID8099C,ID8099C,ID8099C,ID8099C,3,062-37359,NaN,WHATSAPP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,062-37359,062-37359,062-37359
2,ID8099D,ID8099D,ID8099D,ID8099D,4,062-37361,NaN,WHATSAPP,NaN,NaN,...,NaN,FIRMADO Y ENVIADO 21/11/24,NaN,NaN,2024-11-21 00:00:00,2024-12-21 00:00:00,NaN,062-37361,062-37361,062-37361
3,ID8099E,ID8099E,ID8099E,ID8099E,5,062-37354,NaN,WHATSAPP,NaN,NaN,...,2025-02-24 00:00:00,APROBACIÓN COORDINACIÓN,2025-02-24 00:00:00,2025-02-24 00:00:00,2025-02-25 00:00:00,NaN,NaN,062-37354,062-37354,062-37354
4,ID9080A,ID9080A,ID9080A,ID9080A,6,142-23909,NaN,WHATSAPP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142-23909,142-23909,142-23909


## No.Expediente

In [8]:
import pandas as pd
import re
import unicodedata

# -------------------------------------------------------------------
# 1. Asegurar columna de fila aproximada de Excel (encabezado en fila 2)
# -------------------------------------------------------------------
if 'Fila_Excel_aprox' not in df.columns:
    df['Fila_Excel_aprox'] = df.index + 2

# -------------------------------------------------------------------
# 2. Renombrar la columna original y conservar valor crudo
# -------------------------------------------------------------------
# Si aún se llama "No. Expediente", la renombramos a "NumeroExpediente"
if 'No. Expediente' in df.columns and 'NumeroExpediente' not in df.columns:
    df = df.rename(columns={"No. Expediente": "NumeroExpediente"})

# Guardamos la columna original tal cual viene del Excel
df['NumeroExpediente_RAW'] = df['NumeroExpediente']

# -------------------------------------------------------------------
# 3. Limpiar espacios / NBSP y generar columna LIMPIA
# -------------------------------------------------------------------
def limpiar_basico_expediente(valor):
    if pd.isna(valor):
        return None
    # Reemplazar NBSP por espacio normal y recortar
    v = str(valor).replace("\u00A0", " ").strip()
    if v == "":
        return None
    return v

df['NumeroExpediente_LIMPIO'] = df['NumeroExpediente_RAW'].apply(limpiar_basico_expediente)

# -------------------------------------------------------------------
# 4. Normalizar para migrar (SQL): convertir "sin dato" → None
# -------------------------------------------------------------------
VALORES_SIN_DATO = {
    "NA", "N/A",
    "NO APLICA", "NO APLICA.",
    "NO REGISTRA", "NO REGISTRA.",
    "NO REGISTRO",
    "SIN REGISTRO"
}

def normalizar_numero_expediente(valor):
    if pd.isna(valor):
        return None
    # Partimos de la versión ya limpia
    v = str(valor).replace("\u00A0", " ").strip()
    if v == "":
        return None
    vu = v.upper()
    if vu in VALORES_SIN_DATO:
        return None
    # Aquí NO imponemos formato ni unicidad, solo dejamos el texto tal cual
    return v

df['NumeroExpediente_SQL'] = df['NumeroExpediente_RAW'].apply(normalizar_numero_expediente)

# -------------------------------------------------------------------
# 5. Longitud y diagnósticos (para varchar(100) en SQL)
# -------------------------------------------------------------------
df['NumeroExpediente_LEN'] = df['NumeroExpediente_SQL'].astype(str).str.len()

mask_con_dato = df['NumeroExpediente_SQL'].notna()
mask_largos = mask_con_dato & (df['NumeroExpediente_LEN'] > 100)

total = len(df)
sin_dato = df['NumeroExpediente_SQL'].isna().sum()
con_dato = mask_con_dato.sum()

print("=== RESUMEN NumeroExpediente ===")
print(f"Total registros: {total}")
print(f"Sin dato (NumeroExpediente_SQL = NULL): {sin_dato}")
print(f"Con dato (NumeroExpediente_SQL no nulo): {con_dato}")

print("\n=== Top 20 valores distintos (NumeroExpediente_SQL) ===")
print(df['NumeroExpediente_SQL'].value_counts(dropna=True).head(20))

# Registros que exceden la longitud permitida por la columna SQL (varchar(100))
if mask_largos.any():
    print("\n=== Registros con NumeroExpediente_SQL > 100 caracteres (revisar) ===")
    print(
        df.loc[mask_largos,
               ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
                'NumeroExpediente_RAW', 'NumeroExpediente_SQL', 'NumeroExpediente_LEN']
        ].to_string(index=False)
    )
else:
    print("\nNingún NumeroExpediente_SQL supera los 100 caracteres (ok para varchar(100)).")

# -------------------------------------------------------------------
# 6. Ejemplo pequeño para inspección rápida en consola
# -------------------------------------------------------------------
print("\n=== Ejemplo de primeras filas (RAW / LIMPIO / SQL) ===")
print(
    df[['Fila_Excel_aprox',
        'NumeroExpediente_RAW',
        'NumeroExpediente_LIMPIO',
        'NumeroExpediente_SQL',
        'NumeroExpediente_LEN']
      ].head(10).to_string(index=False)
)

# -------------------------------------------------------------------
# 7. Exportar a Excel TODOS los registros (para revisión visual)
# -------------------------------------------------------------------
cols_export = [
    'Fila_Excel_aprox',
    'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'NumeroExpediente_RAW',
    'NumeroExpediente_LIMPIO',
    'NumeroExpediente_SQL',
    'NumeroExpediente_LEN'
]

# Algunos DataFrame pueden no tener todas estas columnas; filtramos las que existan
cols_export = [c for c in cols_export if c in df.columns]

df_export = df[cols_export].copy()
nombre_excel = "NumeroExpediente_raw_limpio_sql.xlsx"
df_export.to_excel(nombre_excel, index=False)

print(f"\nArchivo creado: {nombre_excel}  (todos los registros con valor crudo, limpio y para migrar)")


=== RESUMEN NumeroExpediente ===
Total registros: 24055
Sin dato (NumeroExpediente_SQL = NULL): 15683
Con dato (NumeroExpediente_SQL no nulo): 8372

=== Top 20 valores distintos (NumeroExpediente_SQL) ===
NumeroExpediente_SQL
202440003401515432E    3
202440003401515433E    3
202440003401515471E    3
202440003401520684E    3
202440003401515790E    3
202440003401515982E    3
202440003401519194E    3
202440003401515430E    3
202440003401515363E    3
202440003401519082E    3
202440003401518989E    3
202440003401520680E    3
202440003401521208E    3
202440003401521434E    3
202440003401521435E    3
202440003401521437E    3
202440003401521438E    3
202440003401521439E    3
202440003401517209E    2
202440003401517363E    2
Name: count, dtype: int64

=== Registros con NumeroExpediente_SQL > 100 caracteres (revisar) ===
 Fila_Excel_aprox       FMI DEPARTAMENTO            MUNICIPIO                                                                                                                    

In [9]:
# Filas con NumeroExpediente_SQL más largo que el límite del campo SQL (100)
mask_largos = df["NumeroExpediente_SQL"].notna() & (df["NumeroExpediente_LEN"] > 100)

cols_largos = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "NumeroExpediente_RAW",
    "NumeroExpediente_LIMPIO",
    "NumeroExpediente_SQL",
    "NumeroExpediente_LEN",
]

df_largos = df.loc[mask_largos, cols_largos].copy()

print("Total NumeroExpediente > 100 caracteres:", len(df_largos))

df_largos.to_excel(
    "NumeroExpediente_largos_para_corregir.xlsx",
    index=False
)
print("Archivo creado: NumeroExpediente_largos_para_corregir.xlsx")


Total NumeroExpediente > 100 caracteres: 10
Archivo creado: NumeroExpediente_largos_para_corregir.xlsx


## FUENTE

In [10]:
import pandas as pd
import unicodedata

# ==========================================
# 0. Se asume que df ya está cargado
#    y que la hoja es "VIABILIDAD JURÍDICA"
# ==========================================
# Ejemplo (si aún no lo has cargado en esta sesión):
# df = pd.read_excel(
#     "Matriz Viabilidad Juridica DAT-2.xlsx",
#     sheet_name="VIABILIDAD JURÍDICA",
#     header=1,
#     dtype={"ID": "string"},
# )

# Aseguramos columna de fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2   # encabezado empieza en fila 2

# ===============================
# 1. Función de normalización
# ===============================
def normalizar_texto_fuente(s):
    """
    Normaliza textos de la columna FUENTE:
    - None / NaN / NA / N/A / NO APLICA / SIN DATO -> None
    - Mayúsculas, sin tildes, espacios comprimidos
    - Reemplaza guiones y guiones bajos por espacio
    """
    if pd.isna(s):
        return None
    s = str(s).strip()
    if s == "":
        return None

    # Valores que significan "sin dato"
    s_up = s.upper()
    if s_up in {"NA", "N/A", "NO APLICA", "SIN DATO"}:
        return None

    # Quitar tildes
    s_norm = unicodedata.normalize("NFD", s_up)
    s_norm = "".join(c for c in s_norm if unicodedata.category(c) != "Mn")

    # Unificar guiones y guiones bajos a espacio
    for ch in ("_", "-", "–", "—"):
        s_norm = s_norm.replace(ch, " ")

    # Comprimir espacios múltiples
    s_norm = " ".join(s_norm.split())

    return s_norm


# ==========================================
# 2. Catálogo final FuenteProceso (ID -> nombre)
#    (debe coincidir con la tabla:
#     ViabilidadJuridica.Postulacion.FuenteProceso)
# ==========================================
CATALOGO_FUENTE_ID = {
    1:  "ARGOS",
    2:  "BANCO AGRARIO DE COLOMBIA",
    3:  "CELSIA - DONACIÓN",
    4:  "CISA",
    5:  "COMPROMISOS ORGANIZACIONES SOCIALES",
    6:  "COMPROMISOS PMU - DAE ROSAS",
    7:  "CONTRALORÍA",
    8:  "CORREO COMPRAS",
    9:  "DAT",
    10: "DIAN",
    11: "DIRECCIÓN GENERAL",
    12: "DONACIÓN PARTICULAR",
    13: "ECOPETROL",
    14: "FEDEGÁN",
    15: "FISCALÍA",
    16: "FONDO ADAPTACIÓN",
    17: "LA MOJANA",
    18: "MADR",
    19: "MIN. DEFENSA",
    20: "OC - AGROVAR",
    21: "OC - ANUC CAUCA",
    22: "OC - ANUC CÓRDOBA",
    23: "OC - ANUC FENSUAGRO",
    24: "OC - ANUC NACIONAL",
    25: "OC - BELLACRUZ DIÁLOGO SOCIAL",
    26: "OC - CACEP ANAFRO",
    27: "OC - CACEP ANZORC",
    28: "OC - CACEP CONGRESO DE LOS PUEBLOS",
    29: "OC - CACEP MARCHA PATRIÓTICA",
    30: "OC - CATATUMBO",
    31: "OC - CIMA",
    32: "OC - CIMA CAUCA",
    33: "OC - CONPES NARIÑO",
    34: "OC - CUMBRE AGRARIA",
    35: "OC - FENSUAGRO",
    36: "OC - PARQUES NACIONALES",
    37: "OC - PUPSOC",
    38: "OC - PUPSOC CAUCA",
    39: "OC - QUIMBO",
    40: "OC - REINCORPORACIÓN",
    41: "OC - SOS COMPROMISOS ORGANIZACIONES",
    42: "ORFEO",
    43: "ORGANIZACIÓN CAMPESINA",
    44: "ROSAS",
    45: "SENTENCIA",
    46: "SIT",
    47: "SOMOS TIERRA",
    48: "UNGRD",
    49: "WHATSAPP",
    50: "WHATSAPP FEDEGÁN",
    # Si en SQL tienes SAE / SAE TRANSFERENCIA con Id propios,
    # los agregas aquí, por ejemplo:
    # 51: "SAE",
    # 52: "SAE TRANSFERENCIA",
}

# Diccionarios auxiliares: texto normalizado -> (Id, nombre catálogo)
MAP_FUENTE_NORM_TO_ID = {}
MAP_FUENTE_NORM_TO_NOMBRE = {}

for id_fuente, nombre in CATALOGO_FUENTE_ID.items():
    clave_norm = normalizar_texto_fuente(nombre)
    MAP_FUENTE_NORM_TO_ID[clave_norm] = id_fuente
    MAP_FUENTE_NORM_TO_NOMBRE[clave_norm] = nombre


# ===================================================
# 3. Crear columnas FUENTE_RAW / TXT_LIMPIO / SQL
# ===================================================

# 3.1. Columna cruda
df["FUENTE_RAW"] = df["FUENTE"].astype("string")

# 3.2. Columna de texto limpio (normalizado)
df["FUENTE_TXT_LIMPIO"] = df["FUENTE_RAW"].apply(normalizar_texto_fuente)

# 3.3. Nombre de catálogo y FK para SQL
df["FUENTE_NOMBRE_SQL"] = df["FUENTE_TXT_LIMPIO"].map(MAP_FUENTE_NORM_TO_NOMBRE)
df["IdFuenteProceso_SQL"] = df["FUENTE_TXT_LIMPIO"].map(MAP_FUENTE_NORM_TO_ID).astype("Int64")

# Opcional: reordenar columnas para que queden juntas junto a FUENTE
cols = list(df.columns)
if "FUENTE" in cols:
    pos = cols.index("FUENTE") + 1
    for c in ["FUENTE_RAW", "FUENTE_TXT_LIMPIO", "FUENTE_NOMBRE_SQL", "IdFuenteProceso_SQL"]:
        if c in cols:
            cols.remove(c)
    cols[pos:pos] = ["FUENTE_RAW", "FUENTE_TXT_LIMPIO", "FUENTE_NOMBRE_SQL", "IdFuenteProceso_SQL"]
    df = df[cols]


# ============================
# 4. Diagnósticos por consola
# ============================

print("=== Valores CRUDOS de FUENTE (top 30) ===")
print(df["FUENTE_RAW"].value_counts(dropna=False).head(30))

print("\n=== Valores NORMALIZADOS de FUENTE_TXT_LIMPIO (top 30) ===")
print(df["FUENTE_TXT_LIMPIO"].value_counts(dropna=False).head(30))

print("\n=== Conteo IdFuenteProceso_SQL (incluyendo nulos) ===")
print(df["IdFuenteProceso_SQL"].value_counts(dropna=False))

# Filas donde hay texto limpio pero NO se pudo mapear a catálogo
mask_sin_mapeo = df["FUENTE_TXT_LIMPIO"].notna() & df["IdFuenteProceso_SQL"].isna()
fuentes_sin_id = (
    df.loc[mask_sin_mapeo, ["FUENTE_RAW", "FUENTE_TXT_LIMPIO"]]
      .drop_duplicates()
      .sort_values("FUENTE_TXT_LIMPIO")
)

print("\n=== FUENTE con texto limpio pero SIN IdFuenteProceso_SQL (para revisar a mano) ===")
print(fuentes_sin_id.to_string(index=False))
print("Total sin IdFuenteProceso_SQL:", len(fuentes_sin_id))

# Pequeña muestra fila a fila para ver el mapeo
print("\n=== Ejemplo de mapeo FUENTE (primeras 10 filas) ===")
print(
    df[[
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "FUENTE_RAW",
        "FUENTE_TXT_LIMPIO",
        "FUENTE_NOMBRE_SQL",
        "IdFuenteProceso_SQL",
    ]].head(10).to_string(index=False)
)


# ===========================================
# 5. Excels para revisión manual / documentación
# ===========================================

# 5.1. Excel con TODO el mapeo
cols_export_full = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "FUENTE_RAW",
    "FUENTE_TXT_LIMPIO",
    "FUENTE_NOMBRE_SQL",
    "IdFuenteProceso_SQL",
]

df[cols_export_full].to_excel(
    "FUENTE_mapeo_completo.xlsx",
    index=False
)

# 5.2. Excel solo con las FUENTE que NO se pudieron mapear al catálogo
df[mask_sin_mapeo][cols_export_full].to_excel(
    "FUENTE_sin_mapeo.xlsx",
    index=False
)

print("\nArchivos generados:")
print(" - FUENTE_mapeo_completo.xlsx")
print(" - FUENTE_sin_mapeo.xlsx")


=== Valores CRUDOS de FUENTE (top 30) ===
FUENTE_RAW
WHATSAPP                               11478
SIT                                     6570
ORFEO                                   1561
FEDEGAN                                 1377
CORREO_COMPRAS                          1100
OC-REINCORPORACION                       525
OC-ANUC NACIONAL                         289
WHATSAPP-FEDEGAN                         178
UNGRD                                    120
OC-QUIMBO                                114
OC-AGROVAR                                71
OC-ANUC CAUCA                             64
DAT                                       57
OC-CATATUMBO                              53
<NA>                                      43
LA MOJANA                                 43
OC-CONPES-NARIÑO                          38
ARGOS                                     34
MIN. DEFENSA                              31
CISA                                      28
OC-PUPSOC-CAUCA                           27
OC

## Tipo de proceso agrario

In [11]:
import pandas as pd
import unicodedata

# ============================================================
# 1. Cargar archivo (ajusta ruta / sheet si es necesario)
# ============================================================
df = pd.read_excel(
    "Matriz Viabilidad Juridica DAT-2.xlsx",
    sheet_name="VIABILIDAD JURÍDICA",
    header=1,
    dtype={"ID": "string"},
)

# Número de fila de Excel (encabezado en fila 2)
df["Fila_Excel_aprox"] = df.index + 2

# ============================================================
# 2. Función de normalización genérica
#    (Si ya la tienes definida igual, puedes omitir este bloque)
# ============================================================
def normalizar_texto(s):
    """
    Normaliza texto:
    - Si es NaN o vacío / 'sin dato' -> None
    - Mayúsculas
    - Sin tildes
    - Guiones y '_' a espacio
    - Espacios múltiples comprimidos
    """
    if pd.isna(s):
        return None
    s = str(s).strip()
    if s == "":
        return None

    s_up = s.upper()

    # textos típicos de "sin dato"
    if s_up in {"NA", "N/A", "NO APLICA", "SIN DATO"}:
        return None

    # quitar tildes
    s_norm = unicodedata.normalize("NFD", s_up)
    s_norm = "".join(c for c in s_norm if unicodedata.category(c) != "Mn")

    # unificar guiones y guiones bajos a espacio
    for ch in ("_", "-", "–", "—"):
        s_norm = s_norm.replace(ch, " ")

    # comprimir espacios múltiples
    s_norm = " ".join(s_norm.split())
    return s_norm

# ============================================================
# 3. Catálogo definitivo TipoProceso (como en SQL)
# ============================================================
CATALOGO_TIPO_PROCESO = [
    (1, "CLARIFICACIÓN"),
    (2, "DESLINDE"),
    (3, "EXTINCIÓN"),
    (4, "NUEVO_SU_288"),
    (5, "BIENES FISCALES PATRIMONIALES"),
    (6, "RECUPERACIÓN"),
    (7, "TITULACIÓN DE BALDÍO A PERSONA NATURAL"),
    (8, "RECONOCIMIENTO DE SENTENCIA SU-288"),
    (9, "TITULACIÓN DE LA POSESIÓN"),
]

# Diccionarios: clave normalizada -> (Id, nombre catálogo)
MAP_TIPO_PROC_NORM_A_ID = {}
MAP_TIPO_PROC_NORM_A_NOMBRE = {}

for id_tipo, nombre in CATALOGO_TIPO_PROCESO:
    clave_norm = normalizar_texto(nombre)
    MAP_TIPO_PROC_NORM_A_ID[clave_norm] = id_tipo
    MAP_TIPO_PROC_NORM_A_NOMBRE[clave_norm] = nombre

# ============================================================
# 4. Normalizar columna "TIPO DE PROCESO AGRARIO"
# ============================================================

col_raw = "TIPO DE PROCESO AGRARIO"

# Copia cruda por si luego quieres mantenerla con otro nombre
df["TipoProceso_RAW"] = df[col_raw]

# Texto limpio (normalizado)
df["TIPO_PROCESO_TXT_LIMPIO"] = df[col_raw].apply(normalizar_texto)

# Nombre catálogo y ID para SQL
df["TipoProceso_NOMBRE_SQL"] = df["TIPO_PROCESO_TXT_LIMPIO"].map(MAP_TIPO_PROC_NORM_A_NOMBRE)
df["IdTipoProceso_SQL"] = df["TIPO_PROCESO_TXT_LIMPIO"].map(MAP_TIPO_PROC_NORM_A_ID).astype("Int64")

# ============================================================
# 5. Reubicar columnas nuevas junto a la columna original
# ============================================================
cols = list(df.columns)
idx_raw = cols.index(col_raw)

# Quitamos las columnas nuevas de donde hayan quedado
for c in ["TipoProceso_RAW", "TIPO_PROCESO_TXT_LIMPIO", "TipoProceso_NOMBRE_SQL", "IdTipoProceso_SQL"]:
    if c in cols:
        cols.remove(c)

# Las volvemos a insertar inmediatamente después de la original
insert_pos = idx_raw + 1
cols[insert_pos:insert_pos] = [
    "TipoProceso_RAW",
    "TIPO_PROCESO_TXT_LIMPIO",
    "TipoProceso_NOMBRE_SQL",
    "IdTipoProceso_SQL",
]

df = df[cols]

# ============================================================
# 6. Resumen por consola
# ============================================================

print("=== Distribución de TIPO_PROCESO_TXT_LIMPIO ===")
print(df["TIPO_PROCESO_TXT_LIMPIO"].value_counts(dropna=False))

print("\n=== Resumen de mapeo a IdTipoProceso_SQL ===")
resumen_mapeo = (
    df.groupby(["TipoProceso_RAW", "TIPO_PROCESO_TXT_LIMPIO", "TipoProceso_NOMBRE_SQL", "IdTipoProceso_SQL"], dropna=False)
      .size()
      .reset_index(name="conteo")
      .sort_values(["IdTipoProceso_SQL", "TipoProceso_NOMBRE_SQL", "TIPO_PROCESO_TXT_LIMPIO"])
)
print(resumen_mapeo.to_string(index=False))

# Casos con texto pero SIN IdTipoProceso_SQL (para revisión manual)
mask_sin_id = df["TIPO_PROCESO_TXT_LIMPIO"].notna() & df["IdTipoProceso_SQL"].isna()

tipo_sin_mapeo = (
    df.loc[mask_sin_id, ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
                         "TipoProceso_RAW", "TIPO_PROCESO_TXT_LIMPIO"]]
      .drop_duplicates()
      .sort_values(["TIPO_PROCESO_TXT_LIMPIO", "TipoProceso_RAW"])
)

print("\n=== TIPO DE PROCESO AGRARIO con texto pero SIN IdTipoProceso_SQL (para revisar) ===")
print(tipo_sin_mapeo.to_string(index=False))
print("Total sin IdTipoProceso_SQL:", len(tipo_sin_mapeo))

# ============================================================
# 7. Exportar a Excel para revisión visual
# ============================================================

# a) Mapeo completo (todos los registros)
cols_export_full = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "TipoProceso_RAW",
    "TIPO_PROCESO_TXT_LIMPIO",
    "TipoProceso_NOMBRE_SQL",
    "IdTipoProceso_SQL",
]

df[cols_export_full].to_excel(
    "TipoProceso_mapeo_completo.xlsx",
    index=False
)

# b) Solo casos sin mapeo (texto pero sin Id)
tipo_sin_mapeo.to_excel(
    "TipoProceso_sin_mapeo.xlsx",
    index=False
)

print("\nArchivos creados:")
print(" - TipoProceso_mapeo_completo.xlsx")
print(" - TipoProceso_sin_mapeo.xlsx")
print("\nTotal filas sin IdTipoProceso_SQL (en df):", df["IdTipoProceso_SQL"].isna().sum())


c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


=== Distribución de TIPO_PROCESO_TXT_LIMPIO ===
TIPO_PROCESO_TXT_LIMPIO
None                                      23229
CLARIFICACION                               365
DESLINDE                                    104
BIENES FISCALES PATRIMONIALES                92
EXTINCION                                    69
RECUPERACION                                 69
TITULACION DE BALDIO A PERSONA NATURAL       68
NUEVO SU 288                                 30
TITULACION DE LA POSESION                    23
RECONOCIMIENTO DE SENTENCIA SU 288            6
Name: count, dtype: int64

=== Resumen de mapeo a IdTipoProceso_SQL ===
                       TipoProceso_RAW                TIPO_PROCESO_TXT_LIMPIO                 TipoProceso_NOMBRE_SQL  IdTipoProceso_SQL  conteo
                         CLARIFICACION                          CLARIFICACION                          CLARIFICACIÓN                  1     365
                              DESLINDE                               DESLINDE           

## ETAPA PROCESAL

In [12]:
import pandas as pd
import unicodedata

# ========================
# 1. Función de normalizar
# ========================
def normalizar_texto(s):
    if pd.isna(s):
        return None
    s = str(s).strip().upper()
    if s in ("", "NA", "N/A", "NO APLICA", "SIN DATO"):
        return None

    # quitar tildes
    s = unicodedata.normalize("NFD", s)
    s = "".join(c for c in s if unicodedata.category(c) != "Mn")

    # unificar separadores y espacios
    for ch in ("_", "-"):
        s = s.replace(ch, " ")
    s = " ".join(s.split())

    return s

# ===============================
# 2. Catálogo ETAPA PROCESAL SQL
# ===============================
CATALOGO_ETAPA = [
    "APERTURA TRAMITE ADMINISTRATIVO",  # 1
    "ARCHIVO FINAL",                    # 2
    "CIERRE ETAPA PROBATORIA",          # 3
    "EJECUTORIADO",                     # 4
    "ETAPA FINAL",                      # 5
    "ETAPA INICIAL",                    # 6
    "ETAPA PRELIMINAR",                 # 7
    "ETAPA PREVIA",                     # 8
    "ETAPA PROBATORIA",                 # 9
    "NUEVO_SU_288",                     #10
    "TRASLADADO",                       #11
    "ETAPA DE APERTURA",                #12
    "ETAPA DE CIERRE",                  #13
    "EN MISIONAL SIN GESTION",          #14
    "SIN INICIAR",                      #15
    "REGISTRO",                         #16
    "SUSPENDIDO",                       #17
    "DECIDIDO",                         #18
]

MAP_ETAPA_NOMBRE = {
    normalizar_texto(nombre): nombre
    for nombre in CATALOGO_ETAPA
}
MAP_ETAPA_ID = {
    normalizar_texto(nombre): idx
    for idx, nombre in enumerate(CATALOGO_ETAPA, start=1)
}

# ===============================
# 3. Sinónimos que trae el Excel
# ===============================
SINONIMOS = {
    "CIERRE PROBATORIA": "CIERRE ETAPA PROBATORIA",
    "APERTURA PROBATORIA": "ETAPA PROBATORIA",
    "ETAPA INICIO": "ETAPA INICIAL",
    "SININICIAR": "SIN INICIAR",
}

for variante, oficial in SINONIMOS.items():
    MAP_ETAPA_ID[normalizar_texto(variante)] = MAP_ETAPA_ID[normalizar_texto(oficial)]
    MAP_ETAPA_NOMBRE[normalizar_texto(variante)] = MAP_ETAPA_NOMBRE[normalizar_texto(oficial)]

# ==========================
# 4. Columnas en el DataFrame
# ==========================
# Aseguramos columna de fila de Excel (si no la tienes ya creada)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2  # porque header=1

# RAW directamente de la matriz
df["EtapaProcesal_RAW"] = df["ETAPA PROCESAL"]

# Texto normalizado
df["EtapaProcesal_TXT_LIMPIO"] = df["EtapaProcesal_RAW"].apply(normalizar_texto)

# Nombre catálogo y FK
df["EtapaProcesal_NOMBRE_SQL"] = df["EtapaProcesal_TXT_LIMPIO"].map(MAP_ETAPA_NOMBRE)
df["IdEtapaProcesal_SQL"] = df["EtapaProcesal_TXT_LIMPIO"].map(MAP_ETAPA_ID).astype("Int64")

# ==============================
# 5. Resumen por combinación
# ==============================
resumen = (
    df.groupby(
        ["EtapaProcesal_RAW",
         "EtapaProcesal_TXT_LIMPIO",
         "EtapaProcesal_NOMBRE_SQL",
         "IdEtapaProcesal_SQL"],
        dropna=False
    )
    .size()
    .reset_index(name="conteo")
    .sort_values("conteo", ascending=False)
)

print("=== Distribución de EtapaProcesal_TXT_LIMPIO ===")
print(df["EtapaProcesal_TXT_LIMPIO"].value_counts(dropna=False))

print("\n=== Resumen de mapeo a IdEtapaProcesal_SQL ===")
print(resumen.to_string(index=False))

# ==========================
# 6. Filas sin Id (para revisar)
# ==========================
mask_sin_id = (
    df["EtapaProcesal_TXT_LIMPIO"].notna()
    & df["IdEtapaProcesal_SQL"].isna()
)

etapa_sin_id = df.loc[
    mask_sin_id,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "EtapaProcesal_RAW",
        "EtapaProcesal_TXT_LIMPIO",
    ],
]

print("\n=== ETAPA PROCESAL con texto pero SIN IdEtapaProcesal_SQL (para revisar) ===")
print(etapa_sin_id.to_string(index=False))
print("Total sin IdEtapaProcesal_SQL:", len(etapa_sin_id))

# ==========================
# 7. Excels de salida
# ==========================
# 7.1 Todas las filas (para inspección visual)
cols_mapeo_filas = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "EtapaProcesal_RAW",
    "EtapaProcesal_TXT_LIMPIO",
    "EtapaProcesal_NOMBRE_SQL",
    "IdEtapaProcesal_SQL",
]
df[cols_mapeo_filas].to_excel(
    "EtapaProcesal_mapeo_completo_filas.xlsx", index=False
)

# 7.2 Solo las filas sin Id (si hubiera)
etapa_sin_id.to_excel(
    "EtapaProcesal_sin_IdEtapaProcesal_SQL.xlsx", index=False
)

# 7.3 Resumen por combinación (como el que ya viste)
resumen.to_excel(
    "EtapaProcesal_resumen_valores.xlsx", index=False
)

print("\nArchivos creados:")
print(" - EtapaProcesal_mapeo_completo_filas.xlsx")
print(" - EtapaProcesal_sin_IdEtapaProcesal_SQL.xlsx")
print(" - EtapaProcesal_resumen_valores.xlsx")


=== Distribución de EtapaProcesal_TXT_LIMPIO ===
EtapaProcesal_TXT_LIMPIO
None                               23223
ETAPA PRELIMINAR                     212
APERTURA TRAMITE ADMINISTRATIVO      112
ETAPA DE CIERRE                       85
EN MISIONAL SIN GESTION               62
ETAPA FINAL                           55
SININICIAR                            49
TRASLADADO                            47
ARCHIVO FINAL                         44
APERTURA PROBATORIA                   42
ETAPA PREVIA                          24
REGISTRO                              24
NUEVO SU 288                          21
ETAPA DE APERTURA                     19
EJECUTORIADO                          19
CIERRE PROBATORIA                      8
ETAPA INICIO                           7
SUSPENDIDO                             1
DECIDIDO                               1
Name: count, dtype: int64

=== Resumen de mapeo a IdEtapaProcesal_SQL ===
              EtapaProcesal_RAW        EtapaProcesal_TXT_LIMPIO        Et

## DEPENDENCIA

In [13]:
# ==========================
# DEPENDENCIA (texto libre)
# ==========================

# Si no existe aún, creamos la columna con la fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2   # porque leíste con header=1

# 1. Asegurar nombre de columna consistente con SQL
df = df.rename(columns={"DEPENDENCIA": "Dependencia"})

# 2. Guardar valor crudo antes de limpiar
df["Dependencia_RAW"] = df["Dependencia"]

# 3. Función de limpieza (respetando las reglas que ya tenías)
def limpiar_dependencia(valor):
    if pd.isna(valor):
        return None

    s = str(valor)

    # NBSP → espacio normal
    s = s.replace("\u00A0", " ")

    # quitar espacios al inicio/fin
    s = s.strip()

    # valores que tratamos como “sin dato”
    if s == "":
        return None

    s_upper = s.upper()
    if s_upper in {
        "NA", "N/A", "NO APLICA", "SIN DATO",
        "NO REGISTRA", "NO REGISTRA."
    }:
        return None

    # comprimir espacios internos múltiples a uno solo
    s = " ".join(s.split())

    return s

# 4. Aplicar limpieza → texto “normalizado” para trabajo
df["Dependencia_TXT_LIMPIO"] = df["Dependencia_RAW"].apply(limpiar_dependencia)

# 5. Valor que se migrará a SQL (respetando longitud de la columna)
#    Ajusta este valor si cambias la definición en SQL.
MAX_LEN_DEPENDENCIA_SQL = 500   # según la regla que venías usando

df["Dependencia_SQL"] = df["Dependencia_TXT_LIMPIO"]

# Longitud y detección de registros que exceden el máximo
df["Dependencia_SQL_longitud"] = df["Dependencia_SQL"].astype("string").str.len()
mask_larga = df["Dependencia_SQL"].notna() & (
    df["Dependencia_SQL_longitud"] > MAX_LEN_DEPENDENCIA_SQL
)

print("Total filas con Dependencia vacía (para migrar como NULL):",
      df["Dependencia_SQL"].isna().sum())
print("Filas con Dependencia_SQL que exceden",
      MAX_LEN_DEPENDENCIA_SQL, "caracteres:", mask_larga.sum())

# Truncar SOLO para la columna que se va a migrar
df.loc[mask_larga, "Dependencia_SQL"] = (
    df.loc[mask_larga, "Dependencia_SQL"]
      .str.slice(0, MAX_LEN_DEPENDENCIA_SQL)
)

# 6. Resumen rápido de valores (ya limpios)
print("\n=== Top 30 valores de Dependencia_TXT_LIMPIO ===")
print(df["Dependencia_TXT_LIMPIO"].value_counts(dropna=False).head(30))

# 7. DataFrame con las filas problemáticas (texto demasiado largo)
dependencia_larga = df.loc[
    mask_larga,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "Dependencia_RAW",
        "Dependencia_TXT_LIMPIO",
        "Dependencia_SQL",
        "Dependencia_SQL_longitud",
    ],
].copy()

print("\n=== Ejemplos de Dependencia que exceden longitud máxima ===")
print(dependencia_larga.head(20).to_string(index=False))

# 8. Excel con TODAS las filas (para inspección visual)
cols_mapeo_filas = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "Dependencia_RAW",
    "Dependencia_TXT_LIMPIO",
    "Dependencia_SQL",
    "Dependencia_SQL_longitud",
]

df[cols_mapeo_filas].to_excel(
    "Dependencia_mapeo_completo_filas.xlsx",
    index=False,
)

# 9. Excel solo con las filas que exceden la longitud (si hay)
dependencia_larga.to_excel(
    "Dependencia_dependencia_larga.xlsx",
    index=False,
)

print("\nArchivos creados:")
print(" - Dependencia_mapeo_completo_filas.xlsx")
print(" - Dependencia_dependencia_larga.xlsx")


Total filas con Dependencia vacía (para migrar como NULL): 23223
Filas con Dependencia_SQL que exceden 500 caracteres: 0

=== Top 30 valores de Dependencia_TXT_LIMPIO ===
Dependencia_TXT_LIMPIO
None                                                            23223
SPAGJ                                                             420
UGT                                                               119
DIRECCION DE ACCESO A TIERRAS -PROGRAMAS ESPECIALES                97
SSJ                                                                92
SUBDIRECCION DE ACCESO A TIERRAS POR ZONAS FOCALIZADAS             61
SUBDIRECCION DE SEGURIDAD JURIDICA                                 23
SUBDIRECCION DE ACCESO A TIERRAS POR DEMANDA Y DESCONGESTION        9
SPAGJ - UGT                                                         5
UGT CORDOBA                                                         2
UGT CAUCA                                                           2
UGT MAGDALENA                       

## ID ORFEO

In [14]:
import pandas as pd

# ==========================
# ID ORFEO  →  RadicadoOrfeo
# ==========================

# 0. Asegurar columna de fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2   # porque leíste con header=1

# 1. Guardar valor crudo de la columna del Excel
df["RadicadoOrfeo_RAW"] = df["ID ORFEO"]

# 2. Función de limpieza
def limpiar_radicado_orfeo(valor):
    if pd.isna(valor):
        return None

    s = str(valor)

    # NBSP → espacio normal
    s = s.replace("\u00A0", " ")

    # quitar espacios al inicio/fin
    s = s.strip()

    # textos que significan "sin dato"
    if s == "":
        return None

    s_upper = s.upper()
    if s_upper in {
        "NA", "N/A", "NO APLICA", "SIN DATO",
        "NO REGISTRA", "NO REGISTRA."
    }:
        return None

    # comprimir espacios internos
    s = " ".join(s.split())

    return s

# 3. Texto limpio
df["RadicadoOrfeo_TXT_LIMPIO"] = df["RadicadoOrfeo_RAW"].apply(limpiar_radicado_orfeo)

# 4. Valor final para migrar a SQL (varchar(50))
MAX_LEN_ORFEO_SQL = 50
df["RadicadoOrfeo_SQL"] = df["RadicadoOrfeo_TXT_LIMPIO"]

df["RadicadoOrfeo_SQL_longitud"] = (
    df["RadicadoOrfeo_SQL"].astype("string").str.len()
)

mask_largos = df["RadicadoOrfeo_SQL"].notna() & (
    df["RadicadoOrfeo_SQL_longitud"] > MAX_LEN_ORFEO_SQL
)

# 5. Duplicados (solo informativo, no se tocan)
mask_no_nulo = df["RadicadoOrfeo_SQL"].notna()
mask_dups = mask_no_nulo & df["RadicadoOrfeo_SQL"].duplicated(keep=False)

# 6. Resumen por pantalla
print("Total filas RadicadoOrfeo_SQL = NULL:",
      df["RadicadoOrfeo_SQL"].isna().sum())
print("Filas con RadicadoOrfeo_SQL > 50 caracteres:", mask_largos.sum())
print("Filas con RadicadoOrfeo_SQL duplicados:", mask_dups.sum())

print("\nDistribución de longitudes RadicadoOrfeo_SQL (top 10):")
print(df["RadicadoOrfeo_SQL_longitud"].value_counts().head(10))

print("\n=== Ejemplo de mapeo (primeras 10 filas) ===")
print(
    df.loc[:, [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "RadicadoOrfeo_RAW",
        "RadicadoOrfeo_TXT_LIMPIO",
        "RadicadoOrfeo_SQL",
        "RadicadoOrfeo_SQL_longitud",
    ]].head(10).to_string(index=False)
)

print("\n=== Ejemplos de RadicadoOrfeo largos (> 50) ===")
radicado_largo = df.loc[
    mask_largos,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "RadicadoOrfeo_RAW",
        "RadicadoOrfeo_TXT_LIMPIO",
        "RadicadoOrfeo_SQL",
        "RadicadoOrfeo_SQL_longitud",
    ],
].copy()
print(radicado_largo.head(20).to_string(index=False))

print("\n=== Ejemplos de RadicadoOrfeo duplicados ===")
radicado_dups = df.loc[
    mask_dups,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "RadicadoOrfeo_SQL",
    ],
].sort_values("RadicadoOrfeo_SQL")
print(radicado_dups.head(20).to_string(index=False))

# 7. (Opcional) truncar SOLO los largos en la columna que migra
df.loc[mask_largos, "RadicadoOrfeo_SQL"] = (
    df.loc[mask_largos, "RadicadoOrfeo_SQL"].str[:MAX_LEN_ORFEO_SQL]
)
df["RadicadoOrfeo_SQL_longitud"] = (
    df["RadicadoOrfeo_SQL"].astype("string").str.len()
)

# 8. Excels por FILA para revisión

# 8.1. Mapeo completo (todas las filas)
cols_mapeo = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "RadicadoOrfeo_RAW",
    "RadicadoOrfeo_TXT_LIMPIO",
    "RadicadoOrfeo_SQL",
    "RadicadoOrfeo_SQL_longitud",
]
df[cols_mapeo].to_excel(
    "RadicadoOrfeo_mapeo_completo_filas.xlsx",
    index=False,
)

# 8.2. Solo valores largos
radicado_largo.to_excel(
    "RadicadoOrfeo_largos.xlsx",
    index=False,
)

# 8.3. Solo duplicados (informativo)
radicado_dups.to_excel(
    "RadicadoOrfeo_duplicados.xlsx",
    index=False,
)

print("\nArchivos creados:")
print(" - RadicadoOrfeo_mapeo_completo_filas.xlsx")
print(" - RadicadoOrfeo_largos.xlsx")
print(" - RadicadoOrfeo_duplicados.xlsx")


Total filas RadicadoOrfeo_SQL = NULL: 23411
Filas con RadicadoOrfeo_SQL > 50 caracteres: 0
Filas con RadicadoOrfeo_SQL duplicados: 163

Distribución de longitudes RadicadoOrfeo_SQL (top 10):
RadicadoOrfeo_SQL_longitud
19    641
18      1
39      1
8       1
Name: count, dtype: Int64

=== Ejemplo de mapeo (primeras 10 filas) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO     MUNICIPIO RadicadoOrfeo_RAW RadicadoOrfeo_TXT_LIMPIO RadicadoOrfeo_SQL  RadicadoOrfeo_SQL_longitud
                2  062-37357      BOLIVAR      ZAMBRANO               NaN                     None              None                        <NA>
                3  062-37359      BOLIVAR      ZAMBRANO               NaN                     None              None                        <NA>
                4  062-37361      BOLIVAR      ZAMBRANO               NaN                     None              None                        <NA>
                5  062-37354      BOLIVAR      ZAMBRANO               NaN                 

## Localización

In [15]:
# DEPARTAMENTO / MUNICIPIO  → versiones limpias

df["DEPARTAMENTO_LIMPIO"] = df["DEPARTAMENTO"].apply(normalizar_texto)
df["MUNICIPIO_LIMPIO"]    = df["MUNICIPIO"].apply(normalizar_texto)

# Ver un resumen de cómo quedaron (opcional, solo para revisar)
loc_revision = (
    df
      .groupby(
          ["DEPARTAMENTO", "MUNICIPIO",
           "DEPARTAMENTO_LIMPIO", "MUNICIPIO_LIMPIO"],
          dropna=False
      )
      .size()
      .reset_index(name="conteo")
)

print("=== Resumen DEPARTAMENTO / MUNICIPIO ===")
print(loc_revision.to_string(index=False))


=== Resumen DEPARTAMENTO / MUNICIPIO ===
                                            DEPARTAMENTO                   MUNICIPIO                                      DEPARTAMENTO_LIMPIO            MUNICIPIO_LIMPIO  conteo
                                                AMAZONAS                     LETICIA                                                 AMAZONAS                     LETICIA       5
                                               ANTIOQUIA                   ABEJORRAL                                                ANTIOQUIA                   ABEJORRAL       9
                                               ANTIOQUIA                    ABRIAQUI                                                ANTIOQUIA                    ABRIAQUI       4
                                               ANTIOQUIA                       AMAGA                                                ANTIOQUIA                       AMAGA       2
                                               ANTIOQUIA             

## Conectarme a la base de datos

In [16]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
import pyodbc


In [17]:
server = r"172.23.92.97\MSSQLSERVER2022"
database = "ViabilidadJuridica"
user = "andres.garciav"
password = "4ndr3s.g4rc14v_3610@"

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={user};"
    f"PWD={password};"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


## Localización

*No hay ningun cordoba bolivar en la base de datos revisar ese caso.*

In [18]:
import pandas as pd
import unicodedata

# -------------------------------------------------------------------
# 0. Fila de Excel aproximada (si aún no existe)
# -------------------------------------------------------------------
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# -------------------------------------------------------------------
# 1. Función de normalización (misma idea que venimos usando)
# -------------------------------------------------------------------
def normalizar_texto(s):
    if pd.isna(s):
        return None
    s = str(s).replace("\u00A0", " ").strip().upper()

    if s in ("", "NA", "N/A", "NO APLICA", "SIN DATO"):
        return None

    # quitar tildes
    s = unicodedata.normalize("NFD", s)
    s = "".join(c for c in s if unicodedata.category(c) != "Mn")
    # comprimir espacios
    s = " ".join(s.split())
    return s

# -------------------------------------------------------------------
# 2. Correcciones puntuales en el Excel (como ya habías hecho)
# -------------------------------------------------------------------
# META / FUENTEDEORO  ->  META / FUENTE DE ORO
mask_meta_fuente = (
    (df["DEPARTAMENTO"] == "META") &
    (df["MUNICIPIO"] == "FUENTEDEORO")
)
df.loc[mask_meta_fuente, "MUNICIPIO"] = "FUENTE DE ORO"

# MAGDALENA / ARIGUIANI  ->  MAGDALENA / ARIGUANÍ
mask_mag_arig = (
    (df["DEPARTAMENTO"] == "MAGDALENA") &
    (df["MUNICIPIO"] == "ARIGUIANI")
)
df.loc[mask_mag_arig, "MUNICIPIO"] = "ARIGUANÍ"

# -------------------------------------------------------------------
# 3. Columnas RAW y LIMPIO en el DataFrame del Excel
# -------------------------------------------------------------------
df["DEPARTAMENTO_RAW"] = df["DEPARTAMENTO"]
df["MUNICIPIO_RAW"]    = df["MUNICIPIO"]

df["DEPARTAMENTO_LIMPIO"] = df["DEPARTAMENTO_RAW"].apply(normalizar_texto)
df["MUNICIPIO_LIMPIO"]    = df["MUNICIPIO_RAW"].apply(normalizar_texto)

df["LLAVE_LOC_TXT_LIMPIO"] = (
    df["DEPARTAMENTO_LIMPIO"] + " || " + df["MUNICIPIO_LIMPIO"]
)

# -------------------------------------------------------------------
# 4. Leer catálogo Localizacion desde SQL Server y normalizar
# -------------------------------------------------------------------
consulta_localizacion = """
SELECT
    IdLocalizacion,
    NombreDepartamento,
    NombreMunicipio
FROM ViabilidadJuridica.Postulacion.Localizacion;
"""

cat_loc = pd.read_sql(consulta_localizacion, con=engine)

cat_loc["DEP_LOC_LIMPIO"] = cat_loc["NombreDepartamento"].apply(normalizar_texto)
cat_loc["MUN_LOC_LIMPIO"] = cat_loc["NombreMunicipio"].apply(normalizar_texto)
cat_loc["LLAVE_LOC_CAT"]  = (
    cat_loc["DEP_LOC_LIMPIO"] + " || " + cat_loc["MUN_LOC_LIMPIO"]
)

# 4.1 Ver posibles duplicados en el catálogo
dup_loc = (
    cat_loc
      .groupby(["LLAVE_LOC_CAT"], dropna=False)["IdLocalizacion"]
      .nunique()
      .reset_index(name="num_IdLocalizacion")
)
print("=== Posibles duplicados en catálogo Localizacion (DEPTO+MUNI) ===")
print(dup_loc[dup_loc["num_IdLocalizacion"] > 1].to_string(index=False))

# -------------------------------------------------------------------
# 5. Diccionarios de mapeo: llave normalizada -> Id + nombre amigable
# -------------------------------------------------------------------
map_llave_to_id = dict(
    zip(cat_loc["LLAVE_LOC_CAT"], cat_loc["IdLocalizacion"])
)
map_llave_to_nombre = dict(
    zip(
        cat_loc["LLAVE_LOC_CAT"],
        cat_loc["NombreMunicipio"] + " (" + cat_loc["NombreDepartamento"] + ")"
    )
)

# -------------------------------------------------------------------
# 6. Columnas finales para migrar a SQL
# -------------------------------------------------------------------
df["IdLocalizacion_SQL"] = df["LLAVE_LOC_TXT_LIMPIO"].map(map_llave_to_id).astype("Int64")
df["Localizacion_NOMBRE_SQL"] = df["LLAVE_LOC_TXT_LIMPIO"].map(map_llave_to_nombre)

# -------------------------------------------------------------------
# 7. Resumen agrupado (como en FUENTE / ETAPA / TIPO PROCESO)
# -------------------------------------------------------------------
resumen_loc = (
    df.groupby(
        [
            "DEPARTAMENTO_RAW",
            "MUNICIPIO_RAW",
            "DEPARTAMENTO_LIMPIO",
            "MUNICIPIO_LIMPIO",
            "Localizacion_NOMBRE_SQL",
            "IdLocalizacion_SQL",
        ],
        dropna=False,
    )
    .size()
    .reset_index(name="conteo")
)

print("\n=== Resumen de mapeo Localización (agrupado) ===")
print(resumen_loc.to_string(index=False))

sin_loc = resumen_loc[
    resumen_loc["DEPARTAMENTO_LIMPIO"].notna()
    & resumen_loc["MUNICIPIO_LIMPIO"].notna()
    & resumen_loc["IdLocalizacion_SQL"].isna()
]

print("\n=== Combinaciones DEPARTAMENTO/MUNICIPIO sin IdLocalizacion_SQL ===")
print(sin_loc.to_string(index=False))
print("Total sin IdLocalizacion_SQL:", int(sin_loc["conteo"].sum()))

# -------------------------------------------------------------------
# 8. Filas con DEPARTAMENTO y MUNICIPIO totalmente vacíos (NaN/NaN)
# -------------------------------------------------------------------
mask_nan = df["DEPARTAMENTO"].isna() & df["MUNICIPIO"].isna()
filas_nan = df.loc[mask_nan, ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"]]

print("\n=== Filas con DEPARTAMENTO y MUNICIPIO en NaN ===")
print(filas_nan.to_string(index=False))
print("Índices filas NaN/NaN:", filas_nan.index.tolist())

# -------------------------------------------------------------------
# 9. Excels de apoyo (mismo estilo que venimos usando)
# -------------------------------------------------------------------
# 9.1 Mapeo agrupado
resumen_loc.to_excel("Localizacion_mapeo_completo.xlsx", index=False)

# 9.2 Solo combinaciones sin mapeo (pendientes por revisar)
sin_loc[[
    "DEPARTAMENTO_RAW",
    "MUNICIPIO_RAW",
    "DEPARTAMENTO_LIMPIO",
    "MUNICIPIO_LIMPIO",
    "conteo",
]].to_excel("pendientes_localizacion.xlsx", index=False)

# 9.3 Detalle fila a fila (crudo / limpio / SQL)
cols_detalle = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO_RAW",
    "DEPARTAMENTO_LIMPIO",
    "MUNICIPIO_RAW",
    "MUNICIPIO_LIMPIO",
    "Localizacion_NOMBRE_SQL",
    "IdLocalizacion_SQL",
]
df[cols_detalle].to_excel("Localizacion_raw_limpio_sql.xlsx", index=False)

print("\nArchivos creados:")
print(" - Localizacion_mapeo_completo.xlsx")
print(" - pendientes_localizacion.xlsx")
print(" - Localizacion_raw_limpio_sql.xlsx")


=== Posibles duplicados en catálogo Localizacion (DEPTO+MUNI) ===
Empty DataFrame
Columns: [LLAVE_LOC_CAT, num_IdLocalizacion]
Index: []

=== Resumen de mapeo Localización (agrupado) ===
                                        DEPARTAMENTO_RAW               MUNICIPIO_RAW                                      DEPARTAMENTO_LIMPIO            MUNICIPIO_LIMPIO                                                Localizacion_NOMBRE_SQL  IdLocalizacion_SQL  conteo
                                                AMAZONAS                     LETICIA                                                 AMAZONAS                     LETICIA                                                     LETICIA (AMAZONAS)                1090       5
                                               ANTIOQUIA                   ABEJORRAL                                                ANTIOQUIA                   ABEJORRAL                                                  ABEJORRAL (ANTIOQUIA)                   2       9
      

*CAUCA / SOTARA PAISPAMBA → conteo = 86 Significa que hay 86 registros en la matriz con DEPARTAMENTO = "CAUCA" y MUNICIPIO = "SOTARA PAISPAMBA"*

*que no encontraron coincidencia en la tabla Postulacion.Localizacion, por eso quedan con IdLocalizacion_SQL = <NA>.*

In [19]:
import pandas as pd

# 1. Asegurar columna con número aproximado de fila en Excel
#    (encabezado empieza en la fila 2)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 2. Filas que NO tienen IdLocalizacion_SQL (son tus 89 registros)
mask_sin_loc = df["IdLocalizacion_SQL"].isna()

# 3. Seleccionar columnas útiles para revisar
columnas_revision = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "DEPARTAMENTO_LIMPIO",
    "MUNICIPIO_LIMPIO",
    "IdLocalizacion_SQL",
]

df_sin_loc = df.loc[mask_sin_loc, columnas_revision].copy()

# 4. Ver un pequeño resumen por combinación (para confirmar los conteos)
resumen_sin_loc = (
    df_sin_loc
      .groupby(["DEPARTAMENTO", "MUNICIPIO",
                "DEPARTAMENTO_LIMPIO", "MUNICIPIO_LIMPIO"],
               dropna=False)
      .size()
      .reset_index(name="conteo")
)

print("=== Combinaciones sin IdLocalizacion_SQL ===")
print(resumen_sin_loc.to_string(index=False))

print("\nTotal filas sin IdLocalizacion_SQL:", len(df_sin_loc))

# 5. Exportar TODO el detalle a Excel para corrección manual
nombre_archivo = "Registros_sin_IdLocalizacion_SQL.xlsx"
df_sin_loc.to_excel(nombre_archivo, index=False)
print(f"\nArchivo creado: {nombre_archivo}")


=== Combinaciones sin IdLocalizacion_SQL ===
DEPARTAMENTO        MUNICIPIO DEPARTAMENTO_LIMPIO MUNICIPIO_LIMPIO  conteo
   ANTIOQUIA   PUERTO WILCHES           ANTIOQUIA   PUERTO WILCHES       1
       CAUCA SOTARA PAISPAMBA               CAUCA SOTARA PAISPAMBA      86
     CORDOBA          BOLIVAR             CORDOBA          BOLIVAR       1
  LA GUAJIRA         SAN JUAN          LA GUAJIRA         SAN JUAN       1
         NaN              NaN                 NaN              NaN      13

Total filas sin IdLocalizacion_SQL: 102

Archivo creado: Registros_sin_IdLocalizacion_SQL.xlsx


## AREA_REGISTRAL_ORIGINAL

In [20]:
import re
import numpy as np
import pandas as pd

# localizar la columna de área registral por nombre
col_area = [c for c in df.columns if "ÁREA REGISTRAL" in c.upper()][0]

# conservar una copia sin tocar
df["AREA_REGISTRAL_ORIGINAL"] = df[col_area]


In [21]:
def limpiar_area(valor):
    if pd.isna(valor):
        return pd.NA
    
    s = str(valor).strip()
    if s == "":
        return pd.NA

    # quitar espacios
    s = s.replace(" ", "")

    # quedarnos solo con la parte numérica (signo, dígitos, . y ,)
    m = re.match(r"^-?[\d\.,]+", s)
    if not m:
        return pd.NA
    s_num = m.group(0)

    # casos de separadores:
    # - si hay . y , → asumimos formato colombiano: . miles, , decimal
    if "," in s_num and "." in s_num:
        s_num = s_num.replace(".", "").replace(",", ".")
    # - solo coma → la tomamos como decimal
    elif "," in s_num:
        s_num = s_num.replace(",", ".")
    # - solo punto → lo usamos como decimal

    try:
        v = float(s_num)
    except ValueError:
        return pd.NA

    # dejarlo ya con 4 decimales
    return round(v, 4)


In [22]:
df["AreaRegistral_LIMPIA"] = df[col_area].apply(limpiar_area)
df["AreaRegistral_LIMPIA"] = pd.to_numeric(df["AreaRegistral_LIMPIA"], errors="coerce")

In [23]:
no_convertidos = df[
    df[col_area].notna() & df["AreaRegistral_LIMPIA"].isna()
][[col_area, "AREA_REGISTRAL_ORIGINAL"]]

print("Total filas no convertidas a número:", len(no_convertidos))
print(no_convertidos.head(20).to_string(index=False))


Total filas no convertidas a número: 4723
ÁREA REGISTRAL\n (0,0000 = Has,M2) AREA_REGISTRAL_ORIGINAL
                         402.2400.               402.2400.
                         325,5,000               325,5,000
                          46,3,000                46,3,000
                          51,2,000                51,2,000
                         27,2,590                27,2,590 
                        43,5092,50              43,5092,50
                         25,2200,0               25,2200,0
                           1,400,0                 1,400,0
                       1,496, 9521             1,496, 9521
                         89, 2,500               89, 2,500
                            250,0,                  250,0,
                        12 , 3,452              12 , 3,452
                          84,6,698                84,6,698
                          7,7500,0                7,7500,0
                        22,8109,65              22,8109,65
              

In [24]:
limite = 99999999999999.9999

fuera_rango = df[
    df["AreaRegistral_LIMPIA"].notna() &
    (df["AreaRegistral_LIMPIA"].abs() > limite)
][[col_area, "AREA_REGISTRAL_ORIGINAL", "AreaRegistral_LIMPIA"]]

print("Filas fuera de rango decimal(18,4):", len(fuera_rango))
print(fuera_rango.head(20).to_string(index=False))


Filas fuera de rango decimal(18,4): 0
Empty DataFrame
Columns: [ÁREA REGISTRAL
 (0,0000 = Has,M2), AREA_REGISTRAL_ORIGINAL, AreaRegistral_LIMPIA]
Index: []


In [25]:
negativos = df[
    df["AreaRegistral_LIMPIA"].notna() &
    (df["AreaRegistral_LIMPIA"] < 0)
][[col_area, "AREA_REGISTRAL_ORIGINAL", "AreaRegistral_LIMPIA"]]

print("Filas con área negativa:", len(negativos))
print(negativos.head(20).to_string(index=False))


Filas con área negativa: 0
Empty DataFrame
Columns: [ÁREA REGISTRAL
 (0,0000 = Has,M2), AREA_REGISTRAL_ORIGINAL, AreaRegistral_LIMPIA]
Index: []


In [26]:
import re
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 0. Asegurar columna de fila Excel aproximada
# ---------------------------------------------------------
if "Fila_Excel_aprox" not in df.columns:
    df.insert(0, "Fila_Excel_aprox", df.index + 2)  # header empieza en fila 2

# ---------------------------------------------------------
# 1. Identificar la columna de área registral en el Excel
#    (la que trae el valor original)
# ---------------------------------------------------------
cols_area = [c for c in df.columns if "ÁREA REGISTRAL" in c.upper()]
if not cols_area:
    raise ValueError("No se encontró ninguna columna que contenga 'ÁREA REGISTRAL' en el nombre.")
col_area = cols_area[0]

# Copia cruda
df["AreaRegistral_RAW"] = df[col_area]

# ---------------------------------------------------------
# 2. Función de limpieza: texto y número
#    Regla SQL: NUMERIC(18,4) → rango ±99999999999999.9999
# ---------------------------------------------------------
LIMITE_DECIMAL = 99999999999999.9999  # DECIMAL(18,4)

def limpiar_area_registral(valor):
    """
    Devuelve una tupla:
      (AreaRegistral_TXT_LIMPIO, AreaRegistral_SQL)

    - TXT_LIMPIO: texto numérico estandarizado (punto como separador decimal)
    - SQL: float redondeado a 4 decimales o NaN si no se puede convertir.
    """
    if pd.isna(valor):
        return (None, pd.NA)

    s_raw = str(valor).strip()
    if s_raw == "":
        return (None, pd.NA)

    # quitar espacios internos
    s = s_raw.replace(" ", "")

    # extraer solo la parte numérica inicial (signo, dígitos, . y ,)
    m = re.match(r"^-?[\d\.,]+", s)
    if not m:
        # no parece número → dejamos texto (crudo) y número NaN
        return (s_raw, pd.NA)

    s_num = m.group(0)

    # normalizar separadores:
    # - si hay punto y coma → asumimos . miles, , decimal → quitamos puntos, coma a punto
    if "," in s_num and "." in s_num:
        s_norm = s_num.replace(".", "").replace(",", ".")
    # - solo coma → la tomamos como decimal
    elif "," in s_num:
        s_norm = s_num.replace(",", ".")
    # - solo punto o solo dígitos
    else:
        s_norm = s_num

    try:
        v = float(s_norm)
    except ValueError:
        # no se pudo convertir a número
        return (s_norm, pd.NA)

    # redondear a 4 decimales
    v_red = round(v, 4)

    # si está fuera del rango de NUMERIC(18,4) lo dejamos como NaN para no migrarlo sin revisar
    if abs(v_red) > LIMITE_DECIMAL:
        return (s_norm, pd.NA)

    return (s_norm, v_red)

# ---------------------------------------------------------
# 3. Aplicar limpieza y crear columnas limpias / para SQL
# ---------------------------------------------------------
df[["AreaRegistral_TXT_LIMPIO", "AreaRegistral_SQL"]] = df["AreaRegistral_RAW"].apply(
    lambda v: pd.Series(limpiar_area_registral(v))
)

df["AreaRegistral_SQL"] = pd.to_numeric(df["AreaRegistral_SQL"], errors="coerce")

# ---------------------------------------------------------
# 4. Diagnósticos (válidos, inválidos, negativos)
# ---------------------------------------------------------
mask_con_texto = (
    df["AreaRegistral_RAW"].notna() &
    (df["AreaRegistral_RAW"].astype(str).str.strip() != "")
)

mask_validos = df["AreaRegistral_SQL"].notna()
mask_invalidos = mask_con_texto & df["AreaRegistral_SQL"].isna()
mask_negativos = mask_validos & (df["AreaRegistral_SQL"] < 0)

print("=== Resumen AREA REGISTRAL ===")
print("Total registros:", len(df))
print("Con dato en columna original:", mask_con_texto.sum())
print("Válidos (AreaRegistral_SQL no nulo):", mask_validos.sum())
print("Inválidos (NaN pero con texto no vacío):", mask_invalidos.sum())
print("Valores negativos (para revisar):", mask_negativos.sum())

# Pequeño ejemplo de cómo quedó el mapeo
cols_demo = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "AreaRegistral_RAW", "AreaRegistral_TXT_LIMPIO", "AreaRegistral_SQL"
]
ejemplo = df.loc[mask_con_texto, cols_demo].head(20)

print("\n=== Ejemplo de AREA REGISTRAL (RAW / LIMPIO / SQL) ===")
print(ejemplo.to_string(index=False))

# ---------------------------------------------------------
# 5. DataFrames para revisión manual
# ---------------------------------------------------------
# a) Registros con problema (texto pero sin número válido)
area_invalidos = df.loc[mask_invalidos, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "AreaRegistral_RAW", "AreaRegistral_TXT_LIMPIO"
]].copy()

# b) Registros con área negativa (numéricamente válida)
area_negativos = df.loc[mask_negativos, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "AreaRegistral_RAW", "AreaRegistral_TXT_LIMPIO", "AreaRegistral_SQL"
]].copy()

# ---------------------------------------------------------
# 6. Excel con mapeo completo + problemáticos
# ---------------------------------------------------------
with pd.ExcelWriter("AreaRegistral_revision.xlsx", engine="openpyxl") as writer:
    # Hoja 1: todos los registros (crudo, limpio y SQL)
    df.loc[:, cols_demo].to_excel(
        writer, sheet_name="Area_raw_limpio_sql", index=False
    )

    # Hoja 2: sólo registros inválidos (no convertidos / fuera de rango)
    area_invalidos.to_excel(
        writer, sheet_name="Area_invalidos", index=False
    )

    # Hoja 3: sólo registros negativos
    area_negativos.to_excel(
        writer, sheet_name="Area_negativos", index=False
    )

print("\nArchivos creados:")
print(" - AreaRegistral_revision.xlsx (3 hojas: mapeo completo, inválidos, negativos)")


=== Resumen AREA REGISTRAL ===
Total registros: 24055
Con dato en columna original: 19939
Válidos (AreaRegistral_SQL no nulo): 19104
Inválidos (NaN pero con texto no vacío): 835
Valores negativos (para revisar): 0

=== Ejemplo de AREA REGISTRAL (RAW / LIMPIO / SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO            MUNICIPIO AreaRegistral_RAW AreaRegistral_TXT_LIMPIO  AreaRegistral_SQL
                2  062-37357      BOLIVAR             ZAMBRANO            3.0034                   3.0034             3.0034
                3  062-37359      BOLIVAR             ZAMBRANO            8.1976                   8.1976             8.1976
                4  062-37361      BOLIVAR             ZAMBRANO             50.33                    50.33            50.3300
                5  062-37354      BOLIVAR             ZAMBRANO           400.504                  400.504           400.5040
                6  142-23909      CORDOBA               AYAPEL           2928825                  2928825

## CÍRCULO REGISTRAL

In [27]:
import pandas as pd

# -------------------------------------------------------------------
# 1. Localizar la columna original de CÍRCULO REGISTRAL en el Excel
# -------------------------------------------------------------------
cols_circ = [
    c for c in df.columns
    if "CÍRCULO REGISTRAL" in c.upper() or "CIRCULO REGISTRAL" in c.upper()
]
if not cols_circ:
    raise ValueError("No se encontró la columna de CÍRCULO REGISTRAL en df.")
col_circulo = cols_circ[0]

# Posición de la columna original para insertar las nuevas al lado
idx = df.columns.get_loc(col_circulo)

# -------------------------------------------------------------------
# 2. Crear columnas estándar: RAW / TXT_LIMPIO / SQL
#    (sin borrar la columna original del Excel)
# -------------------------------------------------------------------
# Copia cruda
if "CirculoRegistral_RAW" not in df.columns:
    df.insert(idx + 1, "CirculoRegistral_RAW", df[col_circulo])
else:
    df["CirculoRegistral_RAW"] = df[col_circulo]

# Inicializamos columnas de texto limpio y para migrar
if "CirculoRegistral_TXT_LIMPIO" not in df.columns:
    df.insert(idx + 2, "CirculoRegistral_TXT_LIMPIO", pd.NA)
if "CirculoRegistral_SQL" not in df.columns:
    df.insert(idx + 3, "CirculoRegistral_SQL", pd.NA)

# -------------------------------------------------------------------
# 3. Limpieza de texto (reutilizando normalizar_texto)
#    - Quita espacios extremos
#    - Aplica normalizar_texto (mayúsculas, sin tildes, etc.)
#    - Deja None cuando es “sin dato”
# -------------------------------------------------------------------
def limpiar_circulo(valor):
    if pd.isna(valor):
        return None
    s = str(valor).strip()
    if s == "":
        return None

    # Usa la misma función global que ya tienes definida
    s_norm = normalizar_texto(s)
    return s_norm

df["CirculoRegistral_TXT_LIMPIO"] = df["CirculoRegistral_RAW"].apply(limpiar_circulo)

# Valor definitivo para migrar a SQL (CirculoRegistral varchar(100) NULL)
df["CirculoRegistral_SQL"] = df["CirculoRegistral_TXT_LIMPIO"]

# -------------------------------------------------------------------
# 4. Validaciones: nulos, inválidos, longitud > 100
# -------------------------------------------------------------------
df["CirculoRegistral_LEN"] = (
    df["CirculoRegistral_SQL"].astype("string").str.len()
)

mask_con_dato = (
    df["CirculoRegistral_RAW"].astype("string").str.strip().ne("")
)
mask_validos = df["CirculoRegistral_SQL"].notna()
mask_invalidos = mask_con_dato & df["CirculoRegistral_SQL"].isna()

# Registros que se pasan de varchar(100)
mask_fuera_long = df["CirculoRegistral_SQL"].notna() & (
    df["CirculoRegistral_LEN"] > 100
)

print("=== RESUMEN CirculoRegistral ===")
print(f"Total registros: {len(df)}")
print(f"Con dato en columna original: {mask_con_dato.sum()}")
print(f"Válidos (CirculoRegistral_SQL no nulo): {mask_validos.sum()}")
print(f"Inválidos (NaN pero con texto no vacío): {mask_invalidos.sum()}")
print(f"Fuera de longitud (>100 caracteres): {mask_fuera_long.sum()}")

print("\n=== Top 30 valores distintos (CirculoRegistral_SQL) ===")
print(df["CirculoRegistral_SQL"].value_counts(dropna=False).head(30))

# -------------------------------------------------------------------
# 5. DataFrames de apoyo: inválidos y fuera de longitud
# -------------------------------------------------------------------
df_invalidos = df.loc[mask_invalidos, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "CirculoRegistral_RAW", "CirculoRegistral_TXT_LIMPIO"
]].copy()

df_fuera_long = df.loc[mask_fuera_long, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "CirculoRegistral_RAW", "CirculoRegistral_TXT_LIMPIO",
    "CirculoRegistral_SQL", "CirculoRegistral_LEN"
]].copy()

print("\n=== Ejemplo de CirculoRegistral (RAW / LIMPIO / SQL) ===")
print(
    df.loc[df["CirculoRegistral_RAW"].notna(), [
        "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
        "CirculoRegistral_RAW", "CirculoRegistral_TXT_LIMPIO",
        "CirculoRegistral_SQL"
    ]]
    .head(20)
    .to_string(index=False)
)

# -------------------------------------------------------------------
# 6. Exportar a Excel con TODO (crudo, limpio, SQL) y casos raros
# -------------------------------------------------------------------
with pd.ExcelWriter("CirculoRegistral_raw_limpio_sql.xlsx", engine="openpyxl") as writer:
    # Hoja principal: todos los registros
    df[[
        "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
        col_circulo,              # columna original tal cual está en el Excel
        "CirculoRegistral_RAW",
        "CirculoRegistral_TXT_LIMPIO",
        "CirculoRegistral_SQL",
        "CirculoRegistral_LEN",
    ]].to_excel(writer, index=False, sheet_name="Circulo_raw_limpio_sql")

    # Hoja de inválidos (si hay)
    df_invalidos.to_excel(writer, index=False, sheet_name="Circulo_invalidos")

    # Hoja de > 100 caracteres (para revisar por longitud)
    df_fuera_long.to_excel(writer, index=False, sheet_name="Circulo_fuera_long")


=== RESUMEN CirculoRegistral ===
Total registros: 24055
Con dato en columna original: 5461
Válidos (CirculoRegistral_SQL no nulo): 5461
Inválidos (NaN pero con texto no vacío): 0
Fuera de longitud (>100 caracteres): 0

=== Top 30 valores distintos (CirculoRegistral_SQL) ===
CirculoRegistral_SQL
None                    18594
VALLEDUPAR                292
EL CARMEN DE BOLIVAR      262
SABANALARGA               250
PLATO                     220
CHIMICHAGUA               212
MONTERIA                  197
CARTAGENA                 159
MONTELIBANO               153
SAN JUAN DEL CESAR        145
RIOHACHA                  144
BARRANCABERMEJA           140
SINCELEJO                 132
SAHAGUN                   106
CHAPARRAL                  94
MAGANGUE                   91
CIENAGA                    87
AGUACHICA                  87
COROZAL                    83
LORICA                     79
POPAYAN                    78
VELEZ                      76
SIMITI                     75
PATIA EL BORDO

# ÁREA CALCULADA


In [28]:
import pandas as pd
import numpy as np
import re

# -------------------------------------------------------------------
# 1. Localizar la columna de ÁREA CALCULADA en el DataFrame
# -------------------------------------------------------------------
cols_area_calc = [
    c for c in df.columns
    if "ÁREA CALCULADA" in c.upper() or "AREA CALCULADA" in c.upper()
]
if not cols_area_calc:
    raise ValueError("No se encontró la columna de ÁREA CALCULADA en df.")

col_area_calc = cols_area_calc[0]
idx = df.columns.get_loc(col_area_calc)

# -------------------------------------------------------------------
# 2. Crear columnas estándar RAW / TXT_LIMPIO / SQL junto a la original
# -------------------------------------------------------------------
# Copia cruda
if "AreaCalculada_RAW" not in df.columns:
    df.insert(idx + 1, "AreaCalculada_RAW", df[col_area_calc])
else:
    df["AreaCalculada_RAW"] = df[col_area_calc]

# Columnas de texto limpio y valor SQL
if "AreaCalculada_TXT_LIMPIO" not in df.columns:
    df.insert(idx + 2, "AreaCalculada_TXT_LIMPIO", pd.NA)
if "AreaCalculada_SQL" not in df.columns:
    df.insert(idx + 3, "AreaCalculada_SQL", np.nan)

# -------------------------------------------------------------------
# 3. Función de parseo: texto → (texto_normalizado, número con 4 decimales)
# -------------------------------------------------------------------
def parsear_area_calculada(valor):
    # Devolver (texto_limpio, numero_float) o (None, NaN) si no se puede
    if pd.isna(valor):
        return (None, np.nan)

    s = str(valor).strip()
    if s == "":
        return (None, np.nan)

    # quitar espacios internos
    s = s.replace(" ", "")

    # quedarnos solo con signo, dígitos, puntos y comas
    m = re.match(r"^-?[\d\.,]+", s)
    if not m:
        return (None, np.nan)
    s_num = m.group(0)

    # manejo de separadores:
    # - si hay punto y coma: decidir quién es decimal
    if "." in s_num and "," in s_num:
        # si la coma está después del último punto -> coma decimal
        if s_num.rfind(",") > s_num.rfind("."):
            # ejemplo: 1.234,56 -> 1234.56
            s_std = s_num.replace(".", "").replace(",", ".")
        else:
            # ejemplo: 1,234.56 -> 1234.56
            s_std = s_num.replace(",", "")
    elif "," in s_num and "." not in s_num:
        # solo coma -> usar coma como decimal
        s_std = s_num.replace(",", ".")
    else:
        # solo punto o solo dígitos
        s_std = s_num

    try:
        num = float(s_std)
    except ValueError:
        return (None, np.nan)

    # Redondear a 4 decimales para DECIMAL(18,4)
    num = round(num, 4)

    # Texto limpio que queremos ver (igual que para área registral)
    txt_limpio = s_std

    return (txt_limpio, num)

# Aplicar al DataFrame
df[["AreaCalculada_TXT_LIMPIO", "AreaCalculada_SQL"]] = df["AreaCalculada_RAW"].apply(
    lambda v: pd.Series(parsear_area_calculada(v))
)

# Asegurar tipo numérico en la columna SQL
df["AreaCalculada_SQL"] = pd.to_numeric(df["AreaCalculada_SQL"], errors="coerce")

# -------------------------------------------------------------------
# 4. Validaciones: nulos, inválidos, negativos, fuera de rango DECIMAL(18,4)
# -------------------------------------------------------------------
mask_con_dato = (
    df["AreaCalculada_RAW"].astype("string").str.strip().ne("")
)

mask_validos = df["AreaCalculada_SQL"].notna()
mask_invalidos = mask_con_dato & df["AreaCalculada_SQL"].isna()

# límite para numeric(18,4): ±99999999999999.9999
limite = 99999999999999.9999

mask_negativos = df["AreaCalculada_SQL"].notna() & (df["AreaCalculada_SQL"] < 0)
mask_fuera_rango = df["AreaCalculada_SQL"].notna() & (df["AreaCalculada_SQL"].abs() > limite)

print("=== RESUMEN Área Calculada ===")
print(f"Total registros: {len(df)}")
print(f"Con dato en columna original: {mask_con_dato.sum()}")
print(f"Válidos (AreaCalculada_SQL no nulo): {mask_validos.sum()}")
print(f"Inválidos (NaN pero con texto no vacío): {mask_invalidos.sum()}")
print(f"Valores negativos: {mask_negativos.sum()}")
print(f"Fuera de rango DECIMAL(18,4): {mask_fuera_rango.sum()}")

print("\n=== Top 20 valores distintos (AreaCalculada_SQL) ===")
print(df["AreaCalculada_SQL"].value_counts(dropna=False).head(20))

# -------------------------------------------------------------------
# 5. DataFrames de apoyo para revisión manual
# -------------------------------------------------------------------
df_invalidos = df.loc[mask_invalidos, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "AreaCalculada_RAW", "AreaCalculada_TXT_LIMPIO"
]].copy()

df_prob_valor = df.loc[mask_negativos | mask_fuera_rango, [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "AreaCalculada_RAW", "AreaCalculada_TXT_LIMPIO", "AreaCalculada_SQL"
]].copy()

print("\n=== Ejemplo de ÁREA CALCULADA (RAW / LIMPIO / SQL) ===")
print(
    df.loc[df["AreaCalculada_RAW"].notna(), [
        "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
        "AreaCalculada_RAW", "AreaCalculada_TXT_LIMPIO", "AreaCalculada_SQL"
    ]]
    .head(20)
    .to_string(index=False)
)

# -------------------------------------------------------------------
# 6. Exportar a Excel: mapeo completo + inválidos + negativos/fuera de rango
# -------------------------------------------------------------------
with pd.ExcelWriter("AreaCalculada_revision.xlsx", engine="openpyxl") as writer:
    # Hoja principal: todos los registros
    df[[
        "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
        col_area_calc,                 # columna original del Excel
        "AreaCalculada_RAW",
        "AreaCalculada_TXT_LIMPIO",
        "AreaCalculada_SQL",
    ]].to_excel(writer, index=False, sheet_name="AreaCalculada_mapeo_completo")

    # Hoja de inválidos
    df_invalidos.to_excel(writer, index=False, sheet_name="AreaCalculada_invalidos")

    # Hoja de negativos / fuera de rango
    df_prob_valor.to_excel(
        writer, index=False, sheet_name="AreaCalculada_negativos_fuera_rango"
    )


=== RESUMEN Área Calculada ===
Total registros: 24055
Con dato en columna original: 24040
Válidos (AreaCalculada_SQL no nulo): 24038
Inválidos (NaN pero con texto no vacío): 2
Valores negativos: 0
Fuera de rango DECIMAL(18,4): 0

=== Top 20 valores distintos (AreaCalculada_SQL) ===
AreaCalculada_SQL
0.0000      9391
NaN           17
152.8984      10
15.1638        6
0.0400         5
10.7528        5
38.7344        4
184.0189       4
3.4982         4
415.4777       4
128.1441       4
616.3764       3
859.1795       3
111.5784       3
0.0307         3
38.8000        3
561.3090       3
206.8845       3
635.1738       3
122.0500       3
Name: count, dtype: int64

=== Ejemplo de ÁREA CALCULADA (RAW / LIMPIO / SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO            MUNICIPIO AreaCalculada_RAW AreaCalculada_TXT_LIMPIO  AreaCalculada_SQL
                2  062-37357      BOLIVAR             ZAMBRANO          2.906059                 2.906059             2.9061
                3  062-3735

c:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Entorno_migracion\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


## "PERSONA TITULAR
(Natural / Jurídica)"


In [29]:
import pandas as pd
import numpy as np

# 1) Detectar automáticamente la columna de PERSONA TITULAR
candidatas = [c for c in df.columns 
              if "PERSONA" in c.upper() and "TITULAR" in c.upper()]

print("Columnas candidatas para PERSONA TITULAR:", candidatas)

# Tomamos la primera candidata (si hay varias, aquí podrías afinar el filtro)
col_persona = candidatas[0]

print("Usando la columna:", col_persona)

# 2) Normalizar texto de PERSONA TITULAR
df["PERSONA_TITULAR_LIMPIA"] = df[col_persona].apply(
    lambda x: normalizar_texto(x) if pd.notna(x) else None
)

# 3) Ver resumen de cómo vienen los valores originales vs limpios
titular_rev = (
    df
      .groupby(
          [col_persona, "PERSONA_TITULAR_LIMPIA"],
          dropna=False
      )
      .size()
      .reset_index(name="conteo")
)

print("=== Resumen PERSONA TITULAR ===")
print(titular_rev.to_string(index=False))

# 4) Función de mapeo a los valores finales para SQL
def mapear_tipo_persona(valor_limpio: str | None) -> str | None:
    if valor_limpio is None or valor_limpio == "":
        return None

    naturales = {
        "NATURAL",
        "PERSONA NATURAL",
        "P NATURAL",
        "P.NATURAL",
        "P NAT",
    }
    juridicas = {
        "JURIDICA",
        "JURÍDICA",
        "PERSONA JURIDICA",
        "PERSONA JURÍDICA",
        "P JURIDICA",
        "P.JURIDICA",
        "P JUR",
    }
    sin_dato = {
        "NA",
        "N/A",
        "NO APLICA",
        "NO REGISTRA",
        "SIN INFORMACION",
        "SIN INFORMACIÓN",
        "NINGUNO",
        "NINGUNA",
        "0",
    }

    if valor_limpio in naturales:
        return "Natural"
    if valor_limpio in juridicas:
        return "Jurídica"
    if valor_limpio in sin_dato:
        return None

    # Cualquier otro valor lo dejamos como NULL para revisión
    return None

# 5) Aplicar el mapeo
df["TipoPersonaTitular_SQL"] = df["PERSONA_TITULAR_LIMPIA"].apply(mapear_tipo_persona)

# 6) Ver qué quedó sin clasificar (para revisar y, si quieres, ampliar los conjuntos)
sin_clasificar = (
    df[
        (df["PERSONA_TITULAR_LIMPIA"].notna())
        & (df["TipoPersonaTitular_SQL"].isna())
    ][[col_persona, "PERSONA_TITULAR_LIMPIA"]]
    .drop_duplicates()
)

print("=== Valores PERSONA TITULAR sin clasificar ===")
print(sin_clasificar.to_string(index=False))
print(
    "Total filas sin TipoPersonaTitular_SQL:",
    len(df[(df["TipoPersonaTitular_SQL"].isna()) & (df["PERSONA_TITULAR_LIMPIA"].notna())])
)


Columnas candidatas para PERSONA TITULAR: ['PERSONA TITULAR\n(Natural / Jurídica)']
Usando la columna: PERSONA TITULAR
(Natural / Jurídica)
=== Resumen PERSONA TITULAR ===
PERSONA TITULAR\n(Natural / Jurídica) PERSONA_TITULAR_LIMPIA  conteo
                             JURÍDICA               JURIDICA      11
                             Jurídica               JURIDICA    1697
                              NATURAL                NATURAL     223
                              Natural                NATURAL    5993
                             jurídica               JURIDICA       1
                              natural                NATURAL       9
                                  NaN                    NaN   16121
=== Valores PERSONA TITULAR sin clasificar ===
Empty DataFrame
Columns: [PERSONA TITULAR
(Natural / Jurídica), PERSONA_TITULAR_LIMPIA]
Index: []
Total filas sin TipoPersonaTitular_SQL: 0


In [30]:
import pandas as pd

# ======================================================
# PERSONA TITULAR → TipoPersonaTitular_SQL (Natural / Jurídica / NULL)
# ======================================================

# 1) Detectar columna de PERSONA TITULAR
candidatas = [c for c in df.columns 
              if "PERSONA" in c.upper() and "TITULAR" in c.upper()]

print("Columnas candidatas para PERSONA TITULAR:", candidatas)
if not candidatas:
    raise ValueError("No se encontró columna para PERSONA TITULAR en el DataFrame.")

col_persona = candidatas[0]
print("Usando la columna:", col_persona)

# 2) Asegurar columna con número de fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2  # encabezado en fila 2

# 3) Crear columnas RAW y LIMPIA
df["PersonaTitular_RAW"] = df[col_persona]

df["PERSONA_TITULAR_TXT_LIMPIO"] = df[col_persona].apply(
    lambda x: normalizar_texto(x) if pd.notna(x) else None
)

# 4) Función de mapeo final a texto para SQL
def mapear_tipo_persona(valor_limpio):
    if valor_limpio is None or valor_limpio == "":
        return None

    naturales = {
        "NATURAL",
        "PERSONA NATURAL",
        "P NATURAL",
        "P.NATURAL",
        "P NAT",
    }
    juridicas = {
        "JURIDICA",
        "JURÍDICA",
        "PERSONA JURIDICA",
        "PERSONA JURÍDICA",
        "P JURIDICA",
        "P.JURIDICA",
        "P JUR",
    }
    sin_dato = {
        "NA",
        "N/A",
        "NO APLICA",
        "NO REGISTRA",
        "SIN INFORMACION",
        "SIN INFORMACIÓN",
        "NINGUNO",
        "NINGUNA",
        "0",
    }

    if valor_limpio in naturales:
        return "Natural"
    if valor_limpio in juridicas:
        return "Jurídica"
    if valor_limpio in sin_dato:
        return None

    # Cualquier otro valor lo dejamos en NULL para revisión
    return None

# 5) Columna final para migrar a SQL (EstudioTerreno.TipoPersonaTitular)
df["TipoPersonaTitular_SQL"] = df["PERSONA_TITULAR_TXT_LIMPIO"].apply(mapear_tipo_persona)

# 6) Resúmenes en consola
print("\n=== Distribución PERSONA_TITULAR_TXT_LIMPIO ===")
print(df["PERSONA_TITULAR_TXT_LIMPIO"].value_counts(dropna=False))

res_mapeo = (
    df
      .groupby(
          ["PersonaTitular_RAW", "PERSONA_TITULAR_TXT_LIMPIO", "TipoPersonaTitular_SQL"],
          dropna=False
      )
      .size()
      .reset_index(name="conteo")
)

print("\n=== Resumen de mapeo a TipoPersonaTitular_SQL ===")
print(res_mapeo.to_string(index=False))

# 7) Casos sin clasificar (texto limpio pero sin tipo asignado)
mask_sin_clasificar = (
    df["PERSONA_TITULAR_TXT_LIMPIO"].notna()
    & df["TipoPersonaTitular_SQL"].isna()
)

sin_clasificar_distintos = (
    df.loc[mask_sin_clasificar, ["PersonaTitular_RAW", "PERSONA_TITULAR_TXT_LIMPIO"]]
      .drop_duplicates()
      .reset_index(drop=True)
)

print("\n=== Valores PERSONA TITULAR sin clasificar (distintos) ===")
print(sin_clasificar_distintos.to_string(index=False))
print(
    "\nTotal filas sin TipoPersonaTitular_SQL (con texto limpio):",
    mask_sin_clasificar.sum()
)

# 8) Detalle de filas sin clasificar (con ubicación)
sin_clasificar_detalle = df.loc[
    mask_sin_clasificar,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "PersonaTitular_RAW",
        "PERSONA_TITULAR_TXT_LIMPIO",
    ],
].copy().sort_values("Fila_Excel_aprox")

print("\n=== Ejemplo de filas sin clasificar (primeras 20) ===")
print(sin_clasificar_detalle.head(20).to_string(index=False))

# 9) Reordenar columnas: dejar LIMPIO y SQL al lado de la original
cols = list(df.columns)
idx = cols.index(col_persona)  # columna original

# Eliminamos si ya están para no duplicar en el reordenamiento
for extra in ["PersonaTitular_RAW", "PERSONA_TITULAR_TXT_LIMPIO", "TipoPersonaTitular_SQL"]:
    if extra in cols:
        cols.remove(extra)

# Insertar en orden: RAW, LIMPIO, SQL
cols[idx+1:idx+1] = ["PersonaTitular_RAW", "PERSONA_TITULAR_TXT_LIMPIO", "TipoPersonaTitular_SQL"]

df = df[cols]

# 10) Excels de salida
# 10.1) Mapeo completo (agrupado)
res_mapeo.to_excel("PersonaTitular_mapeo_completo.xlsx", index=False)

# 10.2) Todas las filas con valores crudos / limpios / SQL
export_completo = df[[
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_persona,                 # original tal cual
    "PersonaTitular_RAW",
    "PERSONA_TITULAR_TXT_LIMPIO",
    "TipoPersonaTitular_SQL",
]].copy()

export_completo.to_excel("PersonaTitular_raw_limpio_sql.xlsx", index=False)

# 10.3) Solo filas sin clasificar (para revisión manual)
sin_clasificar_detalle.to_excel("PersonaTitular_sin_clasificar.xlsx", index=False)

print("\nArchivos creados:")
print(" - PersonaTitular_mapeo_completo.xlsx")
print(" - PersonaTitular_raw_limpio_sql.xlsx")
print(" - PersonaTitular_sin_clasificar.xlsx")


Columnas candidatas para PERSONA TITULAR: ['PERSONA TITULAR\n(Natural / Jurídica)', 'PERSONA_TITULAR_LIMPIA', 'TipoPersonaTitular_SQL']
Usando la columna: PERSONA TITULAR
(Natural / Jurídica)

=== Distribución PERSONA_TITULAR_TXT_LIMPIO ===
PERSONA_TITULAR_TXT_LIMPIO
None        16121
NATURAL      6225
JURIDICA     1709
Name: count, dtype: int64

=== Resumen de mapeo a TipoPersonaTitular_SQL ===
PersonaTitular_RAW PERSONA_TITULAR_TXT_LIMPIO TipoPersonaTitular_SQL  conteo
          JURÍDICA                   JURIDICA               Jurídica      11
          Jurídica                   JURIDICA               Jurídica    1697
           NATURAL                    NATURAL                Natural     223
           Natural                    NATURAL                Natural    5993
          jurídica                   JURIDICA               Jurídica       1
           natural                    NATURAL                Natural       9
               NaN                        NaN                 

## NOMBRE / APELLIDOS

In [31]:
import re
import numpy as np
import pandas as pd

# ============================
# NOMBRE / APELLIDOS
# ============================

# 1) Detectar columna de propietario
cols_prop_candidatas = [c for c in df.columns if "PROPIETARIO" in c.upper()]
print("Columnas candidatas para PROPIETARIO:", cols_prop_candidatas)

if not cols_prop_candidatas:
    raise ValueError("No se encontró columna que contenga 'PROPIETARIO' en el nombre.")

col_prop = cols_prop_candidatas[0]
print("Usando la columna de propietario:", col_prop)

# 2) Fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 3) Columna RAW (cruda)
df["NombrePropietario_RAW"] = df[col_prop]

# 4) Normalizar texto del nombre (similar a otras columnas)
SIN_DATO_NOMBRE = {
    "NA",
    "N/A",
    "NO APLICA",
    "SIN DATO",
    "SIN INFORMACION",
    "SIN INFORMACIÓN",
    "NO REGISTRA",
    "NINGUNO",
    "NINGUNA",
    "0",
}

def normalizar_nombre_propietario(x):
    if pd.isna(x):
        return None
    s = str(x).replace("\u00A0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    if s == "":
        return None

    s_norm = normalizar_texto(s)  # mayúsculas, sin tildes, etc.

    if s_norm in SIN_DATO_NOMBRE:
        return None

    return s_norm

df["NombrePropietario_TXT_LIMPIO"] = df["NombrePropietario_RAW"].apply(normalizar_nombre_propietario)

# 5) Número de palabras del nombre limpio
def contar_palabras(texto):
    if not isinstance(texto, str):
        return 0
    partes = [p for p in texto.strip().split(" ") if p]
    return len(partes)

df["NumPalabras_Nombre"] = df["NombrePropietario_TXT_LIMPIO"].apply(contar_palabras)

# 6) Función para partir según REGLA NUEVA
def partir_nombre_apellido(tipo_persona, nombre_limpio):
    """
    - Jurídica: todo en nombre, apellido NULL.
    - Natural / otros:
        0 palabras -> (None, None)
        1 palabra  -> (palabra, None)
        2 palabras -> (p1 p2, None)
        3 palabras -> (p1 p2, p3)
        4 palabras -> (p1 p2, p3 p4)
        >4 palabras -> dividir por mitad: ceil(n/2) para nombre, resto para apellido
    """
    if nombre_limpio is None:
        return (None, None)

    # Jurídica: todo en nombre
    if tipo_persona == "Jurídica":
        return (nombre_limpio, None)

    # Naturales / otros
    palabras = [p for p in nombre_limpio.split(" ") if p]
    n = len(palabras)

    if n == 0:
        return (None, None)
    elif n == 1:
        nombre = palabras[0]
        apellido = None
    elif n == 2:
        nombre = " ".join(palabras[:2])
        apellido = None
    elif n == 3:
        nombre = " ".join(palabras[:2])
        apellido = palabras[2]
    elif n == 4:
        nombre = " ".join(palabras[:2])
        apellido = " ".join(palabras[2:])
    else:
        # n > 4 -> dividir por mitad (redondeando hacia arriba para el nombre)
        mitad = (n + 1) // 2  # ceil(n/2)
        nombre = " ".join(palabras[:mitad])
        apellido = " ".join(palabras[mitad:])

    return (nombre, apellido)

# 7) Aplicar el particionado a todo el DataFrame
resultado_nombres = df.apply(
    lambda row: partir_nombre_apellido(
        row.get("TipoPersonaTitular_SQL"),
        row.get("NombrePropietario_TXT_LIMPIO"),
    ),
    axis=1,
    result_type="expand",
)

resultado_nombres.columns = ["NombrePropietario_SQL", "ApellidoPropietario_SQL"]
df[["NombrePropietario_SQL", "ApellidoPropietario_SQL"]] = resultado_nombres

# 8) Longitudes para validar contra VARCHAR(100)
df["NombrePropietario_LEN_SQL"] = df["NombrePropietario_SQL"].str.len()
df["ApellidoPropietario_LEN_SQL"] = df["ApellidoPropietario_SQL"].str.len()

mask_longitud = (
    (df["NombrePropietario_LEN_SQL"].fillna(0) > 100)
    | (df["ApellidoPropietario_LEN_SQL"].fillna(0) > 100)
)

exceden_longitud = df.loc[
    mask_longitud,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "TipoPersonaTitular_SQL",
        "NombrePropietario_RAW",
        "NombrePropietario_TXT_LIMPIO",
        "NumPalabras_Nombre",
        "NombrePropietario_SQL",
        "ApellidoPropietario_SQL",
        "NombrePropietario_LEN_SQL",
        "ApellidoPropietario_LEN_SQL",
    ],
].copy().sort_values("Fila_Excel_aprox")

# 9) Resúmenes por consola
print("\n=== Distribución NumPalabras_Nombre (solo NATURALES) ===")
print(
    df.loc[df["TipoPersonaTitular_SQL"] == "Natural", "NumPalabras_Nombre"]
      .value_counts()
      .sort_index()
)

print("\n=== Resumen por tipo de persona y número de palabras ===")
resumen_tipo_palabras = (
    df.groupby(["TipoPersonaTitular_SQL", "NumPalabras_Nombre"], dropna=False)
      .size()
      .reset_index(name="conteo")
      .sort_values(["TipoPersonaTitular_SQL", "NumPalabras_Nombre"])
)
print(resumen_tipo_palabras.to_string(index=False))

print("\nTotal registros con nombre o apellido > 100 caracteres:", exceden_longitud.shape[0])

print("\n=== Ejemplo de primeras filas (RAW / LIMPIO / SQL) ===")
ejemplo = df.loc[
    :19,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "TipoPersonaTitular_SQL",
        "NombrePropietario_RAW",
        "NombrePropietario_TXT_LIMPIO",
        "NumPalabras_Nombre",
        "NombrePropietario_SQL",
        "ApellidoPropietario_SQL",
    ],
]
print(ejemplo.to_string(index=False))

# 10) Reordenar columnas: dejar las nuevas junto a la columna original
nuevas_cols = [
    "NombrePropietario_RAW",
    "NombrePropietario_TXT_LIMPIO",
    "NumPalabras_Nombre",
    "NombrePropietario_SQL",
    "ApellidoPropietario_SQL",
    "NombrePropietario_LEN_SQL",
    "ApellidoPropietario_LEN_SQL",
]

cols = list(df.columns)
for nc in nuevas_cols:
    if nc in cols:
        cols.remove(nc)

idx = cols.index(col_prop)
cols[idx + 1 : idx + 1] = nuevas_cols
df = df[cols]

# 11) Excel con todos los registros
export_completo = df[
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "TipoPersonaTitular_SQL",
        col_prop,  # columna original
        "NombrePropietario_RAW",
        "NombrePropietario_TXT_LIMPIO",
        "NumPalabras_Nombre",
        "NombrePropietario_SQL",
        "ApellidoPropietario_SQL",
        "NombrePropietario_LEN_SQL",
        "ApellidoPropietario_LEN_SQL",
    ]
].copy()

export_completo.to_excel("Propietarios_raw_limpio_sql.xlsx", index=False)

# 12) Excel de casos que exceden longitud (para revisar)
if not exceden_longitud.empty:
    exceden_longitud.to_excel("Propietarios_longitud_superior_100.xlsx", index=False)

print("\nArchivos creados:")
print(" - Propietarios_raw_limpio_sql.xlsx (todos los registros)")
if not exceden_longitud.empty:
    print(" - Propietarios_longitud_superior_100.xlsx (casos donde nombre o apellido > 100)")


Columnas candidatas para PROPIETARIO: ['NOMBRE DEL (OS) PROPIETARIO (S)']
Usando la columna de propietario: NOMBRE DEL (OS) PROPIETARIO (S)

=== Distribución NumPalabras_Nombre (solo NATURALES) ===
NumPalabras_Nombre
0        45
1         7
2        66
3      1211
4      2729
       ... 
96        1
97        1
102       1
186       1
271       1
Name: count, Length: 78, dtype: int64

=== Resumen por tipo de persona y número de palabras ===
TipoPersonaTitular_SQL  NumPalabras_Nombre  conteo
              Jurídica                   0       2
              Jurídica                   1      10
              Jurídica                   2     239
              Jurídica                   3     403
              Jurídica                   4     342
              Jurídica                   5     212
              Jurídica                   6     141
              Jurídica                   7      97
              Jurídica                   8      83
              Jurídica                   9   

## "NUMERO DE IDENTIFICACIÓN (CC / NIT)"


In [32]:
import re
import numpy as np
import pandas as pd

# Nombre de la columna original de Excel
col_id = 'NUMERO DE IDENTIFICACIÓN \n(CC / NIT)'

def parse_num_doc_simple(valor):
    """
    Clasifica cada valor de identificación en:
    - 'ok'        -> una sola línea, un solo grupo de dígitos, con o sin prefijo.
    - 'multiple'  -> más de un grupo de dígitos o varias líneas.
    - 'vacio'     -> vacío o solo espacios.
    - 'sin_digitos' -> texto sin ningún número.
    - 'demasiado_largo' -> un solo grupo de dígitos pero excesivamente largo.
    Devuelve: (TipoDoc_TXT, Identificacion_LIMPIA, Estado_NumeroID)
    """
    # 1) Nulos / vacíos
    if pd.isna(valor):
        return (None, None, 'vacio')

    texto = str(valor).strip()
    if texto == '':
        return (None, None, 'vacio')

    t_up = texto.upper()

    # 2) Si tiene salto de línea, NO es un caso simple
    if '\n' in texto:
        return (None, None, 'multiple')

    # 3) Detectar prefijo de tipo de documento (solo referencia)
    tipo = None
    if 'NIT' in t_up:
        tipo = 'NIT'
    elif 'C.C' in t_up or ' CC' in t_up:
        tipo = 'CC'
    elif 'T.I' in t_up or ' TI' in t_up:
        tipo = 'TI'
    # (Si luego necesitas más tipos, se agregan aquí)

    # 4) Extraer todos los grupos de dígitos
    grupos = re.findall(r'\d+', texto)

    if len(grupos) == 0:
        # Hay texto pero sin ningún número
        return (tipo, None, 'sin_digitos')

    # 5) Si hay más de un grupo de dígitos, lo pasamos a otra categoría
    #    Ejemplos:
    #    - "NIT. 900240437 - 9"
    #    - "16917289 31575109"
    #    - "NIT. 900.432.349-3"
    if len(grupos) > 1:
        return (tipo, None, 'multiple')

    # 6) Un solo grupo de dígitos -> candidato "simple"
    num = grupos[0]

    # Control de longitud (ajusta si quieres otro límite)
    if len(num) > 15:
        return (tipo, None, 'demasiado_largo')

    # Este es el caso que SÍ vamos a trabajar primero
    # Ejemplos:
    #  - "CC 10539786"         -> tipo='CC',  num='10539786',   estado='ok'
    #  - "76302336"            -> tipo=None, num='76302336',    estado='ok'
    #  - "CC# 10692678"        -> tipo='CC',  num='10692678',   estado='ok'
    #  - "CC No 12124796"      -> tipo='CC',  num='12124796',   estado='ok'
    return (tipo, num, 'ok')


# === Aplicar la función a TODO el DataFrame ===
df[['TipoDoc_TXT', 'Identificacion_LIMPIA', 'Estado_NumeroID']] = (
    df[col_id].apply(lambda v: pd.Series(parse_num_doc_simple(v)))
)

# Resumen de estados
print("=== Conteo por Estado_NumeroID ===")
print(df['Estado_NumeroID'].value_counts(dropna=False))

# Ver algunos ejemplos de los casos 'ok' (los simples que sí vamos a usar ahora)
mask_ok = df['Estado_NumeroID'] == 'ok'

ejemplo_ok = df.loc[mask_ok,
                    ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
                     col_id, 'TipoDoc_TXT', 'Identificacion_LIMPIA', 'Estado_NumeroID']
                   ].head(50)

print("\n=== Ejemplos de filas 'ok' (una sola línea, un solo número, con/sin prefijo) ===")
print(ejemplo_ok.to_string(index=False))

# (OPCIONAL) Exportar a Excel solo los casos 'ok' para compararlos cómodamente
ruta_salida = r"NUM_IDENT_OK_vs_ORIGINAL.xlsx"
df.loc[mask_ok,
       ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
        col_id, 'TipoDoc_TXT', 'Identificacion_LIMPIA']
      ].to_excel(ruta_salida, index=False)

print(f"\nArchivo con casos 'ok' exportado a: {ruta_salida}")


=== Conteo por Estado_NumeroID ===
Estado_NumeroID
vacio          16281
ok              5249
multiple        2208
sin_digitos      317
Name: count, dtype: int64

=== Ejemplos de filas 'ok' (una sola línea, un solo número, con/sin prefijo) ===
 Fila_Excel_aprox        FMI       DEPARTAMENTO           MUNICIPIO NUMERO DE IDENTIFICACIÓN \n(CC / NIT) TipoDoc_TXT Identificacion_LIMPIA Estado_NumeroID
                2  062-37357            BOLIVAR            ZAMBRANO                            9005427093        None            9005427093              ok
                4  062-37361            BOLIVAR            ZAMBRANO                               5200583        None               5200583              ok
                5  062-37354            BOLIVAR            ZAMBRANO                              17095448        None              17095448              ok
                6  142-23909            CORDOBA              AYAPEL                              71367123        None              71

In [33]:
import pandas as pd

# Nombre de la columna original (ya la deberías tener definida así)
col_id = 'NUMERO DE IDENTIFICACIÓN \n(CC / NIT)'

# Filtrar los registros marcados como 'multiple'
mask_multiple = df['Estado_NumeroID'] == 'multiple'

print("Total registros con Estado_NumeroID = 'multiple':", mask_multiple.sum())

# Ver algunos ejemplos en pantalla (puedes subir el head si quieres ver más)
mult_ejemplo = df.loc[mask_multiple,
                      ['Fila_Excel_aprox',
                       'FMI',
                       'DEPARTAMENTO',
                       'MUNICIPIO',
                       col_id,
                       'TipoPersonaTitular_SQL',
                       'TipoDoc_TXT',
                       'Identificacion_LIMPIA',
                       'Estado_NumeroID']
                     ].head(100)

print("\n=== Ejemplos de registros 'multiple' ===")
print(mult_ejemplo.to_string(index=False))

# Exportar TODOS los registros 'multiple' a Excel para revisión detallada
ruta_salida_multiple = r"NUM_IDENT_multiple_pendientes.xlsx"

df.loc[mask_multiple,
       ['Fila_Excel_aprox',
        'FMI',
        'DEPARTAMENTO',
        'MUNICIPIO',
        col_id,
        'TipoPersonaTitular_SQL',
        'TipoDoc_TXT',
        'Identificacion_LIMPIA',
        'Estado_NumeroID']
      ].to_excel(ruta_salida_multiple, index=False)

print(f"\nArchivo con todos los registros 'multiple' exportado a: {ruta_salida_multiple}")


Total registros con Estado_NumeroID = 'multiple': 2208

=== Ejemplos de registros 'multiple' ===
 Fila_Excel_aprox        FMI       DEPARTAMENTO                   MUNICIPIO                                                                                                                 NUMERO DE IDENTIFICACIÓN \n(CC / NIT) TipoPersonaTitular_SQL TipoDoc_TXT Identificacion_LIMPIA Estado_NumeroID
                3  062-37359            BOLIVAR                    ZAMBRANO                                                                                                                                         900542709 – 3               Jurídica        None                  None        multiple
               13  148-16873            CORDOBA                PLANETA RICA                                                                                                                                           800204353-3               Jurídica        None                  None        multiple
      

In [34]:
import pandas as pd

col_id = 'NUMERO DE IDENTIFICACIÓN \n(CC / NIT)'

# Filtro de registros multiple
mask_multiple = df['Estado_NumeroID'] == 'multiple'

# Columnas que quieres ver en el Excel
cols_multiple = [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    col_id,
    'TipoPersonaTitular_SQL',
    'TipoDoc_TXT',
    'Identificacion_LIMPIA',
    'Estado_NumeroID'
]

# Ruta de salida (puedes cambiarla)
ruta_salida = r"NUM_IDENT_multiple_pendientes.xlsx"

# Exportar a Excel
df.loc[mask_multiple, cols_multiple].to_excel(ruta_salida, index=False)

print(f"Archivo generado: {ruta_salida}")


Archivo generado: NUM_IDENT_multiple_pendientes.xlsx


## ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN	


In [35]:
import pandas as pd
import numpy as np
import re

# 1) Localizar la columna de Excel
candidatas_natjur = [c for c in df.columns if 'NATURALEZA' in c.upper()]
print("Columnas candidatas para NATURALEZA JURÍDICA:", candidatas_natjur)

if not candidatas_natjur:
    raise ValueError("No se encontró columna que contenga 'NATURALEZA' en el encabezado.")

col_natjur = candidatas_natjur[0]
print("Usando columna de Excel:", col_natjur)

# 2) Columna de trabajo para SQL (sin cambiar el significado del texto)
def limpiar_natjur(v):
    if pd.isna(v):
        return np.nan
    t = str(v).strip()
    # compactar espacios
    t = re.sub(r'\s+', ' ', t)
    return t

df['NaturalezaJuridica_SQL'] = df[col_natjur].apply(limpiar_natjur)

# Versión normalizada solo para análisis (si ya tienes normalizar_texto definida)
try:
    df['NaturalezaJuridica_LIMPIA'] = df['NaturalezaJuridica_SQL'].apply(
        lambda x: normalizar_texto(x) if pd.notna(x) else None
    )
except NameError:
    print("Aviso: normalizar_texto no está definido en este notebook; se omite la columna _LIMPIA.")
    df['NaturalezaJuridica_LIMPIA'] = df['NaturalezaJuridica_SQL']

# 3) Revisar longitudes (tope 100 caracteres en SQL)
df['NaturalezaJuridica_len'] = df['NaturalezaJuridica_SQL'].fillna('').str.len()

print("=== Resumen de longitudes NaturalezaJuridica_SQL ===")
print(df['NaturalezaJuridica_len'].describe())

# Registros que exceden 100 caracteres → pendientes de decisión
pendientes_longitud = df[df['NaturalezaJuridica_len'] > 100][[
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    col_natjur,
    'NaturalezaJuridica_SQL',
    'NaturalezaJuridica_len'
]]

if not pendientes_longitud.empty:
    print("\n=== Registros que exceden los 100 caracteres (PENDIENTES de ajustar antes de migrar) ===")
    print(pendientes_longitud.to_string(index=False))
else:
    print("\nTodos los valores de NaturalezaJuridica_SQL caben en varchar(100).")

# 4) Resumen de valores distintos (para entender qué viene en la columna)
resumen_valores = (
    df
      .groupby('NaturalezaJuridica_LIMPIA', dropna=False)
      .size()
      .reset_index(name='conteo')
      .sort_values('conteo', ascending=False)
)

print("\n=== Principales valores de NaturalezaJuridica_LIMPIA (top 50) ===")
print(resumen_valores.head(50).to_string(index=False))


Columnas candidatas para NATURALEZA JURÍDICA: ['ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN\xa0']
Usando columna de Excel: ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN 
=== Resumen de longitudes NaturalezaJuridica_SQL ===
count    24055.000000
mean         3.195261
std          3.483393
min          0.000000
25%          0.000000
50%          0.000000
75%          7.000000
max         21.000000
Name: NaturalezaJuridica_len, dtype: float64

Todos los valores de NaturalezaJuridica_SQL caben en varchar(100).

=== Principales valores de NaturalezaJuridica_LIMPIA (top 50) ===
NaturalezaJuridica_LIMPIA  conteo
                      NaN   12930
                  PRIVADO    9176
                   BALDIO    1699
             SIN REGISTRO     164
                    MIXTO      82
    FORMULA TRANSACCIONAL       2
                PENDIENTE       1
                  PUBLICO       1


In [36]:
import pandas as pd
import numpy as np
import re

# ============================
# ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN
# -> NaturalezaJuridica (varchar(100) NULL)
# ============================

# 0) Fila aproximada (por si aún no existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Localizar la columna del Excel
candidatas_natjur = [c for c in df.columns if "NATURALEZA" in c.upper()]
print("Columnas candidatas para NATURALEZA JURÍDICA:", candidatas_natjur)

if not candidatas_natjur:
    raise ValueError("No se encontró columna que contenga 'NATURALEZA' en el encabezado.")

col_natjur = candidatas_natjur[0]
print("Usando columna de Excel:", col_natjur)

# 2) Columna RAW: lo que viene directo del Excel
df["NaturalezaJuridica_RAW"] = df[col_natjur]

# 3) Versión para SQL: texto limpio pero conservando forma (no se fuerzan mayúsculas)
def limpiar_natjur_sql(v):
    if pd.isna(v):
        return np.nan
    t = str(v)

    # quitar NBSP y similares
    t = t.replace("\u00A0", " ")

    # quitar espacios al inicio/fin
    t = t.strip()

    # compactar espacios internos
    t = re.sub(r"\s+", " ", t)

    # valores típicos de "sin dato"
    if t == "" or t.upper() in {"NA", "N/A", "NO APLICA", "SIN DATO"}:
        return np.nan

    return t

df["NaturalezaJuridica_SQL"] = df["NaturalezaJuridica_RAW"].apply(limpiar_natjur_sql)

# 4) Versión normalizada solo para análisis (mayúsculas, sin tildes, etc.)
#    usando la misma normalizar_texto que ya tienes definida
def normalizar_natjur_analisis(v):
    if pd.isna(v):
        return None
    return normalizar_texto(v)

df["NaturalezaJuridica_TXT_LIMPIO"] = df["NaturalezaJuridica_SQL"].apply(
    lambda x: normalizar_natjur_analisis(x) if pd.notna(x) else None
)

# 5) Longitud para validar contra varchar(100)
df["NaturalezaJuridica_LEN_SQL"] = df["NaturalezaJuridica_SQL"].fillna("").str.len()

print("=== Resumen de longitudes NaturalezaJuridica_SQL ===")
print(df["NaturalezaJuridica_LEN_SQL"].describe())

# Registros que exceden 100 caracteres → pendientes de decisión
pendientes_longitud = df[df["NaturalezaJuridica_LEN_SQL"] > 100][[
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_natjur,
    "NaturalezaJuridica_RAW",
    "NaturalezaJuridica_SQL",
    "NaturalezaJuridica_LEN_SQL",
]]

if not pendientes_longitud.empty:
    print("\n=== Registros que exceden los 100 caracteres (PENDIENTES de ajustar antes de migrar) ===")
    print(pendientes_longitud.to_string(index=False))
else:
    print("\nTodos los valores de NaturalezaJuridica_SQL caben en varchar(100).")

# 6) Resumen de valores distintos (para entender qué viene en la columna)
resumen_valores = (
    df.groupby("NaturalezaJuridica_TXT_LIMPIO", dropna=False)
      .size()
      .reset_index(name="conteo")
      .sort_values("conteo", ascending=False)
)

print("\n=== Principales valores de NaturalezaJuridica_TXT_LIMPIO (top 50) ===")
print(resumen_valores.head(50).to_string(index=False))

# 7) Reordenar columnas: dejar las nuevas junto a la columna original
nuevas_cols = [
    "NaturalezaJuridica_RAW",
    "NaturalezaJuridica_SQL",
    "NaturalezaJuridica_TXT_LIMPIO",
    "NaturalezaJuridica_LEN_SQL",
]

cols = list(df.columns)
for nc in nuevas_cols:
    if nc in cols:
        cols.remove(nc)

idx = cols.index(col_natjur)
cols[idx + 1 : idx + 1] = nuevas_cols
df = df[cols]

# 8) Excel con todos los registros (raw / limpio / sql)
export_completo = df[[
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_natjur,  # columna original del Excel
    "NaturalezaJuridica_RAW",
    "NaturalezaJuridica_SQL",
    "NaturalezaJuridica_TXT_LIMPIO",
    "NaturalezaJuridica_LEN_SQL",
]].copy()

nombre_archivo_completo = "NaturalezaJuridica_raw_limpio_sql.xlsx"
export_completo.to_excel(nombre_archivo_completo, index=False)

# 9) Excel solo con los que exceden 100 caracteres (si los hay)
if not pendientes_longitud.empty:
    nombre_archivo_largos = "NaturalezaJuridica_longitud_superior_100.xlsx"
    pendientes_longitud.to_excel(nombre_archivo_largos, index=False)
else:
    nombre_archivo_largos = None

print("\nArchivos creados:")
print(f" - {nombre_archivo_completo}")
if nombre_archivo_largos:
    print(f" - {nombre_archivo_largos} (casos con longitud > 100)")


Columnas candidatas para NATURALEZA JURÍDICA: ['ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN\xa0', 'NaturalezaJuridica_SQL', 'NaturalezaJuridica_LIMPIA', 'NaturalezaJuridica_len']
Usando columna de Excel: ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN 
=== Resumen de longitudes NaturalezaJuridica_SQL ===
count    24055.000000
mean         3.195261
std          3.483393
min          0.000000
25%          0.000000
50%          0.000000
75%          7.000000
max         21.000000
Name: NaturalezaJuridica_LEN_SQL, dtype: float64

Todos los valores de NaturalezaJuridica_SQL caben en varchar(100).

=== Principales valores de NaturalezaJuridica_TXT_LIMPIO (top 50) ===
NaturalezaJuridica_TXT_LIMPIO  conteo
                          NaN   12930
                      PRIVADO    9176
                       BALDIO    1699
                 SIN REGISTRO     164
                        MIXTO      82
        FORMULA TRANSACCIONAL       2
                    PENDIENTE       1
          

## Título originario

In [37]:
import numpy as np
import pandas as pd
import unicodedata

# --- 1. Localizar columna de "Título originario" / Acreditación ---

cols_candidatas = [
    c for c in df.columns
    if "TITULO" in c.upper() or "TÍTULO" in c.upper() or "ORIGINARIO" in c.upper()
]
print("Columnas candidatas para TÍTULO ORIGINARIO / ACREDITACIÓN:", cols_candidatas)

col_tit = cols_candidatas[0]
print("Usando columna:", col_tit)

# --- 2. Función de normalización (por si no está definida) ---

def normalizar_texto(s):
    if pd.isna(s):
        return None
    s = str(s).strip().upper()
    # quitar tildes
    s = "".join(
        ch for ch in unicodedata.normalize("NFD", s)
        if unicodedata.category(ch) != "Mn"
    )
    # colapsar espacios
    s = " ".join(s.split())
    return s

# --- 3. Columna normalizada base ---

df["AcreditacionPropiedad_LIMPIA"] = df[col_tit].apply(normalizar_texto)

# --- 4. Mapeo con correcciones de escritura ---

mapa_acreditacion = {
    # TÍTULO ORIGINARIO (todas las variantes que has visto)
    "TITULO ORIGINARIO": "TÍTULO ORIGINARIO",
    "TTITULO ORIGINARIO": "TÍTULO ORIGINARIO",

    # FÓRMULA TRANSACCIONAL y variantes mal escritas
    "FORMULA TRANSACCIONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRANSACCIONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRANSACCIOONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRANSACIONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRASNSACCIONAL": "FÓRMULA TRANSACCIONAL",

    # SIN REGISTRO y equivalentes claros
    "SIN REGISTRO": "SIN REGISTRO",
    "NO REGISTRA": "SIN REGISTRO",
}

def mapear_acreditacion(valor_limpio):
    """
    valor_limpio: texto ya normalizado (mayúsculas, sin tildes).
    Devuelve:
      - 'TÍTULO ORIGINARIO' / 'FÓRMULA TRANSACCIONAL' / 'SIN REGISTRO', o
      - np.nan si se deja vacío o para revisión.
    """
    if valor_limpio is None or pd.isna(valor_limpio):
        return np.nan

    # Donde el Excel tiene '0' lo dejamos como vacío
    if valor_limpio == "0":
        return np.nan

    # Mapeos que sí queremos corregir automáticamente
    if valor_limpio in mapa_acreditacion:
        return mapa_acreditacion[valor_limpio]

    # El resto (PENDIENTE, PRIVADO, SON REGISTRO, BALDIO, SI, etc.)
    # se deja como NaN para revisión manual
    return np.nan

# --- 5. Crear / sobrescribir columna AcreditacionPropiedad_SQL ---

df["AcreditacionPropiedad_SQL"] = df["AcreditacionPropiedad_LIMPIA"].apply(
    mapear_acreditacion
)

# --- 6. Resumen general para verificar ---

resumen = (
    df.groupby([col_tit, "AcreditacionPropiedad_LIMPIA", "AcreditacionPropiedad_SQL"],
               dropna=False)
      .size()
      .reset_index(name="conteo")
)

print("=== Resumen ACREDITACIÓN DE LA PROPIEDAD / TÍTULO ORIGINARIO ===")
print(resumen.to_string(index=False))

# --- 7. Registros NO clasificados (excluyendo los '0', que ya decidimos dejar vacíos) ---

mask_no_clas = (
    df["AcreditacionPropiedad_SQL"].isna()
    & df["AcreditacionPropiedad_LIMPIA"].notna()
    & (df["AcreditacionPropiedad_LIMPIA"] != "0")
)

pendientes = df.loc[
    mask_no_clas,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        col_tit,
        "AcreditacionPropiedad_LIMPIA",
    ],
]

print("\n=== Valores de ACREDITACIÓN no clasificados (revisar manualmente) ===")
print(pendientes.to_string(index=False))

# Además, listado compacto de los textos distintos que quedan pendientes
print("\n=== Textos distintos pendientes de clasificar ===")
print(df.loc[mask_no_clas, "AcreditacionPropiedad_LIMPIA"]
        .value_counts(dropna=False)
        .to_string())


Columnas candidatas para TÍTULO ORIGINARIO / ACREDITACIÓN: ['Título originario\xa0']
Usando columna: Título originario 
=== Resumen ACREDITACIÓN DE LA PROPIEDAD / TÍTULO ORIGINARIO ===
    Título originario  AcreditacionPropiedad_LIMPIA AcreditacionPropiedad_SQL  conteo
                     0                            0                       NaN      68
                BALDÍO                       BALDIO                       NaN       2
FORMULA  TRANSACCIONAL        FORMULA TRANSACCIONAL     FÓRMULA TRANSACCIONAL       1
 FORMULA TRANSACCIONAL        FORMULA TRANSACCIONAL     FÓRMULA TRANSACCIONAL    3374
FORMULA TRANSACCIONAL         FORMULA TRANSACCIONAL     FÓRMULA TRANSACCIONAL       1
FORMULA TRANSACCIOONAL       FORMULA TRANSACCIOONAL     FÓRMULA TRANSACCIONAL       3
  FORMULA TRANSACIONAL         FORMULA TRANSACIONAL     FÓRMULA TRANSACCIONAL       1
 Formula Transaccional        FORMULA TRANSACCIONAL     FÓRMULA TRANSACCIONAL       2
 Formula transaccional        FORMULA TRA

In [38]:
# 1) Ver los valores distintos (sin contar vacíos)
valores_distintos = df["AcreditacionPropiedad_SQL"].dropna().unique()
print("Valores distintos en AcreditacionPropiedad_SQL:")
for v in valores_distintos:
    print("-", v)

# 2) Ver cuántas veces aparece cada uno (incluyendo vacíos)
print("\nConteo por valor (incluye vacíos):")
print(df["AcreditacionPropiedad_SQL"].value_counts(dropna=False))

# 3) Si quieres ver algunos registros de ejemplo
cols_ver = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_tit, "AcreditacionPropiedad_LIMPIA", "AcreditacionPropiedad_SQL"
]
print("\nMuestra de registros con la acreditación ya clasificada:")
print(df[cols_ver].head(50).to_string(index=False))


Valores distintos en AcreditacionPropiedad_SQL:
- FÓRMULA TRANSACCIONAL
- TÍTULO ORIGINARIO
- SIN REGISTRO

Conteo por valor (incluye vacíos):
AcreditacionPropiedad_SQL
NaN                      15031
TÍTULO ORIGINARIO         5144
FÓRMULA TRANSACCIONAL     3387
SIN REGISTRO               493
Name: count, dtype: int64

Muestra de registros con la acreditación ya clasificada:
 Fila_Excel_aprox        FMI DEPARTAMENTO                   MUNICIPIO    Título originario  AcreditacionPropiedad_LIMPIA AcreditacionPropiedad_SQL
                2  062-37357      BOLIVAR                    ZAMBRANO FORMULA TRANSACCIONAL        FORMULA TRANSACCIONAL     FÓRMULA TRANSACCIONAL
                3  062-37359      BOLIVAR                    ZAMBRANO FORMULA TRANSACCIONAL        FORMULA TRANSACCIONAL     FÓRMULA TRANSACCIONAL
                4  062-37361      BOLIVAR                    ZAMBRANO FORMULA TRANSACCIONAL        FORMULA TRANSACCIONAL     FÓRMULA TRANSACCIONAL
                5  062-37354      B

In [39]:
import numpy as np
import pandas as pd

# ============================
# TÍTULO ORIGINARIO / ACREDITACIÓN DE LA PROPIEDAD
# -> AcreditacionPropiedad (varchar(100) NULL)
# ============================

if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Localizar columna del Excel
cols_candidatas = [
    c for c in df.columns
    if "TITULO" in c.upper() or "TÍTULO" in c.upper() or "ORIGINARIO" in c.upper()
]
print("Columnas candidatas para TÍTULO ORIGINARIO / ACREDITACIÓN:", cols_candidatas)

if not cols_candidatas:
    raise ValueError("No se encontró columna que contenga 'TITULO', 'TÍTULO' u 'ORIGINARIO' en el encabezado.")

col_tit = cols_candidatas[0]
print("Usando columna:", col_tit)

# 2) RAW
df["AcreditacionPropiedad_RAW"] = df[col_tit]

# 3) TXT_LIMPIO (normalizado)
def normalizar_acreditacion_txt(v):
    if pd.isna(v):
        return None
    return normalizar_texto(v)

df["AcreditacionPropiedad_TXT_LIMPIO"] = df["AcreditacionPropiedad_RAW"].apply(
    lambda x: normalizar_acreditacion_txt(x) if pd.notna(x) else None
)

# 4) Catálogo y sinónimos
CATALOGO_ACREDITACION = [
    "TÍTULO ORIGINARIO",
    "FÓRMULA TRANSACCIONAL",
    "SIN REGISTRO",
]

MAP_NORM_TO_NOMBRE = {
    normalizar_texto(nombre): nombre
    for nombre in CATALOGO_ACREDITACION
}

SINONIMOS = {
    "TITULO ORIGINARIO": "TÍTULO ORIGINARIO",
    "TTITULO ORIGINARIO": "TÍTULO ORIGINARIO",

    "FORMULA TRANSACCIONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRANSACCIOONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRANSACIONAL": "FÓRMULA TRANSACCIONAL",
    "FORMULA TRASNSACCIONAL": "FÓRMULA TRANSACCIONAL",

    "SIN REGISTRO": "SIN REGISTRO",
    "NO REGISTRA": "SIN REGISTRO",
}

for variante, canonico in SINONIMOS.items():
    clave_norm = normalizar_texto(variante)
    MAP_NORM_TO_NOMBRE[clave_norm] = canonico

# 5) Mapeo a SQL
def mapear_acreditacion_sql(valor_limpio_txt):
    if valor_limpio_txt is None or pd.isna(valor_limpio_txt):
        return np.nan
    if valor_limpio_txt == "0":
        return np.nan

    nombre_canonico = MAP_NORM_TO_NOMBRE.get(valor_limpio_txt)
    if nombre_canonico is not None:
        return nombre_canonico

    return np.nan  # pendiente de revisión

df["AcreditacionPropiedad_SQL"] = df["AcreditacionPropiedad_TXT_LIMPIO"].apply(
    mapear_acreditacion_sql
)

# 6) Longitud
df["AcreditacionPropiedad_LEN_SQL"] = df["AcreditacionPropiedad_SQL"].fillna("").str.len()

print("=== Resumen de longitudes AcreditacionPropiedad_SQL ===")
print(df["AcreditacionPropiedad_LEN_SQL"].describe())

pendientes_longitud = df[df["AcreditacionPropiedad_LEN_SQL"] > 100][[
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_tit,
    "AcreditacionPropiedad_RAW",
    "AcreditacionPropiedad_SQL",
    "AcreditacionPropiedad_LEN_SQL",
]]

if not pendientes_longitud.empty:
    print("\n=== Registros que exceden los 100 caracteres (PENDIENTES) ===")
    print(pendientes_longitud.to_string(index=False))
else:
    print("\nTodos los valores de AcreditacionPropiedad_SQL caben en varchar(100).")

# 7) Resumen de mapeo
resumen = (
    df
      .groupby(
          [col_tit, "AcreditacionPropiedad_TXT_LIMPIO", "AcreditacionPropiedad_SQL"],
          dropna=False
      )
      .size()
      .reset_index(name="conteo")
      .sort_values("conteo", ascending=False)
)

print("\n=== Resumen ACREDITACIÓN/TÍTULO ORIGINARIO (RAW / TXT_LIMPIO / SQL) ===")
print(resumen.to_string(index=False))

# 8) Pendientes de clasificar
mask_pendientes = (
    df["AcreditacionPropiedad_SQL"].isna()
    & df["AcreditacionPropiedad_TXT_LIMPIO"].notna()
    & (df["AcreditacionPropiedad_TXT_LIMPIO"] != "0")
)

pendientes = df.loc[
    mask_pendientes,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        col_tit,
        "AcreditacionPropiedad_RAW",
        "AcreditacionPropiedad_TXT_LIMPIO",
    ],
].copy()

print("\n=== Valores de ACREDITACIÓN no clasificados (revisar manualmente) ===")
if pendientes.empty:
    print("No hay valores pendientes de clasificar.")
else:
    print(pendientes.to_string(index=False))
    print("\n=== Textos distintos pendientes de clasificar ===")
    print(
        pendientes["AcreditacionPropiedad_TXT_LIMPIO"]
        .value_counts(dropna=False)
        .to_string()
    )

# 9) Reordenar columnas para dejar RAW/LIMPIO/SQL junto a la original
nuevas_cols = [
    "AcreditacionPropiedad_RAW",
    "AcreditacionPropiedad_TXT_LIMPIO",
    "AcreditacionPropiedad_SQL",
    "AcreditacionPropiedad_LEN_SQL",
]

cols = list(df.columns)
for nc in nuevas_cols:
    if nc in cols:
        cols.remove(nc)

idx = cols.index(col_tit)
cols[idx + 1 : idx + 1] = nuevas_cols
df = df[cols]

# 10) Excels

# 10.1) Completo (todos los registros)
export_completo = df[[
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_tit,
    "AcreditacionPropiedad_RAW",
    "AcreditacionPropiedad_TXT_LIMPIO",
    "AcreditacionPropiedad_SQL",
    "AcreditacionPropiedad_LEN_SQL",
]].copy()

archivo_completo = "AcreditacionPropiedad_raw_limpio_sql.xlsx"
export_completo.to_excel(archivo_completo, index=False)

# 10.2) Solo pendientes de clasificar (si los hay)
if not pendientes.empty:
    archivo_pendientes = "AcreditacionPropiedad_pendientes_clasificar.xlsx"
    pendientes.to_excel(archivo_pendientes, index=False)
else:
    archivo_pendientes = None

print("\nArchivos creados:")
print(f" - {archivo_completo}")
if archivo_pendientes:
    print(f" - {archivo_pendientes} (valores pendientes de clasificar)")
if not pendientes_longitud.empty:
    archivo_largos = "AcreditacionPropiedad_longitud_superior_100.xlsx"
    pendientes_longitud.to_excel(archivo_largos, index=False)
    print(f" - {archivo_largos} (longitud > 100 caracteres)")


Columnas candidatas para TÍTULO ORIGINARIO / ACREDITACIÓN: ['Título originario\xa0']
Usando columna: Título originario 
=== Resumen de longitudes AcreditacionPropiedad_SQL ===
count    24055.000000
mean         6.838121
std          8.949257
min          0.000000
25%          0.000000
50%          0.000000
75%         17.000000
max         21.000000
Name: AcreditacionPropiedad_LEN_SQL, dtype: float64

Todos los valores de AcreditacionPropiedad_SQL caben en varchar(100).

=== Resumen ACREDITACIÓN/TÍTULO ORIGINARIO (RAW / TXT_LIMPIO / SQL) ===
    Título originario  AcreditacionPropiedad_TXT_LIMPIO AcreditacionPropiedad_SQL  conteo
                   NaN                              NaN                       NaN   14952
     TÍTULO ORIGINARIO                TITULO ORIGINARIO         TÍTULO ORIGINARIO    5036
 FORMULA TRANSACCIONAL            FORMULA TRANSACCIONAL     FÓRMULA TRANSACCIONAL    3374
          SIN REGISTRO                     SIN REGISTRO              SIN REGISTRO     490
  

## Medida procesal

*Hipoteca*

In [40]:
import pandas as pd
import numpy as np

cols = list(df.columns)

# 1) Buscar todas las columnas que parezcan "Si / No" y "Anotación"
cols_si = [c for c in cols if 'SI' in str(c).upper() and 'NO' in str(c).upper()]
cols_anot = [c for c in cols if 'ANOTAC' in str(c).upper()]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

# Ordenar por posición para saber cuál es la PRIMERA de cada tipo
pos = {c: i for i, c in enumerate(cols)}

cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

col_hip_si = cols_si_ordenadas[0]      # primer 'Si / No'  → HIPOTECA
col_hip_anot = cols_anot_ordenadas[0]  # primera 'Anotación' → HIPOTECA

print("\nSe asume que HIPOTECA usa estas columnas:")
print("  HIPOTECA Si/No   :", col_hip_si)
print("  HIPOTECA Anotac. :", col_hip_anot)

# 2) Copiar a columnas más claras, solo para trabajar
df['HIPOTECA_SI_RAW'] = df[col_hip_si]
df['HIPOTECA_ANOT_RAW'] = df[col_hip_anot]

# 3) Ver distribución de valores y algunos ejemplos

print("\n=== Valores originales de HIPOTECA (Si/No) ===")
print(df['HIPOTECA_SI_RAW'].value_counts(dropna=False))

print("\n=== Ejemplos HIPOTECA (Si/No + Anotación) ===")
cols_mostrar = ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
                'HIPOTECA_SI_RAW', 'HIPOTECA_ANOT_RAW']
cols_mostrar = [c for c in cols_mostrar if c in df.columns]

print(df[cols_mostrar].head(30).to_string(index=False))


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6', 'ENTREGÓ CARPETA DE SOPORTES\n(SI / NO)']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

Se asume que HIPOTECA usa estas columnas:
  HIPOTECA Si/No   : Si / No
  HIPOTECA Anotac. : Anotación

=== Valores originales de HIPOTECA (Si/No) ===
HIPOTECA_SI_RAW
NaN          14909
NO            7296
SI            1810
no              25
si               8
Si               3
Si               2
No               1
PENDIENTE        1
Name: count, dtype: int64

=== Ejemplos HIPOTECA (Si/No + Anotación) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO            MUNICIPIO HIPOTECA_SI_RAW HIPOTECA_ANOT_RAW
                2  062-37357      BOLIVAR             ZAMBRANO              NO               NaN
                3  062-37359      BOLIVAR             ZAMBRANO              NO               NaN
          

In [41]:
import pandas as pd
import numpy as np

# Ya teníamos identificadas estas columnas:
#   col_hip_si   = 'Si / No'
#   col_hip_anot = 'Anotación'

col_hip_si   = 'Si / No'
col_hip_anot = 'Anotación'

# Copia a columnas de trabajo (por claridad)
df['HIPOTECA_SI_RAW']   = df[col_hip_si]
df['HIPOTECA_ANOT_RAW'] = df[col_hip_anot]

# --- 1) Normalizar SI/NO/PENDIENTE para HIPOTECA ---

def normalizar_hipoteca_valor(v):
    if pd.isna(v):
        return None  # lo dejamos como "sin diligenciar"
    t = str(v).strip().upper()
    if t in ('SI', 'SÍ'):
        return 'SI'
    if t == 'NO':
        return 'NO'
    if 'PEND' in t:           # PENDIENTE, PEND, etc.
        return 'PENDIENTE'
    # Cualquier cosa rara queda como None para revisar manualmente
    return None

df['HIPOTECA_VALOR_SQL'] = df['HIPOTECA_SI_RAW'].apply(normalizar_hipoteca_valor)

print("=== Conteo HIPOTECA_VALOR_SQL normalizado ===")
print(df['HIPOTECA_VALOR_SQL'].value_counts(dropna=False))


# --- 2) Limpiar anotación de HIPOTECA y revisar longitud ---

# Texto limpio (sin espacios de más)
df['HIPOTECA_ANOT_SQL'] = df['HIPOTECA_ANOT_RAW'].astype(str).str.strip()
df.loc[df['HIPOTECA_ANOT_RAW'].isna(), 'HIPOTECA_ANOT_SQL'] = None

# Longitud para comparar con varchar(4000)
df['HIPOTECA_ANOT_LEN'] = df['HIPOTECA_ANOT_SQL'].fillna('').str.len()

print("\n=== Resumen longitudes anotación HIPOTECA ===")
print(df['HIPOTECA_ANOT_LEN'].describe())

# Registros que excedan 4000 caracteres (no deberían, pero los listamos)
hipoteca_anot_largas = df[df['HIPOTECA_ANOT_LEN'] > 4000]

if not hipoteca_anot_largas.empty:
    print("\n*** OJO: anotaciones HIPOTECA que superan 4000 caracteres ***")
    cols_ver = ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
                'HIPOTECA_VALOR_SQL', 'HIPOTECA_ANOT_LEN', 'HIPOTECA_ANOT_SQL']
    cols_ver = [c for c in cols_ver if c in hipoteca_anot_largas.columns]
    print(hipoteca_anot_largas[cols_ver].to_string(index=False))
else:
    print("\nNo hay anotaciones de HIPOTECA que superen los 4000 caracteres (ok para varchar(4000)).")


# --- 3) Muestra rápida para revisar que todo tenga sentido ---

cols_muestra = ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
                col_hip_si, 'HIPOTECA_VALOR_SQL', col_hip_anot, 'HIPOTECA_ANOT_SQL']
cols_muestra = [c for c in cols_muestra if c in df.columns]

print("\n=== Muestra de HIPOTECA normalizada ===")
print(df[cols_muestra].head(40).to_string(index=False))


=== Conteo HIPOTECA_VALOR_SQL normalizado ===
HIPOTECA_VALOR_SQL
None         14909
NO            7322
SI            1823
PENDIENTE        1
Name: count, dtype: int64

=== Resumen longitudes anotación HIPOTECA ===
count    24055.000000
mean         0.393515
std          9.214527
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        867.000000
Name: HIPOTECA_ANOT_LEN, dtype: float64

No hay anotaciones de HIPOTECA que superen los 4000 caracteres (ok para varchar(4000)).

=== Muestra de HIPOTECA normalizada ===
 Fila_Excel_aprox        FMI DEPARTAMENTO            MUNICIPIO Si / No HIPOTECA_VALOR_SQL Anotación HIPOTECA_ANOT_SQL
                2  062-37357      BOLIVAR             ZAMBRANO      NO                 NO       NaN              None
                3  062-37359      BOLIVAR             ZAMBRANO      NO                 NO       NaN              None
                4  062-37361      BOLIVAR             ZAMBRANO      NO                

In [42]:
import pandas as pd
import numpy as np

# ================================
# MEDIDA PROCESAL - HIPOTECA
# ================================

# 0) Fila aproximada de Excel, por si aún no existe
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)

# 1) Detectar columnas de HIPOTECA (Si/No y Anotación)
cols_si = [c for c in cols if "SI" in str(c).upper() and "NO" in str(c).upper()]
cols_anot = [c for c in cols if "ANOTAC" in str(c).upper()]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

# Ordenar por posición para tomar la PRIMERA de cada tipo (HIPOTECA)
pos = {c: i for i, c in enumerate(cols)}
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

col_hip_si   = cols_si_ordenadas[0]
col_hip_anot = cols_anot_ordenadas[0]

print("\nSe asume que HIPOTECA usa estas columnas:")
print("  HIPOTECA Si/No   :", col_hip_si)
print("  HIPOTECA Anotac. :", col_hip_anot)

# 2) Copias RAW para trabajar
df["HIPOTECA_SI_RAW"]   = df[col_hip_si]
df["HIPOTECA_ANOT_RAW"] = df[col_hip_anot]

# 3) Normalizar valor SI/NO/PENDIENTE → texto limpio + valor SQL
def normalizar_hipoteca_valor(v):
    if pd.isna(v):
        return None  # sin diligenciar
    t = str(v).strip().upper()
    # Por si viene con tilde
    t = t.replace("Í", "I")

    if t in ("SI", "SÍ"):
        return "SI"
    if t == "NO":
        return "NO"
    if "PEND" in t:   # PENDIENTE, PEND, etc.
        return "PENDIENTE"

    # Cualquier cosa rara queda como None para revisar manualmente
    return None

df["HIPOTECA_VALOR_TXT_LIMPIO"] = df["HIPOTECA_SI_RAW"].apply(normalizar_hipoteca_valor)
# Para SQL, usamos directamente el mismo valor limpio (varchar(10))
df["HIPOTECA_VALOR_SQL"] = df["HIPOTECA_VALOR_TXT_LIMPIO"]

print("\n=== Conteo HIPOTECA_VALOR_TXT_LIMPIO ===")
print(df["HIPOTECA_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

# Valores raros: hay algo en la columna original, pero no se pudo mapear a SI/NO/PENDIENTE
mask_valor_raro = df["HIPOTECA_SI_RAW"].notna() & df["HIPOTECA_VALOR_TXT_LIMPIO"].isna()

hipoteca_valores_raros = df.loc[
    mask_valor_raro,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "HIPOTECA_SI_RAW",
    ],
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal filas HIPOTECA con valor Si/No raro (sin mapeo):", len(hipoteca_valores_raros))
if not hipoteca_valores_raros.empty:
    print("=== Ejemplos de valores HIPOTECA_SI_RAW sin mapeo ===")
    print(hipoteca_valores_raros.head(30).to_string(index=False))

# 4) Limpiar ANOTACIÓN HIPOTECA y validar longitud (varchar(4000))

def limpiar_anotacion_hipoteca(v):
    if pd.isna(v):
        return None
    txt = str(v)
    # quitar saltos de línea y tabs, compactar espacios
    txt = txt.replace("\r", " ").replace("\n", " ")
    txt = " ".join(txt.split())
    txt = txt.strip()
    return txt if txt != "" else None

df["HIPOTECA_ANOT_SQL"] = df["HIPOTECA_ANOT_RAW"].apply(limpiar_anotacion_hipoteca)
df["HIPOTECA_ANOT_LEN"] = df["HIPOTECA_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes HIPOTECA_ANOT_SQL ===")
print(df["HIPOTECA_ANOT_LEN"].describe())

mask_anot_larga = df["HIPOTECA_ANOT_LEN"] > 4000
hipoteca_anot_largas = df.loc[
    mask_anot_larga,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "HIPOTECA_VALOR_SQL",
        "HIPOTECA_ANOT_LEN",
        "HIPOTECA_ANOT_SQL",
    ],
].copy()

print("\nTotal filas con HIPOTECA_ANOT_SQL > 4000 caracteres:", len(hipoteca_anot_largas))
if not hipoteca_anot_largas.empty:
    print("=== Ejemplos de anotaciones HIPOTECA demasiado largas ===")
    print(hipoteca_anot_largas.head(20).to_string(index=False))

# 5) Reordenar columnas nuevas al lado de las originales
cols_all = list(df.columns)

for nc in [
    "HIPOTECA_SI_RAW",
    "HIPOTECA_VALOR_TXT_LIMPIO",
    "HIPOTECA_VALOR_SQL",
    "HIPOTECA_ANOT_RAW",
    "HIPOTECA_ANOT_SQL",
    "HIPOTECA_ANOT_LEN",
]:
    if nc in cols_all:
        cols_all.remove(nc)

# Después de la columna Si/No
idx_si = cols_all.index(col_hip_si)
cols_all[idx_si + 1 : idx_si + 1] = [
    "HIPOTECA_SI_RAW",
    "HIPOTECA_VALOR_TXT_LIMPIO",
    "HIPOTECA_VALOR_SQL",
]

# Después de la columna de Anotación
idx_anot = cols_all.index(col_hip_anot)
cols_all[idx_anot + 1 : idx_anot + 1] = [
    "HIPOTECA_ANOT_RAW",
    "HIPOTECA_ANOT_SQL",
    "HIPOTECA_ANOT_LEN",
]

df = df[cols_all]

# 6) Excel con TODOS los registros (RAW / LIMPIO / SQL)
cols_export = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_hip_si,
    "HIPOTECA_SI_RAW",
    "HIPOTECA_VALOR_TXT_LIMPIO",
    "HIPOTECA_VALOR_SQL",
    col_hip_anot,
    "HIPOTECA_ANOT_RAW",
    "HIPOTECA_ANOT_SQL",
    "HIPOTECA_ANOT_LEN",
]
cols_export = [c for c in cols_export if c in df.columns]

df_hipoteca_all = df[cols_export].copy()

archivo_completo = "MedidaProcesal_Hipoteca_raw_limpio_sql.xlsx"
df_hipoteca_all.to_excel(archivo_completo, index=False)

# 7) Excels adicionales para revisión (si aplica)
archivo_valores_raros = None
archivo_anot_largas = None

if not hipoteca_valores_raros.empty:
    archivo_valores_raros = "MedidaProcesal_Hipoteca_valores_sin_mapeo.xlsx"
    hipoteca_valores_raros.to_excel(archivo_valores_raros, index=False)

if not hipoteca_anot_largas.empty:
    archivo_anot_largas = "MedidaProcesal_Hipoteca_anotaciones_largas.xlsx"
    hipoteca_anot_largas.to_excel(archivo_anot_largas, index=False)

print("\nArchivos creados:")
print(f" - {archivo_completo}")
if archivo_valores_raros:
    print(f" - {archivo_valores_raros} (valores Si/No sin mapeo)")
if archivo_anot_largas:
    print(f" - {archivo_anot_largas} (anotaciones > 4000 caracteres)")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6', 'ENTREGÓ CARPETA DE SOPORTES\n(SI / NO)']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

Se asume que HIPOTECA usa estas columnas:
  HIPOTECA Si/No   : Si / No
  HIPOTECA Anotac. : Anotación

=== Conteo HIPOTECA_VALOR_TXT_LIMPIO ===
HIPOTECA_VALOR_TXT_LIMPIO
None         14909
NO            7322
SI            1823
PENDIENTE        1
Name: count, dtype: int64

Total filas HIPOTECA con valor Si/No raro (sin mapeo): 0

=== Resumen longitudes HIPOTECA_ANOT_SQL ===
count    24055.000000
mean         0.393182
std          9.209103
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        866.000000
Name: HIPOTECA_ANOT_LEN, dtype: float64

Total filas con HIPOTECA_ANOT_SQL > 4000 caracteres: 0

Archivos creados:
 - MedidaProcesal_Hipoteca_raw_limpio_sql.xl

## SERVIDUMBRES	


In [43]:
import pandas as pd
import numpy as np

# ======================================
# MEDIDA PROCESAL - SERVIDUMBRES
# ======================================

# 0) Fila aproximada de Excel (por si aún no existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)

# 1) Localizar columnas de SERVIDUMBRES (Si/No + Anotación)
cols_si = [c for c in cols if "SI" in str(c).upper() and "NO" in str(c).upper()]
cols_anot = [c for c in cols if "ANOTAC" in str(c).upper()]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

# Ordenar por posición (en el Excel las columnas van en orden)
pos = {c: i for i, c in enumerate(cols)}
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# SERVIDUMBRES = segunda pareja Si/No + Anotación
if "Si / No.1" in df.columns:
    col_serv_si = "Si / No.1"
else:
    if len(cols_si_ordenadas) < 2:
        raise ValueError("No se encontró una segunda columna 'Si / No' para SERVIDUMBRES.")
    col_serv_si = cols_si_ordenadas[1]

if "Anotación.1" in df.columns:
    col_serv_anot = "Anotación.1"
else:
    if len(cols_anot_ordenadas) < 2:
        raise ValueError("No se encontró una segunda columna 'Anotación' para SERVIDUMBRES.")
    col_serv_anot = cols_anot_ordenadas[1]

print("\nSERVIDUMBRES Si/No   :", col_serv_si)
print("SERVIDUMBRES Anotac. :", col_serv_anot)

# 2) Copias RAW de trabajo
df["SERVIDUMBRES_SI_RAW"]   = df[col_serv_si]
df["SERVIDUMBRES_ANOT_RAW"] = df[col_serv_anot]

# 3) Normalizar valor SI / NO / PENDIENTE

def normalizar_servidumbre_valor(v):
    if pd.isna(v):
        return None
    txt = str(v).strip().upper()

    # Quitar tildes comunes
    txt = (txt
           .replace("Á", "A")
           .replace("É", "E")
           .replace("Í", "I")
           .replace("Ó", "O")
           .replace("Ú", "U"))

    if txt in {"SI", "S", "SI.", "SI "}:
        return "SI"
    if txt in {"NO", "N", "NO.", "NO "}:
        return "NO"
    if "PEND" in txt:      # PENDIENTE, PEND, etc.
        return "PENDIENTE"

    # Cualquier cosa rara se deja como None para revisar
    return None

df["SERVIDUMBRES_VALOR_TXT_LIMPIO"] = df["SERVIDUMBRES_SI_RAW"].apply(normalizar_servidumbre_valor)
df["SERVIDUMBRES_VALOR_SQL"] = df["SERVIDUMBRES_VALOR_TXT_LIMPIO"]  # va a MedidaProcesal.Valor (varchar(10))

print("\n=== Conteo SERVIDUMBRES_VALOR_TXT_LIMPIO ===")
print(df["SERVIDUMBRES_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

# Valores raros: hay algo en la columna original, pero no se pudo mapear
mask_valor_raro = df["SERVIDUMBRES_SI_RAW"].notna() & df["SERVIDUMBRES_VALOR_TXT_LIMPIO"].isna()

servid_valores_raros = df.loc[
    mask_valor_raro,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "SERVIDUMBRES_SI_RAW",
    ],
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal filas SERVIDUMBRES con valor Si/No raro (sin mapeo):", len(servid_valores_raros))
if not servid_valores_raros.empty:
    print("=== Ejemplos de valores SERVIDUMBRES_SI_RAW sin mapeo ===")
    print(servid_valores_raros.head(30).to_string(index=False))

# 4) Limpiar anotación SERVIDUMBRES y revisar longitud (varchar(4000))

def limpiar_anotacion_servidumbre(v):
    if pd.isna(v):
        return None
    txt = str(v)
    # quitar saltos de línea y tabs, compactar espacios
    txt = txt.replace("\r", " ").replace("\n", " ")
    txt = " ".join(txt.split())
    txt = txt.strip()
    return txt if txt != "" else None

df["SERVIDUMBRES_ANOT_SQL"] = df["SERVIDUMBRES_ANOT_RAW"].apply(limpiar_anotacion_servidumbre)
df["SERVIDUMBRES_ANOT_LEN"] = df["SERVIDUMBRES_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes SERVIDUMBRES_ANOT_SQL ===")
print(df["SERVIDUMBRES_ANOT_LEN"].describe())

mask_anot_larga = df["SERVIDUMBRES_ANOT_LEN"] > 4000
servid_anot_largas = df.loc[
    mask_anot_larga,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "SERVIDUMBRES_VALOR_SQL",
        "SERVIDUMBRES_ANOT_LEN",
        "SERVIDUMBRES_ANOT_SQL",
    ],
].copy()

print("\nTotal filas con SERVIDUMBRES_ANOT_SQL > 4000 caracteres:", len(servid_anot_largas))
if not servid_anot_largas.empty:
    print("=== Ejemplos de anotaciones SERVIDUMBRES demasiado largas ===")
    print(servid_anot_largas.head(20).to_string(index=False))

# 5) Reordenar columnas nuevas al lado de las originales
cols_all = list(df.columns)

for nc in [
    "SERVIDUMBRES_SI_RAW",
    "SERVIDUMBRES_VALOR_TXT_LIMPIO",
    "SERVIDUMBRES_VALOR_SQL",
    "SERVIDUMBRES_ANOT_RAW",
    "SERVIDUMBRES_ANOT_SQL",
    "SERVIDUMBRES_ANOT_LEN",
]:
    if nc in cols_all:
        cols_all.remove(nc)

# Insertar después de la columna Si/No de SERVIDUMBRES
idx_si = cols_all.index(col_serv_si)
cols_all[idx_si + 1 : idx_si + 1] = [
    "SERVIDUMBRES_SI_RAW",
    "SERVIDUMBRES_VALOR_TXT_LIMPIO",
    "SERVIDUMBRES_VALOR_SQL",
]

# Insertar después de la columna de Anotación de SERVIDUMBRES
idx_anot = cols_all.index(col_serv_anot)
cols_all[idx_anot + 1 : idx_anot + 1] = [
    "SERVIDUMBRES_ANOT_RAW",
    "SERVIDUMBRES_ANOT_SQL",
    "SERVIDUMBRES_ANOT_LEN",
]

df = df[cols_all]

# 6) Excel con TODOS los registros (RAW / LIMPIO / SQL)
cols_export = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_serv_si,
    "SERVIDUMBRES_SI_RAW",
    "SERVIDUMBRES_VALOR_TXT_LIMPIO",
    "SERVIDUMBRES_VALOR_SQL",
    col_serv_anot,
    "SERVIDUMBRES_ANOT_RAW",
    "SERVIDUMBRES_ANOT_SQL",
    "SERVIDUMBRES_ANOT_LEN",
]
cols_export = [c for c in cols_export if c in df.columns]

df_servid_all = df[cols_export].copy()

archivo_completo = "MedidaProcesal_Servidumbres_raw_limpio_sql.xlsx"
df_servid_all.to_excel(archivo_completo, index=False)

# 7) Excels adicionales para revisión
archivo_valores_raros = None
archivo_anot_largas = None

if not servid_valores_raros.empty:
    archivo_valores_raros = "MedidaProcesal_Servidumbres_valores_sin_mapeo.xlsx"
    servid_valores_raros.to_excel(archivo_valores_raros, index=False)

if not servid_anot_largas.empty:
    archivo_anot_largas = "MedidaProcesal_Servidumbres_anotaciones_largas.xlsx"
    servid_anot_largas.to_excel(archivo_anot_largas, index=False)

print("\nArchivos creados:")
print(f" - {archivo_completo}")
if archivo_valores_raros:
    print(f" - {archivo_valores_raros} (valores Si/No sin mapeo)")
if archivo_anot_largas:
    print(f" - {archivo_anot_largas} (anotaciones > 4000 caracteres)")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6', 'ENTREGÓ CARPETA DE SOPORTES\n(SI / NO)']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

SERVIDUMBRES Si/No   : Si / No.1
SERVIDUMBRES Anotac. : Anotación.1

=== Conteo SERVIDUMBRES_VALOR_TXT_LIMPIO ===
SERVIDUMBRES_VALOR_TXT_LIMPIO
None         15317
NO            8011
SI             726
PENDIENTE        1
Name: count, dtype: int64

Total filas SERVIDUMBRES con valor Si/No raro (sin mapeo): 30
=== Ejemplos de valores SERVIDUMBRES_SI_RAW sin mapeo ===
 Fila_Excel_aprox       FMI DEPARTAMENTO             MUNICIPIO SERVIDUMBRES_SI_RAW
             2814 015-15791    ANTIOQUIA              CAUCASIA                    
            14117 303-71380    ANTIOQUIA                 YONDO                    
            14118 303-71383    ANTIOQUIA                 YONDO                    
            

## MEDIDAS CAUTELARES	


In [44]:
import pandas as pd
import numpy as np

# ======================================
# MEDIDA PROCESAL - MEDIDAS CAUTELARES
# ======================================

# 0) Fila aproximada de Excel (por si aún no existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)

# 1) Localizar columnas de MEDIDAS CAUTELARES (Si/No + Anotación)
cols_si = [c for c in cols if "SI" in str(c).upper() and "NO" in str(c).upper()]
cols_anot = [c for c in cols if "ANOTAC" in str(c).upper()]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

# Ordenar por posición en el Excel
pos = {c: i for i, c in enumerate(cols)}
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# MEDIDAS CAUTELARES = tercer par Si/No + Anotación
if "Si / No.2" in df.columns:
    col_med_si = "Si / No.2"
else:
    if len(cols_si_ordenadas) < 3:
        raise ValueError("No se encontró una tercera columna 'Si / No' para MEDIDAS CAUTELARES.")
    col_med_si = cols_si_ordenadas[2]

if "Anotación.2" in df.columns:
    col_med_anot = "Anotación.2"
else:
    if len(cols_anot_ordenadas) < 3:
        raise ValueError("No se encontró una tercera columna 'Anotación' para MEDIDAS CAUTELARES.")
    col_med_anot = cols_anot_ordenadas[2]

print("\nMEDIDAS CAUTELARES Si/No   :", col_med_si)
print("MEDIDAS CAUTELARES Anotac. :", col_med_anot)

# 2) Copias RAW de trabajo
df["MEDIDAS_CAUT_SI_RAW"]   = df[col_med_si]
df["MEDIDAS_CAUT_ANOT_RAW"] = df[col_med_anot]

# 3) Normalizar valor SI / NO / PENDIENTE

def normalizar_medidas_caut_valor(v):
    if pd.isna(v):
        return None
    txt = str(v).strip().upper()

    # quitar tildes comunes
    txt = (txt
           .replace("Á", "A")
           .replace("É", "E")
           .replace("Í", "I")
           .replace("Ó", "O")
           .replace("Ú", "U"))

    if txt in {"SI", "S", "SI.", "SI "}:
        return "SI"
    if txt in {"NO", "N", "NO.", "NO "}:
        return "NO"
    if "PEND" in txt:   # PENDIENTE, PEND, etc.
        return "PENDIENTE"

    # Cualquier otro valor se deja como None para revisión manual
    return None

df["MEDIDAS_CAUT_VALOR_TXT_LIMPIO"] = df["MEDIDAS_CAUT_SI_RAW"].apply(normalizar_medidas_caut_valor)
df["MEDIDAS_CAUT_VALOR_SQL"] = df["MEDIDAS_CAUT_VALOR_TXT_LIMPIO"]  # para MedidaProcesal.Valor (varchar(10))

print("\n=== Conteo MEDIDAS_CAUT_VALOR_TXT_LIMPIO ===")
print(df["MEDIDAS_CAUT_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

# Valores raros: hay algo en la columna original, pero no se pudo mapear
mask_valor_raro = df["MEDIDAS_CAUT_SI_RAW"].notna() & df["MEDIDAS_CAUT_VALOR_TXT_LIMPIO"].isna()

medidas_caut_valores_raros = df.loc[
    mask_valor_raro,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "MEDIDAS_CAUT_SI_RAW",
    ],
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal filas MEDIDAS CAUTELARES con valor Si/No raro (sin mapeo):",
      len(medidas_caut_valores_raros))
if not medidas_caut_valores_raros.empty:
    print("=== Ejemplos de valores MEDIDAS_CAUT_SI_RAW sin mapeo ===")
    print(medidas_caut_valores_raros.head(30).to_string(index=False))

# 4) Limpiar anotación y revisar longitud (varchar(4000))

def limpiar_anotacion_medidas_caut(v):
    if pd.isna(v):
        return None
    txt = str(v)
    # quitar saltos de línea y tabs, compactar espacios
    txt = txt.replace("\r", " ").replace("\n", " ")
    txt = " ".join(txt.split())
    txt = txt.strip()
    return txt if txt != "" else None

df["MEDIDAS_CAUT_ANOT_SQL"] = df["MEDIDAS_CAUT_ANOT_RAW"].apply(limpiar_anotacion_medidas_caut)
df["MEDIDAS_CAUT_ANOT_LEN"] = df["MEDIDAS_CAUT_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes MEDIDAS_CAUT_ANOT_SQL ===")
print(df["MEDIDAS_CAUT_ANOT_LEN"].describe())

mask_anot_larga_caut = df["MEDIDAS_CAUT_ANOT_LEN"] > 4000
medidas_caut_anot_largas = df.loc[
    mask_anot_larga_caut,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "MEDIDAS_CAUT_VALOR_SQL",
        "MEDIDAS_CAUT_ANOT_LEN",
        "MEDIDAS_CAUT_ANOT_SQL",
    ],
].copy()

print("\nTotal filas con MEDIDAS_CAUT_ANOT_SQL > 4000 caracteres:",
      len(medidas_caut_anot_largas))
if not medidas_caut_anot_largas.empty:
    print("=== Ejemplos de anotaciones MEDIDAS CAUTELARES demasiado largas ===")
    print(medidas_caut_anot_largas.head(20).to_string(index=False))

# 5) Reordenar columnas nuevas al lado de las originales
cols_all = list(df.columns)

for nc in [
    "MEDIDAS_CAUT_SI_RAW",
    "MEDIDAS_CAUT_VALOR_TXT_LIMPIO",
    "MEDIDAS_CAUT_VALOR_SQL",
    "MEDIDAS_CAUT_ANOT_RAW",
    "MEDIDAS_CAUT_ANOT_SQL",
    "MEDIDAS_CAUT_ANOT_LEN",
]:
    if nc in cols_all:
        cols_all.remove(nc)

# Insertar después de la columna Si/No de MEDIDAS CAUTELARES
idx_si = cols_all.index(col_med_si)
cols_all[idx_si + 1 : idx_si + 1] = [
    "MEDIDAS_CAUT_SI_RAW",
    "MEDIDAS_CAUT_VALOR_TXT_LIMPIO",
    "MEDIDAS_CAUT_VALOR_SQL",
]

# Insertar después de la columna de Anotación de MEDIDAS CAUTELARES
idx_anot = cols_all.index(col_med_anot)
cols_all[idx_anot + 1 : idx_anot + 1] = [
    "MEDIDAS_CAUT_ANOT_RAW",
    "MEDIDAS_CAUT_ANOT_SQL",
    "MEDIDAS_CAUT_ANOT_LEN",
]

df = df[cols_all]

# 6) Excel con TODOS los registros (RAW / LIMPIO / SQL)
cols_export = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_med_si,
    "MEDIDAS_CAUT_SI_RAW",
    "MEDIDAS_CAUT_VALOR_TXT_LIMPIO",
    "MEDIDAS_CAUT_VALOR_SQL",
    col_med_anot,
    "MEDIDAS_CAUT_ANOT_RAW",
    "MEDIDAS_CAUT_ANOT_SQL",
    "MEDIDAS_CAUT_ANOT_LEN",
]
cols_export = [c for c in cols_export if c in df.columns]

df_medidas_caut_all = df[cols_export].copy()
archivo_completo = "MedidaProcesal_MedidasCautelares_raw_limpio_sql.xlsx"
df_medidas_caut_all.to_excel(archivo_completo, index=False)

# 7) Excels adicionales para revisión
archivo_valores_raros = None
archivo_anot_largas = None

if not medidas_caut_valores_raros.empty:
    archivo_valores_raros = "MedidaProcesal_MedidasCautelares_valores_sin_mapeo.xlsx"
    medidas_caut_valores_raros.to_excel(archivo_valores_raros, index=False)

if not medidas_caut_anot_largas.empty:
    archivo_anot_largas = "MedidaProcesal_MedidasCautelares_anotaciones_largas.xlsx"
    medidas_caut_anot_largas.to_excel(archivo_anot_largas, index=False)

print("\nArchivos creados:")
print(f" - {archivo_completo}")
if archivo_valores_raros:
    print(f" - {archivo_valores_raros} (valores Si/No sin mapeo)")
if archivo_anot_largas:
    print(f" - {archivo_anot_largas} (anotaciones > 4000 caracteres)")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6', 'ENTREGÓ CARPETA DE SOPORTES\n(SI / NO)']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

MEDIDAS CAUTELARES Si/No   : Si / No.2
MEDIDAS CAUTELARES Anotac. : Anotación.2

=== Conteo MEDIDAS_CAUT_VALOR_TXT_LIMPIO ===
MEDIDAS_CAUT_VALOR_TXT_LIMPIO
None         12688
NO           10125
SI            1241
PENDIENTE        1
Name: count, dtype: int64

Total filas MEDIDAS CAUTELARES con valor Si/No raro (sin mapeo): 0

=== Resumen longitudes MEDIDAS_CAUT_ANOT_SQL ===
count    24055.000000
mean         0.170692
std          3.314977
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        281.000000
Name: MEDIDAS_CAUT_ANOT_LEN, dtype: float64

Total filas con MEDIDAS_CAUT_ANOT_SQL > 4000 caracteres: 0

Archivos creados:
 - MedidaProcesal_MedidasCautelares_

## RUPTA

In [45]:
import numpy as np
import pandas as pd

# ======================================
# MEDIDA PROCESAL - RUPTA
# ======================================

# 0) Fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)

# 1) Localizar columnas RUPTA (Si/No + Anotación + Clasificación)
cols_si = [c for c in cols if c.startswith("Si / No")]
cols_anot = [c for c in cols if c.startswith("Anotación")]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

# Ordenar por posición en el Excel
pos = {c: i for i, c in enumerate(cols)}
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# Según el orden:
# HIPOTECA      -> Si / No,   Anotación
# SERVIDUMBRES  -> Si / No.1, Anotación.1
# MED. CAUT.    -> Si / No.2, Anotación.2
# RUPTA         -> Si / No.3, Anotación.3
if "Si / No.3" in df.columns:
    rupta_si_col = "Si / No.3"
else:
    if len(cols_si_ordenadas) < 4:
        raise ValueError("No se encontró una cuarta columna 'Si / No' para RUPTA.")
    rupta_si_col = cols_si_ordenadas[3]

if "Anotación.3" in df.columns:
    rupta_anot_col = "Anotación.3"
else:
    if len(cols_anot_ordenadas) < 4:
        raise ValueError("No se encontró una cuarta columna 'Anotación' para RUPTA.")
    rupta_anot_col = cols_anot_ordenadas[3]

rupta_clasif_col = "Colectivo / Individual"

print("\nRUPTA Si/No   :", rupta_si_col)
print("RUPTA Anotac. :", rupta_anot_col)
print("RUPTA Clasif. :", rupta_clasif_col)

# 2) Copias RAW
df["RUPTA_VALOR_RAW"]  = df[rupta_si_col]
df["RUPTA_ANOT_RAW"]   = df[rupta_anot_col]
df["RUPTA_CLASIF_RAW"] = df[rupta_clasif_col]

# 3) Normalizar valor SI / NO / PENDIENTE (RUPTA_VALOR_SQL)

def normalizar_rupta_si_no(v):
    if pd.isna(v):
        return None
    t = str(v).strip().upper()
    t = (t.replace("Á", "A")
           .replace("É", "E")
           .replace("Í", "I")
           .replace("Ó", "O")
           .replace("Ú", "U"))

    if t in {"SI", "S", "SI.", "SI "}:
        return "SI"
    if t in {"NO", "N", "NO.", "NO "}:
        return "NO"
    if "PEND" in t:  # PEND, PENDIENTE, etc.
        return "PENDIENTE"

    # cualquier otro valor queda sin clasificar
    return None

df["RUPTA_VALOR_TXT_LIMPIO"] = df["RUPTA_VALOR_RAW"].apply(normalizar_rupta_si_no)
df["RUPTA_VALOR_SQL"] = df["RUPTA_VALOR_TXT_LIMPIO"]  # varchar(10) en MedidaProcesal.Valor

print("\n=== Conteo RUPTA_VALOR_TXT_LIMPIO ===")
print(df["RUPTA_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

mask_valor_raro = df["RUPTA_VALOR_RAW"].notna() & df["RUPTA_VALOR_TXT_LIMPIO"].isna()

rupta_valores_raros = df.loc[
    mask_valor_raro,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
     "RUPTA_VALOR_RAW"]
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal filas RUPTA con valor Si/No raro (sin mapeo):",
      len(rupta_valores_raros))
if not rupta_valores_raros.empty:
    print("=== Ejemplos RUPTA_VALOR_RAW sin mapeo ===")
    print(rupta_valores_raros.head(30).to_string(index=False))

# 4) Normalizar clasificación Colectivo / Individual

def limpiar_rupta_clasif(v):
    if pd.isna(v):
        return None
    txt = str(v).strip().upper()
    txt = (txt.replace("Á", "A")
             .replace("É", "E")
             .replace("Í", "I")
             .replace("Ó", "O")
             .replace("Ú", "U"))
    txt = " ".join(txt.split())
    return txt if txt else None

df["RUPTA_CLASIF_TXT_LIMPIO"] = df["RUPTA_CLASIF_RAW"].apply(limpiar_rupta_clasif)

def mapear_rupta_clasif(txt):
    if txt is None:
        return None
    if txt.startswith("COLECTIVO"):
        return "COLECTIVO"
    if txt.startswith("INDIVIDUAL"):
        return "INDIVIDUAL"
    if "PEND" in txt:
        return "PENDIENTE"
    # COLECTIVA, SI, NO, etc. quedan sin clasificar
    return None

df["RUPTA_TIPOCLASIF_SQL"] = df["RUPTA_CLASIF_TXT_LIMPIO"].apply(mapear_rupta_clasif)

print("\n=== Conteo RUPTA_TIPOCLASIF_SQL normalizado ===")
print(df["RUPTA_TIPOCLASIF_SQL"].value_counts(dropna=False))

mask_clasif_raro = df["RUPTA_CLASIF_TXT_LIMPIO"].notna() & df["RUPTA_TIPOCLASIF_SQL"].isna()

rupta_clasif_raros = df.loc[
    mask_clasif_raro,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
     "RUPTA_CLASIF_RAW", "RUPTA_CLASIF_TXT_LIMPIO"]
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\nTotal filas RUPTA con clasificación fuera de catálogo:",
      len(rupta_clasif_raros))
if not rupta_clasif_raros.empty:
    print("=== Ejemplos RUPTA_CLASIF_RAW fuera de catálogo ===")
    print(rupta_clasif_raros.head(30).to_string(index=False))

# 5) Limpiar anotación RUPTA y revisar longitud (varchar(4000))

def limpiar_rupta_anot(v):
    if pd.isna(v):
        return None
    txt = str(v)
    txt = txt.replace("\r", " ").replace("\n", " ")
    txt = " ".join(txt.split())
    txt = txt.strip()
    return txt if txt else None

df["RUPTA_ANOT_SQL"] = df["RUPTA_ANOT_RAW"].apply(limpiar_rupta_anot)
df["RUPTA_ANOT_LEN"] = df["RUPTA_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes RUPTA_ANOT_SQL ===")
print(df["RUPTA_ANOT_LEN"].describe())

mask_anot_larga = df["RUPTA_ANOT_LEN"] > 4000
rupta_anot_largas = df.loc[
    mask_anot_larga,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
     "RUPTA_VALOR_SQL", "RUPTA_ANOT_LEN", "RUPTA_ANOT_SQL"]
].copy()

print("\nTotal filas con RUPTA_ANOT_SQL > 4000 caracteres:",
      len(rupta_anot_largas))
if not rupta_anot_largas.empty:
    print("=== Ejemplos de anotaciones RUPTA demasiado largas ===")
    print(rupta_anot_largas.head(20).to_string(index=False))

# 6) Reordenar columnas nuevas cerca de las originales
cols_all = list(df.columns)

for nc in [
    "RUPTA_VALOR_RAW",
    "RUPTA_VALOR_TXT_LIMPIO",
    "RUPTA_VALOR_SQL",
    "RUPTA_ANOT_RAW",
    "RUPTA_ANOT_SQL",
    "RUPTA_ANOT_LEN",
    "RUPTA_CLASIF_RAW",
    "RUPTA_CLASIF_TXT_LIMPIO",
    "RUPTA_TIPOCLASIF_SQL",
]:
    if nc in cols_all:
        cols_all.remove(nc)

# Insertar después de la columna Si/No de RUPTA
idx_si = cols_all.index(rupta_si_col)
cols_all[idx_si + 1 : idx_si + 1] = [
    "RUPTA_VALOR_RAW",
    "RUPTA_VALOR_TXT_LIMPIO",
    "RUPTA_VALOR_SQL",
]

# Insertar después de la columna Anotación de RUPTA
idx_anot = cols_all.index(rupta_anot_col)
cols_all[idx_anot + 1 : idx_anot + 1] = [
    "RUPTA_ANOT_RAW",
    "RUPTA_ANOT_SQL",
    "RUPTA_ANOT_LEN",
]

# Insertar después de la columna Colectivo / Individual
idx_clasif = cols_all.index(rupta_clasif_col)
cols_all[idx_clasif + 1 : idx_clasif + 1] = [
    "RUPTA_CLASIF_RAW",
    "RUPTA_CLASIF_TXT_LIMPIO",
    "RUPTA_TIPOCLASIF_SQL",
]

df = df[cols_all]

# 7) Excel con TODOS los registros (RAW / LIMPIO / SQL)
cols_export = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    rupta_si_col,
    "RUPTA_VALOR_RAW",
    "RUPTA_VALOR_TXT_LIMPIO",
    "RUPTA_VALOR_SQL",
    rupta_anot_col,
    "RUPTA_ANOT_RAW",
    "RUPTA_ANOT_SQL",
    "RUPTA_ANOT_LEN",
    rupta_clasif_col,
    "RUPTA_CLASIF_RAW",
    "RUPTA_CLASIF_TXT_LIMPIO",
    "RUPTA_TIPOCLASIF_SQL",
]
cols_export = [c for c in cols_export if c in df.columns]

df_rupta_all = df[cols_export].copy()
archivo_completo = "MedidaProcesal_RUPTA_raw_limpio_sql.xlsx"
df_rupta_all.to_excel(archivo_completo, index=False)

# 8) Excels para casos especiales
mask_casos_raros = mask_valor_raro | mask_clasif_raro

rupta_casos_raros = df.loc[
    mask_casos_raros,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "RUPTA_VALOR_RAW",
        "RUPTA_VALOR_TXT_LIMPIO",
        "RUPTA_VALOR_SQL",
        "RUPTA_CLASIF_RAW",
        "RUPTA_CLASIF_TXT_LIMPIO",
        "RUPTA_TIPOCLASIF_SQL",
    ],
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

archivo_valores_raros = None
archivo_anot_largas = None

if not rupta_casos_raros.empty:
    archivo_valores_raros = "MedidaProcesal_RUPTA_valores_sin_mapeo.xlsx"
    rupta_casos_raros.to_excel(archivo_valores_raros, index=False)

if not rupta_anot_largas.empty:
    archivo_anot_largas = "MedidaProcesal_RUPTA_anotaciones_largas.xlsx"
    rupta_anot_largas.to_excel(archivo_anot_largas, index=False)

print("\nArchivos creados:")
print(f" - {archivo_completo}")
if archivo_valores_raros:
    print(f" - {archivo_valores_raros}  (Si/No o clasificación fuera de catálogo)")
if archivo_anot_largas:
    print(f" - {archivo_anot_largas}  (anotaciones > 4000 caracteres)")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']

RUPTA Si/No   : Si / No.3
RUPTA Anotac. : Anotación.3
RUPTA Clasif. : Colectivo / Individual

=== Conteo RUPTA_VALOR_TXT_LIMPIO ===
RUPTA_VALOR_TXT_LIMPIO
None         12710
NO           10903
SI             441
PENDIENTE        1
Name: count, dtype: int64

Total filas RUPTA con valor Si/No raro (sin mapeo): 0

=== Conteo RUPTA_TIPOCLASIF_SQL normalizado ===
RUPTA_TIPOCLASIF_SQL
None          23898
INDIVIDUAL       90
COLECTIVO        67
Name: count, dtype: int64

Total filas RUPTA con clasificación fuera de catálogo: 222
=== Ejemplos RUPTA_CLASIF_RAW fuera de catálogo ===
 Fila_Excel_aprox       FMI DEPARTAMENTO MUNICIPIO RUPTA_CLASIF_RAW RUPTA_CLASIF_TXT_LIMPIO
             4947  035-1505    ANTIOQUIA   BETULIA               NO              

## RTDAF

In [46]:
import numpy as np
import pandas as pd

# ======================================
# MEDIDA PROCESAL - RTDAF LEY 1448
# ======================================

# 0) Fila aproximada de Excel (por si acaso)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)

# 1) Identificar columnas de Si/No y Anotación
cols_si = [c for c in cols if c.startswith("Si / No")]
cols_anot = [c for c in cols if c.startswith("Anotación")]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

# Ordenar por posición real en el Excel
pos = {c: i for i, c in enumerate(cols)}
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# Según el orden del Excel:
#   HIPOTECA            -> Si / No      , Anotación
#   SERVIDUMBRES        -> Si / No.1    , Anotación.1
#   MEDIDAS CAUTELARES  -> Si / No.2    , Anotación.2
#   RUPTA               -> Si / No.3    , Anotación.3
#   RTDAF LEY 1448      -> Si / No.4    , Anotación.4
if "Si / No.4" in df.columns:
    rtdaf_si_col = "Si / No.4"
else:
    if len(cols_si_ordenadas) < 5:
        raise ValueError("No se encontró una quinta columna 'Si / No' para RTDAF.")
    rtdaf_si_col = cols_si_ordenadas[4]

if "Anotación.4" in df.columns:
    rtdaf_anot_col = "Anotación.4"
else:
    if len(cols_anot_ordenadas) < 5:
        raise ValueError("No se encontró una quinta columna 'Anotación' para RTDAF.")
    rtdaf_anot_col = cols_anot_ordenadas[4]

print("RTDAF Si/No   :", rtdaf_si_col)
print("RTDAF Anotac. :", rtdaf_anot_col)

# 2) Crear columnas RAW para RTDAF
df["RTDAF_VALOR_RAW"] = df[rtdaf_si_col]
df["RTDAF_ANOT_RAW"]  = df[rtdaf_anot_col]

print("\n=== Valores originales de RTDAF (Si/No) ===")
print(df["RTDAF_VALOR_RAW"].value_counts(dropna=False))

# 3) Normalizar Si/No de RTDAF -> RTDAF_VALOR_TXT_LIMPIO / RTDAF_VALOR_SQL

def normalizar_rtdaf_si_no(v):
    if pd.isna(v):
        return None
    t = str(v).strip().upper()
    t = (t.replace("Á", "A")
           .replace("É", "E")
           .replace("Í", "I")
           .replace("Ó", "O")
           .replace("Ú", "U"))
    if t in {"SI", "S", "SI.", "SI "}:
        return "SI"
    if t in {"NO", "N", "NO.", "NO "}:
        return "NO"
    if "PEND" in t:   # PEND, PENDIENTE, etc.
        return "PENDIENTE"
    # cualquier otra cosa queda sin clasificar para poder revisarla
    return None

df["RTDAF_VALOR_TXT_LIMPIO"] = df["RTDAF_VALOR_RAW"].apply(normalizar_rtdaf_si_no)
df["RTDAF_VALOR_SQL"] = df["RTDAF_VALOR_TXT_LIMPIO"]  # varchar(10) en MedidaProcesal.Valor

print("\n=== Conteo RTDAF_VALOR_TXT_LIMPIO ===")
print(df["RTDAF_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

# 4) Detectar valores raros en la columna Si/No de RTDAF (fuera de catálogo)
mask_sin_mapeo = df["RTDAF_VALOR_RAW"].notna() & df["RTDAF_VALOR_TXT_LIMPIO"].isna()

rtdaf_valores_sin_mapeo = df.loc[
    mask_sin_mapeo,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", "RTDAF_VALOR_RAW"]
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\n=== RTDAF valores Si/No fuera de catálogo (revisar manualmente) ===")
print("Total fuera de catálogo:", len(rtdaf_valores_sin_mapeo))
if not rtdaf_valores_sin_mapeo.empty:
    print(rtdaf_valores_sin_mapeo.head(30).to_string(index=False))

# 5) Limpiar anotación RTDAF y revisar longitud (varchar(4000) en SQL)

def limpiar_rtdaf_anot(v):
    if pd.isna(v):
        return None
    txt = str(v)
    # quitar saltos de línea y espacios raros
    txt = txt.replace("\r", " ").replace("\n", " ")
    txt = " ".join(txt.split())
    txt = txt.strip()
    return txt if txt else None

df["RTDAF_ANOT_SQL"] = df["RTDAF_ANOT_RAW"].apply(limpiar_rtdaf_anot)
df["RTDAF_ANOT_LEN"] = df["RTDAF_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes RTDAF_ANOT_SQL ===")
print(df["RTDAF_ANOT_LEN"].describe())

RTDAF_ANOT_MAX = 4000
mask_anot_larga = df["RTDAF_ANOT_LEN"] > RTDAF_ANOT_MAX

rtdaf_anot_largas = df.loc[
    mask_anot_larga,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
     "RTDAF_VALOR_SQL", "RTDAF_ANOT_LEN", "RTDAF_ANOT_SQL"]
].copy()

print("\nTotal anotaciones RTDAF > 4000 caracteres:", len(rtdaf_anot_largas))
if not rtdaf_anot_largas.empty:
    print("=== Ejemplos de anotaciones RTDAF demasiado largas ===")
    print(rtdaf_anot_largas.head(20).to_string(index=False))

# 6) Reordenar columnas nuevas cerca de las originales

cols_all = list(df.columns)

for nc in [
    "RTDAF_VALOR_RAW",
    "RTDAF_VALOR_TXT_LIMPIO",
    "RTDAF_VALOR_SQL",
    "RTDAF_ANOT_RAW",
    "RTDAF_ANOT_SQL",
    "RTDAF_ANOT_LEN",
]:
    if nc in cols_all:
        cols_all.remove(nc)

# Insertar después de la columna Si/No de RTDAF
idx_si = cols_all.index(rtdaf_si_col)
cols_all[idx_si + 1 : idx_si + 1] = [
    "RTDAF_VALOR_RAW",
    "RTDAF_VALOR_TXT_LIMPIO",
    "RTDAF_VALOR_SQL",
]

# Insertar después de la columna Anotación de RTDAF
idx_anot = cols_all.index(rtdaf_anot_col)
cols_all[idx_anot + 1 : idx_anot + 1] = [
    "RTDAF_ANOT_RAW",
    "RTDAF_ANOT_SQL",
    "RTDAF_ANOT_LEN",
]

df = df[cols_all]

# 7) Excel con TODOS los registros RTDAF (raw / limpio / sql)
cols_export = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    rtdaf_si_col,
    "RTDAF_VALOR_RAW",
    "RTDAF_VALOR_TXT_LIMPIO",
    "RTDAF_VALOR_SQL",
    rtdaf_anot_col,
    "RTDAF_ANOT_RAW",
    "RTDAF_ANOT_SQL",
    "RTDAF_ANOT_LEN",
]
cols_export = [c for c in cols_export if c in df.columns]

df_rtdaf_all = df[cols_export].copy()
archivo_completo = "MedidaProcesal_RTDAF_raw_limpio_sql.xlsx"
df_rtdaf_all.to_excel(archivo_completo, index=False)

# 8) Excels adicionales: valores sin mapeo y anotaciones largas
archivo_valores_sin_mapeo = None
archivo_anot_largas = None

if not rtdaf_valores_sin_mapeo.empty:
    archivo_valores_sin_mapeo = "MedidaProcesal_RTDAF_valores_sin_mapeo.xlsx"
    rtdaf_valores_sin_mapeo.to_excel(archivo_valores_sin_mapeo, index=False)

if not rtdaf_anot_largas.empty:
    archivo_anot_largas = "MedidaProcesal_RTDAF_anotaciones_largas.xlsx"
    rtdaf_anot_largas.to_excel(archivo_anot_largas, index=False)

print("\nArchivos creados:")
print(f" - {archivo_completo}")
if archivo_valores_sin_mapeo:
    print(f" - {archivo_valores_sin_mapeo}  (Si/No fuera de catálogo)")
if archivo_anot_largas:
    print(f" - {archivo_anot_largas}  (anotaciones > 4000 caracteres)")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']
RTDAF Si/No   : Si / No.4
RTDAF Anotac. : Anotación.4

=== Valores originales de RTDAF (Si/No) ===
RTDAF_VALOR_RAW
NaN          15278
NO            8652
SI              95
                29
PENDIENTE        1
Name: count, dtype: int64

=== Conteo RTDAF_VALOR_TXT_LIMPIO ===
RTDAF_VALOR_TXT_LIMPIO
None         15307
NO            8652
SI              95
PENDIENTE        1
Name: count, dtype: int64

=== RTDAF valores Si/No fuera de catálogo (revisar manualmente) ===
Total fuera de catálogo: 29
 Fila_Excel_aprox       FMI DEPARTAMENTO             MUNICIPIO RTDAF_VALOR_RAW
            14117 303-71380    ANTIOQUIA                 YONDO                
            14118 303-71383    ANTIOQUIA                 YONDO                
             6787 30

## ofertas otra entidad

In [47]:
import numpy as np
import pandas as pd

# ======================================
# MEDIDA PROCESAL - OFERTA OTRAS ENTIDADES
# ======================================

# 0) Fila aproximada de Excel (por si acaso)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)

# 1) Identificar columnas de Si/No y Anotación
cols_si = [c for c in cols if c.startswith("Si / No")]
cols_anot = [c for c in cols if c.startswith("Anotación")]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

# Ordenar por posición real (para respetar el orden del Excel)
pos = {c: i for i, c in enumerate(cols)}
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# Según el orden del Excel:
#   HIPOTECA            -> Si / No      , Anotación
#   SERVIDUMBRES        -> Si / No.1    , Anotación.1
#   MEDIDAS CAUTELARES  -> Si / No.2    , Anotación.2
#   RUPTA               -> Si / No.3    , Anotación.3
#   RTDAF LEY 1448      -> Si / No.4    , Anotación.4
#   OFERTA OTRAS ENT.   -> Si / No.5    , Anotación.5
if "Si / No.5" in df.columns:
    oferta_si_col = "Si / No.5"
else:
    if len(cols_si_ordenadas) < 6:
        raise ValueError("No se encontró una sexta columna 'Si / No' para OFERTA OTRAS ENTIDADES.")
    oferta_si_col = cols_si_ordenadas[5]

if "Anotación.5" in df.columns:
    oferta_anot_col = "Anotación.5"
else:
    if len(cols_anot_ordenadas) < 6:
        raise ValueError("No se encontró una sexta columna 'Anotación' para OFERTA OTRAS ENTIDADES.")
    oferta_anot_col = cols_anot_ordenadas[5]

print("OFERTA Si/No   :", oferta_si_col)
print("OFERTA Anotac. :", oferta_anot_col)

# 2) Crear columnas RAW para OFERTA OTRAS ENTIDADES
df["OFERTA_VALOR_RAW"] = df[oferta_si_col]
df["OFERTA_ANOT_RAW"]  = df[oferta_anot_col]

print("\n=== Valores originales de OFERTA (Si/No) ===")
print(df["OFERTA_VALOR_RAW"].value_counts(dropna=False))

# 3) Normalizar Si/No de OFERTA -> TXT_LIMPIO / SQL

def normalizar_oferta_si_no(v):
    if pd.isna(v):
        return None
    t = str(v).strip().upper()
    t = (t.replace("Á", "A")
           .replace("É", "E")
           .replace("Í", "I")
           .replace("Ó", "O")
           .replace("Ú", "U"))
    if t in {"SI", "S", "SI.", "SI "}:
        return "SI"
    if t in {"NO", "N", "NO.", "NO "}:
        return "NO"
    if "PEND" in t:   # PEND, PENDIENTE, etc.
        return "PENDIENTE"
    # cualquier otra cosa queda sin clasificar (para revisar)
    return None

df["OFERTA_VALOR_TXT_LIMPIO"] = df["OFERTA_VALOR_RAW"].apply(normalizar_oferta_si_no)
df["OFERTA_VALOR_SQL"] = df["OFERTA_VALOR_TXT_LIMPIO"]  # para MedidaProcesal.Valor (varchar(10))

print("\n=== Conteo OFERTA_VALOR_TXT_LIMPIO ===")
print(df["OFERTA_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

# 4) Detectar valores raros en Si/No (fuera de catálogo)
mask_sin_mapeo_oferta = df["OFERTA_VALOR_RAW"].notna() & df["OFERTA_VALOR_TXT_LIMPIO"].isna()

oferta_valores_sin_mapeo = df.loc[
    mask_sin_mapeo_oferta,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", "OFERTA_VALOR_RAW"]
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\n=== OFERTA valores Si/No fuera de catálogo (revisar manualmente) ===")
print("Total fuera de catálogo:", len(oferta_valores_sin_mapeo))
if not oferta_valores_sin_mapeo.empty:
    print(oferta_valores_sin_mapeo.head(30).to_string(index=False))

# 5) Limpiar anotación OFERTA y revisar longitud (varchar(4000) en SQL)

def limpiar_oferta_anot(v):
    if pd.isna(v):
        return None
    txt = str(v)
    # quitar saltos de línea y espacios raros
    txt = txt.replace("\r", " ").replace("\n", " ")
    txt = " ".join(txt.split())
    txt = txt.strip()
    return txt if txt else None

df["OFERTA_ANOT_SQL"] = df["OFERTA_ANOT_RAW"].apply(limpiar_oferta_anot)
df["OFERTA_ANOT_LEN"] = df["OFERTA_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes OFERTA_ANOT_SQL ===")
print(df["OFERTA_ANOT_LEN"].describe())

OFERTA_ANOT_MAX = 4000
mask_anot_larga_oferta = df["OFERTA_ANOT_LEN"] > OFERTA_ANOT_MAX

oferta_anot_largas = df.loc[
    mask_anot_larga_oferta,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
     "OFERTA_VALOR_SQL", "OFERTA_ANOT_LEN", "OFERTA_ANOT_SQL"]
].copy()

print("\nTotal anotaciones OFERTA > 4000 caracteres:", len(oferta_anot_largas))
if not oferta_anot_largas.empty:
    print("=== Ejemplos de anotaciones OFERTA demasiado largas ===")
    print(oferta_anot_largas.head(20).to_string(index=False))

# 6) Reordenar columnas nuevas cerca de las originales

cols_all = list(df.columns)

for nc in [
    "OFERTA_VALOR_RAW",
    "OFERTA_VALOR_TXT_LIMPIO",
    "OFERTA_VALOR_SQL",
    "OFERTA_ANOT_RAW",
    "OFERTA_ANOT_SQL",
    "OFERTA_ANOT_LEN",
]:
    if nc in cols_all:
        cols_all.remove(nc)

# Insertar después de la columna Si/No de OFERTA
idx_si = cols_all.index(oferta_si_col)
cols_all[idx_si + 1 : idx_si + 1] = [
    "OFERTA_VALOR_RAW",
    "OFERTA_VALOR_TXT_LIMPIO",
    "OFERTA_VALOR_SQL",
]

# Insertar después de la columna Anotación de OFERTA
idx_anot = cols_all.index(oferta_anot_col)
cols_all[idx_anot + 1 : idx_anot + 1] = [
    "OFERTA_ANOT_RAW",
    "OFERTA_ANOT_SQL",
    "OFERTA_ANOT_LEN",
]

df = df[cols_all]

# 7) Excel con TODOS los registros OFERTA (raw / limpio / sql)
cols_export = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    oferta_si_col,
    "OFERTA_VALOR_RAW",
    "OFERTA_VALOR_TXT_LIMPIO",
    "OFERTA_VALOR_SQL",
    oferta_anot_col,
    "OFERTA_ANOT_RAW",
    "OFERTA_ANOT_SQL",
    "OFERTA_ANOT_LEN",
]
cols_export = [c for c in cols_export if c in df.columns]

df_oferta_all = df[cols_export].copy()
archivo_completo = "MedidaProcesal_OFERTA_raw_limpio_sql.xlsx"
df_oferta_all.to_excel(archivo_completo, index=False)

# 8) Excels adicionales: valores sin mapeo y anotaciones largas
archivo_valores_sin_mapeo = None
archivo_anot_largas = None

if not oferta_valores_sin_mapeo.empty:
    archivo_valores_sin_mapeo = "MedidaProcesal_OFERTA_valores_sin_mapeo.xlsx"
    oferta_valores_sin_mapeo.to_excel(archivo_valores_sin_mapeo, index=False)

if not oferta_anot_largas.empty:
    archivo_anot_largas = "MedidaProcesal_OFERTA_anotaciones_largas.xlsx"
    oferta_anot_largas.to_excel(archivo_anot_largas, index=False)

print("\nArchivos creados:")
print(f" - {archivo_completo}")
if archivo_valores_sin_mapeo:
    print(f" - {archivo_valores_sin_mapeo}  (Si/No fuera de catálogo)")
if archivo_anot_largas:
    print(f" - {archivo_anot_largas}  (anotaciones > 4000 caracteres)")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']
OFERTA Si/No   : Si / No.5
OFERTA Anotac. : Anotación.5

=== Valores originales de OFERTA (Si/No) ===
OFERTA_VALOR_RAW
NaN          12746
NO           11223
SI              52
no              30
No               2
si               1
PENDIENTE        1
Name: count, dtype: int64

=== Conteo OFERTA_VALOR_TXT_LIMPIO ===
OFERTA_VALOR_TXT_LIMPIO
None         12746
NO           11255
SI              53
PENDIENTE        1
Name: count, dtype: int64

=== OFERTA valores Si/No fuera de catálogo (revisar manualmente) ===
Total fuera de catálogo: 0

=== Resumen longitudes OFERTA_ANOT_SQL ===
count    24055.000000
mean         0.010434
std          0.998885
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        154.

## Proceso de clasificaion

In [48]:
import numpy as np
import pandas as pd

# ======================================
# MEDIDA PROCESAL - PROCESOS DE CLARIFICACIÓN
# ======================================

# 0) Fila aproximada de Excel (por si acaso)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

cols = list(df.columns)

# 1) Identificar columnas de Si/No y Anotación
cols_si = [c for c in cols if c.startswith("Si / No")]
cols_anot = [c for c in cols if c.startswith("Anotación")]

print("Columnas tipo 'Si / No':", cols_si)
print("Columnas tipo 'Anotación':", cols_anot)

if not cols_si or not cols_anot:
    raise ValueError("No se encontraron columnas 'Si / No' o 'Anotación' en el DataFrame.")

# Ordenar por posición real en el Excel
pos = {c: i for i, c in enumerate(cols)}
cols_si_ordenadas = sorted(cols_si, key=lambda c: pos[c])
cols_anot_ordenadas = sorted(cols_anot, key=lambda c: pos[c])

# Según el orden:
#   HIPOTECA                   -> Si / No      , Anotación
#   SERVIDUMBRES               -> Si / No.1    , Anotación.1
#   MEDIDAS CAUTELARES         -> Si / No.2    , Anotación.2
#   RUPTA                      -> Si / No.3    , Anotación.3
#   RTDAF LEY 1448             -> Si / No.4    , Anotación.4
#   OFERTA OTRAS ENTIDADES     -> Si / No.5    , Anotación.5
#   PROCESOS DE CLARIFICACIÓN  -> Si / No.6    , Anotación.6
if "Si / No.6" in df.columns:
    proc_si_col = "Si / No.6"
else:
    if len(cols_si_ordenadas) < 7:
        raise ValueError("No se encontró una séptima columna 'Si / No' para PROCESOS DE CLARIFICACIÓN.")
    proc_si_col = cols_si_ordenadas[6]

if "Anotación.6" in df.columns:
    proc_anot_col = "Anotación.6"
else:
    if len(cols_anot_ordenadas) < 7:
        raise ValueError("No se encontró una séptima columna 'Anotación' para PROCESOS DE CLARIFICACIÓN.")
    proc_anot_col = cols_anot_ordenadas[6]

print("PROC. CLARIF Si/No   :", proc_si_col)
print("PROC. CLARIF Anotac. :", proc_anot_col)

# 2) Crear columnas RAW para PROCESOS DE CLARIFICACIÓN
df["PROC_CLARIF_VALOR_RAW"] = df[proc_si_col]
df["PROC_CLARIF_ANOT_RAW"]  = df[proc_anot_col]

print("\n=== Valores originales de PROCESOS DE CLARIFICACIÓN (Si/No) ===")
print(df["PROC_CLARIF_VALOR_RAW"].value_counts(dropna=False))

# 3) Normalizar Si/No -> TXT_LIMPIO / SQL

def normalizar_proc_si_no(v):
    if pd.isna(v):
        return None
    t = str(v).strip().upper()
    t = (t.replace("Á", "A")
           .replace("É", "E")
           .replace("Í", "I")
           .replace("Ó", "O")
           .replace("Ú", "U"))
    if t in {"SI", "S", "SI.", "SI "}:
        return "SI"
    if t in {"NO", "N", "NO.", "NO "}:
        return "NO"
    if "PEND" in t:   # PEND, PENDIENTE, etc.
        return "PENDIENTE"
    # cualquier otra cosa queda sin clasificar (para revisión)
    return None

df["PROC_CLARIF_VALOR_TXT_LIMPIO"] = df["PROC_CLARIF_VALOR_RAW"].apply(normalizar_proc_si_no)
df["PROC_CLARIF_VALOR_SQL"] = df["PROC_CLARIF_VALOR_TXT_LIMPIO"]

print("\n=== Conteo PROC_CLARIF_VALOR_TXT_LIMPIO ===")
print(df["PROC_CLARIF_VALOR_TXT_LIMPIO"].value_counts(dropna=False))

# 4) Valores Si/No fuera de catálogo
mask_fuera_proc = df["PROC_CLARIF_VALOR_RAW"].notna() & df["PROC_CLARIF_VALOR_TXT_LIMPIO"].isna()

reg_fuera_proc = df.loc[
    mask_fuera_proc,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO", "PROC_CLARIF_VALOR_RAW"]
].copy().sort_values(["DEPARTAMENTO", "MUNICIPIO", "FMI"])

print("\n=== PROCESOS DE CLARIFICACIÓN valores Si/No fuera de catálogo (revisar manualmente) ===")
print("Total fuera de catálogo:", len(reg_fuera_proc))
if not reg_fuera_proc.empty:
    print(reg_fuera_proc.head(30).to_string(index=False))

# 5) Limpiar anotación y revisar longitud (varchar(4000))

def limpiar_proc_anot(v):
    if pd.isna(v):
        return None
    txt = str(v)
    txt = txt.replace("\r", " ").replace("\n", " ")
    txt = " ".join(txt.split())
    txt = txt.strip()
    return txt if txt else None

df["PROC_CLARIF_ANOT_SQL"] = df["PROC_CLARIF_ANOT_RAW"].apply(limpiar_proc_anot)
df["PROC_CLARIF_ANOT_LEN"] = df["PROC_CLARIF_ANOT_SQL"].fillna("").str.len()

print("\n=== Resumen longitudes PROC_CLARIF_ANOT_SQL ===")
print(df["PROC_CLARIF_ANOT_LEN"].describe())

PROC_CLARIF_ANOT_MAX = 4000
mask_largas_proc = df["PROC_CLARIF_ANOT_LEN"] > PROC_CLARIF_ANOT_MAX

largas_proc = df.loc[
    mask_largas_proc,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "PROC_CLARIF_VALOR_SQL",
        "PROC_CLARIF_ANOT_LEN",
        "PROC_CLARIF_ANOT_SQL",
    ],
].copy()

print("\nTotal anotaciones PROCESOS DE CLARIFICACIÓN > 4000 caracteres:", len(largas_proc))
if not largas_proc.empty:
    print("=== Ejemplos de anotaciones PROCESOS DE CLARIFICACIÓN demasiado largas ===")
    print(largas_proc.head(20).to_string(index=False))

# 6) Reordenar columnas nuevas cerca de las originales

cols_all = list(df.columns)

for nc in [
    "PROC_CLARIF_VALOR_RAW",
    "PROC_CLARIF_VALOR_TXT_LIMPIO",
    "PROC_CLARIF_VALOR_SQL",
    "PROC_CLARIF_ANOT_RAW",
    "PROC_CLARIF_ANOT_SQL",
    "PROC_CLARIF_ANOT_LEN",
]:
    if nc in cols_all:
        cols_all.remove(nc)

# Insertar después de la columna Si/No de PROCESOS DE CLARIFICACIÓN
idx_si = cols_all.index(proc_si_col)
cols_all[idx_si + 1 : idx_si + 1] = [
    "PROC_CLARIF_VALOR_RAW",
    "PROC_CLARIF_VALOR_TXT_LIMPIO",
    "PROC_CLARIF_VALOR_SQL",
]

# Insertar después de la columna Anotación de PROCESOS DE CLARIFICACIÓN
idx_anot = cols_all.index(proc_anot_col)
cols_all[idx_anot + 1 : idx_anot + 1] = [
    "PROC_CLARIF_ANOT_RAW",
    "PROC_CLARIF_ANOT_SQL",
    "PROC_CLARIF_ANOT_LEN",
]

df = df[cols_all]

# 7) Excel con TODOS los registros (raw / limpio / sql)
cols_export = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    proc_si_col,
    "PROC_CLARIF_VALOR_RAW",
    "PROC_CLARIF_VALOR_TXT_LIMPIO",
    "PROC_CLARIF_VALOR_SQL",
    proc_anot_col,
    "PROC_CLARIF_ANOT_RAW",
    "PROC_CLARIF_ANOT_SQL",
    "PROC_CLARIF_ANOT_LEN",
]
cols_export = [c for c in cols_export if c in df.columns]

df_proc_all = df[cols_export].copy()
archivo_completo = "MedidaProcesal_PROC_CLARIF_raw_limpio_sql.xlsx"
df_proc_all.to_excel(archivo_completo, index=False)

# 8) Excels adicionales: valores sin mapeo y anotaciones largas
archivo_valores_sin_mapeo = None
archivo_anot_largas = None

if not reg_fuera_proc.empty:
    archivo_valores_sin_mapeo = "MedidaProcesal_PROC_CLARIF_valores_sin_mapeo.xlsx"
    reg_fuera_proc.to_excel(archivo_valores_sin_mapeo, index=False)

if not largas_proc.empty:
    archivo_anot_largas = "MedidaProcesal_PROC_CLARIF_anotaciones_largas.xlsx"
    largas_proc.to_excel(archivo_anot_largas, index=False)

print("\nArchivos creados:")
print(f" - {archivo_completo}")
if archivo_valores_sin_mapeo:
    print(f" - {archivo_valores_sin_mapeo}  (Si/No fuera de catálogo)")
if archivo_anot_largas:
    print(f" - {archivo_anot_largas}  (anotaciones > 4000 caracteres)")


Columnas tipo 'Si / No': ['Si / No', 'Si / No.1', 'Si / No.2', 'Si / No.3', 'Si / No.4', 'Si / No.5', 'Si / No.6']
Columnas tipo 'Anotación': ['Anotación', 'Anotación.1', 'Anotación.2', 'Anotación.3', 'Anotación.4', 'Anotación.5', 'Anotación.6']
PROC. CLARIF Si/No   : Si / No.6
PROC. CLARIF Anotac. : Anotación.6

=== Valores originales de PROCESOS DE CLARIFICACIÓN (Si/No) ===
PROC_CLARIF_VALOR_RAW
NaN          15449
NO            8505
no              36
SI              33
                30
PENDIENTE        1
No               1
Name: count, dtype: int64

=== Conteo PROC_CLARIF_VALOR_TXT_LIMPIO ===
PROC_CLARIF_VALOR_TXT_LIMPIO
None         15479
NO            8542
SI              33
PENDIENTE        1
Name: count, dtype: int64

=== PROCESOS DE CLARIFICACIÓN valores Si/No fuera de catálogo (revisar manualmente) ===
Total fuera de catálogo: 30
 Fila_Excel_aprox       FMI DEPARTAMENTO             MUNICIPIO PROC_CLARIF_VALOR_RAW
            14117 303-71380    ANTIOQUIA                 YONDO

## Fecha de informe previo

In [49]:
import pandas as pd
import numpy as np

# 1) Identificar la columna de fecha de informe previo
cols_fecha_prev = [
    c for c in df.columns
    if 'FECHA' in c.upper() and ('PREVIO' in c.upper() or 'INFORME' in c.upper())
]

print("Columnas candidatas para FECHA INFORME PREVIO:", cols_fecha_prev)

if 'FECHA INFORME PREVIO REPORTADA' in df.columns:
    col_fecha_prev = 'FECHA INFORME PREVIO REPORTADA'
else:
    if not cols_fecha_prev:
        raise ValueError("No se encontró columna para FECHA INFORME PREVIO; revisa encabezados.")
    col_fecha_prev = cols_fecha_prev[0]

print("Usando columna de fecha:", col_fecha_prev)

# 2) Guardar versión cruda y convertir a datetime
df['FechaInformePrevio_RAW'] = df[col_fecha_prev]

# Conversion a datetime (formato día/mes primero)
df['FechaInformePrevio_SQL'] = pd.to_datetime(
    df[col_fecha_prev],
    errors='coerce',   # lo que no se pueda convertir queda como NaT
    dayfirst=True
)

# 3) Resumen de conversión
print("\n=== Resumen conversión FECHA INFORME PREVIO ===")
print("Total filas:", len(df))
print("Fechas válidas (no NaT):", df['FechaInformePrevio_SQL'].notna().sum())
print("Fechas inválidas (NaT):", df['FechaInformePrevio_SQL'].isna().sum())

print("\nRango de fechas válidas:")
print("  Mínima:", df['FechaInformePrevio_SQL'].min())
print("  Máxima:", df['FechaInformePrevio_SQL'].max())

# 4) Ver ejemplos de valores problemáticos (texto que no se pudo convertir)
problemas = df[
    df['FechaInformePrevio_RAW'].notna()
    & df['FechaInformePrevio_SQL'].isna()
][['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO', 'FechaInformePrevio_RAW']]

print("\n=== Ejemplos de FECHA INFORME PREVIO que NO se pudieron convertir (máx. 50) ===")
print(problemas.head(50).to_string(index=False))

# 5) (Opcional) Detectar fechas raras, por ejemplo muy antiguas o muy futuras
#   Ajusta los límites si quieres
limite_inf = pd.Timestamp('1900-01-01')
limite_sup = pd.Timestamp('2030-12-31')

fechas_raras = df[
    df['FechaInformePrevio_SQL'].notna() &
    ((df['FechaInformePrevio_SQL'] < limite_inf) |
     (df['FechaInformePrevio_SQL'] > limite_sup))
][['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO', 'FechaInformePrevio_RAW', 'FechaInformePrevio_SQL']]

print("\n=== Fechas fuera de rango razonable (para revisar) ===")
if not fechas_raras.empty:
    print(fechas_raras.to_string(index=False))
else:
    print("No hay fechas fuera del rango 1900–2030.")


Columnas candidatas para FECHA INFORME PREVIO: ['FECHA INFORME PREVIO REPORTADA', 'FECHA DE INFORME', 'FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL', 'FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO']
Usando columna de fecha: FECHA INFORME PREVIO REPORTADA

=== Resumen conversión FECHA INFORME PREVIO ===
Total filas: 24055
Fechas válidas (no NaT): 299
Fechas inválidas (NaT): 23756

Rango de fechas válidas:
  Mínima: 1970-01-01 00:00:00.000045429
  Máxima: 2026-05-24 00:00:00

=== Ejemplos de FECHA INFORME PREVIO que NO se pudieron convertir (máx. 50) ===
 Fila_Excel_aprox        FMI       DEPARTAMENTO         MUNICIPIO                                                                                                                                                                                                                                                                                                                                                                                       

In [50]:
import pandas as pd
import numpy as np

# 1) Identificar la columna de fecha
if 'FECHA INFORME PREVIO REPORTADA' in df.columns:
    col_fecha_prev = 'FECHA INFORME PREVIO REPORTADA'
else:
    # Si el encabezado viene levemente distinto, ajusta aquí
    posibles = [c for c in df.columns if 'FECHA' in c.upper() and 'INFORME' in c.upper()]
    print("Columnas candidatas para FECHA INFORME PREVIO:", posibles)
    col_fecha_prev = posibles[0]

print("Usando columna de fecha:", col_fecha_prev)

# 2) Guardar original
df['FechaInformePrevio_RAW'] = df[col_fecha_prev]

# 3) Limpiar textos tipo NO / NO REGISTRA / NO APLICA -> dejar vacío
def limpiar_texto_fecha(v):
    if pd.isna(v):
        return None
    t = str(v).strip().upper()
    # Normalizar espacios internos
    t_sin_esp = " ".join(t.split())

    # Frases que significan "no hay fecha" => se dejan vacías
    frases_no = {
        'NO',
        'NO REGISTRA',
        'NO APLICA',
        'NO APLICA.',
        'NO SE APLICA',
        'NO APLICA N/A',
        'NO REPORTA',
        'NO REGISTRA INFORMACION',
        'NO REGISTRA INFORMACIÓN'
    }

    if t_sin_esp in frases_no:
        return None  # se deja vacío para que en SQL quede NULL

    # Cualquier otro texto se conserva para intentar convertirlo a fecha
    return v

df['FechaInformePrevio_TXT_LIMPIO'] = df[col_fecha_prev].apply(lpiar_texto_fecha := limpiar_texto_fecha)

# 4) Convertir a datetime usando el texto ya limpio
df['FechaInformePrevio_SQL'] = pd.to_datetime(
    df['FechaInformePrevio_TXT_LIMPIO'],
    errors='coerce',   # lo que no se pueda convertir queda como NaT
    dayfirst=True
)

# 5) Resumen general
print("\n=== Resumen conversión FECHA INFORME PREVIO (con NO/NO REGISTRA/NO APLICA limpiados) ===")
print("Total filas:", len(df))
print("Fechas válidas (no NaT):", df['FechaInformePrevio_SQL'].notna().sum())
print("Fechas inválidas (NaT):", df['FechaInformePrevio_SQL'].isna().sum())

print("\nRango de fechas válidas:")
print("  Mínima:", df['FechaInformePrevio_SQL'].min())
print("  Máxima:", df['FechaInformePrevio_SQL'].max())

# 6) Ver qué textos quedaron sin poder convertirse (distintos de NO/NO REGISTRA/NO APLICA)
problemas = df[
    df['FechaInformePrevio_TXT_LIMPIO'].notna() &
    df['FechaInformePrevio_SQL'].isna()
][['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
   'FechaInformePrevio_RAW', 'FechaInformePrevio_TXT_LIMPIO']]

print("\n=== Ejemplos de FECHA INFORME PREVIO que NO se pudieron convertir (máx. 50) ===")
print(problemas.head(50).to_string(index=False))


Usando columna de fecha: FECHA INFORME PREVIO REPORTADA

=== Resumen conversión FECHA INFORME PREVIO (con NO/NO REGISTRA/NO APLICA limpiados) ===
Total filas: 24055
Fechas válidas (no NaT): 299
Fechas inválidas (NaT): 23756

Rango de fechas válidas:
  Mínima: 1970-01-01 00:00:00.000045429
  Máxima: 2026-05-24 00:00:00

=== Ejemplos de FECHA INFORME PREVIO que NO se pudieron convertir (máx. 50) ===
 Fila_Excel_aprox       FMI DEPARTAMENTO         MUNICIPIO                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [51]:
import re
import pandas as pd
import numpy as np

# Suponemos que ya tienes estas columnas creadas previamente:
#   - 'FechaInformePrevio_TXT_LIMPIO'
#   - 'FechaInformePrevio_SQL'

# 1) Definir función para corregir formatos raros
def corregir_formato_fecha_raro(v):
    if pd.isna(v):
        return v
    s = str(v).strip()

    # Caso 1: "2103/2024" -> "21/03/2024" (ddmm/aaaa)
    m1 = re.fullmatch(r'(\d{2})(\d{2})/(\d{4})', s)
    if m1:
        d, mth, y = m1.groups()
        return f"{d}/{mth}/{y}"

    # Caso 2: "25/042024" -> "25/04/2024" (dd/mmAAAA)
    m2 = re.fullmatch(r'(\d{2})/(\d{2})(\d{4})', s)
    if m2:
        d, mth, y = m2.groups()
        return f"{d}/{mth}/{y}"

    # Si no coincide con ninguno de los patrones, se deja igual
    return s

# 2) Tomar solo las filas que aún no se pudieron convertir
mask_problema = (
    df['FechaInformePrevio_TXT_LIMPIO'].notna() &
    df['FechaInformePrevio_SQL'].isna()
)

# Aplicar corrección solo sobre esas filas problemáticas
df.loc[mask_problema, 'FechaInformePrevio_TXT_LIMPIO'] = (
    df.loc[mask_problema, 'FechaInformePrevio_TXT_LIMPIO']
      .apply(corregir_formato_fecha_raro)
)

# 3) Volver a convertir todo a datetime con el texto corregido
df['FechaInformePrevio_SQL'] = pd.to_datetime(
    df['FechaInformePrevio_TXT_LIMPIO'],
    errors='coerce',
    dayfirst=True
)

# 4) Resumen después de la corrección
print("=== Resumen FECHA INFORME PREVIO después de corrección extra ===")
print("Fechas válidas (no NaT):", df['FechaInformePrevio_SQL'].notna().sum())
print("Fechas inválidas (NaT):", df['FechaInformePrevio_SQL'].isna().sum())

# 5) Ver qué sigue sin convertirse (ahí debería seguir apareciendo por ejemplo el 'SI')
problemas_restantes = df[
    df['FechaInformePrevio_TXT_LIMPIO'].notna() &
    df['FechaInformePrevio_SQL'].isna()
][['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
   'FechaInformePrevio_RAW', 'FechaInformePrevio_TXT_LIMPIO']]

print("\n=== Valores que todavía NO se convierten a fecha (muestra) ===")
print(problemas_restantes.head(50).to_string(index=False))


=== Resumen FECHA INFORME PREVIO después de corrección extra ===
Fechas válidas (no NaT): 303
Fechas inválidas (NaT): 23752

=== Valores que todavía NO se convierten a fecha (muestra) ===
 Fila_Excel_aprox       FMI DEPARTAMENTO MUNICIPIO                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [52]:
col_txt = 'FechaInformePrevio_TXT_LIMPIO'    # la columna de texto ya limpiada

# 1) Filas donde la fecha quedó NaT
mask_nat = df['FechaInformePrevio_SQL'].isna()

# 2) Dentro de esas, separar vacíos de texto
#    (NaN o solo espacios en blanco)
mask_vacio = (
    df[col_txt].isna()
    | (df[col_txt].astype(str).str.strip() == '')
)

# 2.a) Conteo de vacíos
print("=== Vacíos (NaN o cadena vacía) ===")
print((mask_nat & mask_vacio).sum())

# 2.b) Valores NO vacíos pero que igual quedaron NaT
print("\n=== Valores de texto que no se pudieron convertir a fecha ===")
restantes = (
    df.loc[mask_nat & ~mask_vacio, col_txt]
      .value_counts(dropna=False)
)
print(restantes.head(50).to_string())


=== Vacíos (NaN o cadena vacía) ===
23749

=== Valores de texto que no se pudieron convertir a fecha ===
FechaInformePrevio_TXT_LIMPIO
Estudio realizado el 08 de mayo de 2024\n\n1. El inmueble identificado con el Folio de Matrícula Inmobiliaria No. 045-3048, que registra en la primera anotación la Escritura Pública de compraventa No. 212 de 14 e agosto de 1947 de la Notaría de Sabanalarga. Teniendo en cuenta lo anterior, el predio acredita propiedad privada por FORMULA TRANSACCIONAL.\n2. El estado del folio es ACTIVO, de tipo RURAL.\n3. Registra inscritas 7 anotaciones.\n4. No registra folio matriz. \n5. No registra folio derivado.\n6. No registra complementación. \n7. Registra medida de protección colectiva RUPTA\n8. No registra medidas cautelares ni gravámenes.\n9. Registra como propietarios a HECTOR EMILIO VIZCAINO SANJUAN y JOSE JIMENEZ S.  Sin embargo, es de mencionar que en la anotación No. 7 se registra compraventa de derechos herenciales en falsa tradición, situación que implic

In [53]:
import pandas as pd
import numpy as np
import re

# ======================================
# FECHA INFORME PREVIO
# Usando columna: FECHA INFORME PREVIO REPORTADA
# ======================================

# 0) Fila aproximada de Excel
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Identificar la columna de fecha de informe previo
cols_fecha_prev = [
    c for c in df.columns
    if "FECHA" in c.upper() and ("PREVIO" in c.upper() or "INFORME" in c.upper())
]

print("Columnas candidatas para FECHA INFORME PREVIO:", cols_fecha_prev)

if "FECHA INFORME PREVIO REPORTADA" in df.columns:
    col_fecha_prev = "FECHA INFORME PREVIO REPORTADA"
else:
    if not cols_fecha_prev:
        raise ValueError("No se encontró columna para FECHA INFORME PREVIO; revisa encabezados.")
    col_fecha_prev = cols_fecha_prev[0]

print("Usando columna de fecha:", col_fecha_prev)

# 2) Guardar versión cruda
df["FechaInformePrevio_RAW"] = df[col_fecha_prev]

# 3) Limpiar textos que significan "no hay fecha"
def limpiar_texto_fecha(v):
    if pd.isna(v):
        return None
    t = str(v).strip().upper()
    t_sin_esp = " ".join(t.split())

    frases_no = {
        "NO",
        "NO REGISTRA",
        "NO APLICA",
        "NO APLICA.",
        "NO SE APLICA",
        "NO APLICA N/A",
        "NO REPORTA",
        "NO REGISTRA INFORMACION",
        "NO REGISTRA INFORMACIÓN",
    }

    if t_sin_esp in frases_no:
        return None  # quedará NULL en SQL
    return v  # cualquier otra cosa se intenta convertir a fecha

df["FechaInformePrevio_TXT_LIMPIO"] = df[col_fecha_prev].apply(limpiar_texto_fecha)

# 4) Primera conversión a datetime
df["FechaInformePrevio_SQL"] = pd.to_datetime(
    df["FechaInformePrevio_TXT_LIMPIO"],
    errors="coerce",   # lo que no se pueda convertir queda como NaT
    dayfirst=True
)

print("\n=== Resumen conversión FECHA INFORME PREVIO (1ª pasada) ===")
print("Total filas:", len(df))
print("Fechas válidas (no NaT):", df["FechaInformePrevio_SQL"].notna().sum())
print("Fechas inválidas (NaT):", df["FechaInformePrevio_SQL"].isna().sum())

print("\nRango de fechas válidas (1ª pasada):")
print("  Mínima:", df["FechaInformePrevio_SQL"].min())
print("  Máxima:", df["FechaInformePrevio_SQL"].max())

# 5) Corrección de formatos raros en los que siguen fallando
def corregir_formato_fecha_raro(v):
    if pd.isna(v):
        return v
    s = str(v).strip()

    # Caso 1: "2103/2024" -> "21/03/2024" (ddmm/aaaa)
    m1 = re.fullmatch(r"(\d{2})(\d{2})/(\d{4})", s)
    if m1:
        d, mth, y = m1.groups()
        return f"{d}/{mth}/{y}"

    # Caso 2: "25/042024" -> "25/04/2024" (dd/mmAAAA)
    m2 = re.fullmatch(r"(\d{2})/(\d{2})(\d{4})", s)
    if m2:
        d, mth, y = m2.groups()
        return f"{d}/{mth}/{y}"

    # Otros formatos se dejan igual
    return s

mask_problema = (
    df["FechaInformePrevio_TXT_LIMPIO"].notna()
    & df["FechaInformePrevio_SQL"].isna()
)

df.loc[mask_problema, "FechaInformePrevio_TXT_LIMPIO"] = (
    df.loc[mask_problema, "FechaInformePrevio_TXT_LIMPIO"]
      .apply(corregir_formato_fecha_raro)
)

# 6) Segunda conversión después de correcciones
df["FechaInformePrevio_SQL"] = pd.to_datetime(
    df["FechaInformePrevio_TXT_LIMPIO"],
    errors="coerce",
    dayfirst=True
)

print("\n=== Resumen FECHA INFORME PREVIO después de corrección extra ===")
print("Fechas válidas (no NaT):", df["FechaInformePrevio_SQL"].notna().sum())
print("Fechas inválidas (NaT):", df["FechaInformePrevio_SQL"].isna().sum())

print("\nRango de fechas válidas (2ª pasada):")
print("  Mínima:", df["FechaInformePrevio_SQL"].min())
print("  Máxima:", df["FechaInformePrevio_SQL"].max())

# 7) Casos que todavía NO se convierten a fecha
col_txt = "FechaInformePrevio_TXT_LIMPIO"
mask_nat = df["FechaInformePrevio_SQL"].isna()

# Vacíos (NaN o cadena vacía)
mask_vacio = (
    df[col_txt].isna()
    | (df[col_txt].astype(str).str.strip() == "")
)

print("\n=== Vacíos (NaN o cadena vacía) con FechaInformePrevio_SQL = NaT ===")
print((mask_nat & mask_vacio).sum())

# No vacíos que igualmente no se convirtieron
restantes = (
    df.loc[mask_nat & ~mask_vacio, col_txt]
      .value_counts(dropna=False)
)

print("\n=== Valores de texto que no se pudieron convertir a fecha (top 50) ===")
print(restantes.head(50).to_string())

problemas_detalle = df.loc[
    mask_nat & ~mask_vacio,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "FechaInformePrevio_RAW",
        "FechaInformePrevio_TXT_LIMPIO",
    ],
].copy()

# 8) Fechas fuera de rango razonable (por si hay errores groseros)
limite_inf = pd.Timestamp("1900-01-01")
limite_sup = pd.Timestamp("2030-12-31")

fechas_raras = df[
    df["FechaInformePrevio_SQL"].notna()
    & (
        (df["FechaInformePrevio_SQL"] < limite_inf)
        | (df["FechaInformePrevio_SQL"] > limite_sup)
    )
][[
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "FechaInformePrevio_RAW",
    "FechaInformePrevio_TXT_LIMPIO",
    "FechaInformePrevio_SQL",
]]

print("\n=== Fechas fuera de rango razonable (1900–2030) ===")
print("Total fuera de rango:", len(fechas_raras))
if not fechas_raras.empty:
    print(fechas_raras.head(50).to_string(index=False))

# 9) Excels de salida

# 9.1) Todos los registros (raw / limpio / sql)
cols_export = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    col_fecha_prev,
    "FechaInformePrevio_RAW",
    "FechaInformePrevio_TXT_LIMPIO",
    "FechaInformePrevio_SQL",
]
cols_export = [c for c in cols_export if c in df.columns]

df_fechas_all = df[cols_export].copy()
archivo_completo = "FechaInformePrevio_raw_limpio_sql.xlsx"
df_fechas_all.to_excel(archivo_completo, index=False)

# 9.2) Textos no convertidos a fecha
archivo_no_convertidos = None
if not problemas_detalle.empty:
    archivo_no_convertidos = "FechaInformePrevio_valores_no_convertidos.xlsx"
    problemas_detalle.to_excel(archivo_no_convertidos, index=False)

# 9.3) Fechas fuera de rango razonable
archivo_fechas_raras = None
if not fechas_raras.empty:
    archivo_fechas_raras = "FechaInformePrevio_fechas_fuera_rango.xlsx"
    fechas_raras.to_excel(archivo_fechas_raras, index=False)

print("\nArchivos creados:")
print(f" - {archivo_completo}")
if archivo_no_convertidos:
    print(f" - {archivo_no_convertidos}  (textos que no se convierten a fecha)")
if archivo_fechas_raras:
    print(f" - {archivo_fechas_raras}  (fechas fuera de 1900–2030)")


Columnas candidatas para FECHA INFORME PREVIO: ['FECHA INFORME PREVIO REPORTADA', 'FECHA DE INFORME', 'FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL', 'FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO', 'FechaInformePrevio_RAW', 'FechaInformePrevio_SQL', 'FechaInformePrevio_TXT_LIMPIO']
Usando columna de fecha: FECHA INFORME PREVIO REPORTADA

=== Resumen conversión FECHA INFORME PREVIO (1ª pasada) ===
Total filas: 24055
Fechas válidas (no NaT): 299
Fechas inválidas (NaT): 23756

Rango de fechas válidas (1ª pasada):
  Mínima: 1970-01-01 00:00:00.000045429
  Máxima: 2026-05-24 00:00:00

=== Resumen FECHA INFORME PREVIO después de corrección extra ===
Fechas válidas (no NaT): 303
Fechas inválidas (NaT): 23752

Rango de fechas válidas (2ª pasada):
  Mínima: 1970-01-01 00:00:00.000045429
  Máxima: 2026-05-24 00:00:00

=== Vacíos (NaN o cadena vacía) con FechaInformePrevio_SQL = NaT ===
23749

=== Valores de texto que no se pudieron convertir a fecha (top 50) ===
FechaInformePrevio_TXT_LIMPIO
E

## CONCEPTO ANTIGUO

In [54]:
import pandas as pd
import numpy as np

# 1) Buscar la columna de Excel que contiene el concepto previo
candidatas = [c for c in df.columns if 'CONCEPTO' in c.upper()]
print("Columnas candidatas para CONCEPTO:", candidatas)

# Ajusta este nombre si ves que en tu archivo es ligeramente distinto
col_conc = 'CONCEPTO ANTIGUO'

# Copia del texto original (por si luego quieres comparar)
df['ConceptoPrevio_TXT_ORIG'] = df[col_conc]

# 2) Limpieza básica de texto y mapeo de "sin concepto" a NULL
def limpiar_concepto(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    t_up = t.upper()

    # Lista de textos que interpretamos como "sin concepto"
    SIN_CONCEPTO = {
        'NO', 'NO REGISTRA', 'NO APLICA',
        'NINGUNO', 'SIN CONCEPTO'
    }

    if t_up in SIN_CONCEPTO:
        return None

    # En cualquier otro caso dejamos el texto como está
    return t

df['ConceptoPrevio_SQL'] = df['ConceptoPrevio_TXT_ORIG'].apply(limpiar_concepto)

# 3) Analizar longitudes para ver si hay cosas raras (aunque varchar(MAX) lo permite)
df['ConceptoPrevio_LEN'] = df['ConceptoPrevio_SQL'].fillna('').str.len()

print("=== Resumen longitudes ConceptoPrevio_SQL ===")
print(df['ConceptoPrevio_LEN'].describe())

# 4) Algunos ejemplos de conceptos muy cortos (<= 20 caracteres, distintos de vacío)
print("\n=== Ejemplos de conceptos muy cortos (<= 20 caracteres) ===")
ej_cortos = df[(df['ConceptoPrevio_LEN'] > 0) &
               (df['ConceptoPrevio_LEN'] <= 20)][
    ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO', 'ConceptoPrevio_SQL']
]
print(ej_cortos.head(30).to_string(index=False))

# 5) Algunos ejemplos de conceptos muy largos (>= 500 caracteres)
print("\n=== Ejemplos de conceptos largos (>= 500 caracteres) ===")
ej_largos = df[df['ConceptoPrevio_LEN'] >= 500][
    ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
     'ConceptoPrevio_SQL', 'ConceptoPrevio_LEN']
]
print(ej_largos.head(20).to_string(index=False))


Columnas candidatas para CONCEPTO: ['CONCEPTO ANTIGUO', 'ANALISIS JURÍDICO / CONCEPTO FINAL\n(DEFINICIÓN DE VIABILIDAD JURÍDICA)']
=== Resumen longitudes ConceptoPrevio_SQL ===
count    24055.000000
mean       815.260195
std       2119.108417
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      32767.000000
Name: ConceptoPrevio_LEN, dtype: float64

=== Ejemplos de conceptos muy cortos (<= 20 caracteres) ===
 Fila_Excel_aprox              FMI DEPARTAMENTO              MUNICIPIO   ConceptoPrevio_SQL
             1041 CÓDIGO CATASTRAL      VICHADA               CUMARIBO      No registra FMI
             1072        ESCRITURA      CAQUETA SAN VICENTE DEL CAGUAN      No registra FMI
             3026        076-23960       BOYACA                  CHITA  FOLIO NO ENCONTRADO
             3110         540-4764      VICHADA         PUERTO CARRENO           CONCLUSIÓN
             3342       120-137376        CAUCA       SOTARA PAISPAMBA            PEN

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4105696512.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['ConceptoPrevio_SQL'] = df['ConceptoPrevio_TXT_ORIG'].apply(limpiar_concepto)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4105696512.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['ConceptoPrevio_LEN'] = df['ConceptoPrevio_SQL'].fillna('').str.len()


In [55]:
import pandas as pd
import numpy as np
import re

# ======================================
# CONCEPTO ANTIGUO  → ConceptosPrevio.Concepto (varchar(MAX))
# ======================================

# 0) Asegurar Fila_Excel_aprox
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Ubicar la columna de concepto
if "CONCEPTO ANTIGUO" in df.columns:
    col_conc = "CONCEPTO ANTIGUO"
else:
    # Respaldo, por si algún día cambian el encabezado
    candidatas = [c for c in df.columns if "CONCEPTO" in c.upper()]
    print("Columnas candidatas para CONCEPTO:", candidatas)
    if not candidatas:
        raise ValueError("No se encontró columna de CONCEPTO; revisa encabezados.")
    col_conc = candidatas[0]

print("Usando columna de concepto previo:", col_conc)

# 2) Copia del texto original
df["ConceptoPrevio_TXT_ORIG"] = df[col_conc]

# 3) Limpieza básica y mapeo de "sin concepto" a NULL
SIN_CONCEPTO = {
    "NO",
    "NO REGISTRA",
    "NO REGISTRA.",
    "NO APLICA",
    "NO APLICA.",
    "NO SE APLICA",
    "SIN CONCEPTO",
    "NINGUNO",
    "NINGUNA",
    "N/A",
    "NA",
    "NO REPORTA",
    "NO REGISTRA INFORMACION",
    "NO REGISTRA INFORMACIÓN",
}

def limpiar_concepto(v):
    if pd.isna(v):
        return None

    t = str(v).strip()
    if t == "":
        return None

    # Normalizamos sólo para comparar contra la lista SIN_CONCEPTO
    t_up = " ".join(t.upper().split())

    if t_up in SIN_CONCEPTO:
        return None

    # En cualquier otro caso dejamos el texto como está
    return t

df["ConceptoPrevio_SQL"] = df["ConceptoPrevio_TXT_ORIG"].apply(limpiar_concepto)

# 4) Versión de texto "limpio" sólo para análisis (espacios consolidados)
df["ConceptoPrevio_TXT_LIMPIO"] = (
    df["ConceptoPrevio_SQL"]
      .astype("string")
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
)
df.loc[df["ConceptoPrevio_SQL"].isna(), "ConceptoPrevio_TXT_LIMPIO"] = None

# 5) Longitud del texto
df["ConceptoPrevio_LEN"] = df["ConceptoPrevio_TXT_LIMPIO"].fillna("").str.len()

print("=== Resumen longitudes ConceptoPrevio_SQL ===")
print(df["ConceptoPrevio_LEN"].describe())

# 6) Ejemplos de conceptos muy cortos (0 < len <= 20)
print("\n=== Ejemplos de conceptos muy cortos (0 < longitud <= 20) ===")
conceptos_cortos = df[
    (df["ConceptoPrevio_LEN"] > 0) &
    (df["ConceptoPrevio_LEN"] <= 20)
][[
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "ConceptoPrevio_TXT_ORIG", "ConceptoPrevio_SQL", "ConceptoPrevio_LEN"
]]

print(conceptos_cortos.head(30).to_string(index=False))

# 7) Ejemplos de conceptos largos (>= 500 caracteres)
print("\n=== Ejemplos de conceptos largos (>= 500 caracteres) ===")
conceptos_largos = df[
    df["ConceptoPrevio_LEN"] >= 500
][[
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "ConceptoPrevio_TXT_ORIG", "ConceptoPrevio_SQL", "ConceptoPrevio_LEN"
]]

print(conceptos_largos.head(20).to_string(index=False))

# 8) Archivo Excel de revisión
cols_base = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_conc, "ConceptoPrevio_TXT_ORIG",
    "ConceptoPrevio_TXT_LIMPIO", "ConceptoPrevio_SQL", "ConceptoPrevio_LEN"
]
cols_base = [c for c in cols_base if c in df.columns]

nombre_archivo = "ConceptoPrevio_revision.xlsx"

with pd.ExcelWriter(nombre_archivo) as writer:
    # Hoja 1: todos los conceptos
    df[cols_base].to_excel(writer, sheet_name="Todos", index=False)

    # Hoja 2: conceptos cortos
    if not conceptos_cortos.empty:
        conceptos_cortos.to_excel(writer, sheet_name="Cortos_<=20", index=False)

    # Hoja 3: conceptos largos
    if not conceptos_largos.empty:
        conceptos_largos.to_excel(writer, sheet_name="Largos_>=500", index=False)

print(f"\nArchivo creado: {nombre_archivo}")


Usando columna de concepto previo: CONCEPTO ANTIGUO


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\1429210873.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["ConceptoPrevio_TXT_LIMPIO"] = (


=== Resumen longitudes ConceptoPrevio_SQL ===
count       24055.0
mean      811.09075
std      2106.94731
min             0.0
25%             0.0
50%             0.0
75%             0.0
max         32629.0
Name: ConceptoPrevio_LEN, dtype: Float64

=== Ejemplos de conceptos muy cortos (0 < longitud <= 20) ===
 Fila_Excel_aprox              FMI DEPARTAMENTO              MUNICIPIO ConceptoPrevio_TXT_ORIG   ConceptoPrevio_SQL  ConceptoPrevio_LEN
             1041 CÓDIGO CATASTRAL      VICHADA               CUMARIBO         No registra FMI      No registra FMI                  15
             1072        ESCRITURA      CAQUETA SAN VICENTE DEL CAGUAN         No registra FMI      No registra FMI                  15
             3026        076-23960       BOYACA                  CHITA     FOLIO NO ENCONTRADO  FOLIO NO ENCONTRADO                  19
             3110         540-4764      VICHADA         PUERTO CARRENO              CONCLUSIÓN           CONCLUSIÓN                  10
          

## "ANALISIS JURÍDICO / CONCEPTO FINAL (DEFINICIÓN DE VIABILIDAD JURÍDICA)"


In [56]:
import pandas as pd
import numpy as np
import re

# ===========================================
# CONCEPTO FINAL VIABILIDAD JURÍDICA
# → AnalisisJuridico.ConceptoFinal.ConceptoActualDeViabilidadJuridica (varchar(MAX))
# ===========================================

# 0) Asegurar columna de fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Ubicar la columna de Excel correspondiente al concepto final
candidatas = [
    c for c in df.columns
    if "CONCEPTO FINAL" in str(c).upper()
       or "ANALISIS JURIDICO" in str(c).upper()
       or "ANÁLISIS JURÍDICO" in str(c).upper()
]

print("Columnas candidatas para CONCEPTO FINAL:", candidatas)

if not candidatas:
    raise ValueError(
        "No se encontró una columna que parezca ser el CONCEPTO FINAL "
        "(revisa encabezados en el Excel)."
    )

col_conc_final = candidatas[0]
print("Usando columna de concepto final:", col_conc_final)

# 2) Copia del texto original (por trazabilidad)
df["ConceptoFinal_TXT_ORIG"] = df[col_conc_final]

# 3) Limpieza básica y mapeo de textos que significan “sin concepto” a NULL
NO_INFO = {
    "NO",
    "NO.",
    "NO REGISTRA",
    "NO REGISTRA.",
    "NO APLICA",
    "NO APLICA.",
    "NO SE APLICA",
    "NO SE APLICA.",
    "NO REPORTA",
    "N/A",
    "NA",
    "N.A.",
    "SIN INFORMACION",
    "SIN INFORMACIÓN",
    "SIN REGISTRO",
    "SIN REGISTRO.",
    "NINGUNO",
    "NINGUNA",
    "0",
}

def limpiar_concepto_final(v):
    if pd.isna(v):
        return None

    t = str(v).strip()
    if t == "":
        return None

    # Normalizar sólo para comparar contra NO_INFO
    t_up = " ".join(t.upper().split())

    if t_up in NO_INFO:
        return None

    # Cualquier otro texto se conserva tal cual
    return t

# Columna lista para migrar a SQL
df["ConceptoActualDeViabilidadJuridica_SQL"] = df["ConceptoFinal_TXT_ORIG"].apply(
    limpiar_concepto_final
)

# 4) Versión “limpia” sólo para análisis (espacios consolidados)
df["ConceptoFinal_TXT_LIMPIO"] = (
    df["ConceptoActualDeViabilidadJuridica_SQL"]
      .astype("string")
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
)
df.loc[df["ConceptoActualDeViabilidadJuridica_SQL"].isna(),
       "ConceptoFinal_TXT_LIMPIO"] = None

# 5) Longitud del texto (aunque en SQL sea varchar(MAX), sirve para revisar extremos)
df["ConceptoFinal_LEN"] = df["ConceptoFinal_TXT_LIMPIO"].fillna("").str.len()

print("=== Resumen longitudes ConceptoActualDeViabilidadJuridica_SQL ===")
print(df["ConceptoFinal_LEN"].describe())

# 6) Ejemplos de conceptos muy cortos (0 < longitud <= 20)
print("\n=== Ejemplos de conceptos muy cortos (0 < longitud <= 20) ===")
conceptos_cortos = df[
    (df["ConceptoFinal_LEN"] > 0) &
    (df["ConceptoFinal_LEN"] <= 20)
][[
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "ConceptoFinal_TXT_ORIG",
    "ConceptoActualDeViabilidadJuridica_SQL",
    "ConceptoFinal_LEN"
]]

print(conceptos_cortos.head(30).to_string(index=False))

# 7) Ejemplos de conceptos muy largos (>= 500 caracteres)
print("\n=== Ejemplos de conceptos largos (>= 500 caracteres) ===")
conceptos_largos = df[
    df["ConceptoFinal_LEN"] >= 500
][[
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    "ConceptoFinal_TXT_ORIG",
    "ConceptoActualDeViabilidadJuridica_SQL",
    "ConceptoFinal_LEN"
]]

print(conceptos_largos.head(20).to_string(index=False))

# 8) Archivo Excel de revisión (mismo esquema que en ConceptoPrevio)
cols_base = [
    "Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
    col_conc_final,
    "ConceptoFinal_TXT_ORIG",
    "ConceptoFinal_TXT_LIMPIO",
    "ConceptoActualDeViabilidadJuridica_SQL",
    "ConceptoFinal_LEN",
]
cols_base = [c for c in cols_base if c in df.columns]

nombre_archivo = "ConceptoFinal_ViabilidadJuridica_revision.xlsx"

with pd.ExcelWriter(nombre_archivo) as writer:
    # Hoja 1: todos los registros
    df[cols_base].to_excel(writer, sheet_name="Todos", index=False)

    # Hoja 2: cortos
    if not conceptos_cortos.empty:
        conceptos_cortos.to_excel(writer, sheet_name="Cortos_<=20", index=False)

    # Hoja 3: largos
    if not conceptos_largos.empty:
        conceptos_largos.to_excel(writer, sheet_name="Largos_>=500", index=False)

print(f"\nArchivo creado: {nombre_archivo}")


Columnas candidatas para CONCEPTO FINAL: ['ANALISIS JURÍDICO / CONCEPTO FINAL\n(DEFINICIÓN DE VIABILIDAD JURÍDICA)']
Usando columna de concepto final: ANALISIS JURÍDICO / CONCEPTO FINAL
(DEFINICIÓN DE VIABILIDAD JURÍDICA)


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\3637432935.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["ConceptoFinal_TXT_ORIG"] = df[col_conc_final]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\3637432935.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["ConceptoActualDeViabilidadJuridica_SQL"] = df["ConceptoFinal_TXT_ORIG"].apply(
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\3637432935.py:82: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resul

=== Resumen longitudes ConceptoActualDeViabilidadJuridica_SQL ===
count        24055.0
mean     3447.021077
std      5044.179609
min              0.0
25%              0.0
50%             75.0
75%           7030.0
max          32678.0
Name: ConceptoFinal_LEN, dtype: Float64

=== Ejemplos de conceptos muy cortos (0 < longitud <= 20) ===
 Fila_Excel_aprox         FMI       DEPARTAMENTO              MUNICIPIO ConceptoFinal_TXT_ORIG ConceptoActualDeViabilidadJuridica_SQL  ConceptoFinal_LEN
             2743    355-4203             TOLIMA               PLANADAS          FOLIO CERRADO                          FOLIO CERRADO                 13
             3013    540-4198            VICHADA               CUMARIBO  V.    RONDAS HÍDRICAS                  V.    RONDAS HÍDRICAS                 18
             3029     540-647            VICHADA               CUMARIBO             CONCLUSIÓN                             CONCLUSIÓN                 10
             5913   034-15786          ANTIOQUIA   

## Fechainforme

In [57]:
import pandas as pd
import numpy as np
from datetime import datetime

# ===============================
# FECHA DE INFORME (CONCEPTO FINAL)
# Tabla destino: AnalisisJuridico.ConceptoFinal.FechaInforme (datetime NULL)
# ===============================

# 0) Asegurar columna de fila aproximada
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

# 1) Ubicar la columna adecuada de "FECHA DE INFORME" (NO la de informe PREVIO)
cols_fecha_candidatas = [
    c for c in df.columns
    if ("FECHA" in str(c).upper()) and ("INFORME" in str(c).upper())
]

print("Columnas candidatas para FECHA DE INFORME:", cols_fecha_candidatas)

# Preferir una que NO contenga "PREVIO"
candidatas_sin_prev = [c for c in cols_fecha_candidatas if "PREVIO" not in str(c).upper()]

if candidatas_sin_prev:
    col_fecha_informe = candidatas_sin_prev[0]
else:
    # si no hay otra, tomar la primera candidata
    col_fecha_informe = cols_fecha_candidatas[0]

print("Usando columna para ConceptoFinal.FechaInforme:", col_fecha_informe)

# 2) Columnas RAW y de texto limpio
df["FECHA_INFORME_RAW"] = df[col_fecha_informe]

df["FECHA_INFORME_TXT_LIMPIO"] = (
    df["FECHA_INFORME_RAW"]
      .astype(str)
      .str.strip()
)

# 3) Función de normalización -> datetime (FechaInforme_SQL)
VALORES_VACIOS_INFORME = {
    "NO",
    "NO REGISTRA",
    "NO REGISTRA.",
    "NO APLICA",
    "NO APLICA.",
    "NO EXISTE",
    "NO EXISTE.",
    "N/A",
    "NA"
}

def normalizar_fecha_informe(v):
    """Devuelve un datetime (Timestamp) o NaT para la fecha de concepto final."""
    if pd.isna(v):
        return pd.NaT

    # Si ya viene como datetime/Timestamp se respeta
    if isinstance(v, (pd.Timestamp, datetime)):
        return pd.to_datetime(v, errors="coerce")

    t = str(v).strip()
    if t == "":
        return pd.NaT

    t_up = t.upper()

    # Textos que significan 'sin registro'
    if t_up in VALORES_VACIOS_INFORME:
        return pd.NaT

    # Intento de parseo flexible (soporta 25/04/2024, 2024-04-25, etc.)
    fecha = pd.to_datetime(t, dayfirst=True, errors="coerce")
    if pd.isna(fecha):
        return pd.NaT

    return fecha

df["FechaInforme_SQL"] = df["FECHA_INFORME_RAW"].apply(normalizar_fecha_informe)

# 4) Resumen de resultados
total_filas = len(df)
n_validas = df["FechaInforme_SQL"].notna().sum()
n_nulas = df["FechaInforme_SQL"].isna().sum()

mask_texto_no_vacio = df["FECHA_INFORME_TXT_LIMPIO"].str.len() > 0
mask_invalidas_con_texto = mask_texto_no_vacio & df["FechaInforme_SQL"].isna()
n_invalidas_con_texto = mask_invalidas_con_texto.sum()

print("\n=== Resumen FECHA DE INFORME (ConceptoFinal) ===")
print("Total filas:", total_filas)
print("Fechas válidas (no NaT):", n_validas)
print("Valores nulos (NaT) en FechaInforme_SQL:", n_nulas)
print("Fechas inválidas (NaT con texto no vacío):", n_invalidas_con_texto)

# 5) Ejemplos correcto vs normalizado
ejemplos_ok = df[df["FechaInforme_SQL"].notna()].head(30)
print("\n=== Ejemplos de FECHA DE INFORME original vs normalizada ===")
print(
    ejemplos_ok[
        ["Fila_Excel_aprox",
         "FMI",
         "DEPARTAMENTO",
         "MUNICIPIO",
         "FECHA_INFORME_RAW",
         "FechaInforme_SQL"]
    ].to_string(index=False)
)

# 6) Si hay inválidas con texto, mostrar algunos ejemplos
if n_invalidas_con_texto > 0:
    ejemplos_invalidos = df[mask_invalidas_con_texto].head(100)
    print("\n=== Ejemplos de filas con texto que NO se pudo convertir a fecha ===")
    print(
        ejemplos_invalidos[
            ["Fila_Excel_aprox",
             "FMI",
             "DEPARTAMENTO",
             "MUNICIPIO",
             "FECHA_INFORME_RAW",
             "FECHA_INFORME_TXT_LIMPIO"]
        ].to_string(index=False)
    )
else:
    print("\nNo quedan filas con texto en FECHA DE INFORME que no se hayan podido convertir a fecha.")


Columnas candidatas para FECHA DE INFORME: ['FECHA INFORME PREVIO REPORTADA', 'FECHA DE INFORME', 'FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL', 'FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO', 'FechaInformePrevio_RAW', 'FechaInformePrevio_SQL', 'FechaInformePrevio_TXT_LIMPIO']
Usando columna para ConceptoFinal.FechaInforme: FECHA DE INFORME

=== Resumen FECHA DE INFORME (ConceptoFinal) ===
Total filas: 24055
Fechas válidas (no NaT): 9885
Valores nulos (NaT) en FechaInforme_SQL: 14170
Fechas inválidas (NaT con texto no vacío): 14170

=== Ejemplos de FECHA DE INFORME original vs normalizada ===
 Fila_Excel_aprox        FMI DEPARTAMENTO           MUNICIPIO FECHA_INFORME_RAW FechaInforme_SQL
                2  062-37357      BOLIVAR            ZAMBRANO        2024-10-17       2024-10-17
                3  062-37359      BOLIVAR            ZAMBRANO        2025-11-25       2025-11-25
                4  062-37361      BOLIVAR            ZAMBRANO        2024-09-30       2024-09-30
         

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\701586686.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FECHA_INFORME_RAW"] = df[col_fecha_informe]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\701586686.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["FECHA_INFORME_TXT_LIMPIO"] = (
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\701586686.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

In [58]:
import pandas as pd

# ============================
# EXCEL DE REVISIÓN DE FECHAS
# ============================

# Hojas para FECHA INFORME PREVIO (si ya fueron creadas esas columnas)
tiene_prev = all(
    col in df.columns
    for col in ["FechaInformePrevio_RAW", "FechaInformePrevio_TXT_LIMPIO", "FechaInformePrevio_SQL"]
)

if tiene_prev:
    df_prev_mapeo = df[
        [
            "Fila_Excel_aprox",
            "FMI",
            "DEPARTAMENTO",
            "MUNICIPIO",
            "FechaInformePrevio_RAW",
            "FechaInformePrevio_TXT_LIMPIO",
            "FechaInformePrevio_SQL",
        ]
    ].copy()
else:
    df_prev_mapeo = None

# Hojas para FECHA INFORME (ConceptoFinal)
df_final_mapeo = df[
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "FECHA_INFORME_RAW",
        "FECHA_INFORME_TXT_LIMPIO",
        "FechaInforme_SQL",
    ]
].copy()

mask_final_invalidas = (
    df["FECHA_INFORME_TXT_LIMPIO"].str.len() > 0
) & df["FechaInforme_SQL"].isna()

df_final_invalidas = df.loc[
    mask_final_invalidas,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "FECHA_INFORME_RAW",
        "FECHA_INFORME_TXT_LIMPIO",
    ],
].copy()

# Ruta de salida (ajústala si quieres)
ruta_salida = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Fechas_Conceptos_revision.xlsx"

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    # Hoja Informe Previo (si existe)
    if df_prev_mapeo is not None:
        df_prev_mapeo.to_excel(writer, sheet_name="InformePrevio_mapeo", index=False)

    # Hoja Informe Final (mapeo completo)
    df_final_mapeo.to_excel(writer, sheet_name="InformeFinal_mapeo", index=False)

    # Hoja Informe Final – inválidas
    if not df_final_invalidas.empty:
        df_final_invalidas.to_excel(writer, sheet_name="InformeFinal_invalidas", index=False)

print("Archivo de revisión generado en:", ruta_salida)


Archivo de revisión generado en: C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Fechas_Conceptos_revision.xlsx


## VIABLE / NO VIABLE / PENDIENTE

*Los nulos se pasaron como Sin definir*

In [59]:
import pandas as pd
import numpy as np
import unicodedata

# ======================================================
# VIABILIDAD: VIABLE / NO VIABLE / PENDIENTE / PARCIAL
# Campos destino lógicos:
#   - IdTipoViabilidad_SQL (0..4)
#   - NombreTipoViabilidad_SQL (texto catálogo)
# ======================================================

# Asegurar columna de fila aproximada (si no existe)
if "Fila_Excel_aprox" not in df.columns:
    df["Fila_Excel_aprox"] = df.index + 2

#-------------------------------------------
# 1) Ubicar la columna de Excel
#-------------------------------------------
cols_viab = [
    c for c in df.columns
    if ("VIABLE" in str(c).upper()) and ("PENDIENTE" in str(c).upper())
]

if not cols_viab:
    # Respaldo por si el nombre es distinto
    cols_viab = [c for c in df.columns if "VIABLE" in str(c).upper()]

print("Columnas candidatas para VIABILIDAD:", cols_viab)
col_viab = cols_viab[0]
print("Usando columna:", col_viab)

# Guardamos el valor original
df["Viabilidad_RAW"] = df[col_viab]

#-------------------------------------------
# 2) Funciones de normalización / mapeo
#-------------------------------------------
def quitar_acentos(txt: str) -> str:
    return "".join(
        ch for ch in unicodedata.normalize("NFD", txt)
        if unicodedata.category(ch) != "Mn"
    )

def mapear_viabilidad(v):
    """
    Devuelve (IdTipoViabilidad, NombreTipoViabilidad)

      0: Sin definir
      1: Pendiente
      2: Viable
      3: Viabilidad parcial
      4: No viable
    """
    if pd.isna(v):
        return 0, "Sin definir"

    t = str(v).strip()
    if t == "":
        return 0, "Sin definir"

    t_up = quitar_acentos(t.upper())

    # Textos claros de "sin información"
    if (
        "SIN REGISTRO" in t_up or
        "NO REGISTRA" in t_up or
        "NO APLICA" in t_up or
        "SIN INFORMACION" in t_up
    ):
        return 0, "Sin definir"

    # 1) Pendiente
    if "PEND" in t_up:   # PENDIENTE, EN PENDIENTE, PENDIENTE POR..., etc.
        return 1, "Pendiente"

    # 3) Viabilidad parcial
    if ("VIABILIDAD PARCIAL" in t_up or
        "PARCIALMENTE" in t_up or
        ("PARCIAL" in t_up and "VIABLE" in t_up)):
        return 3, "Viabilidad parcial"

    # 4) No viable
    if "NO VIABLE" in t_up or (" NO " in f" {t_up} " and "VIABLE" in t_up):
        return 4, "No viable"

    # 2) Viable (cualquier texto que contenga VIABLE, pero no caiga en los casos anteriores)
    if "VIABLE" in t_up:
        return 2, "Viable"

    # Cualquier otra cosa: dejar como "Sin definir" pero marcado para revisión
    return 0, "OTRO: " + t

#-------------------------------------------
# 3) Aplicar mapeo al DataFrame
#-------------------------------------------
df[["IdTipoViabilidad_SQL", "NombreTipoViabilidad_SQL"]] = df["Viabilidad_RAW"].apply(
    lambda v: pd.Series(mapear_viabilidad(v))
)

#-------------------------------------------
# 4) Resumen y casos raros (OTRO)
#-------------------------------------------
print("\n=== Resumen IdTipoViabilidad_SQL ===")
print(df["IdTipoViabilidad_SQL"].value_counts(dropna=False))

print("\n=== Resumen NombreTipoViabilidad_SQL ===")
print(df["NombreTipoViabilidad_SQL"].value_counts(dropna=False))

# Filas con texto fuera de catálogo (NombreTipoViabilidad_SQL comienza por 'OTRO:')
mask_otros = df["NombreTipoViabilidad_SQL"].str.startswith("OTRO:", na=False)
otros = df.loc[
    mask_otros,
    ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO",
     "Viabilidad_RAW", "NombreTipoViabilidad_SQL"]
]

print("\n=== Casos con texto de viabilidad fuera de catálogo (OTRO) ===")
if not otros.empty:
    print(otros.to_string(index=False))
else:
    print("No hay textos fuera de catálogo.")


Columnas candidatas para VIABILIDAD: ['VIABLE / NO VIABLE / PENDIENTE']
Usando columna: VIABLE / NO VIABLE / PENDIENTE


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4089030005.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Viabilidad_RAW"] = df[col_viab]



=== Resumen IdTipoViabilidad_SQL ===
IdTipoViabilidad_SQL
2    10081
0     7251
4     5411
1     1249
3       63
Name: count, dtype: int64

=== Resumen NombreTipoViabilidad_SQL ===
NombreTipoViabilidad_SQL
Viable                10081
Sin definir            7251
No viable              5411
Pendiente              1249
Viabilidad parcial       63
Name: count, dtype: int64

=== Casos con texto de viabilidad fuera de catálogo (OTRO) ===
No hay textos fuera de catálogo.


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4089030005.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[["IdTipoViabilidad_SQL", "NombreTipoViabilidad_SQL"]] = df["Viabilidad_RAW"].apply(
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4089030005.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[["IdTipoViabilidad_SQL", "NombreTipoViabilidad_SQL"]] = df["Viabilidad_RAW"].apply(


In [60]:
import pandas as pd

# ============================
# EXCEL DE REVISIÓN VIABILIDAD
# ============================

viab_mapeo = df[
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "Viabilidad_RAW",
        "IdTipoViabilidad_SQL",
        "NombreTipoViabilidad_SQL",
    ]
].copy()

mask_otros = df["NombreTipoViabilidad_SQL"].str.startswith("OTRO:", na=False)

viab_otros = df.loc[
    mask_otros,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "Viabilidad_RAW",
        "NombreTipoViabilidad_SQL",
    ],
].copy()

ruta_salida_viab = (
    r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos"
    r"\Migracion base de datos viablidad juridica\Viabilidad_revision.xlsx"
)

with pd.ExcelWriter(ruta_salida_viab, engine="openpyxl") as writer:
    viab_mapeo.to_excel(writer, sheet_name="Viabilidad_mapeo", index=False)
    if not viab_otros.empty:
        viab_otros.to_excel(writer, sheet_name="Viabilidad_OTROS", index=False)

print("Archivo de revisión de viabilidad generado en:", ruta_salida_viab)


Archivo de revisión de viabilidad generado en: C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\Viabilidad_revision.xlsx


## TIPO DE INFORME

In [61]:
import pandas as pd
import numpy as np
import unicodedata

# ============================================================
# 1) UBICAR COLUMNA DE TIPO DE INFORME Y CREAR COLUMNA RAW
# ============================================================

candidatas = [
    c for c in df.columns
    if "TIPO" in c.upper() and "INFORME" in c.upper()
]

print("Columnas candidatas para TIPO DE INFORME:", candidatas)

if not candidatas:
    raise ValueError("No se encontró columna para TIPO DE INFORME; revisa encabezados.")

col_tipo_inf = candidatas[0]
print("Usando columna:", col_tipo_inf)

# Copia EXACTA de lo que viene del Excel (no tocar nunca esta columna)
df["TipoInforme_RAW"] = df[col_tipo_inf]

# ============================================================
# 2) TEXTO LIMPIO PARA CLASIFICAR (mayúsculas, sin tildes, sin espacios extra)
# ============================================================

def limpiar_texto_tipo_inf(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None
    # colapsar espacios internos
    t = " ".join(t.split())
    # pasar a mayúsculas y quitar tildes
    t_up = t.upper()
    t_norm = unicodedata.normalize("NFD", t_up)
    t_sin_tilde = "".join(ch for ch in t_norm if unicodedata.category(ch) != "Mn")
    return t_sin_tilde

df["TipoInforme_TXT_LIMPIO"] = df["TipoInforme_RAW"].apply(limpiar_texto_tipo_inf)

print("\n=== Valores originales de TIPO DE INFORME (RAW) ===")
print(df["TipoInforme_RAW"].value_counts(dropna=False))

print("\n=== Valores normalizados (TXT_LIMPIO) de TIPO DE INFORME ===")
print(df["TipoInforme_TXT_LIMPIO"].value_counts(dropna=False))

# ============================================================
# 3) MAPEAR A CATÁLOGO (NUEVO / ACTUALIZACION) + TRATAR "SIN DATO"
# ============================================================

# Textos que significan "no tiene tipo de informe" -> se dejan como NULL
SIN_DATO = {
    "NO APLICA",
    "NO APLICA.",
    "NO REGISTRA",
    "NO REGISTRA.",
    "SIN INFORME",
    "SIN INFORMACION",
    "SIN INFORMACIÓN",
    "NINGUNO",
    "N/A",
    "NA"
}

def normalizar_tipo_informe(t):
    if t is None:
        return None

    # Casos de "sin dato"
    if t in SIN_DATO:
        return None

    # NUEVO: agrupar variantes
    if t in {
        "NUEVO",
        "INICIAL",
        "PRIMERA VEZ",
        "INFORME NUEVO",
        "NVO"
    }:
        return "NUEVO"

    # ACTUALIZACION: agrupar variantes
    if t in {
        "ACTUALIZACION",
        "ACTUALIZACION DEL INFORME",
        "ACTUALIZADO",
        "INFORME ACTUALIZADO",
        "ACTUALIZACION INFORME"
    }:
        return "ACTUALIZACION"

    # Cualquier otro texto queda sin clasificar (para revisión)
    return None

df["NombreTipoInforme_SQL"] = df["TipoInforme_TXT_LIMPIO"].apply(normalizar_tipo_informe)

# Ids de catálogo (ajustados a tu tabla Catalogo.TipoInforme)
mapa_tipo_informe = {
    "NUEVO": 1,
    "ACTUALIZACION": 2,
}

df["IdTipoInforme_SQL"] = df["NombreTipoInforme_SQL"].map(mapa_tipo_informe)

# ============================================================
# 4) RESÚMENES Y PENDIENTES DE CLASIFICAR
# ============================================================

print("\n=== Resumen NombreTipoInforme_SQL (catálogo) ===")
print(df["NombreTipoInforme_SQL"].value_counts(dropna=False))

print("\n=== Conteo IdTipoInforme_SQL ===")
print(df["IdTipoInforme_SQL"].value_counts(dropna=False))

# Filas NO clasificadas:
#   - tienen texto limpio
#   - pero NombreTipoInforme_SQL es None
mask_no_clas = (
    df["TipoInforme_TXT_LIMPIO"].notna()
    & (df["TipoInforme_TXT_LIMPIO"].str.len() > 0)
    & df["NombreTipoInforme_SQL"].isna()
)

no_clasificados = df.loc[
    mask_no_clas,
    [
        "Fila_Excel_aprox",
        "FMI",
        "DEPARTAMENTO",
        "MUNICIPIO",
        "TipoInforme_RAW",
        "TipoInforme_TXT_LIMPIO"
    ]
].copy()

print("\n=== Valores de TIPO DE INFORME no clasificados (para revisar) ===")
if not no_clasificados.empty:
    print(no_clasificados.to_string(index=False))
else:
    print("No hay valores pendientes de clasificar (todos NULL o mapeados a catálogo).")

# Resumen compacto de los textos pendientes
pendientes_resumen = (
    df.loc[mask_no_clas, "TipoInforme_TXT_LIMPIO"]
      .value_counts(dropna=False)
      .reset_index()
      .rename(columns={"index": "TipoInforme_TXT_LIMPIO", "TipoInforme_TXT_LIMPIO": "conteo"})
)

# ============================================================
# 5) EXCEL DE REVISIÓN (UN SOLO ARCHIVO, VARIAS HOJAS)
# ============================================================

# Mapeo completo (agrupado) para ver cómo quedan las combinaciones
resumen_mapeo = (
    df
      .groupby(
          ["TipoInforme_RAW",
           "TipoInforme_TXT_LIMPIO",
           "NombreTipoInforme_SQL",
           "IdTipoInforme_SQL"],
          dropna=False
      )
      .size()
      .reset_index(name="conteo")
)

ruta_salida = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\TipoInforme_revision.xlsx"

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    # Hoja 1: resumen del mapeo (RAW → limpio → SQL)
    resumen_mapeo.to_excel(writer, sheet_name="mapeo_tipo_informe", index=False)

    # Hoja 2: filas pendientes de clasificar
    no_clasificados.to_excel(writer, sheet_name="pendientes_clasificacion", index=False)

    # Hoja 3: resumen de textos pendientes (únicos + conteo)
    pendientes_resumen.to_excel(writer, sheet_name="pendientes_resumen", index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)


Columnas candidatas para TIPO DE INFORME: ['TIPO DE INFORME']
Usando columna: TIPO DE INFORME

=== Valores originales de TIPO DE INFORME (RAW) ===
TipoInforme_RAW
NaN              17144
NUEVO             5107
ACTUALIZACION     1800
NUEVO                4
Name: count, dtype: int64

=== Valores normalizados (TXT_LIMPIO) de TIPO DE INFORME ===
TipoInforme_TXT_LIMPIO
None             17144
NUEVO             5111
ACTUALIZACION     1800
Name: count, dtype: int64

=== Resumen NombreTipoInforme_SQL (catálogo) ===
NombreTipoInforme_SQL
None             17144
NUEVO             5111
ACTUALIZACION     1800
Name: count, dtype: int64

=== Conteo IdTipoInforme_SQL ===
IdTipoInforme_SQL
NaN    17144
1.0     5111
2.0     1800
Name: count, dtype: int64

=== Valores de TIPO DE INFORME no clasificados (para revisar) ===
No hay valores pendientes de clasificar (todos NULL o mapeados a catálogo).

Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documento

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4223604505.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["TipoInforme_RAW"] = df[col_tipo_inf]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4223604505.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["TipoInforme_TXT_LIMPIO"] = df["TipoInforme_RAW"].apply(limpiar_texto_tipo_inf)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4223604505.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

## "CAUSAL NO VIABILIDAD"


In [62]:
import pandas as pd
import numpy as np
import unicodedata

# ============================================
# CAUSAL NO VIABILIDAD  → IdTipoNoViabilidad_SQL
# ============================================

# 1) Localizar la columna de Excel
candidatas = [c for c in df.columns
              if 'CAUSAL' in c.upper() and 'VIABILIDAD' in c.upper()]

print("Columnas candidatas para CAUSAL NO VIABILIDAD:", candidatas)
if not candidatas:
    raise ValueError("No se encontró columna para CAUSAL NO VIABILIDAD.")

col_causal = candidatas[0]         # debería ser 'CAUSAL \n NO VIABILIDAD'
print("Usando columna:", col_causal)

# 2) Columna RAW = exactamente igual al Excel
df["CausalNoViab_RAW"] = df[col_causal]

# 3) Texto limpio para análisis/mapeo
def limpiar_causal(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if not t:
        return None
    # quitar tildes
    t_norm = unicodedata.normalize("NFD", t)
    t_norm = "".join(ch for ch in t_norm if unicodedata.category(ch) != "Mn")
    # normalizar espacios
    t_norm = " ".join(t_norm.split())
    return t_norm.upper()

df["CausalNoViab_TXT_LIMPIO"] = df["CausalNoViab_RAW"].apply(limpiar_causal)

print("\n=== Valores CRUDOS (RAW) de CAUSAL NO VIABILIDAD (top 30) ===")
print(df["CausalNoViab_RAW"].value_counts(dropna=False).head(30))

print("\n=== Valores NORMALIZADOS (TXT_LIMPIO) de CAUSAL NO VIABILIDAD (top 30) ===")
print(df["CausalNoViab_TXT_LIMPIO"].value_counts(dropna=False).head(30))

# 4) Diccionario → IdTipoNoViabilidad según catálogo de SQL Server
map_causal_a_id = {
    "PREDIO PRIVADO CON FALSA TRADICION": 1,
    "PRESUNTO BALDIO CON FALSA TRADICION": 2,
    "NUMERO DE FMI INCONSISTENTE": 3,
    "FOLIO CERRADO": 4,
    "PERTENENCIA - ART. 48 L160/94": 5,
    "ES UN URBANO": 6,
    "PRESENTA UNA MEDIDA CAUTELAR": 7,
    "URT": 8,
    "ESTA EN PROCESO DE EXTINCION DE DOMINIO": 9,
    "PROPIETARIO TITULAR FALLECIDO": 10,
    "SOCIEDAD COMERCIAL LIQUIDADA Y CANCELADA": 11,
    "ANTECEDENTES QUE INHABILITAN PARA CONTRATAR": 12,
    "FIDEICOMISO NO COMERCIALIZABLE": 13,
    "CON PROCESO AGRARIO": 14,
    # 15 no se usa
    "OTROS": 16,
    "PERTENENCIA - SU 288/2022": 17,
    "BALDIO": 18,
    "DEMANDA INSCRITA": 19,
}

# (opcional) nombre del catálogo para entender el Id
catalogo_tipo_no_viab = {
    0:  "Sin definir",
    1:  "PREDIO PRIVADO CON FALSA TRADICIÓN",
    2:  "PRESUNTO BALDÍO CON FALSA TRADICIÓN",
    3:  "NÚMERO DE FMI INCONSISTENTE",
    4:  "FOLIO CERRADO",
    5:  "PERTENENCIA - ART. 48 L160/94",
    6:  "ES UN URBANO",
    7:  "PRESENTA UNA MEDIDA CAUTELAR",
    8:  "URT",
    9:  "ESTÁ EN PROCESO DE EXTINCIÓN DE DOMINIO",
    10: "PROPIETARIO TITULAR FALLECIDO",
    11: "SOCIEDAD COMERCIAL LIQUIDADA Y CANCELADA",
    12: "ANTECEDENTES QUE INHABILITAN PARA CONTRATAR",
    13: "FIDEICOMISO NO COMERCIALIZABLE",
    14: "CON PROCESO AGRARIO",
    16: "OTROS",
    17: "PERTENENCIA - SU 288/2022",
    18: "BALDÍO",
    19: "DEMANDA INSCRITA",
}

# 5) Asignar IdTipoNoViabilidad_SQL
def asignar_id_tipo_no_viab(txt_norm):
    # txt_norm ya viene en mayúsculas sin tildes ni espacios raros
    if txt_norm is None or (isinstance(txt_norm, str) and txt_norm.strip() == ""):
        # Sin dato → 0 (sin causal / sin registro)
        return 0
    # Si está en el catálogo, usar ese Id
    if txt_norm in map_causal_a_id:
        return map_causal_a_id[txt_norm]
    # Si no está, queda NaN para marcar “texto pero sin clasificar”
    return np.nan

df["IdTipoNoViabilidad_SQL"] = df["CausalNoViab_TXT_LIMPIO"].apply(asignar_id_tipo_no_viab)
df["NombreTipoNoViabilidad_SQL"] = df["IdTipoNoViabilidad_SQL"].map(catalogo_tipo_no_viab)

print("\n=== Resumen IdTipoNoViabilidad_SQL ===")
print(df["IdTipoNoViabilidad_SQL"].value_counts(dropna=False).sort_index())

# 6) Preparar dataframes para Excel de revisión

# a) Valores distintos RAW / LIMPIO / Id
valores_distintos = (
    df[["CausalNoViab_RAW", "CausalNoViab_TXT_LIMPIO", "IdTipoNoViabilidad_SQL"]]
    .drop_duplicates()
    .sort_values(["CausalNoViab_TXT_LIMPIO", "CausalNoViab_RAW"], na_position="last")
)

# b) Bien clasificados: tienen Id y Id != 0
mask_bien = df["IdTipoNoViabilidad_SQL"].notna() & (df["IdTipoNoViabilidad_SQL"] != 0)
cols_base = [
    "Fila_Excel_aprox",
    "FMI",
    "DEPARTAMENTO",
    "MUNICIPIO",
    "CausalNoViab_RAW",
    "CausalNoViab_TXT_LIMPIO",
    "IdTipoNoViabilidad_SQL",
    "NombreTipoNoViabilidad_SQL",
]
cols_base = [c for c in cols_base if c in df.columns]

df_bien = df.loc[mask_bien, cols_base].copy().sort_values(
    by=["IdTipoNoViabilidad_SQL", "CausalNoViab_TXT_LIMPIO"],
    ascending=[True, True],
)

# c) Con texto pero sin clasificar: Id es NaN y texto limpio no vacío
mask_sin_clasif = (
    df["IdTipoNoViabilidad_SQL"].isna()
    & df["CausalNoViab_TXT_LIMPIO"].notna()
    & (df["CausalNoViab_TXT_LIMPIO"] != "")
)

df_sin = df.loc[mask_sin_clasif, cols_base].copy().sort_values(
    by=["CausalNoViab_TXT_LIMPIO", "FMI"],
    ascending=[True, True],
)

print("\nFilas bien clasificadas (Id != 0):", len(df_bien))
print("Filas con texto pero sin clasificar (Id NaN):", len(df_sin))

# 7) Exportar a un solo Excel con varias hojas
ruta_salida = (
    r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT"
    r"\Documentos\Migracion base de datos viablidad juridica\CausalNoViab_revision.xlsx"
)

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    valores_distintos.to_excel(writer, sheet_name="valores_distintos", index=False)
    df_bien.to_excel(writer, sheet_name="bien_clasificados", index=False)
    df_sin.to_excel(writer, sheet_name="sin_clasificar", index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)


Columnas candidatas para CAUSAL NO VIABILIDAD: ['CAUSAL \n NO VIABILIDAD']
Usando columna: CAUSAL 
 NO VIABILIDAD

=== Valores CRUDOS (RAW) de CAUSAL NO VIABILIDAD (top 30) ===
CausalNoViab_RAW
NaN                                            20944
PERTENENCIA - ART. 48 L160/94                    475
Propietario titular fallecido                    458
Presunto baldío con falsa tradición              402
Folio cerrado                                    234
Es un presunto baldío con falsa tradición        214
Predio privado con falsa tradición               166
Número de FMI inconsistente                      163
Presenta una medida cautelar                     155
Es un urbano                                     154
Otros                                            143
PERTENENCIA - SU 288/2022                        127
BALDIO                                           116
URT                                              107
DEMANDA INSCRITA                                  74
Con proceso

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\389897886.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["CausalNoViab_RAW"] = df[col_causal]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\389897886.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["CausalNoViab_TXT_LIMPIO"] = df["CausalNoViab_RAW"].apply(limpiar_causal)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\389897886.py:103: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram


Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\CausalNoViab_revision.xlsx


## INSUMOS PENDIENTES 

In [63]:
import pandas as pd
import numpy as np

# =====================================================
# INSUMOS PENDIENTES  → InsumosPendientes_SQL (varchar(MAX))
# =====================================================

# 1) Detectar la columna ORIGINAL de Excel para INSUMOS PENDIENTES
candidatas_insumos = [
    c for c in df.columns
    if ('INSUMOS' in c.upper() and 'PENDIENT' in c.upper())
]

print("Columnas candidatas para INSUMOS PENDIENTES (antes de filtrar):", candidatas_insumos)

# Excluir columnas derivadas que ya creamos en Python
candidatas_insumos = [
    c for c in candidatas_insumos
    if c not in [
        'InsumosPendientes_RAW',
        'InsumosPendientes_SQL',
        'InsumosPendientes_LEN',
    ]
    and not c.upper().startswith('INSUMOSPENDIENTES_')
]

# Dar prioridad a la que coincide con el encabezado original del Excel
prioritarias = [c for c in candidatas_insumos if c.strip().upper() == 'INSUMOS PENDIENTES']

if prioritarias:
    col_insumos = prioritarias[0]
else:
    if not candidatas_insumos:
        raise ValueError("No se encontró la columna ORIGINAL de INSUMOS PENDIENTES.")
    col_insumos = candidatas_insumos[0]

print("Usando columna de Excel:", col_insumos)

# 2) Columna RAW = exactamente igual a la del Excel
df['InsumosPendientes_RAW'] = df[col_insumos]

# 3) Normalización ligera para la columna que va a SQL
def normalizar_insumos(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None
    
    t_up = t.upper()
    # Textos que significan "no tiene insumos pendientes" → NULL en SQL
    placeholders = {
        "NO",
        "NO APLICA",
        "NO APLICA.",
        "NO REGISTRA",
        "NINGUNO",
        "NINGUNA",
    }
    if t_up in placeholders:
        return None  # se migra como NULL

    # Si llega aquí es texto “real”, se deja tal cual
    return t

df['InsumosPendientes_SQL'] = df['InsumosPendientes_RAW'].apply(normalizar_insumos)

# 4) Longitud (para revisar textos muy largos, aunque sea varchar(MAX))
df['InsumosPendientes_LEN'] = df['InsumosPendientes_SQL'].fillna('').astype(str).str.len()

print("=== Resumen INSUMOS PENDIENTES (SQL no nulo vs nulo) ===")
print(df['InsumosPendientes_SQL'].isna().value_counts(dropna=False))

print("\n=== Resumen estadístico de longitudes (InsumosPendientes_LEN) ===")
print(df['InsumosPendientes_LEN'].describe())

# 5) Asegurar columna de fila aproximada
if 'Fila_Excel_aprox' not in df.columns:
    df['Fila_Excel_aprox'] = df.index + 2

# 6) DataFrames para revisión

# a) Valores distintos RAW / SQL (evitando el problema de tipos al ordenar)
valores_distintos = (
    df[['InsumosPendientes_RAW', 'InsumosPendientes_SQL']]
      .drop_duplicates()
)

valores_distintos = valores_distintos.sort_values(
    by=['InsumosPendientes_SQL', 'InsumosPendientes_RAW'],
    key=lambda col: col.astype(str),   # forzar a string para evitar mezcla int/datetime
    na_position='last'
)

# Columnas de contexto si existen
contexto = ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO']
contexto = [c for c in contexto if c in df.columns]

cols_detalle = contexto + [
    'InsumosPendientes_RAW',
    'InsumosPendientes_SQL',
    'InsumosPendientes_LEN',
]

# b) Registros con texto real (SQL no nulo)
df_con_texto = df[df['InsumosPendientes_SQL'].notna()][cols_detalle].copy()

# c) Registros donde en el Excel hay algo pero terminó como NULL (placeholders)
mask_placeholders = (
    df['InsumosPendientes_RAW'].notna()
    & df['InsumosPendientes_SQL'].isna()
)

df_placeholders = df[mask_placeholders][cols_detalle].copy()

print("\nFilas con texto real (SQL no nulo):", len(df_con_texto))
print("Filas con placeholders (RAW no nulo pero SQL nulo):", len(df_placeholders))

# 7) Exportar a un solo Excel con varias hojas de revisión
ruta_salida = (
    r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT"
    r"\Documentos\Migracion base de datos viablidad juridica\InsumosPendientes_revision.xlsx"
)

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    valores_distintos.to_excel(writer, sheet_name="valores_distintos", index=False)
    df_con_texto.to_excel(writer, sheet_name="con_texto_SQL", index=False)
    df_placeholders.to_excel(writer, sheet_name="placeholders_a_NULL", index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)


Columnas candidatas para INSUMOS PENDIENTES (antes de filtrar): ['INSUMOS PENDIENTES ']
Usando columna de Excel: INSUMOS PENDIENTES 
=== Resumen INSUMOS PENDIENTES (SQL no nulo vs nulo) ===
InsumosPendientes_SQL
True     19559
False     4496
Name: count, dtype: int64

=== Resumen estadístico de longitudes (InsumosPendientes_LEN) ===
count    24055.000000
mean        29.703637
std        203.930420
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      16831.000000
Name: InsumosPendientes_LEN, dtype: float64

Filas con texto real (SQL no nulo): 4496
Filas con placeholders (RAW no nulo pero SQL nulo): 81


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4233240920.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['InsumosPendientes_RAW'] = df[col_insumos]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4233240920.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['InsumosPendientes_SQL'] = df['InsumosPendientes_RAW'].apply(normalizar_insumos)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\4233240920.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o


Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\InsumosPendientes_revision.xlsx


## FECHA DE ENTREGA A REVISOR

In [64]:
import pandas as pd
import numpy as np
import datetime as dt

# 1) Detectar columna de Excel para FECHA DE ENTREGA A REVISOR
candidatas = [
    c for c in df.columns
    if 'ENTREGA' in c.upper() and 'REVISOR' in c.upper()
]

print("Columnas candidatas para FECHA ENTREGA A REVISOR:", candidatas)

if not candidatas:
    raise ValueError("No se encontró la columna de FECHA DE ENTREGA A REVISOR en el DataFrame.")

col_fecha_entrega = candidatas[0]
print("Usando columna:", col_fecha_entrega)

# 2) Columna cruda
df['FechaEntregaRevisor_RAW'] = df[col_fecha_entrega]

# 3) Función para pasar a texto “limpio”
def a_texto_fecha(v):
    if pd.isna(v):
        return ""
    # Si viene como datetime / Timestamp de Excel
    if isinstance(v, (pd.Timestamp, dt.datetime, dt.date)):
        # La guardamos en formato DD/MM/YYYY para visualizar
        return v.strftime("%d/%m/%Y")
    return str(v).strip()

df['FECHA_ENTREGA_REV_TXT'] = df['FechaEntregaRevisor_RAW'].apply(a_texto_fecha)

# 4) Normalizar texto: quitar placeholders y “00:00:00”, etc.
PLACEHOLDERS_NO_FECHA = {
    "NO", "NO.", "NO APLICA", "NO APLICA.", "NO REGISTRA", "NO REGISTRA.",
    "SIN FECHA", "SIN INFORME", "N/A", "NA", "NINGUNO", "NINGUNA"
}

def limpiar_texto_fecha_entrega(s: str) -> str:
    s = s.strip()
    if not s:
        return ""
    su = s.upper()
    # Placeholders que significan "no hay fecha"
    if su in PLACEHOLDERS_NO_FECHA:
        return ""
    # Si viene con hora al final tipo "2024-04-25 00:00:00"
    if " " in s:
        # nos quedamos con la primera parte (fecha)
        s = s.split()[0].strip()
    return s

df['FECHA_ENTREGA_REV_TXT_LIMPIO'] = df['FECHA_ENTREGA_REV_TXT'].apply(limpiar_texto_fecha_entrega)

# 5) Parsear a datetime seguro (día primero)
df['FechaEntregaARevisor_SQL'] = pd.to_datetime(
    df['FECHA_ENTREGA_REV_TXT_LIMPIO'],
    dayfirst=True,
    errors='coerce'
)

# 6) Clasificar motivo de nulos (para revisar qué pasa con los NaT)
def motivo_fecha_nula(raw, txt_limpio, fecha_sql):
    # fecha válida
    if pd.notna(fecha_sql):
        return None
    # todo vacío
    if (raw is None or (isinstance(raw, float) and pd.isna(raw))) and txt_limpio == "":
        return "NaN"
    # era placeholder tipo NO / NO APLICA / etc.
    if txt_limpio == "" and isinstance(raw, str) and raw.strip() != "":
        return "Placeholder (NO/NO APLICA/NO REGISTRA/...)"
    # texto raro que no se pudo convertir
    if txt_limpio != "":
        return "Otro texto raro (posible fecha mal escrita)"
    return "NaN"

df['MotivoFechaEntregaRevisorNula'] = df.apply(
    lambda r: motivo_fecha_nula(
        r['FechaEntregaRevisor_RAW'],
        r['FECHA_ENTREGA_REV_TXT_LIMPIO'],
        r['FechaEntregaARevisor_SQL']
    ),
    axis=1
)

# 7) Resumen
total = len(df)
validas = df['FechaEntregaARevisor_SQL'].notna().sum()
nulas = df['FechaEntregaARevisor_SQL'].isna().sum()
print(f"Total filas: {total}")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas nulas/NaT       : {nulas}")

print("\n=== Resumen de motivos para FechaEntregaARevisor_SQL = NaT ===")
print(df.loc[df['FechaEntregaARevisor_SQL'].isna(), 'MotivoFechaEntregaRevisorNula']
        .value_counts(dropna=False))

# 8) Muestra de valores (original vs normalizado) para revisar que la lógica esté bien
muestra = df.loc[
    df['FechaEntregaRevisor_RAW'].notna() | df['FechaEntregaARevisor_SQL'].notna(),
    [
        'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
        'FechaEntregaRevisor_RAW',
        'FECHA_ENTREGA_REV_TXT_LIMPIO',
        'FechaEntregaARevisor_SQL'
    ]
].head(40)

print("\n=== Muestra FECHA ENTREGA A REVISOR (RAW vs TXT_LIMPIO vs SQL) ===")
print(muestra.to_string(index=False))

# 9) Filas con texto pero que no se pudieron convertir (para revisión manual)
invalidas = df[
    df['FechaEntregaARevisor_SQL'].isna()
    & (df['FECHA_ENTREGA_REV_TXT_LIMPIO'] != "")
]

print("\n=== Ejemplos de FECHA ENTREGA A REVISOR con texto que no se pudo parsear ===")
print(
    invalidas[
        ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
         'FechaEntregaRevisor_RAW', 'FECHA_ENTREGA_REV_TXT_LIMPIO']
    ].head(40).to_string(index=False)
)


Columnas candidatas para FECHA ENTREGA A REVISOR: ['FECHA DE ENTREGA A REVISOR', 'PLAZO PARA ENTREGA A REVISOR']
Usando columna: FECHA DE ENTREGA A REVISOR


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\1340245575.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaEntregaRevisor_RAW'] = df[col_fecha_entrega]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\1340245575.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FECHA_ENTREGA_REV_TXT'] = df['FechaEntregaRevisor_RAW'].apply(a_texto_fecha)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_35928\1340245575.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

Total filas: 24055
Fechas válidas (no NaT): 6223
Fechas nulas/NaT       : 17832

=== Resumen de motivos para FechaEntregaARevisor_SQL = NaT ===
MotivoFechaEntregaRevisorNula
NaN                                            17830
Placeholder (NO/NO APLICA/NO REGISTRA/...)         1
Otro texto raro (posible fecha mal escrita)        1
Name: count, dtype: int64

=== Muestra FECHA ENTREGA A REVISOR (RAW vs TXT_LIMPIO vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO                   MUNICIPIO FechaEntregaRevisor_RAW FECHA_ENTREGA_REV_TXT_LIMPIO FechaEntregaARevisor_SQL
                2  062-37357      BOLIVAR                    ZAMBRANO     2024-10-17 00:00:00                   17/10/2024               2024-10-17
                4  062-37361      BOLIVAR                    ZAMBRANO     2024-09-30 00:00:00                   30/09/2024               2024-09-30
                5  062-37354      BOLIVAR                    ZAMBRANO     2025-01-29 00:00:00                   29/01/2025       

In [65]:
import pandas as pd
import numpy as np
import datetime as dt

# ======================================================
# FECHA DE ENTREGA A REVISOR → FechaEntregaARevisor_SQL
# ======================================================

# 1) Detectar columna de Excel para FECHA DE ENTREGA A REVISOR
candidatas = [
    c for c in df.columns
    if 'ENTREGA' in c.upper() and 'REVISOR' in c.upper()
]

print("Columnas candidatas para FECHA ENTREGA A REVISOR:", candidatas)

if not candidatas:
    raise ValueError("No se encontró la columna de FECHA DE ENTREGA A REVISOR en el DataFrame.")

col_fecha_entrega = candidatas[0]
print("Usando columna:", col_fecha_entrega)

# 2) Columna RAW: EXACTAMENTE igual al Excel
df['FechaEntregaRevisor_RAW'] = df[col_fecha_entrega]

# 3) Pasar a texto “visible”
def a_texto_fecha(v):
    if pd.isna(v):
        return ""
    # Si viene como datetime / Timestamp de Excel
    if isinstance(v, (pd.Timestamp, dt.datetime, dt.date)):
        # La guardamos en formato DD/MM/YYYY solo para revisarla
        return v.strftime("%d/%m/%Y")
    return str(v).strip()

df['FECHA_ENTREGA_REV_TXT'] = df['FechaEntregaRevisor_RAW'].apply(a_texto_fecha)

# 4) Limpiar texto: quitar placeholders y horas
PLACEHOLDERS_NO_FECHA = {
    "NO", "NO.", "NO APLICA", "NO APLICA.", "NO REGISTRA", "NO REGISTRA.",
    "SIN FECHA", "SIN INFORME", "N/A", "NA", "NINGUNO", "NINGUNA"
}

def limpiar_texto_fecha_entrega(s: str) -> str:
    s = s.strip()
    if not s:
        return ""
    su = s.upper()
    # Placeholders que significan "no hay fecha"
    if su in PLACEHOLDERS_NO_FECHA:
        return ""
    # Si viene con hora al final tipo "2024-04-25 00:00:00" o "25/04/2024 0:00:00"
    if " " in s:
        s = s.split()[0].strip()
    return s

df['FECHA_ENTREGA_REV_TXT_LIMPIO'] = df['FECHA_ENTREGA_REV_TXT'].apply(limpiar_texto_fecha_entrega)

# 5) Parsear a datetime (día primero)
df['FechaEntregaARevisor_SQL'] = pd.to_datetime(
    df['FECHA_ENTREGA_REV_TXT_LIMPIO'],
    dayfirst=True,
    errors='coerce'
)

# 6) Clasificar motivo de nulos
def motivo_fecha_nula(raw, txt_limpio, fecha_sql):
    # fecha válida
    if pd.notna(fecha_sql):
        return None
    # todo vacío
    if ((raw is None or (isinstance(raw, float) and pd.isna(raw))) and txt_limpio == ""):
        return "NaN"
    # era placeholder tipo NO / NO APLICA / etc.
    if txt_limpio == "" and isinstance(raw, str) and raw.strip() != "":
        return "Placeholder (NO/NO APLICA/NO REGISTRA/...)"
    # texto raro que no se pudo convertir
    if txt_limpio != "":
        return "Otro texto raro (posible fecha mal escrita)"
    return "NaN"

df['MotivoFechaEntregaRevisorNula'] = df.apply(
    lambda r: motivo_fecha_nula(
        r['FechaEntregaRevisor_RAW'],
        r['FECHA_ENTREGA_REV_TXT_LIMPIO'],
        r['FechaEntregaARevisor_SQL']
    ),
    axis=1
)

# 7) Resumen rápido en consola
total = len(df)
validas = df['FechaEntregaARevisor_SQL'].notna().sum()
nulas = df['FechaEntregaARevisor_SQL'].isna().sum()
print(f"Total filas: {total}")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas nulas/NaT       : {nulas}")

print("\n=== Resumen de motivos para FechaEntregaARevisor_SQL = NaT ===")
print(
    df.loc[df['FechaEntregaARevisor_SQL'].isna(), 'MotivoFechaEntregaRevisorNula']
      .value_counts(dropna=False)
)

# 8) DataFrames para revisión y Excel con varias hojas

# Asegurar columna de fila aproximada
if 'Fila_Excel_aprox' not in df.columns:
    df['Fila_Excel_aprox'] = df.index + 2

cols_contexto = [c for c in ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO'] if c in df.columns]

cols_rev = cols_contexto + [
    'FechaEntregaRevisor_RAW',
    'FECHA_ENTREGA_REV_TXT_LIMPIO',
    'FechaEntregaARevisor_SQL',
    'MotivoFechaEntregaRevisorNula'
]

# a) Fechas válidas
df_validas = df[df['FechaEntregaARevisor_SQL'].notna()][cols_rev].copy()

# b) Todas las filas nulas
df_nulas = df[df['FechaEntregaARevisor_SQL'].isna()][cols_rev].copy()

# c) Solo texto raro (no placeholder y no vacío)
mask_texto_raro = (
    df['FechaEntregaARevisor_SQL'].isna()
    & (df['FECHA_ENTREGA_REV_TXT_LIMPIO'] != "")
    & (df['MotivoFechaEntregaRevisorNula'] == "Otro texto raro (posible fecha mal escrita)")
)
df_texto_raro = df[mask_texto_raro][cols_rev].copy()

# d) Resumen de motivos en tabla
resumen_motivos = (
    df.loc[df['FechaEntregaARevisor_SQL'].isna(), 'MotivoFechaEntregaRevisorNula']
      .value_counts(dropna=False)
      .rename_axis('MotivoFechaEntregaRevisorNula')
      .reset_index(name='conteo')
)

# 9) Exportar a un solo Excel con varias hojas
ruta_salida = (
    r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT"
    r"\Documentos\Migracion base de datos viablidad juridica\FechaEntregaRevisor_revision.xlsx"
)

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    df_validas.to_excel(writer, sheet_name="fechas_validas", index=False)
    df_nulas.to_excel(writer, sheet_name="fechas_nulas", index=False)
    df_texto_raro.to_excel(writer, sheet_name="texto_raro", index=False)
    resumen_motivos.to_excel(writer, sheet_name="resumen_motivos", index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)


Columnas candidatas para FECHA ENTREGA A REVISOR: ['FECHA DE ENTREGA A REVISOR', 'PLAZO PARA ENTREGA A REVISOR', 'FechaEntregaRevisor_RAW', 'FechaEntregaARevisor_SQL', 'MotivoFechaEntregaRevisorNula']
Usando columna: FECHA DE ENTREGA A REVISOR
Total filas: 24055
Fechas válidas (no NaT): 6223
Fechas nulas/NaT       : 17832

=== Resumen de motivos para FechaEntregaARevisor_SQL = NaT ===
MotivoFechaEntregaRevisorNula
NaN                                            17830
Placeholder (NO/NO APLICA/NO REGISTRA/...)         1
Otro texto raro (posible fecha mal escrita)        1
Name: count, dtype: int64

Archivo de revisión generado en:
C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\FechaEntregaRevisor_revision.xlsx


## ABOGADO  SUSTANCIADOR ASIGNADO


In [66]:
import pandas as pd
import numpy as np

# Opcional pero recomendable para reducir los PerformanceWarning
df = df.copy()

# 1) Detectar la columna del Excel
candidatas = [
    c for c in df.columns
    if 'ABOGADO' in str(c).upper() and 'SUSTANCIADOR' in str(c).upper()
]

print("Columnas candidatas para ABOGADO SUSTANCIADOR:", candidatas)
if not candidatas:
    raise ValueError("No se encontró la columna de ABOGADO SUSTANCIADOR ASIGNADO en el DataFrame.")

col_abog = candidatas[0]
print("Usando columna:", col_abog)

# 2) Guardar valor crudo (tal cual viene del Excel)
df['AbogadoSustanciadorAsignado_RAW'] = df[col_abog]

# 3) Normalizar texto para la columna SQL
def normalizar_abogado_sust(v):
    if pd.isna(v):
        return None      # quedará como NULL en SQL
    t = str(v).strip()
    if t == "":
        return None
    # colapsar espacios internos múltiples
    t = " ".join(t.split())
    # aquí NO cambiamos a mayúsculas si quieres conservar el formato original de nombres
    # si prefieres todo en mayúscula, descomenta la siguiente línea:
    # t = t.upper()
    return t

df['AbogadoSustanciadorAsignado_SQL'] = df['AbogadoSustanciadorAsignado_RAW'].apply(normalizar_abogado_sust)

# 4) Longitud para verificar contra varchar(1000)
df['AbogadoSustanciadorAsignado_Len'] = df['AbogadoSustanciadorAsignado_SQL'].fillna("").str.len()

print("=== Resumen longitudes ABOGADO SUSTANCIADOR ASIGNADO ===")
print(df['AbogadoSustanciadorAsignado_Len'].describe())

# 5) Registros que superen los 1000 caracteres (si existen)
mas_de_1000 = df[df['AbogadoSustanciadorAsignado_Len'] > 1000][
    [
        'Fila_Excel_aprox',
        'FMI',
        'DEPARTAMENTO',
        'MUNICIPIO',
        'AbogadoSustanciadorAsignado_RAW',
        'AbogadoSustanciadorAsignado_SQL',
        'AbogadoSustanciadorAsignado_Len'
    ]
]

if not mas_de_1000.empty:
    print("\n=== Registros que superan 1000 caracteres (revisar) ===")
    print(mas_de_1000.to_string(index=False))
else:
    print("\nNingún registro supera los 1000 caracteres (ok para varchar(1000)).")

# 6) Muestra rápida RAW vs SQL
muestra = df.loc[
    df['AbogadoSustanciadorAsignado_RAW'].notna(),
    [
        'Fila_Excel_aprox',
        'FMI',
        'DEPARTAMENTO',
        'MUNICIPIO',
        'AbogadoSustanciadorAsignado_RAW',
        'AbogadoSustanciadorAsignado_SQL',
        'AbogadoSustanciadorAsignado_Len'
    ]
].head(30)

print("\n=== Muestra ABOGADO SUSTANCIADOR (RAW vs SQL) ===")
print(muestra.to_string(index=False))

# 7) Excel de revisión (consolidado)
columnas_exportar = [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    'AbogadoSustanciadorAsignado_RAW',
    'AbogadoSustanciadorAsignado_SQL',
    'AbogadoSustanciadorAsignado_Len'
]

columnas_exportar = [c for c in columnas_exportar if c in df.columns]
df_export = df[columnas_exportar].copy()

ruta_salida = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\AbogadoSustanciador_revision.xlsx"
df_export.to_excel(ruta_salida, index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)


Columnas candidatas para ABOGADO SUSTANCIADOR: ['ABOGADO  SUSTANCIADOR ASIGNADO']
Usando columna: ABOGADO  SUSTANCIADOR ASIGNADO
=== Resumen longitudes ABOGADO SUSTANCIADOR ASIGNADO ===
count    24055.000000
mean        19.189981
std         12.901946
min          0.000000
25%          0.000000
50%         24.000000
75%         30.000000
max         37.000000
Name: AbogadoSustanciadorAsignado_Len, dtype: float64

Ningún registro supera los 1000 caracteres (ok para varchar(1000)).

=== Muestra ABOGADO SUSTANCIADOR (RAW vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO            MUNICIPIO   AbogadoSustanciadorAsignado_RAW   AbogadoSustanciadorAsignado_SQL  AbogadoSustanciadorAsignado_Len
                2  062-37357      BOLIVAR             ZAMBRANO   VICTOR EDUARDO TORRES HERNANDEZ   VICTOR EDUARDO TORRES HERNANDEZ                               31
                3  062-37359      BOLIVAR             ZAMBRANO            ALBEIRO ROJAS AHUMADA              ALBEIRO ROJAS AHUMADA      

## ABOGADO REVISOR ASIGNADO


In [67]:
import pandas as pd
import numpy as np

# Opcional: ayuda un poco con las PerformanceWarning de fragmentación
df = df.copy()

# 1) Detectar la columna del Excel
candidatas_rev = [
    c for c in df.columns
    if 'ABOGADO' in str(c).upper() and 'REVISOR' in str(c).upper()
]

print("Columnas candidatas para ABOGADO REVISOR:", candidatas_rev)
if not candidatas_rev:
    raise ValueError("No se encontró la columna de ABOGADO REVISOR ASIGNADO en el DataFrame.")

col_abog_rev = candidatas_rev[0]
print("Usando columna:", col_abog_rev)

# 2) Guardar valor crudo (tal cual viene del Excel)
df['AbogadoRevisorAsignado_RAW'] = df[col_abog_rev]

# 3) Normalizar texto para la columna SQL
def normalizar_abogado_revisor(v):
    if pd.isna(v):
        return None      # quedará como NULL en SQL
    t = str(v).strip()
    if t == "":
        return None
    # colapsar espacios internos múltiples
    t = " ".join(t.split())
    # NO cambiamos a mayúsculas para conservar el formato de nombres
    # si quisieras todo en mayúsculas, descomenta:
    # t = t.upper()
    return t

df['AbogadoRevisorAsignado_SQL'] = df['AbogadoRevisorAsignado_RAW'].apply(normalizar_abogado_revisor)

# 4) Longitud para verificar contra varchar(1000)
df['AbogadoRevisor_Len'] = df['AbogadoRevisorAsignado_SQL'].fillna("").str.len()

print("=== Resumen longitudes ABOGADO REVISOR ASIGNADO ===")
print(df['AbogadoRevisor_Len'].describe())

# 5) Registros que superen los 1000 caracteres (si existen)
largos_rev = df[df['AbogadoRevisor_Len'] > 1000][
    [
        'Fila_Excel_aprox',
        'FMI',
        'DEPARTAMENTO',
        'MUNICIPIO',
        'AbogadoRevisorAsignado_RAW',
        'AbogadoRevisorAsignado_SQL',
        'AbogadoRevisor_Len'
    ]
]

if not largos_rev.empty:
    print("\n=== Registros que superan 1000 caracteres (REVISOR) ===")
    print(largos_rev.to_string(index=False))
else:
    print("\nNingún registro de ABOGADO REVISOR supera los 1000 caracteres (ok para varchar(1000)).")

# 6) Muestra rápida RAW vs SQL
muestra_rev = df.loc[
    df['AbogadoRevisorAsignado_RAW'].notna(),
    [
        'Fila_Excel_aprox',
        'FMI',
        'DEPARTAMENTO',
        'MUNICIPIO',
        'AbogadoRevisorAsignado_RAW',
        'AbogadoRevisorAsignado_SQL',
        'AbogadoRevisor_Len'
    ]
].head(30)

print("\n=== Muestra ABOGADO REVISOR (RAW vs SQL) ===")
print(muestra_rev.to_string(index=False))

# 7) Excel de revisión (consolidado)
columnas_exportar = [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    'AbogadoRevisorAsignado_RAW',
    'AbogadoRevisorAsignado_SQL',
    'AbogadoRevisor_Len'
]

columnas_exportar = [c for c in columnas_exportar if c in df.columns]
df_export = df[columnas_exportar].copy()

ruta_salida = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\AbogadoRevisor_revision.xlsx"
df_export.to_excel(ruta_salida, index=False)

print("\nArchivo de revisión generado en:")
print(ruta_salida)


Columnas candidatas para ABOGADO REVISOR: ['ABOGADO REVISOR ASIGNADO']
Usando columna: ABOGADO REVISOR ASIGNADO
=== Resumen longitudes ABOGADO REVISOR ASIGNADO ===
count    24055.000000
mean        12.432052
std         14.495511
min          0.000000
25%          0.000000
50%          0.000000
75%         30.000000
max         33.000000
Name: AbogadoRevisor_Len, dtype: float64

Ningún registro de ABOGADO REVISOR supera los 1000 caracteres (ok para varchar(1000)).

=== Muestra ABOGADO REVISOR (RAW vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO           MUNICIPIO       AbogadoRevisorAsignado_RAW      AbogadoRevisorAsignado_SQL  AbogadoRevisor_Len
                2  062-37357      BOLIVAR            ZAMBRANO     CINDY ALEXANDRA MORA FAJARDO    CINDY ALEXANDRA MORA FAJARDO                  28
                3  062-37359      BOLIVAR            ZAMBRANO   JEYSON RAUL BERMUDEZ ECHEVERRI  JEYSON RAUL BERMUDEZ ECHEVERRI                  30
                4  062-37361      BOLIVAR   

## # REPARTO


In [68]:
import pandas as pd
import numpy as np
import re

# 1) Ubicar la columna de Excel para # REPARTO
candidatas_rep = [c for c in df.columns if 'REPARTO' in c.upper()]
print("Columnas candidatas para # REPARTO:", candidatas_rep)

if not candidatas_rep:
    raise ValueError("No se encontró la columna de # REPARTO en el DataFrame.")

# Si solo hay una, la usamos; si hay varias, puedes ajustar el índice
col_rep = candidatas_rep[0]
print("Usando columna:", col_rep)

# 2) Guardar valor crudo
df['NroReparto_RAW'] = df[col_rep]

# 3) Función para normalizar / parsear a entero
def parse_nro_reparto(v):
    # Vacío o nulo
    if pd.isna(v):
        return (None, 'vacio')
    t = str(v).strip()
    if t == '':
        return (None, 'vacio')
    
    t_up = t.upper()
    # Textos que significan "no aplica / sin reparto"
    if t_up in [
        'NO APLICA', 'NO', 'N/A', 'NA',
        'NINGUNO', 'SIN REPARTO', 'SIN REGISTRO'
    ]:
        return (None, 'sin_reparto_texto')
    
    # Extraer solo dígitos
    solo_digitos = re.sub(r'[^0-9]', '', t)
    if solo_digitos == '':
        return (None, 'sin_digitos')
    
    try:
        n = int(solo_digitos)
    except Exception:
        return (None, 'error_conversion')
    
    # Reglas simples de validación numérica
    if n <= 0:
        return (None, 'no_positivo')
    # Por si acaso aparece algo absurdo
    if n > 2_147_483_647:  # límite INT de SQL Server
        return (None, 'demasiado_grande')
    
    return (n, 'ok')


# 4) Aplicar al DataFrame: NroReparto_SQL (entero) + Estado_NroReparto (clasificación)
df[['NroReparto_SQL', 'Estado_NroReparto']] = df['NroReparto_RAW'].apply(
    lambda v: pd.Series(parse_nro_reparto(v))
)

# 5) Resumen de estados
print("=== Conteo por estado de NroReparto ===")
print(df['Estado_NroReparto'].value_counts(dropna=False))

# 6) Estadísticos solo para los valores OK (los que se van a migrar como INT)
ok_mask = df['Estado_NroReparto'] == 'ok'
print("\n=== Estadísticos NroReparto_SQL (solo estado 'ok') ===")
print(df.loc[ok_mask, 'NroReparto_SQL'].describe())

# 7) Ejemplos de registros OK (para ver original vs normalizado)
ej_ok = df.loc[ok_mask, [
    'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'NroReparto_RAW', 'NroReparto_SQL'
]].head(30)

print("\n=== Ejemplos NroReparto OK (RAW vs SQL) ===")
print(ej_ok.to_string(index=False))

# 8) Ejemplos de registros problemáticos, por cada categoría distinta de 'ok' y 'vacio'
categorias_problema = df['Estado_NroReparto'].unique()
categorias_problema = [c for c in categorias_problema if c not in ['ok', 'vacio']]

for cat in categorias_problema:
    sub = df.loc[df['Estado_NroReparto'] == cat, [
        'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
        'NroReparto_RAW', 'Estado_NroReparto'
    ]].head(20)
    print(f"\n=== Ejemplos estado '{cat}' ===")
    if not sub.empty:
        print(sub.to_string(index=False))
    else:
        print("Sin ejemplos.")


Columnas candidatas para # REPARTO: ['# REPARTO', 'FECHA ASIGNACIÓN REPARTO']
Usando columna: # REPARTO
=== Conteo por estado de NroReparto ===
Estado_NroReparto
vacio    20388
ok        3667
Name: count, dtype: int64

=== Estadísticos NroReparto_SQL (solo estado 'ok') ===
count    3667.000000
mean       33.199073
std        10.925212
min         1.000000
25%        29.000000
50%        36.000000
75%        40.000000
max        43.000000
Name: NroReparto_SQL, dtype: float64

=== Ejemplos NroReparto OK (RAW vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO                   MUNICIPIO NroReparto_RAW  NroReparto_SQL
                3  062-37359      BOLIVAR                    ZAMBRANO             43            43.0
               10  148-28711      CORDOBA                PUEBLO NUEVO             43            43.0
               13  148-16873      CORDOBA                PLANETA RICA            PA2             2.0
               14  340-14462        SUCRE                  SAN ONOFRE   

In [69]:
import pandas as pd
import numpy as np
import re

# Opcional: ayuda un poco con los PerformanceWarning de fragmentación
df = df.copy()

# 1) Ubicar la columna de Excel para # REPARTO
candidatas_rep = [c for c in df.columns if 'REPARTO' in str(c).upper()]
print("Columnas candidatas para # REPARTO:", candidatas_rep)

if not candidatas_rep:
    raise ValueError("No se encontró la columna de # REPARTO en el DataFrame.")

col_rep = candidatas_rep[0]
print("Usando columna:", col_rep)

# 2) Función para normalizar / parsear a entero
def parse_nro_reparto(v):
    # Vacío o nulo
    if pd.isna(v):
        return (None, 'vacio')
    t = str(v).strip()
    if t == '':
        return (None, 'vacio')
    
    t_up = t.upper()
    # Textos que significan "no aplica / sin reparto"
    if t_up in [
        'NO APLICA', 'NO', 'N/A', 'NA',
        'NINGUNO', 'SIN REPARTO', 'SIN REGISTRO'
    ]:
        return (None, 'sin_reparto_texto')
    
    # Extraer solo dígitos
    solo_digitos = re.sub(r'[^0-9]', '', t)
    if solo_digitos == '':
        return (None, 'sin_digitos')
    
    try:
        n = int(solo_digitos)
    except Exception:
        return (None, 'error_conversion')
    
    # Reglas simples de validación numérica
    if n <= 0:
        return (None, 'no_positivo')
    # Límite INT de SQL Server
    if n > 2_147_483_647:
        return (None, 'demasiado_grande')
    
    return (n, 'ok')

# 3) Construir columnas nuevas en bloque para evitar fragmentación
nuevas = pd.DataFrame(index=df.index)
nuevas['NroReparto_RAW'] = df[col_rep]

parsed = nuevas['NroReparto_RAW'].apply(parse_nro_reparto)
parsed_df = parsed.apply(pd.Series)
parsed_df.columns = ['NroReparto_SQL', 'Estado_NroReparto']

nuevas = pd.concat([nuevas, parsed_df], axis=1)
df = pd.concat([df, nuevas], axis=1)

# 4) Resumen general (incluyendo nulos)
total_filas = len(df)
no_nulos = df['NroReparto_SQL'].notna().sum()
nulos = df['NroReparto_SQL'].isna().sum()

print("\n=== Resumen NroReparto_SQL ===")
print("Total filas:", total_filas)
print("Con NroReparto_SQL NO nulo:", no_nulos)
print("Con NroReparto_SQL nulo   :", nulos)

print("\n=== Conteo por Estado_NroReparto ===")
print(df['Estado_NroReparto'].value_counts(dropna=False))

# 5) Estadísticos solo para los valores OK
ok_mask = df['Estado_NroReparto'] == 'ok'
print("\n=== Estadísticos NroReparto_SQL (solo estado 'ok') ===")
print(df.loc[ok_mask, 'NroReparto_SQL'].describe())

# 6) Ejemplos de registros OK (original vs normalizado)
ej_ok = df.loc[ok_mask, [
    'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'NroReparto_RAW', 'NroReparto_SQL'
]].head(30)

print("\n=== Ejemplos NroReparto OK (RAW vs SQL) ===")
print(ej_ok.to_string(index=False))

# 7) Ejemplos de registros problemáticos por categoría distinta de 'ok' y 'vacio'
categorias_problema = df['Estado_NroReparto'].unique()
categorias_problema = [c for c in categorias_problema if c not in ['ok', 'vacio']]

for cat in categorias_problema:
    sub = df.loc[df['Estado_NroReparto'] == cat, [
        'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
        'NroReparto_RAW', 'Estado_NroReparto'
    ]].head(20)
    print(f"\n=== Ejemplos estado '{cat}' ===")
    if not sub.empty:
        print(sub.to_string(index=False))
    else:
        print("Sin ejemplos.")

# 8) Excel de revisión general
cols_revision = [
    'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'NroReparto_RAW', 'NroReparto_SQL', 'Estado_NroReparto'
]
cols_revision = [c for c in cols_revision if c in df.columns]

df_revision = df[cols_revision].copy()

ruta_rev = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\NroReparto_revision.xlsx"
df_revision.to_excel(ruta_rev, index=False)
print("\nArchivo de revisión generado en:")
print(ruta_rev)

# 9) Excel solo con problemáticos (Estado distinto de 'ok' y con texto original)
mask_prob = df['Estado_NroReparto'].isin(categorias_problema)
df_prob = df.loc[mask_prob, cols_revision].copy()

ruta_prob = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica\NroReparto_problemas.xlsx"
df_prob.to_excel(ruta_prob, index=False)
print("Archivo de problemáticos generado en:")
print(ruta_prob)


Columnas candidatas para # REPARTO: ['# REPARTO', 'FECHA ASIGNACIÓN REPARTO', 'NroReparto_RAW', 'NroReparto_SQL', 'Estado_NroReparto']
Usando columna: # REPARTO

=== Resumen NroReparto_SQL ===
Total filas: 24055
Con NroReparto_SQL NO nulo: NroReparto_SQL    3667
NroReparto_SQL    3667
dtype: int64
Con NroReparto_SQL nulo   : NroReparto_SQL    20388
NroReparto_SQL    20388
dtype: int64

=== Conteo por Estado_NroReparto ===


ValueError: Grouper for 'Estado_NroReparto' not 1-dimensional

## FECHA ASIGNACIÓN REPARTO


In [ ]:
import pandas as pd
import numpy as np
import re
import unicodedata
from datetime import datetime

# Para reducir fragmentación
df = df.copy()

# ------------------------------
# 1) Buscar columna FECHA ASIGNACIÓN REPARTO
#     ignorando tildes y saltos de línea
# ------------------------------
def norm_header(s: str) -> str:
    s = str(s)
    # quitar tildes
    s_norm = unicodedata.normalize("NFD", s)
    s_sin_tildes = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    return s_sin_tildes.upper()

candidatas_asig = []
for c in df.columns:
    nh = norm_header(c)
    if "FECHA" in nh and "ASIGNACION" in nh and "REPARTO" in nh:
        candidatas_asig.append(c)

print("Columnas candidatas para FECHA ASIGNACIÓN REPARTO:", candidatas_asig)

if not candidatas_asig:
    raise ValueError("No se encontró columna para FECHA ASIGNACIÓN REPARTO; revisa encabezado exacto.")
col_asig = candidatas_asig[0]
print("Usando columna:", repr(col_asig))

# ------------------------------
# 2) DataFrame auxiliar con columnas nuevas
# ------------------------------
nuevas = pd.DataFrame(index=df.index)

# RAW = tal cual viene del Excel
nuevas["FechaAsignacionReparto_RAW"] = df[col_asig]

# 3) Pasar RAW a texto base
def a_texto_fecha(v):
    if pd.isna(v):
        return ""
    if isinstance(v, (pd.Timestamp, datetime)):
        return v.strftime("%d/%m/%Y")
    return str(v).strip()

nuevas["FechaAsignacionReparto_TXT"] = nuevas["FechaAsignacionReparto_RAW"].apply(a_texto_fecha)

# 4) Limpiar texto (placeholders, horas, formatos raros)
PLACEHOLDERS_NO_FECHA = {
    "NO", "NO.", "NO APLICA", "NO APLICA.", "NO REGISTRA", "NO REGISTRA.",
    "SIN FECHA", "SIN INFORME", "N/A", "NA", "NINGUNO", "NINGUNA"
}

def limpiar_texto_fecha_asig(s: str) -> str:
    s = str(s).strip()
    if s == "":
        return ""
    su = s.upper()
    if su in PLACEHOLDERS_NO_FECHA:
        return ""
    # quitar hora tipo "2024-04-25 00:00:00"
    if " " in s:
        s = s.split()[0].strip()
    # compactar espacios/saltos
    s = re.sub(r"\s+", " ", s)

    # corregir "20/112024" -> "20/11/2024"
    m1 = re.fullmatch(r"(\d{1,2})/(\d{2})(\d{4})", s)
    if m1:
        d, mm, yyyy = m1.groups()
        return f"{d}/{mm}/{yyyy}"

    # corregir "2103/2024" -> "21/03/2024"
    m2 = re.fullmatch(r"(\d{2})(\d{2})/(\d{4})", s)
    if m2:
        d, mm, yyyy = m2.groups()
        return f"{d}/{mm}/{yyyy}"

    return s

nuevas["FechaAsignacionReparto_TXT_LIMPIO"] = nuevas["FechaAsignacionReparto_TXT"].apply(
    limpiar_texto_fecha_asig
)

# 5) Parsear a datetime + estado
def parse_fecha_asig(t_str: str):
    t_str = "" if t_str is None else str(t_str).strip()
    if t_str == "":
        return pd.NaT, "vacio"

    su = t_str.upper()
    if su in PLACEHOLDERS_NO_FECHA:
        return pd.NaT, "placeholder"

    fecha = pd.to_datetime(t_str, dayfirst=True, errors="coerce")
    if pd.isna(fecha):
        return pd.NaT, "no_parseable"

    return fecha, "ok"

parsed = nuevas["FechaAsignacionReparto_TXT_LIMPIO"].apply(parse_fecha_asig)
parsed_df = parsed.apply(pd.Series)
parsed_df.columns = ["FechaAsignacionReparto_SQL", "Estado_FechaAsignacionReparto"]

nuevas = pd.concat([nuevas, parsed_df], axis=1)

# 6) Unir al df original
df = pd.concat([df, nuevas], axis=1)

# 7) Resumen (incluyendo nulos)
total = len(df)
validas = df["FechaAsignacionReparto_SQL"].notna().sum()
nulas = df["FechaAsignacionReparto_SQL"].isna().sum()

print("\n=== Resumen FECHA ASIGNACIÓN REPARTO ===")
print("Total filas:", total)
print("Fechas válidas (no NaT):", validas)
print("Fechas nulas/NaT       :", nulas)

print("\n=== Conteo por Estado_FechaAsignacionReparto ===")
print(df["Estado_FechaAsignacionReparto"].value_counts(dropna=False))

# 8) Ejemplos válidos
cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

cols_muestra_ok = cols_ctx + [
    "FechaAsignacionReparto_RAW",
    "FechaAsignacionReparto_TXT_LIMPIO",
    "FechaAsignacionReparto_SQL",
]

ej_validas = df.loc[df["FechaAsignacionReparto_SQL"].notna(), cols_muestra_ok].head(30)

print("\n=== Ejemplos de fechas válidas (RAW vs TXT_LIMPIO vs SQL) ===")
print(ej_validas.to_string(index=False))

# 9) Textos que no se pudieron parsear
mask_raras = (
    df["FechaAsignacionReparto_SQL"].isna()
    & (df["FechaAsignacionReparto_TXT_LIMPIO"].astype(str).str.strip() != "")
)

print("\n=== Valores de FechaAsignacionReparto_TXT_LIMPIO que producen NaT (top 20) ===")
print(
    df.loc[mask_raras, "FechaAsignacionReparto_TXT_LIMPIO"]
      .value_counts()
      .head(20)
)

cols_muestra_raras = cols_ctx + [
    "FechaAsignacionReparto_RAW",
    "FechaAsignacionReparto_TXT_LIMPIO",
    "Estado_FechaAsignacionReparto",
]

ej_raras = df.loc[mask_raras, cols_muestra_raras].head(50)

print("\n=== Ejemplos de filas con texto que no se pudo convertir a fecha ===")
if not ej_raras.empty:
    print(ej_raras.to_string(index=False))
else:
    print("No hay textos raros; sólo nulos o placeholders (NO APLICA / NO REGISTRA / etc.).")

# 10) Excels de revisión
base_path = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica"

df_revision = df[cols_muestra_ok + ["Estado_FechaAsignacionReparto"]].copy()
ruta_rev = fr"{base_path}\FechaAsignacionReparto_revision.xlsx"
df_revision.to_excel(ruta_rev, index=False)
print("\nArchivo de revisión generado en:")
print(ruta_rev)

df_problemas = df.loc[mask_raras, cols_muestra_raras].copy()
ruta_prob = fr"{base_path}\FechaAsignacionReparto_problemas.xlsx"
df_problemas.to_excel(ruta_prob, index=False)
print("Archivo de problemáticos generado en:")
print(ruta_prob)


Columnas candidatas para FECHA ASIGNACIÓN REPARTO: ['FECHA ASIGNACIÓN REPARTO']
Usando columna: 'FECHA ASIGNACIÓN REPARTO'

=== Resumen FECHA ASIGNACIÓN REPARTO ===
Total filas: 24055
Fechas válidas (no NaT): 9830
Fechas nulas/NaT       : 14225

=== Conteo por Estado_FechaAsignacionReparto ===
Estado_FechaAsignacionReparto
vacio           14224
ok               9830
no_parseable        1
Name: count, dtype: int64

=== Ejemplos de fechas válidas (RAW vs TXT_LIMPIO vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO           MUNICIPIO FechaAsignacionReparto_RAW FechaAsignacionReparto_TXT_LIMPIO FechaAsignacionReparto_SQL
                2  062-37357      BOLIVAR            ZAMBRANO        2024-10-16 00:00:00                        16/10/2024                 2024-10-16
                3  062-37359      BOLIVAR            ZAMBRANO        2025-10-31 00:00:00                        31/10/2025                 2025-10-31
                4  062-37361      BOLIVAR            ZAMBRANO        2

## PLAZO PARA ENTREGA A REVISOR


In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# Para reducir fragmentación (opcional, pero ayuda)
df = df.copy()

# ===========================
# 1) Localizar columna PLAZO PARA ENTREGA A REVISOR
# ===========================
candidatas_plazo = [
    c for c in df.columns
    if ("PLAZO"   in str(c).upper()
        and "ENTREGA" in str(c).upper()
        and "REVISOR" in str(c).upper())
]

print("Columnas candidatas para PLAZO PARA ENTREGA A REVISOR:", candidatas_plazo)

if not candidatas_plazo:
    raise ValueError("No se encontró la columna de PLAZO PARA ENTREGA A REVISOR en el DataFrame.")

col_plazo = candidatas_plazo[0]
print("Usando columna:", repr(col_plazo))

# ===========================
# 2) DataFrame auxiliar con columnas nuevas
# ===========================
nuevas = pd.DataFrame(index=df.index)

# Valor crudo tal cual viene del Excel
nuevas["FechaPlazoEntregaARevisor_RAW"] = df[col_plazo]

# ===========================
# 3) Pasar RAW a texto base
# ===========================
def a_texto_fecha(v):
    if pd.isna(v):
        return ""
    if isinstance(v, (pd.Timestamp, datetime)):
        return v.strftime("%d/%m/%Y")
    return str(v).strip()

nuevas["FechaPlazoEntregaARevisor_TXT"] = nuevas["FechaPlazoEntregaARevisor_RAW"].apply(a_texto_fecha)

# ===========================
# 4) Limpiar texto: placeholders, horas, formatos raros
# ===========================
PLACEHOLDERS_NO_FECHA = {
    "NO", "NO.", "NO APLICA", "NO APLICA.",
    "NO REGISTRA", "NO REGISTRA.",
    "SIN FECHA", "SIN INFORME",
    "N/A", "NA", "NINGUNO", "NINGUNA"
}

def limpiar_texto_fecha_plazo(s: str) -> str:
    s = str(s).strip()
    if s == "":
        return ""
    su = s.upper()
    # textos que significan "no hay fecha"
    if su in PLACEHOLDERS_NO_FECHA:
        return ""

    # si viene con hora: "2024-04-25 00:00:00" -> "2024-04-25"
    if " " in s:
        s = s.split()[0].strip()

    # compactar espacios/saltos
    s = re.sub(r"\s+", " ", s)

    # Caso 20/112024 -> 20/11/2024
    m1 = re.fullmatch(r"(\d{1,2})/(\d{2})(\d{4})", s)
    if m1:
        d, mm, yyyy = m1.groups()
        return f"{d}/{mm}/{yyyy}"

    # Caso 2103/2024 -> 21/03/2024
    m2 = re.fullmatch(r"(\d{2})(\d{2})/(\d{4})", s)
    if m2:
        d, mm, yyyy = m2.groups()
        return f"{d}/{mm}/{yyyy}"

    return s

nuevas["FechaPlazoEntregaARevisor_TXT_LIMPIO"] = nuevas["FechaPlazoEntregaARevisor_TXT"].apply(
    limpiar_texto_fecha_plazo
)

# ===========================
# 5) Parsear a datetime + estado
# ===========================
def parse_fecha_plazo(t_str: str):
    t_str = "" if t_str is None else str(t_str).strip()
    if t_str == "":
        return pd.NaT, "vacio"

    su = t_str.upper()
    if su in PLACEHOLDERS_NO_FECHA:
        return pd.NaT, "placeholder"

    fecha = pd.to_datetime(t_str, dayfirst=True, errors="coerce")
    if pd.isna(fecha):
        return pd.NaT, "no_parseable"

    return fecha, "ok"

parsed = nuevas["FechaPlazoEntregaARevisor_TXT_LIMPIO"].apply(parse_fecha_plazo)
parsed_df = parsed.apply(pd.Series)
parsed_df.columns = ["FechaPlazoEntregaARevisor_SQL", "Estado_FechaPlazoEntregaARevisor"]

nuevas = pd.concat([nuevas, parsed_df], axis=1)

# ===========================
# 6) Unir al DataFrame principal
# ===========================
df = pd.concat([df, nuevas], axis=1)

# ===========================
# 7) Resumen general (incluye nulos)
# ===========================
total = len(df)
validas = df["FechaPlazoEntregaARevisor_SQL"].notna().sum()
nulas = df["FechaPlazoEntregaARevisor_SQL"].isna().sum()

print("\n=== Resumen PLAZO PARA ENTREGA A REVISOR ===")
print("Total filas:", total)
print("Fechas válidas (no NaT):", validas)
print("Fechas nulas/NaT       :", nulas)

print("\n=== Conteo por Estado_FechaPlazoEntregaARevisor ===")
print(df["Estado_FechaPlazoEntregaARevisor"].value_counts(dropna=False))

# ===========================
# 8) Ejemplos de fechas válidas
# ===========================
cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

cols_muestra_ok = cols_ctx + [
    "FechaPlazoEntregaARevisor_RAW",
    "FechaPlazoEntregaARevisor_TXT_LIMPIO",
    "FechaPlazoEntregaARevisor_SQL",
]

ej_validas = df.loc[df["FechaPlazoEntregaARevisor_SQL"].notna(), cols_muestra_ok].head(30)

print("\n=== Ejemplos de fechas válidas (RAW vs TXT_LIMPIO vs SQL) ===")
print(ej_validas.to_string(index=False))

# ===========================
# 9) Textos que no se pudieron parsear (para revisión)
# ===========================
mask_raras = (
    df["FechaPlazoEntregaARevisor_SQL"].isna()
    & (df["FechaPlazoEntregaARevisor_TXT_LIMPIO"].astype(str).str.strip() != "")
)

print("\n=== Valores de FechaPlazoEntregaARevisor_TXT_LIMPIO que producen NaT (top 20) ===")
print(
    df.loc[mask_raras, "FechaPlazoEntregaARevisor_TXT_LIMPIO"]
      .value_counts()
      .head(20)
)

cols_muestra_raras = cols_ctx + [
    "FechaPlazoEntregaARevisor_RAW",
    "FechaPlazoEntregaARevisor_TXT_LIMPIO",
    "Estado_FechaPlazoEntregaARevisor",
]

ej_raras = df.loc[mask_raras, cols_muestra_raras].head(50)

print("\n=== Ejemplos de filas con texto que no se pudo convertir a fecha ===")
if not ej_raras.empty:
    print(ej_raras.to_string(index=False))
else:
    print("No hay textos raros; sólo nulos o placeholders (NO APLICA / NO REGISTRA / etc.).")

# ===========================
# 10) Excels de revisión
# ===========================
base_path = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica"

df_revision = df[cols_muestra_ok + ["Estado_FechaPlazoEntregaARevisor"]].copy()
ruta_rev = fr"{base_path}\FechaPlazoEntregaARevisor_revision.xlsx"
df_revision.to_excel(ruta_rev, index=False)
print("\nArchivo de revisión generado en:")
print(ruta_rev)

df_problemas = df.loc[mask_raras, cols_muestra_raras].copy()
ruta_prob = fr"{base_path}\FechaPlazoEntregaARevisor_problemas.xlsx"
df_problemas.to_excel(ruta_prob, index=False)
print("Archivo de problemáticos generado en:")
print(ruta_prob)


Columnas candidatas para PLAZO PARA ENTREGA A REVISOR: ['PLAZO PARA ENTREGA A REVISOR']
Usando columna: 'PLAZO PARA ENTREGA A REVISOR'

=== Resumen PLAZO PARA ENTREGA A REVISOR ===
Total filas: 24055
Fechas válidas (no NaT): 7493
Fechas nulas/NaT       : 16562

=== Conteo por Estado_FechaPlazoEntregaARevisor ===
Estado_FechaPlazoEntregaARevisor
vacio    16562
ok        7493
Name: count, dtype: int64

=== Ejemplos de fechas válidas (RAW vs TXT_LIMPIO vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO           MUNICIPIO FechaPlazoEntregaARevisor_RAW FechaPlazoEntregaARevisor_TXT_LIMPIO FechaPlazoEntregaARevisor_SQL
                2  062-37357      BOLIVAR            ZAMBRANO                    2024-10-17                           17/10/2024                    2024-10-17
                3  062-37359      BOLIVAR            ZAMBRANO                    2025-11-18                           18/11/2025                    2025-11-18
                4  062-37361      BOLIVAR            ZAMB

## "ESTADO REVISION APROBADO / DEVUELTO / EN REVISION"


In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re

# Para reducir fragmentación (opcional)
df = df.copy()

# =========================
# 1) Detectar columna ESTADO REVISION
# =========================
candidatas_estado = [
    c for c in df.columns
    if ("ESTADO" in str(c).upper()) and ("REVISION" in str(c).upper())
]

print("Columnas candidatas para ESTADO REVISION:", candidatas_estado)

if not candidatas_estado:
    raise ValueError("No se encontró la columna de ESTADO REVISION en el DataFrame.")

col_estado = candidatas_estado[0]
print("Usando columna:", repr(col_estado))

# =========================
# 2) DataFrame auxiliar con columnas nuevas
# =========================
nuevas = pd.DataFrame(index=df.index)

# Valor crudo tal como viene del Excel
nuevas["EstadoRevision_RAW"] = df[col_estado]

# =========================
# 3) Texto limpio (sin tildes, mayúsculas, espacios normalizados)
# =========================
def limpiar_estado_revision(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None

    # quitar tildes
    t_norm = unicodedata.normalize("NFD", t)
    t_sin = "".join(
        ch for ch in t_norm
        if unicodedata.category(ch) != "Mn"
    )
    # mayúsculas y espacios compactados
    t_sin = re.sub(r"\s+", " ", t_sin).strip().upper()
    return t_sin

nuevas["EstadoRevision_TXT_LIMPIO"] = nuevas["EstadoRevision_RAW"].apply(limpiar_estado_revision)

print("\n=== Valores CRUDOS ESTADO REVISION (top 30) ===")
print(nuevas["EstadoRevision_RAW"].value_counts(dropna=False).head(30))

print("\n=== Valores NORMALIZADOS ESTADO REVISION (top 30) ===")
print(nuevas["EstadoRevision_TXT_LIMPIO"].value_counts(dropna=False).head(30))

# =========================
# 4) Clasificar contra catálogo (APROBADO / DEVUELTO / EN REVISION)
# =========================
# Catálogo esperado en SQL:
# 1 -> APROBADO
# 2 -> DEVUELTO
# 3 -> EN REVISION

def clasificar_estado_revision(t):
    """
    t llega ya MAYÚSCULA, sin tildes y con espacios normalizados.
    Devuelve (IdTipoEstadoRevision, NombreTipoEstadoRevision)
    """
    if t is None or t == "":
        return (None, None)

    # APROBADO / APROBADA / APROBADO PARA...
    if "APROBAD" in t:
        return (1, "APROBADO")

    # DEVUELTO / DEVUELTA / DEVOLUCION / DEVUELVE...
    if "DEVUELT" in t or "DEVOLUCION" in t:
        return (2, "DEVUELTO")

    # EN REVISION, EN PROCESO DE REVISION, etc.
    if "REVISION" in t:
        return (3, "EN REVISION")

    # Cualquier otro texto: queda sin clasificar para revisar
    return (None, "SIN_CLASIFICAR")

clasif = nuevas["EstadoRevision_TXT_LIMPIO"].apply(clasificar_estado_revision)
clasif_df = clasif.apply(pd.Series)
clasif_df.columns = ["IdTipoEstadoRevision_SQL", "EstadoRevision_NOMBRE_SQL"]

nuevas = pd.concat([nuevas, clasif_df], axis=1)

# =========================
# 5) Unir al DataFrame principal
# =========================
df = pd.concat([df, nuevas], axis=1)

# =========================
# 6) Resumen (incluye nulos)
# =========================
total = len(df)
no_nulos_id = df["IdTipoEstadoRevision_SQL"].notna().sum()
nulos_id = df["IdTipoEstadoRevision_SQL"].isna().sum()

print("\n=== Resumen IdTipoEstadoRevision_SQL ===")
print("Total filas:", total)
print("Con IdTipoEstadoRevision_SQL NO nulo:", no_nulos_id)
print("Con IdTipoEstadoRevision_SQL nulo   :", nulos_id)

print("\nDistribución IdTipoEstadoRevision_SQL:")
print(df["IdTipoEstadoRevision_SQL"].value_counts(dropna=False).sort_index())

print("\n=== Valores normalizados EstadoRevision_NOMBRE_SQL (top 20) ===")
print(df["EstadoRevision_NOMBRE_SQL"].value_counts(dropna=False).head(20))

# =========================
# 7) Filas con texto pero SIN clasificar
# =========================
mask_sin_clasif = (
    df["EstadoRevision_TXT_LIMPIO"].notna()
    & (df["EstadoRevision_TXT_LIMPIO"] != "")
    & df["IdTipoEstadoRevision_SQL"].isna()
)

cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

ej_sin_clasif = df.loc[
    mask_sin_clasif,
    cols_ctx + ["EstadoRevision_RAW", "EstadoRevision_TXT_LIMPIO", "EstadoRevision_NOMBRE_SQL"]
].head(50)

print("\n=== Ejemplos de ESTADO REVISION con TEXTO pero SIN clasificar ===")
if not ej_sin_clasif.empty:
    print(ej_sin_clasif.to_string(index=False))
else:
    print("Todos los textos se clasificaron en 1=APROBADO, 2=DEVUELTO o 3=EN REVISION (o son nulos).")

# =========================
# 8) (Opcional) Excels de revisión
# =========================
base_path = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica"

df_revision = df[cols_ctx + [
    "EstadoRevision_RAW",
    "EstadoRevision_TXT_LIMPIO",
    "IdTipoEstadoRevision_SQL",
    "EstadoRevision_NOMBRE_SQL"
]].copy()

ruta_rev = fr"{base_path}\EstadoRevision_raw_y_normalizado.xlsx"
df_revision.to_excel(ruta_rev, index=False)
print("\nArchivo de revisión generado en:")
print(ruta_rev)

df_sin_clasif = df.loc[mask_sin_clasif, cols_ctx + [
    "EstadoRevision_RAW",
    "EstadoRevision_TXT_LIMPIO",
    "EstadoRevision_NOMBRE_SQL"
]].copy()

ruta_sin = fr"{base_path}\EstadoRevision_sin_clasificar.xlsx"
df_sin_clasif.to_excel(ruta_sin, index=False)
print("Archivo de no clasificados generado en:")
print(ruta_sin)


Columnas candidatas para ESTADO REVISION: ['ESTADO REVISION\nAPROBADO / DEVUELTO / EN REVISION']
Usando columna: 'ESTADO REVISION\nAPROBADO / DEVUELTO / EN REVISION'

=== Valores CRUDOS ESTADO REVISION (top 30) ===
EstadoRevision_RAW
NaN                                                                                                                                                                                                                                                18964
APROBADO                                                                                                                                                                                                                                            4691
DEVUELTO                                                                                                                                                                                                                                             331
PENDIENTE          

## OBSERVACIONES REVISOR

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re

# Para evitar el warning de fragmentación
df = df.copy()

# =========================
# 1) Detectar columna de OBSERVACIONES REVISOR
# =========================
def normalizar_header(s):
    if s is None:
        return ""
    s_norm = unicodedata.normalize('NFD', str(s))
    s_sin_tildes = ''.join(
        ch for ch in s_norm
        if unicodedata.category(ch) != 'Mn'
    )
    return s_sin_tildes.upper().strip()

candidatas_obs = [
    c for c in df.columns
    if "OBSERVACION" in normalizar_header(c) and "REVISOR" in normalizar_header(c)
]

print("Columnas candidatas para OBSERVACIONES REVISOR:", candidatas_obs)

if not candidatas_obs:
    raise ValueError("No se encontró la columna de OBSERVACIONES REVISOR en el DataFrame.")

col_obs = candidatas_obs[0]
print("Usando columna:", repr(col_obs))

# =========================
# 2) DataFrame auxiliar con columnas nuevas
# =========================
nuevas = pd.DataFrame(index=df.index)

# Valor crudo tal cual viene del Excel
nuevas["ObservacionesRevisor_RAW"] = df[col_obs]

# =========================
# 3) Limpiar texto para versión SQL
# =========================
def limpiar_observacion(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None
    # compactar espacios y saltos de línea a un solo espacio
    t = re.sub(r"\s+", " ", t)
    # NO cambiamos a mayúsculas para conservar legibilidad
    return t

nuevas["ObservacionesRevisor_SQL"] = nuevas["ObservacionesRevisor_RAW"].apply(limpiar_observacion)

# Longitud para revisar extremos
nuevas["ObservacionesRevisor_Len"] = nuevas["ObservacionesRevisor_SQL"].fillna("").astype(str).str.len()

# =========================
# 4) Unir al DataFrame principal
# =========================
df = pd.concat([df, nuevas], axis=1)

# =========================
# 5) Resumen básico (incluye nulos)
# =========================
total = len(df)
con_texto = df["ObservacionesRevisor_SQL"].notna().sum()
sin_texto = df["ObservacionesRevisor_SQL"].isna().sum()
max_len = df["ObservacionesRevisor_Len"].max()

print("\n=== Resumen OBSERVACIONES REVISOR ===")
print(f"Total registros: {total}")
print(f"Con texto (SQL no nulo): {con_texto}")
print(f"Sin texto (SQL nulo)    : {sin_texto}")
print(f"Longitud máxima observada: {max_len}")

print("\n=== Ejemplos (RAW vs SQL) ===")
cols_ctx = [c for c in ["Fila_Excel_aprox", "FMI", "DEPARTAMENTO", "MUNICIPIO"] if c in df.columns]

muestra = df.loc[
    df["ObservacionesRevisor_RAW"].notna() | df["ObservacionesRevisor_SQL"].notna(),
    cols_ctx + [
        "ObservacionesRevisor_RAW",
        "ObservacionesRevisor_SQL",
        "ObservacionesRevisor_Len"
    ]
].head(20)

print(muestra.to_string(index=False))

# =========================
# 6) Observaciones muy largas (umbral orientativo)
# =========================
UMBRAL = 4000
largas = df[df["ObservacionesRevisor_Len"] > UMBRAL]

if not largas.empty:
    print(f"\n=== ObservacionesRevisor que superan {UMBRAL} caracteres ===")
    print(
        largas[
            cols_ctx
            + ["ObservacionesRevisor_Len", "ObservacionesRevisor_SQL"]
        ].to_string(index=False)
    )
else:
    print(f"\nNinguna observación supera los {UMBRAL} caracteres (sin problema para varchar(MAX)).")

# =========================
# 7) (Opcional) Excel de revisión
# =========================
base_path = r"C:\Users\andres.garciav\OneDrive - Agencia Nacional De Tierras - ANT\Documentos\Migracion base de datos viablidad juridica"
ruta_excel = fr"{base_path}\ObservacionesRevisor_raw_y_normalizado.xlsx"

df_export = df[
    cols_ctx
    + [
        col_obs,
        "ObservacionesRevisor_RAW",
        "ObservacionesRevisor_SQL",
        "ObservacionesRevisor_Len",
    ]
]

df_export.to_excel(ruta_excel, index=False)
print("\nArchivo de revisión generado en:")
print(ruta_excel)


Columnas candidatas para OBSERVACIONES REVISOR: ['OBSERVACIONES REVISOR']
Usando columna: 'OBSERVACIONES REVISOR'

=== Resumen OBSERVACIONES REVISOR ===
Total registros: 24055
Con texto (SQL no nulo): 4398
Sin texto (SQL nulo)    : 19657
Longitud máxima observada: 2586

=== Ejemplos (RAW vs SQL) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO           MUNICIPIO                                                                                     ObservacionesRevisor_RAW                                                                                     ObservacionesRevisor_SQL  ObservacionesRevisor_Len
                4  062-37361      BOLIVAR            ZAMBRANO                                                                                         SE REMITE PARA FIRMA                                                                                         SE REMITE PARA FIRMA                        20
                5  062-37354      BOLIVAR            ZAMBRANO                          

## "ENTREGÓ CARPETA DE SOPORTE (SI / NO)"


In [71]:
import pandas as pd
import numpy as np
import unicodedata
import re

# 1) Normalizar encabezados solo para BUSCAR la columna (NO cambia el RAW)
def normalizar_header(s):
    if s is None:
        return ""
    s_norm = unicodedata.normalize('NFD', str(s))
    s_sin_tildes = ''.join(
        ch for ch in s_norm
        if unicodedata.category(ch) != 'Mn'
    )
    # Unificar espacios y saltos de línea
    s_sin_tildes = re.sub(r"\s+", " ", s_sin_tildes)
    return s_sin_tildes.upper().strip()

print("=== Columnas del DataFrame y su versión normalizada ===")
for i, c in enumerate(df.columns):
    print(f"{i:2d} -> {repr(c)}  ||  '{normalizar_header(c)}'")

# 2) Ubicar la columna ENTREGÓ CARPETA DE SOPORTES (SI / NO)
candidatas = []
for c in df.columns:
    norm = normalizar_header(c)
    if "ENTREGO" in norm and "CARPETA" in norm and "SOPORTES" in norm:
        candidatas.append(c)

print("\nColumnas candidatas para ENTREGÓ CARPETA DE SOPORTES:", candidatas)

if not candidatas:
    raise ValueError("No se encontró la columna de ENTREGÓ CARPETA DE SOPORTES en el DataFrame.")

col_entrega = candidatas[0]
print("Usando columna:", repr(col_entrega))

# 3) Columna RAW = tal cual viene del Excel
df['EntregoCarpeta_RAW'] = df[col_entrega]

# 4) Texto limpio (solo espacios / saltos de línea)
def limpiar_basico(v):
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None
    t = re.sub(r"\s+", " ", t)  # compactar espacios y \n
    return t

df['EntregoCarpeta_TXT_LIMPIO'] = df['EntregoCarpeta_RAW'].apply(limpiar_basico)

# 5) Normalizar a SI / NO / PENDIENTE
def quitar_tildes(v):
    if v is None:
        return None
    s_norm = unicodedata.normalize('NFD', str(v))
    return ''.join(ch for ch in s_norm if unicodedata.category(ch) != 'Mn')

def normalizar_entrego(v):
    if v is None:
        return None
    t = quitar_tildes(v).upper().strip()

    # SI
    if t in ("SI", "S", "SI.", "SI "):
        return "SI"
    # NO
    if t in ("NO", "N", "NO.", "NO "):
        return "NO"
    # PENDIENTE
    if "PEND" in t:
        return "PENDIENTE"

    # Cualquier otro texto queda sin clasificar => NULL en SQL
    return None

df['EntregoCarpetaSoportes_SQL'] = df['EntregoCarpeta_TXT_LIMPIO'].apply(normalizar_entrego)

# 6) Resúmenes, incluyendo nulos
print("\n=== Valores CRUDOS ENTREGÓ CARPETA (top 20) ===")
print(df['EntregoCarpeta_RAW'].value_counts(dropna=False).head(20))

print("\n=== Valores NORMALIZADOS ENTREGÓ CARPETA (para SQL) ===")
print(df['EntregoCarpetaSoportes_SQL'].value_counts(dropna=False))

total = len(df)
no_nulos_sql = df['EntregoCarpetaSoportes_SQL'].notna().sum()
nulos_sql = df['EntregoCarpetaSoportes_SQL'].isna().sum()

print("\n=== Resumen nulos / no nulos EntregoCarpetaSoportes_SQL ===")
print(f"Total registros           : {total}")
print(f"Con valor en SQL (no nulo): {no_nulos_sql}")
print(f"Sin valor en SQL (nulo)   : {nulos_sql}")

# 7) Filas con texto fuera de catálogo (para revisar a mano)
cols_ctx = [c for c in ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO'] if c in df.columns]

mask_fuera = (
    df['EntregoCarpeta_RAW'].notna()
    & df['EntregoCarpeta_TXT_LIMPIO'].notna()
    & df['EntregoCarpeta_TXT_LIMPIO'].astype(str).str.strip().ne("")
    & df['EntregoCarpetaSoportes_SQL'].isna()
)

fuera = df.loc[
    mask_fuera,
    cols_ctx + ['EntregoCarpeta_RAW', 'EntregoCarpeta_TXT_LIMPIO']
]

print("\n=== Registros ENTREGÓ CARPETA con valores fuera de catálogo (revisar) ===")
if fuera.empty:
    print("No hay valores fuera de catálogo (solo SI / NO / PENDIENTE o nulos).")
else:
    print(fuera.head(50).to_string(index=False))
    print("\nTotal fuera de catálogo:", len(fuera))


=== Columnas del DataFrame y su versión normalizada ===
 0 -> 'ID'  ||  'ID'
 1 -> 'FMI'  ||  'FMI'
 2 -> 'No. Expediente'  ||  'NO. EXPEDIENTE'
 3 -> 'FUENTE'  ||  'FUENTE'
 4 -> 'TIPO DE PROCESO AGRARIO'  ||  'TIPO DE PROCESO AGRARIO'
 5 -> 'TipoProceso_RAW'  ||  'TIPOPROCESO_RAW'
 6 -> 'TIPO_PROCESO_TXT_LIMPIO'  ||  'TIPO_PROCESO_TXT_LIMPIO'
 7 -> 'TipoProceso_NOMBRE_SQL'  ||  'TIPOPROCESO_NOMBRE_SQL'
 8 -> 'IdTipoProceso_SQL'  ||  'IDTIPOPROCESO_SQL'
 9 -> 'ETAPA PROCESAL'  ||  'ETAPA PROCESAL'
10 -> 'Dependencia'  ||  'DEPENDENCIA'
11 -> 'ID ORFEO'  ||  'ID ORFEO'
12 -> 'DEPARTAMENTO'  ||  'DEPARTAMENTO'
13 -> 'MUNICIPIO'  ||  'MUNICIPIO'
14 -> 'ÁREA REGISTRAL\n (0,0000 = Has,M2)'  ||  'AREA REGISTRAL (0,0000 = HAS,M2)'
15 -> 'CÍRCULO REGISTRAL'  ||  'CIRCULO REGISTRAL'
16 -> 'CirculoRegistral_RAW'  ||  'CIRCULOREGISTRAL_RAW'
17 -> 'CirculoRegistral_TXT_LIMPIO'  ||  'CIRCULOREGISTRAL_TXT_LIMPIO'
18 -> 'CirculoRegistral_SQL'  ||  'CIRCULOREGISTRAL_SQL'
19 -> 'ÁREA CALCULADA'  ||  '

*voy acá*

## FECHA DE ENVÍO A COORDINACIÓN

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re

# ======================================================
# 1. Funciones auxiliares
# ======================================================

def normalizar_header(s: str) -> str:
    if s is None:
        return ""
    s_norm = unicodedata.normalize('NFD', str(s))
    s_sin_tildes = ''.join(ch for ch in s_norm if unicodedata.category(ch) != 'Mn')
    return s_sin_tildes.upper().strip()

def limpiar_fecha_texto(v):
    """Limpia texto de fecha: maneja NO / NO REGISTRA / vacíos / NaT."""
    if pd.isna(v):
        return None
    t = str(v).strip()
    if t == "":
        return None

    # Versión sin tildes y en mayúscula para detectar 'NO', 'NO REGISTRA', etc.
    t_up = unicodedata.normalize('NFD', t)
    t_up = ''.join(ch for ch in t_up if unicodedata.category(ch) != 'Mn').upper()

    # Casos que significan 'sin fecha'
    if t_up in ("NAT", "NONE"):
        return None
    if re.fullmatch(r"NO(\s+REGISTRA| APLICA| INFORMA)?\.?", t_up):
        return None
    if "SIN INFORMACION" in t_up or "SIN INFORMACIÓN" in t_up or "SIN REGISTRO" in t_up:
        return None

    # Normalizar separadores de fecha
    t_norm = t.replace('\\', '/').replace('-', '/').replace('.', '/')
    t_norm = re.sub(r"\s+DE\s+", "/", t_norm, flags=re.IGNORECASE)
    t_norm = re.sub(r"\s+", "/", t_norm)

    return t_norm

def parsear_fecha_flexible(txt):
    """Intenta convertir distintos formatos a Timestamp; si no, devuelve NaT."""
    if txt is None:
        return pd.NaT
    t = str(txt).strip()
    if t == "":
        return pd.NaT
    if t.upper() == "NAT":
        return pd.NaT

    # 1) Intento directo
    fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')
    if pd.notna(fecha):
        return fecha

    # 2) Formato sólo dígitos: DDMMYYYY
    solo_dig = re.sub(r"\D", "", t)
    if len(solo_dig) == 8:
        try:
            d = int(solo_dig[0:2])
            m = int(solo_dig[2:4])
            y = int(solo_dig[4:8])
            if 1 <= d <= 31 and 1 <= m <= 12:
                return pd.Timestamp(year=y, month=m, day=d)
        except ValueError:
            pass

    return pd.NaT


# ======================================================
# 2. Localizar la columna en el Excel
# ======================================================

print("=== Columnas del DataFrame y su versión normalizada ===")
for i, c in enumerate(df.columns):
    print(f"{i:2d} -> '{c}'  ||  '{normalizar_header(c)}'")

candidatas_envio = []
for c in df.columns:
    norm = normalizar_header(c)
    # Ajusta este filtro si tu encabezado viene ligeramente diferente
    if "FECHA" in norm and "ENVIO" in norm and "COORDINACION" in norm:
        candidatas_envio.append(c)

print("\nColumnas candidatas para FECHA DE ENVÍO A COORDINACIÓN:", candidatas_envio)

if not candidatas_envio:
    raise ValueError("No se encontró la columna de FECHA DE ENVÍO A COORDINACIÓN.")

col_envio = candidatas_envio[0]
print("Usando columna:", repr(col_envio))

# ======================================================
# 3. Crear columnas RAW / LIMPIA / SQL
# ======================================================

df['FechaEnvioCoord_RAW'] = df[col_envio]
df['FechaEnvioCoord_TXT_LIMPIO'] = df['FechaEnvioCoord_RAW'].apply(limpiar_fecha_texto)
df['FechaEnvioACoordinacion_SQL'] = df['FechaEnvioCoord_TXT_LIMPIO'].apply(parsear_fecha_flexible)

# ======================================================
# 4. Resumen de calidad
# ======================================================

total = len(df)
validas = df['FechaEnvioACoordinacion_SQL'].notna().sum()
invalidas_con_texto = (
    df['FechaEnvioACoordinacion_SQL'].isna()
    & df['FechaEnvioCoord_TXT_LIMPIO'].notna()
    & (df['FechaEnvioCoord_TXT_LIMPIO'].astype(str).str.strip() != "")
).sum()
sin_fecha = total - validas - invalidas_con_texto

print("\n=== Resumen FECHA DE ENVÍO A COORDINACIÓN ===")
print(f"Total filas: {total}")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")
print(f"Sin fecha (vacío o 'NO/NO REGISTRA/etc.'): {sin_fecha}")

# ======================================================
# 5. Muestra fila a fila: crudo vs limpio vs SQL
# ======================================================

cols_muestra = [
    'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'FechaEnvioCoord_RAW', 'FechaEnvioCoord_TXT_LIMPIO',
    'FechaEnvioACoordinacion_SQL'
]

print("\n=== Muestra de FECHA DE ENVÍO A COORDINACIÓN (primeras 20 filas) ===")
print(df[cols_muestra].head(20).to_string(index=False))

# Filas con texto que no se pudo parsear
mask_invalidos = (
    df['FechaEnvioACoordinacion_SQL'].isna()
    & df['FechaEnvioCoord_TXT_LIMPIO'].notna()
    & (df['FechaEnvioCoord_TXT_LIMPIO'].astype(str).str.strip() != "")
)
invalidos = df.loc[mask_invalidos, cols_muestra]

print("\n=== Filas donde el texto no se pudo convertir a fecha (para revisión manual) ===")
if invalidos.empty:
    print("No hay textos fuera de formato; lo restante son nulos reales.")
else:
    print(invalidos.to_string(index=False))
    print("\nTotal filas con texto no parseado:", len(invalidos))

# ======================================================
# 6. Exportar a Excel para inspección visual
# ======================================================

df[cols_muestra].to_excel(
    "FechaEnvioACoordinacion_todas_filas.xlsx",
    index=False
)

if not invalidos.empty:
    invalidos.to_excel(
        "FechaEnvioACoordinacion_invalidas_texto.xlsx",
        index=False
    )

print("\nArchivos generados:")
print(" - FechaEnvioACoordinacion_todas_filas.xlsx  (todas las filas crudo + limpio + SQL)")
if not invalidos.empty:
    print(" - FechaEnvioACoordinacion_invalidas_texto.xlsx  (solo las 4 filas anómalas)")


=== Columnas del DataFrame y su versión normalizada ===
 0 -> 'ID'  ||  'ID'
 1 -> 'FMI'  ||  'FMI'
 2 -> 'NumeroExpediente'  ||  'NUMEROEXPEDIENTE'
 3 -> 'FUENTE'  ||  'FUENTE'
 4 -> 'TIPO DE PROCESO AGRARIO'  ||  'TIPO DE PROCESO AGRARIO'
 5 -> 'ETAPA PROCESAL'  ||  'ETAPA PROCESAL'
 6 -> 'Dependencia'  ||  'DEPENDENCIA'
 7 -> 'ID ORFEO'  ||  'ID ORFEO'
 8 -> 'DEPARTAMENTO'  ||  'DEPARTAMENTO'
 9 -> 'MUNICIPIO'  ||  'MUNICIPIO'
10 -> 'ÁREA REGISTRAL
 (0,0000 = Has,M2)'  ||  'AREA REGISTRAL
 (0,0000 = HAS,M2)'
11 -> 'CÍRCULO REGISTRAL'  ||  'CIRCULO REGISTRAL'
12 -> 'ÁREA CALCULADA'  ||  'AREA CALCULADA'
13 -> 'PERSONA TITULAR
(Natural / Jurídica)'  ||  'PERSONA TITULAR
(NATURAL / JURIDICA)'
14 -> 'NOMBRE DEL (OS) PROPIETARIO (S)'  ||  'NOMBRE DEL (OS) PROPIETARIO (S)'
15 -> 'NUMERO DE IDENTIFICACIÓN 
(CC / NIT)'  ||  'NUMERO DE IDENTIFICACION 
(CC / NIT)'
16 -> 'ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN '  ||  'ANALISIS DE LA NATURALEZA JURIDICA Y ULTIMA TRADICION'
17 -> 

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\2777423384.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaEnvioCoord_RAW'] = df[col_envio]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\2777423384.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaEnvioCoord_TXT_LIMPIO'] = df['FechaEnvioCoord_RAW'].apply(limpiar_fecha_texto)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\2777423384.py:55: UserWarning: Parsing dates in %Y/%m/%d/%H:%M:%S format when dayfirst=True was s


=== Resumen FECHA DE ENVÍO A COORDINACIÓN ===
Total filas: 24055
Fechas válidas (no NaT): 2544
Fechas inválidas (NaT con texto no vacío): 4
Sin fecha (vacío o 'NO/NO REGISTRA/etc.'): 21507

=== Muestra de FECHA DE ENVÍO A COORDINACIÓN (primeras 20 filas) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO            MUNICIPIO FechaEnvioCoord_RAW FechaEnvioCoord_TXT_LIMPIO FechaEnvioACoordinacion_SQL
                2  062-37357      BOLIVAR             ZAMBRANO                 NaN                       None                         NaT
                3  062-37359      BOLIVAR             ZAMBRANO                 NaN                       None                         NaT
                4  062-37361      BOLIVAR             ZAMBRANO                 NaN                       None                         NaT
                5  062-37354      BOLIVAR             ZAMBRANO 2025-02-24 00:00:00        2025/02/24/00:00:00                  2025-02-24
                6  142-23909      CORDOBA         

## ESTADO APROBACIÓN COORDINADORA


In [ ]:
import pandas as pd
import unicodedata
import re

# ============================================
# 1. Función para normalizar encabezados
# ============================================
def normalizar_header(s: str) -> str:
    if s is None:
        return ""
    s_norm = unicodedata.normalize('NFD', str(s))
    s_sin_tildes = ''.join(ch for ch in s_norm if unicodedata.category(ch) != 'Mn')
    return s_sin_tildes.upper().strip()

# ============================================
# 2. Localizar la columna "ESTADO APROBACIÓN COORDINADORA"
# ============================================
norm_cols = {c: normalizar_header(c) for c in df.columns}

print("=== Columnas del DataFrame y su versión normalizada ===")
for i, (orig, norm) in enumerate(norm_cols.items()):
    print(f"{i:2d} -> '{orig}'  ||  '{norm}'")

candidatas_estado_coord = [
    c for c, norm in norm_cols.items()
    if "ESTADO" in norm and "APROB" in norm and "COORDIN" in norm
]

print("\nColumnas candidatas para ESTADO APROBACIÓN COORDINADORA:", candidatas_estado_coord)

if not candidatas_estado_coord:
    raise ValueError("No se encontró la columna de ESTADO APROBACIÓN COORDINADORA en el DataFrame.")

col_estado = candidatas_estado_coord[0]
print("Usando columna:", repr(col_estado))

# ============================================
# 3. Crear columnas RAW y normalizada (solo MAYÚSCULAS)
# ============================================

df['EstadoAprobCoord_RAW'] = df[col_estado]

def limpiar_estado_sin_catalogo(v):
    """
    - Si viene vacío/NaN -> None
    - Si trae texto -> se deja igual pero:
        * sin tildes
        * espacios normalizados
        * en MAYÚSCULAS
    No se asigna a ningún catálogo.
    """
    if pd.isna(v):
        return None

    t = str(v).strip()
    if t == "":
        return None

    # Quitar tildes
    t_norm = unicodedata.normalize('NFD', t)
    t_norm = ''.join(ch for ch in t_norm if unicodedata.category(ch) != 'Mn')
    # Colapsar espacios y a mayúsculas
    t_norm = re.sub(r"\s+", " ", t_norm).strip().upper()

    return t_norm

df['EstadoAprobacionCoordinadora_SQL'] = df['EstadoAprobCoord_RAW'].apply(limpiar_estado_sin_catalogo)

# Longitud para validar con varchar(100)
df['EstadoAprobCoord_LEN'] = df['EstadoAprobacionCoordinadora_SQL'].astype(str).str.len()

# ============================================
# 4. Resúmenes
# ============================================

print("\n=== Valores CRUDOS (top 30) ESTADO APROBACIÓN COORDINADORA ===")
print(df['EstadoAprobCoord_RAW'].value_counts(dropna=False).head(30))

print("\n=== Valores NORMALIZADOS (top 30, solo MAYÚSCULAS) ===")
print(df['EstadoAprobacionCoordinadora_SQL'].value_counts(dropna=False).head(30))

# Filas que superan los 100 caracteres (no caben en varchar(100))
largos = df[df['EstadoAprobCoord_LEN'] > 100]

if not largos.empty:
    print("\n=== Filas donde el texto supera los 100 caracteres (revisar / recortar) ===")
    print(
        largos[
            ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
             'EstadoAprobCoord_RAW', 'EstadoAprobacionCoordinadora_SQL',
             'EstadoAprobCoord_LEN']
        ].to_string(index=False)
    )
else:
    print("\nNingún valor supera 100 caracteres (ok para varchar(100)).")

# ============================================
# 5. Exportar a Excel para inspección visual
# ============================================

cols_excel = [
    'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'EstadoAprobCoord_RAW', 'EstadoAprobacionCoordinadora_SQL',
    'EstadoAprobCoord_LEN'
]

df[cols_excel].to_excel(
    "EstadoAprobacionCoordinadora_sin_catalogo.xlsx",
    index=False
)

print("\nArchivo generado:")
print(" - EstadoAprobacionCoordinadora_sin_catalogo.xlsx  (crudo + MAYÚSCULAS + longitud)")


=== Columnas del DataFrame y su versión normalizada ===
 0 -> 'ID'  ||  'ID'
 1 -> 'FMI'  ||  'FMI'
 2 -> 'NumeroExpediente'  ||  'NUMEROEXPEDIENTE'
 3 -> 'FUENTE'  ||  'FUENTE'
 4 -> 'TIPO DE PROCESO AGRARIO'  ||  'TIPO DE PROCESO AGRARIO'
 5 -> 'ETAPA PROCESAL'  ||  'ETAPA PROCESAL'
 6 -> 'Dependencia'  ||  'DEPENDENCIA'
 7 -> 'ID ORFEO'  ||  'ID ORFEO'
 8 -> 'DEPARTAMENTO'  ||  'DEPARTAMENTO'
 9 -> 'MUNICIPIO'  ||  'MUNICIPIO'
10 -> 'ÁREA REGISTRAL
 (0,0000 = Has,M2)'  ||  'AREA REGISTRAL
 (0,0000 = HAS,M2)'
11 -> 'CÍRCULO REGISTRAL'  ||  'CIRCULO REGISTRAL'
12 -> 'ÁREA CALCULADA'  ||  'AREA CALCULADA'
13 -> 'PERSONA TITULAR
(Natural / Jurídica)'  ||  'PERSONA TITULAR
(NATURAL / JURIDICA)'
14 -> 'NOMBRE DEL (OS) PROPIETARIO (S)'  ||  'NOMBRE DEL (OS) PROPIETARIO (S)'
15 -> 'NUMERO DE IDENTIFICACIÓN 
(CC / NIT)'  ||  'NUMERO DE IDENTIFICACION 
(CC / NIT)'
16 -> 'ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN '  ||  'ANALISIS DE LA NATURALEZA JURIDICA Y ULTIMA TRADICION'
17 -> 

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\3974494110.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['EstadoAprobCoord_RAW'] = df[col_estado]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\3974494110.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['EstadoAprobacionCoordinadora_SQL'] = df['EstadoAprobCoord_RAW'].apply(limpiar_estado_sin_catalogo)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\3974494110.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usu


Archivo generado:
 - EstadoAprobacionCoordinadora_sin_catalogo.xlsx  (crudo + MAYÚSCULAS + longitud)


## FECHA DE REMISIÓN SOPORTES A GESTORA DOCUMENTAL

In [ ]:
import pandas as pd
import unicodedata
import re
from datetime import date, datetime

# ============================================
# 1. Función para normalizar encabezados
# ============================================
def normalizar_header(s: str) -> str:
    if s is None:
        return ""
    s_norm = unicodedata.normalize('NFD', str(s))
    s_sin_tildes = ''.join(ch for ch in s_norm if unicodedata.category(ch) != 'Mn')
    return s_sin_tildes.upper().strip()

# ============================================
# 2. Localizar la columna de FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL
# ============================================
norm_cols = {c: normalizar_header(c) for c in df.columns}

print("=== Columnas del DataFrame y su versión normalizada ===")
for i, (orig, norm) in enumerate(norm_cols.items()):
    print(f"{i:2d} -> '{orig}'  ||  '{norm}'")

# Buscamos algo tipo: FECHA ... SOPORTES ... GESTORA
candidatas_rem = [
    c for c, norm in norm_cols.items()
    if "FECHA" in norm and "SOPORTES" in norm and "GESTORA" in norm
]

# Plan B (un poco más laxo) si no encontró
if not candidatas_rem:
    candidatas_rem = [
        c for c, norm in norm_cols.items()
        if "FECHA" in norm and "GESTORA" in norm
    ]

print("\nColumnas candidatas para FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL:", candidatas_rem)

if not candidatas_rem:
    raise ValueError("No se encontró la columna de FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL en el DataFrame.")

col_rem = candidatas_rem[0]
print("Usando columna:", repr(col_rem))

# ============================================
# 3. Funciones de limpieza de fecha
# ============================================
def corregir_formato_raro_fecha(t: str) -> str:
    """
    Corrige casos tipo:
      - '20/112024' -> '20/11/2024'
      - '17/052024' -> '17/05/2024'
      - '25/042024' -> '25/04/2024'
      - '2103/2024' -> '21/03/2024'
    """
    t = t.strip()

    # 20/112024 -> 20/11/2024  (dd/mm + aaaa pegado)
    m = re.match(r"^(\d{1,2})/(\d{2})(\d{4})$", t)
    if m:
        d, mm, yyyy = m.groups()
        return f"{d}/{mm}/{yyyy}"

    # 2103/2024 -> 21/03/2024  (ddmm/aaaa)
    m = re.match(r"^(\d{2})(\d{2})/(\d{4})$", t)
    if m:
        dd, mm, yyyy = m.groups()
        return f"{dd}/{mm}/{yyyy}"

    return t

def limpiar_texto_fecha_generica(v):
    """
    Limpieza estándar de fechas:
      - NaN -> None
      - Timestamps -> se devuelven tal cual
      - Textos tipo 'NO REGISTRA', 'NO APLICA', etc. -> None
      - Quita saltos de línea / espacios
      - Aplica corrección de formatos raros
    """
    if pd.isna(v):
        return None

    if isinstance(v, (pd.Timestamp, datetime, date)):
        return v

    t = str(v).strip()
    if t == "":
        return None

    t_upper = t.upper().replace("\u00A0", " ")

    patrones_sin_fecha = [
        "NO REGISTRA",
        "NO APLICA",
        "NO APLICA.",
        "NO REPORTA",
        "NO TIENE",
        "NO",
        "N/A",
        "NA",
        "SIN FECHA",
        "SIN DATO",
        "NO INFORMA",
    ]
    if any(pat in t_upper for pat in patrones_sin_fecha):
        return None

    t_limpio = re.sub(r"\s+", " ", t)
    t_limpio = t_limpio.replace("\n", " ").replace("\t", " ").strip()

    t_limpio = corregir_formato_raro_fecha(t_limpio)

    return t_limpio

# ============================================
# 4. Columnas RAW / LIMPIA / SQL
# ============================================
df['FechaRemSoportes_RAW'] = df[col_rem]
df['FechaRemSoportes_TXT_LIMPIO'] = df['FechaRemSoportes_RAW'].apply(limpiar_texto_fecha_generica)

df['FechaRemisionSoportesAGestoraDocumental_SQL'] = pd.to_datetime(
    df['FechaRemSoportes_TXT_LIMPIO'],
    errors='coerce',
    dayfirst=True
)

# ============================================
# 5. Resumen
# ============================================
validas = df['FechaRemisionSoportesAGestoraDocumental_SQL'].notna().sum()
invalidas_con_texto = (
    df['FechaRemisionSoportesAGestoraDocumental_SQL'].isna()
    & df['FechaRemSoportes_TXT_LIMPIO'].notna()
    & (df['FechaRemSoportes_TXT_LIMPIO'].astype(str).str.strip() != "")
).sum()

print("\n=== Resumen FECHA DE REMISIÓN SOPORTES A GESTORA DOCUMENTAL ===")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")

# ============================================
# 6. Ejemplos de filas todavía inválidas (texto raro)
# ============================================
mask_invalid = (
    df['FechaRemisionSoportesAGestoraDocumental_SQL'].isna()
    & df['FechaRemSoportes_TXT_LIMPIO'].notna()
    & (df['FechaRemSoportes_TXT_LIMPIO'].astype(str).str.strip() != "")
)

ej_invalidas = df.loc[mask_invalid,
    ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
     'FechaRemSoportes_RAW', 'FechaRemSoportes_TXT_LIMPIO']
].head(50)

print("\n=== Ejemplos de FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL que siguen sin parsear (revisar) ===")
if not ej_invalidas.empty:
    print(ej_invalidas.to_string(index=False))
else:
    print("No hay filas con texto 'raro': todo lo que tiene texto se pudo convertir a fecha.")

# ============================================
# 7. Vista general RAW vs LIMPIO vs SQL (como se va a migrar)
# ============================================
cols_ver = [
    'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'FechaRemSoportes_RAW',
    'FechaRemSoportes_TXT_LIMPIO',
    'FechaRemisionSoportesAGestoraDocumental_SQL'
]

print("\n=== MUESTRA (RAW / LIMPIO / SQL) FECHA REMISIÓN SOPORTES A GESTORA DOCUMENTAL ===")
print(df[cols_ver].head(50).to_string(index=False))

# ============================================
# 8. Exportar TODO a Excel para revisión visual
# ============================================
nombre_excel = "FechaRemisionSoportesGestora_raw_limpio_sql.xlsx"
df[cols_ver].to_excel(nombre_excel, index=False)
print(f"\nArchivo creado: {nombre_excel}   Filas: {len(df)}")

# (Opcional) También puedes exportar SOLO las inválidas si las hay:
if not ej_invalidas.empty:
    nombre_excel_inv = "FechaRemisionSoportesGestora_invalidas.xlsx"
    df.loc[mask_invalid, cols_ver].to_excel(nombre_excel_inv, index=False)
    print(f"Archivo con sólo filas inválidas: {nombre_excel_inv}   Filas: {mask_invalid.sum()}")


=== Columnas del DataFrame y su versión normalizada ===
 0 -> 'ID'  ||  'ID'
 1 -> 'FMI'  ||  'FMI'
 2 -> 'NumeroExpediente'  ||  'NUMEROEXPEDIENTE'
 3 -> 'FUENTE'  ||  'FUENTE'
 4 -> 'TIPO DE PROCESO AGRARIO'  ||  'TIPO DE PROCESO AGRARIO'
 5 -> 'ETAPA PROCESAL'  ||  'ETAPA PROCESAL'
 6 -> 'Dependencia'  ||  'DEPENDENCIA'
 7 -> 'ID ORFEO'  ||  'ID ORFEO'
 8 -> 'DEPARTAMENTO'  ||  'DEPARTAMENTO'
 9 -> 'MUNICIPIO'  ||  'MUNICIPIO'
10 -> 'ÁREA REGISTRAL
 (0,0000 = Has,M2)'  ||  'AREA REGISTRAL
 (0,0000 = HAS,M2)'
11 -> 'CÍRCULO REGISTRAL'  ||  'CIRCULO REGISTRAL'
12 -> 'ÁREA CALCULADA'  ||  'AREA CALCULADA'
13 -> 'PERSONA TITULAR
(Natural / Jurídica)'  ||  'PERSONA TITULAR
(NATURAL / JURIDICA)'
14 -> 'NOMBRE DEL (OS) PROPIETARIO (S)'  ||  'NOMBRE DEL (OS) PROPIETARIO (S)'
15 -> 'NUMERO DE IDENTIFICACIÓN 
(CC / NIT)'  ||  'NUMERO DE IDENTIFICACION 
(CC / NIT)'
16 -> 'ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN '  ||  'ANALISIS DE LA NATURALEZA JURIDICA Y ULTIMA TRADICION'
17 -> 

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\2771235217.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaRemSoportes_RAW'] = df[col_rem]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\2771235217.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaRemSoportes_TXT_LIMPIO'] = df['FechaRemSoportes_RAW'].apply(limpiar_texto_fecha_generica)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\2771235217.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually


Archivo creado: FechaRemisionSoportesGestora_raw_limpio_sql.xlsx   Filas: 24055
Archivo con sólo filas inválidas: FechaRemisionSoportesGestora_invalidas.xlsx   Filas: 23


## FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL

In [ ]:
import pandas as pd
import unicodedata
import re
from datetime import date, datetime

# ============================================
# 1. Función para normalizar encabezados
# ============================================
def normalizar_header(s: str) -> str:
    if s is None:
        return ""
    s_norm = unicodedata.normalize('NFD', str(s))
    s_sin_tildes = ''.join(ch for ch in s_norm if unicodedata.category(ch) != 'Mn')
    return s_sin_tildes.upper().strip()

# ============================================
# 2. Localizar la columna "FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL"
# ============================================
norm_cols = {c: normalizar_header(c) for c in df.columns}

print("=== Columnas del DataFrame y su versión normalizada ===")
for i, (orig, norm) in enumerate(norm_cols.items()):
    print(f"{i:2d} -> '{orig}'  ||  '{norm}'")

# Buscamos algo que contenga FECHA + INFORME + GESTORA
candidatas_inf = [
    c for c, norm in norm_cols.items()
    if "FECHA" in norm and "INFORME" in norm and "GESTORA" in norm
]

# Plan B (por si en el encabezado no quedó "GESTORA" explícito)
if not candidatas_inf:
    candidatas_inf = [
        c for c, norm in norm_cols.items()
        if "FECHA" in norm and "INFORME" in norm
    ]

print("\nColumnas candidatas para FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL:", candidatas_inf)

if not candidatas_inf:
    raise ValueError("No se encontró la columna de FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL en el DataFrame.")

col_inf = candidatas_inf[0]
print("Usando columna:", repr(col_inf))

# ============================================
# 3. Funciones de limpieza de fecha (las mismas que antes)
# ============================================
def corregir_formato_raro_fecha(t: str) -> str:
    """
    Corrige casos tipo:
      - '20/112024' -> '20/11/2024'
      - '17/052024' -> '17/05/2024'
      - '25/042024' -> '25/04/2024'
      - '2103/2024' -> '21/03/2024'
    """
    t = t.strip()

    # 20/112024 -> 20/11/2024  (dd/mm + aaaa pegado)
    m = re.match(r"^(\d{1,2})/(\d{2})(\d{4})$", t)
    if m:
        d, mm, yyyy = m.groups()
        return f"{d}/{mm}/{yyyy}"

    # 2103/2024 -> 21/03/2024  (ddmm/aaaa)
    m = re.match(r"^(\d{2})(\d{2})/(\d{4})$", t)
    if m:
        dd, mm, yyyy = m.groups()
        return f"{dd}/{mm}/{yyyy}"

    return t

def limpiar_texto_fecha_generica(v):
    """
    Limpieza estándar de fechas:
      - NaN -> None
      - Timestamps -> se devuelven tal cual
      - Textos tipo 'NO REGISTRA', 'NO APLICA', etc. -> None
      - Quita saltos de línea / espacios
      - Aplica corrección de formatos raros
    """
    if pd.isna(v):
        return None

    if isinstance(v, (pd.Timestamp, datetime, date)):
        return v

    t = str(v).strip()
    if t == "":
        return None

    t_upper = t.upper().replace("\u00A0", " ")

    patrones_sin_fecha = [
        "NO REGISTRA",
        "NO APLICA",
        "NO APLICA.",
        "NO REPORTA",
        "NO TIENE",
        "NO",
        "N/A",
        "NA",
        "SIN FECHA",
        "SIN DATO",
        "NO INFORMA",
    ]
    if any(pat in t_upper for pat in patrones_sin_fecha):
        return None

    t_limpio = re.sub(r"\s+", " ", t)
    t_limpio = t_limpio.replace("\n", " ").replace("\t", " ").strip()

    t_limpio = corregir_formato_raro_fecha(t_limpio)

    return t_limpio

# ============================================
# 4. Columnas RAW / LIMPIA / SQL para FECHA REMISIÓN INFORME
# ============================================
df['FechaRemInforme_RAW'] = df[col_inf]
df['FechaRemInforme_TXT_LIMPIO'] = df['FechaRemInforme_RAW'].apply(limpiar_texto_fecha_generica)

df['FechaRemisionInformeAGestoraDocumental_SQL'] = pd.to_datetime(
    df['FechaRemInforme_TXT_LIMPIO'],
    errors='coerce',
    dayfirst=True
)

# ============================================
# 5. Resumen de calidad de datos
# ============================================
validas = df['FechaRemisionInformeAGestoraDocumental_SQL'].notna().sum()
invalidas_con_texto = (
    df['FechaRemisionInformeAGestoraDocumental_SQL'].isna()
    & df['FechaRemInforme_TXT_LIMPIO'].notna()
    & (df['FechaRemInforme_TXT_LIMPIO'].astype(str).str.strip() != "")
).sum()

print("\n=== Resumen FECHA DE REMISIÓN INFORME A GESTORA DOCUMENTAL ===")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")

# ============================================
# 6. Ejemplos de filas todavía inválidas (texto raro)
# ============================================
mask_invalid = (
    df['FechaRemisionInformeAGestoraDocumental_SQL'].isna()
    & df['FechaRemInforme_TXT_LIMPIO'].notna()
    & (df['FechaRemInforme_TXT_LIMPIO'].astype(str).str.strip() != "")
)

ej_invalidas = df.loc[mask_invalid,
    ['Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
     'FechaRemInforme_RAW', 'FechaRemInforme_TXT_LIMPIO']
].head(50)

print("\n=== Ejemplos de FECHA REMISIÓN INFORME que siguen sin parsear (revisar) ===")
if not ej_invalidas.empty:
    print(ej_invalidas.to_string(index=False))
else:
    print("No hay filas con texto 'raro': todo lo que tiene texto se pudo convertir a fecha.")

# ============================================
# 7. Vista general RAW vs LIMPIO vs SQL (como se va a migrar)
# ============================================
cols_ver = [
    'Fila_Excel_aprox', 'FMI', 'DEPARTAMENTO', 'MUNICIPIO',
    'FechaRemInforme_RAW',
    'FechaRemInforme_TXT_LIMPIO',
    'FechaRemisionInformeAGestoraDocumental_SQL'
]

print("\n=== MUESTRA (RAW / LIMPIO / SQL) FECHA REMISIÓN INFORME A GESTORA DOCUMENTAL ===")
print(df[cols_ver].head(50).to_string(index=False))

# ============================================
# 8. Exportar TODO a Excel para revisión visual
# ============================================
nombre_excel = "FechaRemisionInformeGestora_raw_limpio_sql.xlsx"
df[cols_ver].to_excel(nombre_excel, index=False)
print(f"\nArchivo creado: {nombre_excel}   Filas: {len(df)}")

# (Opcional) también sólo las inválidas, si existen
if not ej_invalidas.empty:
    nombre_excel_inv = "FechaRemisionInformeGestora_invalidas.xlsx"
    df.loc[mask_invalid, cols_ver].to_excel(nombre_excel_inv, index=False)
    print(f"Archivo con sólo filas inválidas: {nombre_excel_inv}   Filas: {mask_invalid.sum()}")


=== Columnas del DataFrame y su versión normalizada ===
 0 -> 'ID'  ||  'ID'
 1 -> 'FMI'  ||  'FMI'
 2 -> 'NumeroExpediente'  ||  'NUMEROEXPEDIENTE'
 3 -> 'FUENTE'  ||  'FUENTE'
 4 -> 'TIPO DE PROCESO AGRARIO'  ||  'TIPO DE PROCESO AGRARIO'
 5 -> 'ETAPA PROCESAL'  ||  'ETAPA PROCESAL'
 6 -> 'Dependencia'  ||  'DEPENDENCIA'
 7 -> 'ID ORFEO'  ||  'ID ORFEO'
 8 -> 'DEPARTAMENTO'  ||  'DEPARTAMENTO'
 9 -> 'MUNICIPIO'  ||  'MUNICIPIO'
10 -> 'ÁREA REGISTRAL
 (0,0000 = Has,M2)'  ||  'AREA REGISTRAL
 (0,0000 = HAS,M2)'
11 -> 'CÍRCULO REGISTRAL'  ||  'CIRCULO REGISTRAL'
12 -> 'ÁREA CALCULADA'  ||  'AREA CALCULADA'
13 -> 'PERSONA TITULAR
(Natural / Jurídica)'  ||  'PERSONA TITULAR
(NATURAL / JURIDICA)'
14 -> 'NOMBRE DEL (OS) PROPIETARIO (S)'  ||  'NOMBRE DEL (OS) PROPIETARIO (S)'
15 -> 'NUMERO DE IDENTIFICACIÓN 
(CC / NIT)'  ||  'NUMERO DE IDENTIFICACION 
(CC / NIT)'
16 -> 'ANÁLISIS DE LA NATURALEZA JURÍDICA Y ÚLTIMA TRADICIÓN '  ||  'ANALISIS DE LA NATURALEZA JURIDICA Y ULTIMA TRADICION'
17 -> 

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\2480942292.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaRemInforme_RAW'] = df[col_inf]
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\2480942292.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['FechaRemInforme_TXT_LIMPIO'] = df['FechaRemInforme_RAW'].apply(limpiar_texto_fecha_generica)
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\2480942292.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually th


Archivo creado: FechaRemisionInformeGestora_raw_limpio_sql.xlsx   Filas: 24055
Archivo con sólo filas inválidas: FechaRemisionInformeGestora_invalidas.xlsx   Filas: 1


## FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO

In [ ]:
import numpy as np
import pandas as pd
import re

# ============================================
# 1) Ubicar la columna de FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO
# ============================================
candidatas_cargue = [
    c for c in df.columns
    if ('FECHA'   in str(c).upper()
        and 'CARG'   in str(c).upper()   # CARGUE / CARGADO
        and 'INFORME' in str(c).upper()
        and 'ORFEO'  in str(c).upper())
]

print("Columnas candidatas para FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO:", candidatas_cargue)

if candidatas_cargue:
    col_cargue_orfeo = candidatas_cargue[0]
else:
    # Ajusta este nombre si en tu df.columns ves algo distinto
    col_cargue_orfeo = 'FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO'

print("Usando columna:", col_cargue_orfeo)

# ============================================
# 2) Columnas auxiliares RAW / TEXTO LIMPIO
# ============================================
df['FechaCargueInformeOrfeo_RAW'] = df[col_cargue_orfeo]

df['FechaCargueInformeOrfeo_TXT_LIMPIO'] = (
    df['FechaCargueInformeOrfeo_RAW']
      .astype(str)
      .str.strip()
)

# Normalizar vacíos típicos
mask_vacio = df['FechaCargueInformeOrfeo_TXT_LIMPIO'].str.upper().isin(['', 'NAN', 'NAT'])
df.loc[mask_vacio, 'FechaCargueInformeOrfeo_TXT_LIMPIO'] = np.nan

df['FechaCargueInformeOrfeo_TXT_UP'] = (
    df['FechaCargueInformeOrfeo_TXT_LIMPIO']
      .astype(str)
      .str.strip()
      .str.upper()
)

# ============================================
# 3) Función de parseo a datetime
# ============================================
def parse_fecha_cargue_orfeo(valor):
    """
    Convierte a datetime valores tipo fecha.
    Deja como NaT textos como 'NO REGISTRA', 'NO APLICA',
    'FIRMADO Y CARGADO EN ORFEO', etc.
    """
    if pd.isna(valor):
        return pd.NaT

    txt = str(valor).strip()
    if txt == "":
        return pd.NaT

    tu = txt.upper()

    # Textos claramente NO fecha -> NaT
    if tu in ("NO", "NO REGISTRA", "NO APLICA", "N/A"):
        return pd.NaT
    if "FIRMADO Y CARGADO EN ORFEO" in tu:
        return pd.NaT
    if "CARGADO EN ORFEO" in tu and not any(ch.isdigit() for ch in tu):
        return pd.NaT

    # Unificar separadores raros a "/"
    t = re.sub(r"[-.]", "/", txt)
    t = re.sub(r"\s+", "/", t)  # "2024 04 25" -> "2024/04/25"

    # Intento genérico con dayfirst
    fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')

    return fecha  # si no puede, queda NaT


# ============================================
# 4) Columna final para SQL
# ============================================
df['FechaCargueInformeJuridicoExpOrfeo_SQL'] = (
    df['FechaCargueInformeOrfeo_TXT_LIMPIO'].apply(parse_fecha_cargue_orfeo)
)

# ============================================
# 5) Resumen y muestras (válidos e inválidos)
# ============================================
mask_valida = df['FechaCargueInformeJuridicoExpOrfeo_SQL'].notna()

mask_con_texto = (
    df['FechaCargueInformeOrfeo_TXT_LIMPIO'].notna()
    & (df['FechaCargueInformeOrfeo_TXT_LIMPIO'].astype(str).str.strip() != '')
)

mask_invalida_con_texto = ~mask_valida & mask_con_texto

validas = mask_valida.sum()
invalidas_con_texto = mask_invalida_con_texto.sum()

print("\n=== Resumen FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO ===")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")

# Muestra de filas listas para migrar (válidas)
ej_validos = df.loc[mask_valida, [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    col_cargue_orfeo,
    'FechaCargueInformeOrfeo_TXT_LIMPIO',
    'FechaCargueInformeJuridicoExpOrfeo_SQL'
]].head(30)

print("\n=== Ejemplos de filas LISTAS PARA MIGRAR (fechas válidas) ===")
if not ej_validos.empty:
    print(ej_validos.to_string(index=False))
else:
    print("No hay fechas válidas para mostrar.")

# Muestra de filas con texto que NO se pudo convertir
ej_invalidos = df.loc[mask_invalida_con_texto, [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    'FechaCargueInformeOrfeo_RAW',
    'FechaCargueInformeOrfeo_TXT_LIMPIO'
]].head(50)

print("\n=== Ejemplos de filas con TEXTO que NO se pudo convertir (para revisar) ===")
if not ej_invalidos.empty:
    print(ej_invalidos.to_string(index=False))
else:
    print("No hay filas con texto no vacío que sigan inválidas.")


# ============================================
# 6) Exportar Excels
# ============================================

# 6.1. Excel con TODOS los registros (crudo vs limpio vs SQL)
cols_export_todos = [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    col_cargue_orfeo,                          # columna original del Excel
    'FechaCargueInformeOrfeo_RAW',             # valor crudo
    'FechaCargueInformeOrfeo_TXT_LIMPIO',      # texto limpio
    'FechaCargueInformeJuridicoExpOrfeo_SQL'   # fecha final para SQL
]

df_export_todos = df[cols_export_todos].copy()
df_export_todos.to_excel(
    'FechaCargueInforme_Juridico_ORFEO_raw_vs_sql.xlsx',
    index=False
)
print("\nArchivo creado (todos): FechaCargueInforme_Juridico_ORFEO_raw_vs_sql.xlsx")

# 6.2. Excel SOLO con registros inválidos (texto que no se pudo convertir)
df_export_invalidos = df.loc[mask_invalida_con_texto, cols_export_todos].copy()
df_export_invalidos.to_excel(
    'FechaCargueInforme_Juridico_ORFEO_invalidos.xlsx',
    index=False
)
print("Archivo creado (inválidos): FechaCargueInforme_Juridico_ORFEO_invalidos.xlsx")


Columnas candidatas para FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO: ['FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO', 'FechaCargueInformeOrfeo_RAW', 'FechaCargueInformeOrfeo_TXT_LIMPIO', 'FechaCargueInformeOrfeo_TXT_UP', 'FechaCargueInformeJuridicoExpOrfeo_SQL']
Usando columna: FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\1362797932.py:79: UserWarning: Parsing dates in %Y/%m/%d/%H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')



=== Resumen FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO ===
Fechas válidas (no NaT): 3370
Fechas inválidas (NaT con texto no vacío): 48

=== Ejemplos de filas LISTAS PARA MIGRAR (fechas válidas) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO                   MUNICIPIO FECHA CARGUE INFORME JURÍDICO EN EXPTE ORFEO FechaCargueInformeOrfeo_TXT_LIMPIO FechaCargueInformeJuridicoExpOrfeo_SQL
                4  062-37361      BOLIVAR                    ZAMBRANO                          2024-11-21 00:00:00                2024-11-21 00:00:00                             2024-11-21
                5  062-37354      BOLIVAR                    ZAMBRANO                          2025-02-25 00:00:00                2025-02-25 00:00:00                             2025-02-25
                7  062-37360      BOLIVAR                    ZAMBRANO                          2024-11-06 00:00:00                2024-11-06 00:00:00                             2024-06-11
                8  062-37356      BOLIVAR  

## FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO

In [ ]:
import numpy as np
import pandas as pd
import re

# ============================================
# 1) Ubicar la columna de FECHA DE CARGUE DOCS Y SOPORTES EN EXPDTE ORFEO
# ============================================
candidatas_docs_orfeo = [
    c for c in df.columns
    if ('FECHA'   in str(c).upper()
        and 'CARG'   in str(c).upper()        # CARGUE / CARGADO
        and ('DOC'   in str(c).upper() or 'SOPORT' in str(c).upper())
        and 'ORFEO'  in str(c).upper())
]

print("Columnas candidatas para FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO:",
      candidatas_docs_orfeo)

if candidatas_docs_orfeo:
    col_cargue_docs_orfeo = candidatas_docs_orfeo[0]
else:
    # Ajusta este nombre si en tu df.columns ves algo distinto
    col_cargue_docs_orfeo = 'FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO'

print("Usando columna:", col_cargue_docs_orfeo)

# ============================================
# 2) Columnas auxiliares RAW / TEXTO LIMPIO
# ============================================
df['FechaCargueDocsSoportesOrfeo_RAW'] = df[col_cargue_docs_orfeo]

df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'] = (
    df['FechaCargueDocsSoportesOrfeo_RAW']
      .astype(str)
      .str.strip()
)

# Normalizar vacíos típicos
mask_vacio = df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'].str.upper().isin(['', 'NAN', 'NAT'])
df.loc[mask_vacio, 'FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'] = np.nan

df['FechaCargueDocsSoportesOrfeo_TXT_UP'] = (
    df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO']
      .astype(str)
      .str.strip()
      .str.upper()
)

# ============================================
# 3) Función de parseo a datetime
# ============================================
def parse_fecha_cargue_docs_orfeo(valor):
    """
    Convierte a datetime valores tipo fecha para
    FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO.
    Deja como NaT textos como 'NO REGISTRA', 'NO APLICA',
    'FIRMADO Y CARGADO EN ORFEO', etc.
    """
    if pd.isna(valor):
        return pd.NaT

    txt = str(valor).strip()
    if txt == "":
        return pd.NaT

    tu = txt.upper()

    # Textos claramente NO fecha -> NaT
    if tu in ("NO", "NO REGISTRA", "NO APLICA", "N/A"):
        return pd.NaT
    if "FIRMADO Y CARGADO EN ORFEO" in tu:
        return pd.NaT
    if "CARGADO EN ORFEO" in tu and not any(ch.isdigit() for ch in tu):
        return pd.NaT

    # Unificar separadores raros a "/"
    t = re.sub(r"[-.]", "/", txt)
    t = re.sub(r"\s+", "/", t)  # "2024 04 25" -> "2024/04/25"

    # Intento genérico con dayfirst
    fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')

    return fecha  # si no puede, queda NaT


# ============================================
# 4) Columna final para SQL
# ============================================
df['FechaDeCargueDocsYSoportesExpOrfeo_SQL'] = (
    df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'].apply(parse_fecha_cargue_docs_orfeo)
)

# ============================================
# 5) Resumen y muestras (válidos e inválidos)
# ============================================
mask_valida = df['FechaDeCargueDocsYSoportesExpOrfeo_SQL'].notna()

mask_con_texto = (
    df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'].notna()
    & (df['FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'].astype(str).str.strip() != '')
)

mask_invalida_con_texto = ~mask_valida & mask_con_texto

validas = mask_valida.sum()
invalidas_con_texto = mask_invalida_con_texto.sum()

print("\n=== Resumen FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO ===")
print(f"Fechas válidas (no NaT): {validas}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto}")

# Muestra de filas listas para migrar (válidas)
ej_validos = df.loc[mask_valida, [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    col_cargue_docs_orfeo,
    'FechaCargueDocsSoportesOrfeo_TXT_LIMPIO',
    'FechaDeCargueDocsYSoportesExpOrfeo_SQL'
]].head(30)

print("\n=== Ejemplos de filas LISTAS PARA MIGRAR (fechas válidas) ===")
if not ej_validos.empty:
    print(ej_validos.to_string(index=False))
else:
    print("No hay fechas válidas para mostrar.")

# Muestra de filas con texto que NO se pudo convertir
ej_invalidos = df.loc[mask_invalida_con_texto, [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    'FechaCargueDocsSoportesOrfeo_RAW',
    'FechaCargueDocsSoportesOrfeo_TXT_LIMPIO'
]].head(50)

print("\n=== Ejemplos de filas con TEXTO que NO se pudo convertir (para revisar) ===")
if not ej_invalidos.empty:
    print(ej_invalidos.to_string(index=False))
else:
    print("No hay filas con texto no vacío que sigan inválidas.")


# ============================================
# 6) Exportar Excels (ORFEO, docs + soportes)
# ============================================

# 6.1. Excel con TODOS los registros (crudo vs limpio vs SQL)
cols_export_todos = [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    col_cargue_docs_orfeo,                       # columna original del Excel
    'FechaCargueDocsSoportesOrfeo_RAW',          # valor crudo
    'FechaCargueDocsSoportesOrfeo_TXT_LIMPIO',   # texto limpio
    'FechaDeCargueDocsYSoportesExpOrfeo_SQL'     # fecha final para SQL
]

df_export_todos = df[cols_export_todos].copy()
df_export_todos.to_excel(
    'FechaCargue_Docs_Soportes_ORFEO_raw_vs_sql.xlsx',
    index=False
)
print("\nArchivo creado (todos): FechaCargue_Docs_Soportes_ORFEO_raw_vs_sql.xlsx")

# 6.2. Excel SOLO con registros inválidos (texto que no se pudo convertir)
df_export_invalidos = df.loc[mask_invalida_con_texto, cols_export_todos].copy()
df_export_invalidos.to_excel(
    'FechaCargue_Docs_Soportes_ORFEO_invalidos.xlsx',
    index=False
)
print("Archivo creado (inválidos): FechaCargue_Docs_Soportes_ORFEO_invalidos.xlsx")


Columnas candidatas para FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO: ['FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO', 'FechaCargueDocsSoportesOrfeo_RAW', 'FechaCargueDocsSoportesOrfeo_TXT_LIMPIO', 'FechaCargueDocsSoportesOrfeo_TXT_UP', 'FechaDeCargueDocsYSoportesExpOrfeo_SQL']
Usando columna: FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO


C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\3836610023.py:81: UserWarning: Parsing dates in %Y/%m/%d/%H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')
C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\3836610023.py:81: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')



=== Resumen FECHA DE CARGUE DOCUMENTOS Y SOPORTES EN EXPDTE ORFEO ===
Fechas válidas (no NaT): 2486
Fechas inválidas (NaT con texto no vacío): 71

=== Ejemplos de filas LISTAS PARA MIGRAR (fechas válidas) ===
 Fila_Excel_aprox        FMI DEPARTAMENTO                   MUNICIPIO FECHA DE CARGUE DOCUEMNTOS Y SOPORTES EN EXPDTE ORFEO FechaCargueDocsSoportesOrfeo_TXT_LIMPIO FechaDeCargueDocsYSoportesExpOrfeo_SQL
                4  062-37361      BOLIVAR                    ZAMBRANO                                   2024-12-21 00:00:00                     2024-12-21 00:00:00                             2024-12-21
                7  062-37360      BOLIVAR                    ZAMBRANO                                   2024-12-21 00:00:00                     2024-12-21 00:00:00                             2024-12-21
                8  062-37356      BOLIVAR                    ZAMBRANO                                   2024-12-21 00:00:00                     2024-12-21 00:00:00                  

## FECHA GESTION DE ETAPA SIT

In [ ]:
import numpy as np
import pandas as pd
import re

# ============================================
# 1) Ubicar la columna de FECHA GESTION ETAPA SIT en el Excel
# ============================================
candidatas_sit = [
    c for c in df.columns
    if ('FECHA' in str(c).upper()
        and 'SIT' in str(c).upper()
        and ('GEST' in str(c).upper() or 'ETAPA' in str(c).upper()))
]

print("Columnas candidatas para FECHA GESTION DE ETAPA SIT:", candidatas_sit)

if candidatas_sit:
    col_sit = candidatas_sit[0]
else:
    # Ajusta este nombre exacto si ves otro en df.columns
    col_sit = 'FECHA GESTION DE ETAPA SIT'

print("Usando columna:", col_sit)

# ============================================
# 2) Columnas auxiliares RAW / TEXTO LIMPIO
# ============================================
df['FechaGestionEtapaSIT_RAW'] = df[col_sit]

df['FechaGestionEtapaSIT_TXT_LIMPIO'] = (
    df['FechaGestionEtapaSIT_RAW']
      .astype(str)
      .str.strip()
)

# Normalizar vacíos típicos
mask_vacio_sit = df['FechaGestionEtapaSIT_TXT_LIMPIO'].str.upper().isin(['', 'NAN', 'NAT'])
df.loc[mask_vacio_sit, 'FechaGestionEtapaSIT_TXT_LIMPIO'] = np.nan

df['FechaGestionEtapaSIT_TXT_UP'] = (
    df['FechaGestionEtapaSIT_TXT_LIMPIO']
      .astype(str)
      .str.strip()
      .str.upper()
)

# ============================================
# 3) Función de parseo a datetime (SIT)
# ============================================
def parse_fecha_sit(valor):
    """
    Convierte a datetime los valores para
    FECHA GESTION DE ETAPA SIT.
    Deja como NaT textos tipo 'NO REGISTRA', 'NO APLICA',
    'NO', 'N/A', etc.
    """
    if pd.isna(valor):
        return pd.NaT

    txt = str(valor).strip()
    if txt == "":
        return pd.NaT

    tu = txt.upper()

    # Textos claramente NO fecha -> NaT
    if tu in ("NO", "NO REGISTRA", "NO APLICA", "N/A"):
        return pd.NaT

    # Unificar separadores a "/"
    t = re.sub(r"[-.]", "/", txt)
    t = re.sub(r"\s+", "/", t)  # "2024 04 25" -> "2024/04/25"

    # Intento genérico con dayfirst
    fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')

    return fecha  # si no puede, NaT


# ============================================
# 4) Columna final para SQL
# ============================================
df['FechaGestionEtapaSIT_SQL'] = (
    df['FechaGestionEtapaSIT_TXT_LIMPIO'].apply(parse_fecha_sit)
)

# ============================================
# 5) Resumen y muestras (válidos e inválidos)
# ============================================
mask_valida_sit = df['FechaGestionEtapaSIT_SQL'].notna()

mask_con_texto_sit = (
    df['FechaGestionEtapaSIT_TXT_LIMPIO'].notna()
    & (df['FechaGestionEtapaSIT_TXT_LIMPIO'].astype(str).str.strip() != '')
)

mask_invalida_con_texto_sit = ~mask_valida_sit & mask_con_texto_sit

validas_sit = mask_valida_sit.sum()
invalidas_con_texto_sit = mask_invalida_con_texto_sit.sum()

print("\n=== Resumen FECHA GESTION DE ETAPA SIT ===")
print(f"Fechas válidas (no NaT): {validas_sit}")
print(f"Fechas inválidas (NaT con texto no vacío): {invalidas_con_texto_sit}")

# Muestra de filas listas para migrar (válidas)
ej_validos_sit = df.loc[mask_valida_sit, [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    col_sit,
    'FechaGestionEtapaSIT_TXT_LIMPIO',
    'FechaGestionEtapaSIT_SQL'
]].head(30)

print("\n=== Ejemplos de filas LISTAS PARA MIGRAR (SIT, fechas válidas) ===")
if not ej_validos_sit.empty:
    print(ej_validos_sit.to_string(index=False))
else:
    print("No hay fechas válidas para mostrar.")

# Muestra de filas con texto que NO se pudo convertir
ej_invalidos_sit = df.loc[mask_invalida_con_texto_sit, [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    'FechaGestionEtapaSIT_RAW',
    'FechaGestionEtapaSIT_TXT_LIMPIO'
]].head(50)

print("\n=== Ejemplos de filas con TEXTO que NO se pudo convertir (SIT, para revisar) ===")
if not ej_invalidos_sit.empty:
    print(ej_invalidos_sit.to_string(index=False))
else:
    print("No hay filas con texto no vacío que sigan inválidas.")


# ============================================
# 6) Exportar Excels (SIT)
# ============================================

# 6.1. Excel con TODOS los registros (crudo vs limpio vs SQL)
cols_export_todos_sit = [
    'Fila_Excel_aprox',
    'FMI',
    'DEPARTAMENTO',
    'MUNICIPIO',
    col_sit,                              # columna original del Excel
    'FechaGestionEtapaSIT_RAW',           # valor crudo
    'FechaGestionEtapaSIT_TXT_LIMPIO',    # texto limpio
    'FechaGestionEtapaSIT_SQL'            # fecha final para SQL
]

df_export_todos_sit = df[cols_export_todos_sit].copy()
df_export_todos_sit.to_excel(
    'FechaGestion_Etapa_SIT_raw_vs_sql.xlsx',
    index=False
)
print("\nArchivo creado (todos): FechaGestion_Etapa_SIT_raw_vs_sql.xlsx")

# 6.2. Excel SOLO con registros inválidos (texto que no se pudo convertir)
df_export_invalidos_sit = df.loc[mask_invalida_con_texto_sit, cols_export_todos_sit].copy()
df_export_invalidos_sit.to_excel(
    'FechaGestion_Etapa_SIT_invalidos.xlsx',
    index=False
)
print("Archivo creado (inválidos): FechaGestion_Etapa_SIT_invalidos.xlsx")


Columnas candidatas para FECHA GESTION DE ETAPA SIT: ['FECHA GESTION DE ETAPA SIT', 'FechaGestionEtapaSIT_RAW', 'FechaGestionEtapaSIT_TXT_LIMPIO', 'FechaGestionEtapaSIT_TXT_UP', 'FechaGestionEtapaSIT_SQL']
Usando columna: FECHA GESTION DE ETAPA SIT

=== Resumen FECHA GESTION DE ETAPA SIT ===
Fechas válidas (no NaT): 422
Fechas inválidas (NaT con texto no vacío): 1

=== Ejemplos de filas LISTAS PARA MIGRAR (SIT, fechas válidas) ===
 Fila_Excel_aprox        FMI       DEPARTAMENTO              MUNICIPIO FECHA GESTION DE ETAPA SIT FechaGestionEtapaSIT_TXT_LIMPIO FechaGestionEtapaSIT_SQL
               29   068-2956            BOLIVAR                MORALES        2025-10-28 00:00:00             2025-10-28 00:00:00               2025-10-28
               76  214-19727         LA GUAJIRA     SAN JUAN DEL CESAR        2025-10-28 00:00:00             2025-10-28 00:00:00               2025-10-28
              111  170-35015       CUNDINAMARCA                  PACHO        2025-10-29 00:00:00   

C:\Users\andres.garciav\AppData\Local\Temp\ipykernel_30824\3301846314.py:75: UserWarning: Parsing dates in %Y/%m/%d/%H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  fecha = pd.to_datetime(t, dayfirst=True, errors='coerce')



Archivo creado (todos): FechaGestion_Etapa_SIT_raw_vs_sql.xlsx
Archivo creado (inválidos): FechaGestion_Etapa_SIT_invalidos.xlsx
